# WSC Project - Data Analysis & NLP

In [ ]:
# Standard library imports
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Data science imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pandas as pd

# NLP imports
from transformers import pipeline, AutoTokenizer, AutoModel
import pandas as pd

# Jupyter settings
%matplotlib inline
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Display settings for full text
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

# Jupyter settings
%matplotlib inline
plt.style.use("seaborn-v0_8")
sns.set_palette("husl")

# Generate labels for actions using LLM

In [18]:
transcrpits_processed = pd.read_csv('data/transcripts_folds.csv')
transcrpits_processed = transcrpits_processed[['sample_id', 'Text']]
row_dicts = transcrpits_processed.apply(lambda row: {"text": row["Text"], "sample_id": row["sample_id"]}, axis=1).tolist()
row_dicts[1050:1150]

# ACTIONS = list(set(pd.read_csv("data/actions.csv")["parameter"]))
# # ACTIONS

[{'text': 'Thompson drives nice fake annaleise at home.', 'sample_id': 1051},
 {'text': 'The lobster panga off the backboard.', 'sample_id': 1052},
 {'text': "Here's Harris little step back over Buddy Hield.",
  'sample_id': 1053},
 {'text': 'A terrific player now heard her steps back.', 'sample_id': 1054},
 {'text': 'I say Cannonball fake stepped into three.', 'sample_id': 1055},
 {'text': 'Take him back to Walker, fakes the three.', 'sample_id': 1056},
 {'text': "He's going to get more relaxed and more comfortable up here in the big step back 3.",
  'sample_id': 1057},
 {'text': 'Step back three for James.', 'sample_id': 1058},
 {'text': 'Fox is going to throw lobs.', 'sample_id': 1059},
 {'text': 'actor Paul George steps back Bob to the Hawks, knock it away and take it away.',
  'sample_id': 1060},
 {'text': "Bird no look passing side and it's good by Gerson.",
  'sample_id': 1061},
 {'text': 'Jordan Bell goes inside, misses the jam, but he got hit.',
  'sample_id': 1062},
 {'text':

In [ ]:
WRITE = False

pseudo_actions_labels = [
{"text": "If you go into that defensive circle and post up, you notice the defensive players behind you and policy and left that jump look a little bit short.", "transcript_id": 0, "action_detected": ["post up"]},
{"text": "He's double teamed.", "transcript_id": 1, "action_detected": ["double team"]},
{"text": "Jordan trying to post up against Oxymora.", "transcript_id": 2, "action_detected": ["post up"]},
{"text": "They're going to come and help on all of his post ups.", "transcript_id": 3, "action_detected": ["post up"]},
{"text": "They're going to come and help on all of his post ups.", "transcript_id": 4, "action_detected": ["post up"]},
{"text": "Maybe tonight not to double team as much.", "transcript_id": 5, "action_detected": ["double team"]},
{"text": "Or for tried to tuck in, but Lowry always takes those post ups personally.", "transcript_id": 6, "action_detected": ["post up"]},
{"text": "Doesn't get the finger roll challenged at the rim.", "transcript_id": 7, "action_detected": ["finger roll"]},
{"text": "Now over tomorrow good pump, fake jokisch.", "transcript_id": 8, "action_detected": ["pump fake"]},
{"text": "I was three shots and made 0 Russell penetration floater.", "transcript_id": 9, "action_detected": ["floater"]},
{"text": "Well, I saw a different type of defense against the Raptors double teamed every time he touched the ball is Westbrook runs is for the midrange shot of fibrosing.", "transcript_id": 10, "action_detected": ["double team"]},
{"text": "Richardson came over from Philly last year and Cauley Stein went for the slam dunk.", "transcript_id": 11, "action_detected": ["slam dunk"]},
{"text": "Richardson came over from Philly last year and Cauley Stein went for the slam dunk.", "transcript_id": 12, "action_detected": ["slam dunk"]},
{"text": "In a pretty good pick and roll team as well.", "transcript_id": 13, "action_detected": ["pick and roll"]},
{"text": "His floater is good.", "transcript_id": 14, "action_detected": ["floater"]},
{"text": "Take that away and Brandon Goodwin takes it coast to coast against Prince.", "transcript_id": 15, "action_detected": ["coast to coast"]},
{"text": "And they just conceded a pick and roll.", "transcript_id": 16, "action_detected": ["pick and roll"]},
{"text": "Coast to coast kissed it off the glass short rebound white.", "transcript_id": 17, "action_detected": ["coast to coast"]},
{"text": "Slam dunk champion.", "transcript_id": 18, "action_detected": ["slam dunk"]},
{"text": "Left handed floater for two, see this little hesitation move.", "transcript_id": 19, "action_detected": ["floater"]},
{"text": "Little pick and roll game Kemba Walker to master at that and Daniel Theis unable to put it down.", "transcript_id": 20, "action_detected": ["pick and roll"]},
{"text": "Downhill in the finger roll is good.", "transcript_id": 21, "action_detected": ["finger roll"]},
{"text": "Just the second time he's had a losing record with Eric at the helm, there's the floater from Trae Young to start thinking.", "transcript_id": 22, "action_detected": ["floater"]},
{"text": "Here's the post up for Danilo.", "transcript_id": 23, "action_detected": ["post up"]},
{"text": "There is an outlet pass.", "transcript_id": 24, "action_detected": ["outlet pass"]},
{"text": "No way, no way here comes out around the corner up a ghost floater counted Mitchell 17.", "transcript_id": 25, "action_detected": ["floater"]},
{"text": "Ben Simmons got all the way to the bucket in the finger roll good.", "transcript_id": 26, "action_detected": ["finger roll"]},
{"text": "DeAngelo Russell leans in and gets the floater.", "transcript_id": 27, "action_detected": ["floater"]},
{"text": "Are underway John Michael Austin Carr an Angel Gray welcome aboard for our first broadcast of a 2021 Darius Garland will find college Sexton Sexton High posted Drummond rubbing works on Capella, finds a country in Larry ***** back out the X of all five Cavaliers touched the ball Garland midrange floater.", "transcript_id": 28, "action_detected": ["floater", "post up"]},
{"text": "Are underway John Michael Austin Carr an Angel Gray welcome aboard for our first broadcast of a 2021 Darius Garland will find college Sexton Sexton High posted Drummond rubbing works on Capella, finds a country in Larry ***** back out the X of all five Cavaliers touched the ball Garland midrange floater.", "transcript_id": 29, "action_detected": ["floater", "post up"]},
{"text": "The flooring goes to work seven to shoot, dancing in pump fakes, spins away from pressure late in.", "transcript_id": 30, "action_detected": ["pump fake"]},
{"text": "He dropped in on angles and Rudy Little late coming over and at high floater for two.", "transcript_id": 31, "action_detected": ["floater"]},
{"text": "You go under that pick and roll.", "transcript_id": 32, "action_detected": ["pick and roll"]},
{"text": "She's strong seasons with Orlando for the last couple of campaigns, a floater by Garland is no good.", "transcript_id": 33, "action_detected": ["floater"]},
{"text": "Nice pick and roll with Dedmon turns cancer challenges.", "transcript_id": 34, "action_detected": ["pick and roll"]},
{"text": "Fournier, chased by Chalmers and the fadeaway, is in.", "transcript_id": 35, "action_detected": ["fadeaway"]},
{"text": "And they're having some issues getting to the weak side and pick and roll so.", "transcript_id": 36, "action_detected": ["pick and roll"]},
{"text": "So post up Robinson, teasing him with that stunt track never came really.", "transcript_id": 37, "action_detected": ["post up"]},
{"text": "Is attempts have exploded with Milwaukee, took him a few extra years after Brooke, but he's there now too, as Robin wins the tip in Milwaukee controls.", "transcript_id": 38, "action_detected": ["tip in"]},
{"text": "So I think the Knicks are gonna go to that more Nixa double teaming.", "transcript_id": 39, "action_detected": []},  # as in your example → None
{"text": "He gets it off of pick and rolls and things like that smart.", "transcript_id": 40, "action_detected": ["pick and roll"]},
{"text": "Josh Tip in the crew chief.", "transcript_id": 41, "action_detected": ["tip in"]},
{"text": "A little pick and roll action between yards and drew and drew his.", "transcript_id": 42, "action_detected": ["pick and roll"]},
{"text": "DeRozan to Belinelli the fadeaway off the window in Good Marco Belinelli.", "transcript_id": 43, "action_detected": ["fadeaway"]},
{"text": "Usually when your back is tight you're not able to get the floaters or go off the dribble like you like him.", "transcript_id": 44, "action_detected": ["floater"]},
{"text": "We'll double team.", "transcript_id": 45, "action_detected": ["double team"]},
{"text": "Just Jackson with Elena little floater.", "transcript_id": 46, "action_detected": ["floater"]},
{"text": "Just Jackson with Elena little floater.", "transcript_id": 47, "action_detected": ["floater"]},
{"text": "Always starts the game posting up.", "transcript_id": 48, "action_detected": ["post up"]},
{"text": "The good pump fake.", "transcript_id": 49, "action_detected": ["pump fake"]},
{"text": "Robert Williams into the game and immediately flushes in an alley Oop pass from Kemba.", "transcript_id": 50, "action_detected": ["alley oop"]},
{"text": "Gallinari posted up against Ingram.", "transcript_id": 51, "action_detected": ["post up"]},
{"text": "Here is MB sons will bring the double team and they forced the early turnover.", "transcript_id": 52, "action_detected": ["double team"]},
{"text": "Jackson drive the floater off the glass.", "transcript_id": 53, "action_detected": ["floater"]},
{"text": "Initially it tried that finger roll in balance units in an effort to save it.", "transcript_id": 54, "action_detected": ["finger roll"]},
{"text": "Middleton to fake the floater?", "transcript_id": 55, "action_detected": ["fake", "floater"]},
{"text": "Adams getting that dunk off a high pick and roll.", "transcript_id": 56, "action_detected": ["pick and roll"]},
{"text": "The young guys trying to find their rhythm play some pick and roll today.", "transcript_id": 57, "action_detected": ["pick and roll"]},
{"text": "Yeah, pick and roll.", "transcript_id": 58, "action_detected": ["pick and roll"]},
{"text": "Nice pick and roll herder.", "transcript_id": 59, "action_detected": ["pick and roll"]},
{"text": "Is Derek with a hot floater cut the rim?", "transcript_id": 60, "action_detected": ["floater"]},
{"text": "Alley Oop intercepted by Embiid Hornets.", "transcript_id": 61, "action_detected": ["alley oop"]},
{"text": "Alley Oop intercepted by Embiid Hornets.", "transcript_id": 62, "action_detected": ["alley oop"]},
{"text": "Alley Oop intercepted by Embiid Hornets.", "transcript_id": 63, "action_detected": ["alley oop"]},
{"text": "Alley Oop intercepted by Embiid Hornets.", "transcript_id": 64, "action_detected": ["alley oop"]},
{"text": "Bring their vice Nights uniform, ever scouted that alley oop very well they'd like to go to Whiteside and Bam, Adebayo on those passes.", "transcript_id": 65, "action_detected": ["alley oop"]},
{"text": "Hard drives another alley oop.", "transcript_id": 66, "action_detected": ["alley oop"]},
{"text": "Santa ran ski with a floater and it's worn on the right.", "transcript_id": 67, "action_detected": ["floater"]},
{"text": "High rainbow shot.", "transcript_id": 68, "action_detected": ["rainbow shot"]},
{"text": "He's been relentless as a post up.", "transcript_id": 69, "action_detected": ["post up"]},
{"text": "They dial up Julius Randle for post up right away.", "transcript_id": 70, "action_detected": ["post up"]},
{"text": "The teardrop couldn't get it to go.", "transcript_id": 71, "action_detected": ["teardrop"]},
{"text": "Thompson also had his first NBA 222 nights ago in that game against Indiana Garlands, exactly the foul line's teardrop is true.", "transcript_id": 72, "action_detected": ["teardrop"]},
{"text": "Gobert could not get the tip in on that end in Putun had it not able to convert.", "transcript_id": 73, "action_detected": ["tip in"]},
{"text": "The floater doesn't go for Simmons.", "transcript_id": 74, "action_detected": ["floater"]},
{"text": "They're going to let these guards shoot these little floaters.", "transcript_id": 75, "action_detected": ["floater"]},
{"text": "You may see some double teams from the wars that you gotta rotate out of it.", "transcript_id": 76, "action_detected": ["double team"]},
{"text": "Tatum double team.", "transcript_id": 77, "action_detected": ["double team"]},
{"text": "So how quick are these double teams coming for Detroit?", "transcript_id": 78, "action_detected": ["double team"]},
{"text": "Comics and Trey Floater from Trey's little law.", "transcript_id": 79, "action_detected": ["floater"]},
{"text": "Millsap wants the post up.", "transcript_id": 80, "action_detected": ["post up"]},
{"text": "15 foot high offer nothing but net and he wanted to back.", "transcript_id": 81, "action_detected": ["nothing but net"]},
{"text": "Gotta post up Cody Zeller.", "transcript_id": 82, "action_detected": ["post up"]},
{"text": "Garrett Temple gets away from Josh Hart slam dunk.", "transcript_id": 83, "action_detected": ["slam dunk"]},
{"text": "Double teaming already thinking about Curry.", "transcript_id": 84, "action_detected": ["double team"]},
{"text": "Now it's Adams wanted to post up favors.", "transcript_id": 85, "action_detected": ["post up"]},
{"text": "Gary Harris turns the corner now Millsap great pump fake.", "transcript_id": 86, "action_detected": ["pump fake"]},
{"text": "Taylor on the run to Bazemore in the corner and he splashes in the tree.", "transcript_id": 87, "action_detected": ["splash"]},
{"text": "A little pick and roll action between yards and drew and drew his.", "transcript_id": 88, "action_detected": ["pick and roll"]},
{"text": "Millsaps, trying to power his way to the basket, little finger, rolled over removed by Millsap, but good position.", "transcript_id": 89, "action_detected": ["finger roll"]},
{"text": "Randall pulls the mist down, brings it to four court NYT up 6 Randall in the Lane double team.", "transcript_id": 90, "action_detected": ["double team"]},
{"text": "Yeah, both teams try to score at the rim, but then there's Splash Mountain taking the team three let's their center.", "transcript_id": 91, "action_detected": ["splash"]},
{"text": "post up the smaller Trae Young and Sweet Junior missing badly there.", "transcript_id": 92, "action_detected": ["post up"]},
{"text": "Player for the Thunder last night to Billy Donovan relied on over 15 minutes is Mitchell gets inside and his floater drops in.", "transcript_id": 93, "action_detected": ["floater"]},
{"text": "Loves that little back down put post up there.", "transcript_id": 94, "action_detected": ["post up"]},
{"text": "Nothing but net.", "transcript_id": 95, "action_detected": ["nothing but net"]},
{"text": "Here a little power throwing his body into Julie's random backs him up and then showing off his little finger roll.", "transcript_id": 96, "action_detected": ["finger roll"]},
{"text": "Pascal look into post up strong take and it doesn't go.", "transcript_id": 97, "action_detected": ["post up"]},
{"text": "Catholic by three paint double team gets it back out to Taj Gibson.", "transcript_id": 98, "action_detected": ["double team"]},
{"text": "Depoorter the floater falls.", "transcript_id": 99, "action_detected": ["floater"]},
{"text": "But back at you with the alley Oop, older Harden gets the ball.", "transcript_id": 100, "action_detected": ["alley oop"]},
{"text": "Nearly had the tip in to get the same shot that Gordon got.", "transcript_id": 101, "action_detected": ["tip in"]},
{"text": "Boston between the legs short.", "transcript_id": 102, "action_detected": ["between the legs"]},
{"text": "Good chemistry between those two, they play a lot of pick and roll together.", "transcript_id": 103, "action_detected": ["pick and roll"]},
{"text": "Here's Rose Derrick Rose with that floater.", "transcript_id": 104, "action_detected": ["floater"]},
{"text": "70 shoot it gets into the paint on a tough fadeaway that's off the front of the iron.", "transcript_id": 105, "action_detected": ["fadeaway"]},
{"text": "Coast to coast now kicks it back out.", "transcript_id": 106, "action_detected": ["coast to coast"]},
{"text": "Coast to coast now kicks it back out.", "transcript_id": 107, "action_detected": ["coast to coast"]},
{"text": "Now over tomorrow good pump, fake jokisch.", "transcript_id": 108, "action_detected": ["pump fake"]},
{"text": "Tristan Thompson guarding Geron Heroes to the bucket on a pick and roll.", "transcript_id": 109, "action_detected": ["pick and roll"]},
{"text": "Partner is going to be middle pick and roll defense.", "transcript_id": 110, "action_detected": ["pick and roll"]},
{"text": "Quick move before the double team had a chance of getting there.", "transcript_id": 111, "action_detected": ["double team"]},
{"text": "Here's Trey Burke, guarded by Bledsoe on the pick and roll.", "transcript_id": 112, "action_detected": ["pick and roll"]},
{"text": "You could play the midrange game, he could post up other guards as Derrick favors.", "transcript_id": 113, "action_detected": ["post up"]},
{"text": "They're going to blitz, which means double team on the pick and roll.", "transcript_id": 114, "action_detected": ["double team", "pick and roll"]},
{"text": "They're going to blitz, which means double team on the pick and roll.", "transcript_id": 115, "action_detected": ["double team", "pick and roll"]},
{"text": "He was double teamed over to McCollum Corner 3.", "transcript_id": 116, "action_detected": ["double team"]},
{"text": "And also Westbrook guys that can defend post up players in this League.", "transcript_id": 117, "action_detected": ["post up"]},
{"text": "He beats her during transition and Tomahawks one arranging Tomahawk from Lehman to get things started here in Atlanta.", "transcript_id": 118, "action_detected": ["tomahawk"]},
{"text": "He beats her during transition and Tomahawks one arranging Tomahawk from Lehman to get things started here in Atlanta.", "transcript_id": 119, "action_detected": ["tomahawk"]},
{"text": "Listed at 51 and 28 third place in the West, parted with between the legs to set up Capella.", "transcript_id": 120, "action_detected": ["between the legs"]},
{"text": "360 with a slam dunk.", "transcript_id": 121, "action_detected": ["slam dunk"]},
{"text": "Bacon pick and roll game with Vucevic off the dribble.", "transcript_id": 122, "action_detected": ["pick and roll"]},
{"text": "Bacon pick and roll game with Vucevic off the dribble.", "transcript_id": 123, "action_detected": ["pick and roll"]},
{"text": "Kowhai Leonard looking to post up Jamal Murray.", "transcript_id": 124, "action_detected": ["post up"]},
{"text": "So can you start faster than pick and roll defense?", "transcript_id": 125, "action_detected": ["pick and roll"]},
{"text": "Double team comms, Domasin Trouble and Harden makes the steal his heart came with double.", "transcript_id": 126, "action_detected": ["double team"]},
{"text": "On the wings of Bradley against Curry shot clocks down to seven pick and roll picked up off the floor nicely by Gortat, Johnson.", "transcript_id": 127, "action_detected": ["pick and roll"]},
{"text": "He steps up on the pick and rolls and make sure that he's not hanging back in the Lane.", "transcript_id": 128, "action_detected": ["pick and roll"]},
{"text": "It appears Lopez for three is nothing but net.", "transcript_id": 129, "action_detected": ["nothing but net"]},
{"text": "Sacoche finger rolling at home.", "transcript_id": 130, "action_detected": ["finger roll"]},
{"text": "Bridges little floater in the Lane, no yellow and offensive rebound.", "transcript_id": 131, "action_detected": ["floater"]},
{"text": "Effectively touches fadeaway jumper.", "transcript_id": 132, "action_detected": ["fadeaway"]},
{"text": "Thomas Bank shot in it.", "transcript_id": 133, "action_detected": ["bank shot"]},
{"text": "Thomas Bank shot in it.", "transcript_id": 134, "action_detected": ["bank shot"]},
{"text": "Because of Josh Richardson's injury, he's already made a game winner this year for Christmas, and The Teardrop by Brogdon missed in.", "transcript_id": 135, "action_detected": ["teardrop"]},
{"text": "Who would you like to Posterize in this League?", "transcript_id": 136, "action_detected": ["poster"]},
{"text": "Murray, the floater, that Mrs Jokic tips it.", "transcript_id": 137, "action_detected": ["floater"]},
{"text": "pick and roll with the big man.", "transcript_id": 138, "action_detected": ["pick and roll"]},
{"text": "He'll take it to the rack.", "transcript_id": 139, "action_detected": ["take it to the rack"]},
{"text": "Richardson, on his hips head, gets the role on the floater.", "transcript_id": 140, "action_detected": ["floater"]},
{"text": "Double team their shot Clock down to two.", "transcript_id": 141, "action_detected": ["double team"]},
{"text": "So tough on the block through double team.", "transcript_id": 142, "action_detected": ["double team"]},
{"text": "Then again, tips in his own mess.", "transcript_id": 143, "action_detected": ["tip in"]},
{"text": "Carlin has been more of the point guard, the penetration, the floater off the rim.", "transcript_id": 144, "action_detected": ["floater"]},
{"text": "The teardrop no.", "transcript_id": 145, "action_detected": ["teardrop"]},
{"text": "Zip surpassed the young finds a little people around the Horn Turner Rainbow shot the mark.", "transcript_id": 146, "action_detected": ["rainbow shot"]},
{"text": "Nice past Cauley Stein attacking little finger roll put in the book and send them to the line.", "transcript_id": 147, "action_detected": ["finger roll"]},
{"text": "Orlando goes to the pick and roll game with bolts driving it to the basket.", "transcript_id": 148, "action_detected": ["pick and roll"]},
{"text": "Booker nearly had the tip in, and then he collides down low.", "transcript_id": 149, "action_detected": ["tip in"]},
{"text": "Very good at the pick and roll, and very bouncy.", "transcript_id": 150, "action_detected": ["pick and roll"]},
{"text": "Thomas Jefferson will go coast to coast.", "transcript_id": 151, "action_detected": ["coast to coast"]},
{"text": "You wanna pick and roll?", "transcript_id": 152, "action_detected": ["pick and roll"]},
{"text": "2 free throws for Josh Jackson and we are tide at four is frameworks that pick and roll this time on the side of steel by Jackson.", "transcript_id": 153, "action_detected": ["pick and roll"]},
{"text": "Double teaming side by the Celtics.", "transcript_id": 154, "action_detected": ["double team"]},
{"text": "Starting down the Lane, little floater tipped up and out.", "transcript_id": 155, "action_detected": ["floater"]},
{"text": "Leslie Matthews inside gets the floater to fall.", "transcript_id": 156, "action_detected": ["floater"]},
{"text": "He had a double team.", "transcript_id": 157, "action_detected": ["double team"]},
{"text": "His forms giving chase gotta contest those jumpers as he'll knocks down the fadeaway jumper.", "transcript_id": 158, "action_detected": ["fadeaway"]},
{"text": "MB pump fakes and drives in on land.", "transcript_id": 159, "action_detected": ["pump fake"]},
{"text": "Little leaner floater that is straight in future.", "transcript_id": 160, "action_detected": ["floater"]},
{"text": "Pope tend to shoot for the Lakers two point jump shot tipped in by LeBron and the Lakers last season led the League in tipping points per possession.", "transcript_id": 161, "action_detected": ["tip in"]},
{"text": "I pick and roll with Jones this time.", "transcript_id": 162, "action_detected": ["pick and roll"]},
{"text": "Alanna factor defensively RHJ going coast to coast for the Deuce.", "transcript_id": 163, "action_detected": ["coast to coast"]},
{"text": "That's on the pick and roll.", "transcript_id": 164, "action_detected": ["pick and roll"]},
{"text": "I pick and roll evolving Brunson in Cleveland and with that shot by Jalen Brunson, Dallas is the first delivered tonight.", "transcript_id": 165, "action_detected": ["pick and roll"]},
{"text": "You'll see him post up.", "transcript_id": 166, "action_detected": ["post up"]},
{"text": "They want to put him in the pick and roll.", "transcript_id": 167, "action_detected": ["pick and roll"]},
{"text": "And here's the Dallas native Myles Turner for one of the things Miles worked on was his post up game.", "transcript_id": 168, "action_detected": ["post up"]},
{"text": "Middleton, great job swish.", "transcript_id": 169, "action_detected": ["swish"]},
{"text": "Conley's teardrop.", "transcript_id": 170, "action_detected": ["teardrop"]},
{"text": "He couldn't find the hell down along the alley Oop.", "transcript_id": 171, "action_detected": ["alley oop"]},
{"text": "He couldn't find the hell down along the alley Oop.", "transcript_id": 172, "action_detected": ["alley oop"]},
{"text": "He couldn't find the hell down along the alley Oop.", "transcript_id": 173, "action_detected": ["alley oop"]},
{"text": "Knox comes to help out this week and doing a lot of dropping and double teaming.", "transcript_id": 174, "action_detected": ["double team"]},
{"text": "Shot Clock down to Claude will post up in a fadeaway here for Sexton.", "transcript_id": 175, "action_detected": ["post up", "fadeaway"]},
{"text": "Shot Clock down to Claude will post up in a fadeaway here for Sexton.", "transcript_id": 176, "action_detected": ["post up", "fadeaway"]},
{"text": "Bogdanovich puts it down on the floor with his left hand, goes to his right and can't finish with the finger roll.", "transcript_id": 177, "action_detected": ["finger roll"]},
{"text": "Driving with the finger roll the rim coming All Star looking moving.", "transcript_id": 178, "action_detected": ["finger roll"]},
{"text": "Wait for the double team.", "transcript_id": 179, "action_detected": ["double team"]},
{"text": "It's the teardrop floater that was about as high as you can throw.", "transcript_id": 180, "action_detected": ["teardrop", "floater"]},
{"text": "It's the teardrop floater that was about as high as you can throw.", "transcript_id": 181, "action_detected": ["teardrop", "floater"]},
{"text": "Fat shoot nothing but net for Casey gotta find more shots for Casey's meeting.", "transcript_id": 182, "action_detected": ["nothing but net"]},
{"text": "He had never had a major injury in his life and he was out for the entire season, as garlands floater doesn't go.", "transcript_id": 183, "action_detected": ["floater"]},
{"text": "He had never had a major injury in his life and he was out for the entire season, as garlands floater doesn't go.", "transcript_id": 184, "action_detected": ["floater"]},
{"text": "Two of the best alley oop growers in the League.", "transcript_id": 185, "action_detected": ["alley oop"]},
{"text": "Try the switching pick and roll.", "transcript_id": 186, "action_detected": ["pick and roll"]},
{"text": "Pick and roll played very nicely.", "transcript_id": 187, "action_detected": ["pick and roll"]},
{"text": "Running floater is good for Serge Ibaka.", "transcript_id": 188, "action_detected": ["floater"]},
{"text": "Tastic pump fakes slash and move without it he put on a show indeed he did and we are underway.", "transcript_id": 189, "action_detected": ["pump fake"]},
{"text": "Nice floater.", "transcript_id": 190, "action_detected": ["floater"]},
{"text": "Nice floater.", "transcript_id": 191, "action_detected": ["floater"]},
{"text": "Miles with a pump fake back to Van Fleet.", "transcript_id": 192, "action_detected": ["pump fake"]},
{"text": "The 4th is Julia wanted to post up ******.", "transcript_id": 193, "action_detected": ["post up"]},
{"text": "Kevin at RIM run thereby Davis forced the mismatching great recognition Rim run to a post up and he asked you shot jump out of DVD.", "transcript_id": 194, "action_detected": ["post up"]},
{"text": "Here's the floater again by Brockton.", "transcript_id": 195, "action_detected": ["floater"]},
{"text": "Current is a double team.", "transcript_id": 196, "action_detected": ["double team"]},
{"text": "Here is a Baka using the jab steps just going on.", "transcript_id": 197, "action_detected": ["jab step"]},
{"text": "Outlet pass broken up and then Capella.", "transcript_id": 198, "action_detected": ["outlet pass"]},
{"text": "Split a double team gets his own miss.", "transcript_id": 199, "action_detected": ["double team"]},
{"text": "After he gets there you can go to the floater Luka Doncic's can do the same thing.", "transcript_id": 200, "action_detected": ["floater"]},
{"text": "Allison goes coast to coast with the left hand layup off the glass.", "transcript_id": 201, "action_detected": ["coast to coast"]},
{"text": "What makes the coast to coast?", "transcript_id": 202, "action_detected": ["coast to coast"]},
{"text": "Chocolate down to two hard double team got found.", "transcript_id": 203, "action_detected": ["double team"]},
{"text": "Pick and roll with Capella.", "transcript_id": 204, "action_detected": ["pick and roll"]},
{"text": "Harden backs off Simmons Simmons with a floater no good.", "transcript_id": 205, "action_detected": ["floater"]},
{"text": "Missing the free throw line floater.", "transcript_id": 206, "action_detected": ["floater"]},
{"text": "Missing the free throw line floater.", "transcript_id": 207, "action_detected": ["floater"]},
{"text": "Point shot is going to be looking to get into the paint post up.", "transcript_id": 208, "action_detected": ["post up"]},
{"text": "He can stretch you out, but also he is one of the best and being active around the glass and in the pick and roll.", "transcript_id": 209, "action_detected": ["pick and roll"]},
{"text": "Davis gets into it quickly on a fadeaway rings out regarded by Aldridge.", "transcript_id": 210, "action_detected": ["fadeaway"]},
{"text": "The floater box Ray rebounded by zoo Patch.", "transcript_id": 211, "action_detected": ["floater"]},
{"text": "Denver on the push and Millsap trying to go coast to coast miss with the left.", "transcript_id": 212, "action_detected": ["coast to coast"]},
{"text": "Coast to coast Simmons lazatin.", "transcript_id": 213, "action_detected": ["coast to coast"]},
{"text": "On the NBA campus, tried to give and go.", "transcript_id": 214, "action_detected": ["give and go"]},
{"text": "Wait posting up Hayward.", "transcript_id": 215, "action_detected": ["post up"]},
{"text": "Thought he was fouled and one day missing on the floater.", "transcript_id": 216, "action_detected": ["floater"]},
{"text": "Inside, Alan spins into a double team.", "transcript_id": 217, "action_detected": ["double team"]},
{"text": "DeMarcus Cousins now Jerrod Jackson table layout to start the game and there's the pick and roll I'm talking about.", "transcript_id": 218, "action_detected": ["pick and roll"]},
{"text": "Dillard's passes deflected ends up in Aminu's hands anyway, out of the double team Feeding Harkless Harkless with five to Shoot Boy active hands by the Grizzlies.", "transcript_id": 219, "action_detected": ["double team"]},
{"text": "Very little floater and who found him.", "transcript_id": 220, "action_detected": ["floater"]},
{"text": "a lot of that is Cameron Payne with a beautiful floater.", "transcript_id": 221, "action_detected": ["floater"]},
{"text": "Now you gotta wear this out if they're not going to double team, they're just milking Kevin Love.", "transcript_id": 222, "action_detected": ["double team"]},
{"text": "But the basically as we see Tobias with the little post up.", "transcript_id": 223, "action_detected": ["post up"]},
{"text": "Murray keeps it, can't hit the floater, batted out.", "transcript_id": 224, "action_detected": ["floater"]},
{"text": "Backdoor Cat Curry picked up on a double team.", "transcript_id": 225, "action_detected": ["backdoor", "double team"]},
{"text": "Double team comes from none for to shoot.", "transcript_id": 226, "action_detected": ["double team"]},
{"text": "We play on simple and easy stealing love outlet passes for the basket.", "transcript_id": 227, "action_detected": ["outlet pass"]},
{"text": "I'll give it another try to Porter the floater falls.", "transcript_id": 228, "action_detected": ["floater"]},
{"text": "Double team aren't Vanvleet goes the other way down to four on the shot Clock on an OB to the bucket.", "transcript_id": 229, "action_detected": ["double team"]},
{"text": "George went through some of his numbers as Kevin Knox misses on the other end, but chases down the rebound and tipping in Robinson.", "transcript_id": 230, "action_detected": ["tip in"]},
{"text": "Patrick Beverley still coming back from that calf injury that floater halfway down and out.", "transcript_id": 231, "action_detected": ["floater"]},
{"text": "Post up for dropping right away, banging with Alex Len.", "transcript_id": 232, "action_detected": ["post up"]},
{"text": "There just shouldn't happen, and you need not double teaming Jokic on the low block if you're gonna stay home with their shooting.", "transcript_id": 233, "action_detected": ["double team"]},
{"text": "Aldridge gets rid before the double team, but trust the shot.", "transcript_id": 234, "action_detected": ["double team"]},
{"text": "Having some trouble in the post up and the loose ball is picked up by the white power pole.", "transcript_id": 235, "action_detected": ["post up"]},
{"text": "Double teams going away by Lonzo.", "transcript_id": 236, "action_detected": ["double team"]},
{"text": "Slam dunk champ.", "transcript_id": 237, "action_detected": ["slam dunk"]},
{"text": "Locked out of five temple looking for the pick, pick and roll without fighting fires.", "transcript_id": 238, "action_detected": ["pick and roll"]},
{"text": "The fadeaway for two.", "transcript_id": 239, "action_detected": ["fadeaway"]},
{"text": "But get to Adams quickly on that pick and roll.", "transcript_id": 240, "action_detected": ["pick and roll"]},
{"text": "Oh, the left hand floater.", "transcript_id": 241, "action_detected": ["floater"]},
{"text": "Now those floaters not falling.", "transcript_id": 242, "action_detected": ["floater"]},
{"text": "Rodney Hood gets inside, Floater, misses ball, knock loose, Hood gets it back.", "transcript_id": 243, "action_detected": ["floater"]},
{"text": "Middleton tries to take advantage double team comp.", "transcript_id": 244, "action_detected": ["double team"]},
{"text": "Full head of Steam Marvin Teardrop in and out.", "transcript_id": 245, "action_detected": ["teardrop"]},
{"text": "Tatum with a nice floater over the top Kemba.", "transcript_id": 246, "action_detected": ["floater"]},
{"text": "can't get the floater to fall foul.", "transcript_id": 247, "action_detected": ["floater"]},
{"text": "Of course in scoring tray with a floater or try again.", "transcript_id": 248, "action_detected": ["floater"]},
{"text": "McKee on the pick and roll and that's how you.", "transcript_id": 249, "action_detected": ["pick and roll"]},
{"text": "The floaters won't go rebound deflected and taken by Peyton.", "transcript_id": 250, "action_detected": ["floater"]},
{"text": "Damien outside double team bounces over to Dirk dumps at Low Brain.", "transcript_id": 251, "action_detected": ["double team"]},
{"text": "Lillard posting up against Brunson finds a cutter for the weak side and Dirks defense against Yusif.", "transcript_id": 252, "action_detected": ["post up"]},
{"text": "We've got flop Cam standing by as well as Kyle Lowry berries at three.", "transcript_id": 253, "action_detected": ["flop"]},
{"text": "Statistically worst defense in the NBA is Isaiah Thomas who misses on the floater.", "transcript_id": 254, "action_detected": ["floater"]},
{"text": "The Left handed player at Bettys shut the majority of those floaters with his right hand.", "transcript_id": 255, "action_detected": ["floater"]},
{"text": "He loves that left side of the floor and a fadeaway jumper over the Magic 7 foot center.", "transcript_id": 256, "action_detected": ["fadeaway"]},
{"text": "Pick and roll every single time as you see KD coming off of Cousins.", "transcript_id": 257, "action_detected": ["pick and roll"]},
{"text": "Fox, the little floater.", "transcript_id": 258, "action_detected": ["floater"]},
{"text": "What makes the coast to coast?", "transcript_id": 259, "action_detected": ["coast to coast"]},
{"text": "The double team finds Trent Junior Swanigan swings it around.", "transcript_id": 260, "action_detected": ["double team"]},
{"text": "A good sign that Conley makes his first floater Simmons, with a little baby hook shot in the paint.", "transcript_id": 261, "action_detected": ["floater"]},
{"text": "Have to take a fadeaway contested shot TJ Warren.", "transcript_id": 262, "action_detected": ["fadeaway"]},
{"text": "Rescreening is Gibson Teague with the floater.", "transcript_id": 263, "action_detected": ["floater"]},
{"text": "Brown with a floater banks it in Bruce Brown the rookie out of Miami of Florida.", "transcript_id": 264, "action_detected": ["floater"]},
{"text": "Little finger roll goals in a chance for three point play and then it's out to a quick corner bead.", "transcript_id": 265, "action_detected": ["finger roll"]},
{"text": "Great touch with the floater.", "transcript_id": 266, "action_detected": ["floater"]},
{"text": "Yeah really can hurt you off the pick and roll game is approved as a 3 point shooter hurt Miami beyond the arc from the dotted line.", "transcript_id": 267, "action_detected": ["pick and roll"]},
{"text": "Great young little floater rolls in.", "transcript_id": 268, "action_detected": ["floater"]},
{"text": "Here comes holiday office screen Double teamed over Rondo into the Lane drops in the more ball cap.", "transcript_id": 269, "action_detected": ["double team"]},
{"text": "Johnson also had his first NBA 222 nights ago in that game against Indiana Garlands, exactly the foul line's teardrop is true.", "transcript_id": 270, "action_detected": ["teardrop"]},
{"text": "But when he comes cross selling off their brush green, he's a good post up player.", "transcript_id": 271, "action_detected": ["post up"]},
{"text": "Here's Quarry, watched by Hood, picked off by Zubats, Hawaii with the floater.", "transcript_id": 272, "action_detected": ["floater"]},
{"text": "Chief works the inline drops it to Nurkic, draws a double team of a couple of seven footers.", "transcript_id": 273, "action_detected": ["double team"]},
{"text": "Don't try and judge us until we are whole and healthy and they double team balance Unison.", "transcript_id": 274, "action_detected": ["double team"]},
{"text": "Thanks for the floater, and that's the first story again.", "transcript_id": 275, "action_detected": ["floater"]},
{"text": "He was six for seven on floaters last night.", "transcript_id": 276, "action_detected": ["floater"]},
{"text": "Veteran head coach in the NBA, Williams off to turn over Anna Randall with a slam dunk.", "transcript_id": 277, "action_detected": ["slam dunk"]},
{"text": "Westbrook drives and kicks Ferguson, who had a couple of trees early in that game, finds Grant into the link to the glass finger, rolled up no Panther defense.", "transcript_id": 278, "action_detected": ["finger roll"]},
{"text": "Welsh posting up.", "transcript_id": 279, "action_detected": ["post up"]},
{"text": "Of course immediately double team Jokic who passes cross court to Murray.", "transcript_id": 280, "action_detected": ["double team"]},
{"text": "A7 footer had never done before, never done and then they created the post up at the free throw line.", "transcript_id": 281, "action_detected": ["post up"]},
{"text": "It's interesting they trying to switch some pick and rolls.", "transcript_id": 282, "action_detected": ["pick and roll"]},
{"text": "Chased by Kuczma pick and roll, it Gobert loss to handle.", "transcript_id": 283, "action_detected": ["pick and roll"]},
{"text": "Two nights ago in San Antonio, Osmonds floater doesn't go Thompson with the offensive rebound.", "transcript_id": 284, "action_detected": ["floater"]},
{"text": "Cunningham swings it forms chased off the line floater for Brent.", "transcript_id": 285, "action_detected": ["floater"]},
{"text": "Well, this is a pick and roll.", "transcript_id": 286, "action_detected": ["pick and roll"]},
{"text": "Well, if you're not going to send the double team and he gives it that deep, there really isn't much you can do about it so long and creative, easy finish thanks.", "transcript_id": 287, "action_detected": ["double team"]},
{"text": "Pick and roll involving Brunson, Kleber, Brunson in the paint looking for the first basket of the game and there it is looks like a vet man.", "transcript_id": 288, "action_detected": ["pick and roll"]},
{"text": "Look and book settle for that fadeaway jump shot.", "transcript_id": 289, "action_detected": ["fadeaway"]},
{"text": "Nice up there and try to get the tip instead.", "transcript_id": 290, "action_detected": ["tip in"]},
{"text": "Carl attack that front foot in Nice give and go right there.", "transcript_id": 291, "action_detected": ["give and go"]},
{"text": "Here's Connolly into the pain and missing the floater.", "transcript_id": 292, "action_detected": ["floater"]},
{"text": "He was in flip flops.", "transcript_id": 293, "action_detected": []},
{"text": "Here comes called her on the high pick and roll rolling inside LeBron James year behind the back floater short.", "transcript_id": 294, "action_detected": ["pick and roll", "behind the back", "floater"]},
{"text": "Here comes called her on the high pick and roll rolling inside LeBron James year behind the back floater short.", "transcript_id": 295, "action_detected": ["pick and roll", "behind the back", "floater"]},
{"text": "It's tough little floater from the baseline over the top of that.", "transcript_id": 296, "action_detected": ["floater"]},
{"text": "Donovan, like the left hand, work behind the screen of Rudy Gobert, the high floater banked it off the glass, and the ball comes loose.", "transcript_id": 297, "action_detected": ["floater"]},
{"text": "Drew holiday in the little floater.", "transcript_id": 298, "action_detected": ["floater"]},
{"text": "They double teaming quickly in the post.", "transcript_id": 299, "action_detected": ["double team"]},
{"text": "Post up for Jokic, dump it into cragan he felt.", "transcript_id": 300, "action_detected": ["post up"]},
{"text": "The first point to the game with a nice floater.", "transcript_id": 301, "action_detected": ["floater"]},
{"text": "Well you seem a double teams right now 'cause the Blazers have number.", "transcript_id": 302, "action_detected": ["double team"]},
{"text": "Doubled Rivers Reborn Rivers defender came over to double team James Horton.", "transcript_id": 303, "action_detected": ["double team"]},
{"text": "JV update drives 18 stays with the bank shot.", "transcript_id": 304, "action_detected": ["bank shot"]},
{"text": "What at this, and I think that went between the legs of Aten.", "transcript_id": 305, "action_detected": ["between the legs"]},
{"text": "And here comes Troy Brown Junior Goes Coast to Coast pizza but swatted away by MTSU.", "transcript_id": 306, "action_detected": ["coast to coast"]},
{"text": "Former slam dunk letter J are put on there with nice men blue.", "transcript_id": 307, "action_detected": ["slam dunk"]},
{"text": "At home we played two of the first three at Home Alley Oop attempt Aaron Drummond, right, there, not a smart pass.", "transcript_id": 308, "action_detected": ["alley oop"]},
{"text": "Looking for the pretty alley Oop Jazz will take it the other way.", "transcript_id": 309, "action_detected": ["alley oop"]},
{"text": "But running the pick and rolls to perfection that last time to Curry Johnson might have gotten away with the travel, but he scores off glass.", "transcript_id": 310, "action_detected": ["pick and roll"]},
{"text": "Will take the three little splash it money.", "transcript_id": 311, "action_detected": ["splash"]},
{"text": "The floater guides it home an one.", "transcript_id": 312, "action_detected": ["floater"]},
{"text": "This is the floater.", "transcript_id": 313, "action_detected": ["floater"]},
{"text": "The alley oop.", "transcript_id": 314, "action_detected": ["alley oop"]},
{"text": "Necessary to throw that high type of floater, but he's just showing off his skills that's involved with Supreme confidence.", "transcript_id": 315, "action_detected": ["floater"]},
{"text": "Are double team on Luka?", "transcript_id": 316, "action_detected": ["double team"]},
{"text": "Remember all the games that Clint's done that just running the floor, forget about pick and roll game, and Harden, throwing lob dunks to him.", "transcript_id": 317, "action_detected": ["pick and roll", "lob"]},
{"text": "Splendid Alley oop that time to pump.", "transcript_id": 318, "action_detected": ["alley oop"]},
{"text": "Will post up against Kobe White.", "transcript_id": 319, "action_detected": ["post up"]},
{"text": "Chandler down the floor so he couldn't bother and got great post up position Jackson going baseline lob it goes high and Hammers at home.", "transcript_id": 320, "action_detected": ["post up", "lob"]},
{"text": "Start with Cody Zeller, PJ Washington trying to post up Prince.", "transcript_id": 321, "action_detected": ["post up"]},
{"text": "post up the smaller Trae Young and Sweet Junior missing badly there.", "transcript_id": 322, "action_detected": ["post up"]},
{"text": "Zatarain's key with the floater.", "transcript_id": 323, "action_detected": ["floater"]},
{"text": "Glad you're bored here on Fox Sports Ohio as they go right away for a post up to Morse, triple team swatted away by Alec Burks.", "transcript_id": 324, "action_detected": ["post up"]},
{"text": "That one is tipped in by McGee.", "transcript_id": 325, "action_detected": ["tip in"]},
{"text": "Oregon State's Gary Payton, defending lunar here's Whiteside runs into the Lane, takes it strong in the Cup to lay it in the pick and roll action right there.", "transcript_id": 326, "action_detected": ["pick and roll"]},
{"text": "Nobody wants to get on that poster.", "transcript_id": 327, "action_detected": ["poster"]},
{"text": "Could help by favors on the pick and roll Hardaway from deep.", "transcript_id": 328, "action_detected": ["pick and roll"]},
{"text": "He's tide up on a double team Good help defense and a travel.", "transcript_id": 329, "action_detected": ["double team"]},
{"text": "Will wait for the double team and then I'll give it up.", "transcript_id": 330, "action_detected": ["double team"]},
{"text": "The floater, the bounce, the bucket.", "transcript_id": 331, "action_detected": ["floater"]},
{"text": "On the post up, Jeffrey Pretty Easy start coming.", "transcript_id": 332, "action_detected": ["post up"]},
{"text": "He puts up a little floater that's no good.", "transcript_id": 333, "action_detected": ["floater"]},
{"text": "He's going against a lot of smaller guards so that post up games becoming important Hemi.", "transcript_id": 334, "action_detected": ["post up"]},
{"text": "Nice dish underneath that ball swatted away baseball pass downfield.", "transcript_id": 335, "action_detected": ["baseball pass"]},
{"text": "And finger roll short, therefore Capella.", "transcript_id": 336, "action_detected": ["finger roll"]},
{"text": "So already 2 forced turnovers from the Clipper defense and their setting pick and rolls with Porzingis.", "transcript_id": 337, "action_detected": ["pick and roll"]},
{"text": "Green comes up short, James on the tip in 27 points for LeBron James.", "transcript_id": 338, "action_detected": ["tip in"]},
{"text": "We've got flop Cam standing by as well as Kyle Lowry berries at three.", "transcript_id": 339, "action_detected": ["flop"]},
{"text": "Ana Steele and Ferguson comes here and finger rolls it up again.", "transcript_id": 340, "action_detected": ["finger roll"]},
{"text": "I still got the sweet shot swish.", "transcript_id": 341, "action_detected": ["swish"]},
{"text": "Yeah, they want a little side pick and roll for who roots.", "transcript_id": 342, "action_detected": ["pick and roll"]},
{"text": "Now you gotta see the double team areas and opportunities for those in there.", "transcript_id": 343, "action_detected": ["double team"]},
{"text": "Is Jackson looking along the post up here to Griffith?", "transcript_id": 344, "action_detected": ["post up"]},
{"text": "Quickest turn around, this fires hurry the outlet pass through Durant.", "transcript_id": 345, "action_detected": ["outlet pass"]},
{"text": "He can go up high, he's got the hops so he's one of those guys on the pick and roll.", "transcript_id": 346, "action_detected": ["pick and roll"]},
{"text": "Play it, but it is tipped in by John Collins.", "transcript_id": 347, "action_detected": ["tip in"]},
{"text": "Post up opportunity for it to white Howard Anna kills in and you seen a lot of that issue.", "transcript_id": 348, "action_detected": ["post up"]},
{"text": "Splash is the 3 pointer.", "transcript_id": 349, "action_detected": ["splash"]},
{"text": "As tears started to show, some promise as a roller to the hoop off pick and rolls, Shotblocker averaged just under 10 points per game.", "transcript_id": 350, "action_detected": ["pick and roll"]},
{"text": "Unable to connect on the floater.", "transcript_id": 351, "action_detected": ["floater"]},
{"text": "Shooting in that game against New York, here's a blocker with a floater inside and he knocks it down.", "transcript_id": 352, "action_detected": ["floater"]},
{"text": "I wasn't a double team so brands like I'm used to that.", "transcript_id": 353, "action_detected": ["double team"]},
{"text": "Little pick and roll game.", "transcript_id": 354, "action_detected": ["pick and roll"]},
{"text": "You get an easy dunk poster.", "transcript_id": 355, "action_detected": ["poster"]},
{"text": "Posting up in the paint.", "transcript_id": 356, "action_detected": ["post up"]},
{"text": "It could be a floater 30 foot 3 pointer, didn't score.", "transcript_id": 357, "action_detected": ["floater"]},
{"text": "Lowry coast to coast rejected inside, but a good follow by Ibaka.", "transcript_id": 358, "action_detected": ["coast to coast"]},
{"text": "Again, Turner will be open, moving and grooving nicely at that time to get the finger roll.", "transcript_id": 359, "action_detected": ["finger roll"]},
{"text": "Again, Turner will be open, moving and grooving nicely at that time to get the finger roll.", "transcript_id": 360, "action_detected": ["finger roll"]},
{"text": "I see there's a double team.", "transcript_id": 361, "action_detected": ["double team"]},
{"text": "No double team.", "transcript_id": 362, "action_detected": ["double team"]},
{"text": "Steven Adams usually likes to get out and guard the pick and roll in attack and Blitz.", "transcript_id": 363, "action_detected": ["pick and roll"]},
{"text": "No help, no double team.", "transcript_id": 364, "action_detected": ["double team"]},
{"text": "Posting up the smaller Lowry no get Lowry for the fouls.", "transcript_id": 365, "action_detected": ["post up"]},
{"text": "Their percentage, low Giannis late getting into the action so Henson has to work through a double team and he draws a foul goddess got popped.", "transcript_id": 366, "action_detected": ["double team"]},
{"text": "That's will certainly run their share of pick and rolls.", "transcript_id": 367, "action_detected": ["pick and roll"]},
{"text": "nothing but net of the bad variety PJ, every goal breaks the seal.", "transcript_id": 368, "action_detected": ["nothing but net"]},
{"text": "Did a good job against the Nets to knockdown floaters in the paint Dillon Brooks pump.", "transcript_id": 369, "action_detected": ["floater"]},
{"text": "Double team on James.", "transcript_id": 370, "action_detected": ["double team"]},
{"text": "Anthony Pick and roll with Vucevic.", "transcript_id": 371, "action_detected": ["pick and roll"]},
{"text": "Covington the floater.", "transcript_id": 372, "action_detected": ["floater"]},
{"text": "Now the beautiful floater.", "transcript_id": 373, "action_detected": ["floater"]},
{"text": "Rozier gets into the paint and he gets the rainbow teardrop.", "transcript_id": 374, "action_detected": ["teardrop"]},
{"text": "There's a floater Jonathan Isaac on the attack.", "transcript_id": 375, "action_detected": ["floater"]},
{"text": "Here's ball floater.", "transcript_id": 376, "action_detected": ["floater"]},
{"text": "Here's the cruder again, an undrafted player out of Kansas State and Richard Super Floater.", "transcript_id": 377, "action_detected": ["floater"]},
{"text": "Utah is a big pick and roll team.", "transcript_id": 378, "action_detected": ["pick and roll"]},
{"text": "No pick and roll action.", "transcript_id": 379, "action_detected": ["pick and roll"]},
{"text": "Tough fadeaway that ones off well defended by Adams with good jobs is not letting into the paint.", "transcript_id": 380, "action_detected": ["fadeaway"]},
{"text": "The floater being one of 'em I think young had 38 in the season opener.", "transcript_id": 381, "action_detected": ["floater"]},
{"text": "He'll take it to the rack.", "transcript_id": 382, "action_detected": ["take it to the rack"]},
{"text": "Horford is is better really difficult match up in post up situations for Karl Anthony Towns.", "transcript_id": 383, "action_detected": ["post up"]},
{"text": "Balance Unix under the pump fake.", "transcript_id": 384, "action_detected": ["pump fake"]},
{"text": "He takes it a post up.", "transcript_id": 385, "action_detected": ["post up"]},
{"text": "Probably very good with those little floaters.", "transcript_id": 386, "action_detected": ["floater"]},
{"text": "fadeaway no short shot.", "transcript_id": 387, "action_detected": ["fadeaway"]},
{"text": "Melo taking it to the ram in the bank shot doesn't go Collins couldn't clean it up but does on the second chance.", "transcript_id": 388, "action_detected": ["bank shot"]},
{"text": "It's better to switch post up dance against pink.", "transcript_id": 389, "action_detected": ["post up"]},
{"text": "Jack turns the corner looking for can or stolen away by Snell Outlet Pass.", "transcript_id": 390, "action_detected": ["outlet pass"]},
{"text": "Booker was very close to an up and down there as pain splashes into three.", "transcript_id": 391, "action_detected": ["splash"]},
{"text": "Friend Forbes delivery that was an ice pick and roll, thereby Lamarcus Aldridge to free up forms.", "transcript_id": 392, "action_detected": ["pick and roll"]},
{"text": "Coast to coast and finishes with the left hand.", "transcript_id": 393, "action_detected": ["coast to coast"]},
{"text": "Pick and roll Whiteside right down the middle of the Lane and nobody here and play it up in a great start from here yet.", "transcript_id": 394, "action_detected": ["pick and roll"]},
{"text": "The Thunder win the tip in here is Westbrook.", "transcript_id": 395, "action_detected": ["tip in"]},
{"text": "In the second meeting, and Wesley Matthews splashes A3.", "transcript_id": 396, "action_detected": ["splash"]},
{"text": "Here's Carmelo Anthony inside on the post up.", "transcript_id": 397, "action_detected": ["post up"]},
{"text": "The other night Nurkic gets inside bank shot won't go the second attempt.", "transcript_id": 398, "action_detected": ["bank shot"]},
{"text": "Kemba Walker, again dealing with some knee discomfort that he's had throughout much of the season, misses his first jumper, but the ball tipped in Matthews in.", "transcript_id": 399, "action_detected": ["tip in"]},
{"text": "Tyler Johnson with the floater is off the mark.", "transcript_id": 400, "action_detected": ["floater"]},
{"text": "Barton little pick and roll with Jokic.", "transcript_id": 401, "action_detected": ["pick and roll"]},
{"text": "Switch pick and rolls between KP and smart.", "transcript_id": 402, "action_detected": ["pick and roll"]},
{"text": "A chance for Simmons post up the smaller Brooks.", "transcript_id": 403, "action_detected": ["post up"]},
{"text": "CJ toying with Wiseman nothing but net.", "transcript_id": 404, "action_detected": ["nothing but net"]},
{"text": "Rudy Gay double team swings too wide.", "transcript_id": 405, "action_detected": ["double team"]},
{"text": "They have to quickly double team Perkins.", "transcript_id": 406, "action_detected": ["double team"]},
{"text": "There they go to eat an early post upon homes.", "transcript_id": 407, "action_detected": ["post up"]},
{"text": "There they go to eat an early post upon homes.", "transcript_id": 408, "action_detected": ["post up"]},
{"text": "There they go to eat an early post upon homes.", "transcript_id": 409, "action_detected": ["post up"]},
{"text": "No double team help from the Kings and which makes them pay router which always looking across that inside shoulder to see where the defense is coming from.", "transcript_id": 410, "action_detected": ["double team"]},
{"text": "Double team.", "transcript_id": 411, "action_detected": ["double team"]},
{"text": "Just buy one tough fadeaway", "transcript_id": 412, "action_detected": ["fadeaway"]},
{"text": "Here's the fadeaways.", "transcript_id": 413, "action_detected": ["fadeaway"]},
{"text": "Andre screens for Reggie Jackson floater in the Lane, fills it up.", "transcript_id": 414, "action_detected": ["floater"]},
{"text": "Middleton floater would not go at Williams who was defending Giannis again, that time with the rebound, got numbers White supporter for free.", "transcript_id": 415, "action_detected": ["floater"]},
{"text": "Point shot is going to be looking to get into the paint post up so you gotta be able to be ready for all situations.", "transcript_id": 416, "action_detected": ["post up"]},
{"text": "See if the Warriors were able to kind of rectify that tonight with some double teaming or creative defensive matchups.", "transcript_id": 417, "action_detected": ["double team"]},
{"text": "fadeaway no short shot.", "transcript_id": 418, "action_detected": ["fadeaway"]},
{"text": "Post up opportunity.", "transcript_id": 419, "action_detected": ["post up"]},
{"text": "Hannah's double teams.", "transcript_id": 420, "action_detected": ["double team"]},
{"text": "Here is Jokic little give and go inside to Millsap.", "transcript_id": 421, "action_detected": ["give and go"]},
{"text": "Joe puts a shoulder down pick and roll.", "transcript_id": 422, "action_detected": ["pick and roll"]},
{"text": "So here's that Lillard Nurkic kind of high pick and roll.", "transcript_id": 423, "action_detected": ["pick and roll"]},
{"text": "Buccament with the floater.", "transcript_id": 424, "action_detected": ["floater"]},
{"text": "floater is no good gets it back.", "transcript_id": 425, "action_detected": ["floater"]},
{"text": "Cook with the alley Oop and Jones with the throw down.", "transcript_id": 426, "action_detected": ["alley oop"]},
{"text": "Again trying to muscle his way up despite the double team.", "transcript_id": 427, "action_detected": ["double team"]},
{"text": "Where he went coast to coast I kept.", "transcript_id": 428, "action_detected": ["coast to coast"]},
{"text": "Conley floater Valentinas taps the rebound but cannot keep it in place.", "transcript_id": 429, "action_detected": ["floater"]},
{"text": "Selden with a tough looking floater.", "transcript_id": 430, "action_detected": ["floater"]},
{"text": "Is Gallinari drives inside a series of pump fakes to get Miguel to Velma go up in the air and he's able to bank it in after a tough shooting night for him?", "transcript_id": 431, "action_detected": ["pump fake", "bank shot"]},
{"text": "Done a nice little floater in and out.", "transcript_id": 432, "action_detected": ["floater"]},
{"text": "Orange splits a double team works his way to the rim, and somehow finagles delay up out of that.", "transcript_id": 433, "action_detected": ["double team"]},
{"text": "Pascal jab step.", "transcript_id": 434, "action_detected": ["jab step"]},
{"text": "Nothing but net.", "transcript_id": 435, "action_detected": ["nothing but net"]},
{"text": "Taking the floater in the Lane hits the backroom goes and gets his own miss.", "transcript_id": 436, "action_detected": ["floater"]},
{"text": "Go coast to coast and flush it in.", "transcript_id": 437, "action_detected": ["coast to coast"]},
{"text": "Post up by Lukas right into the Lane, and hello, nothing like starting your game with an easy basket.", "transcript_id": 438, "action_detected": ["post up"]},
{"text": "Rage on college work the side pick and roll Tray gets into the paint at.", "transcript_id": 439, "action_detected": ["pick and roll"]},
{"text": "And it's nothing but net and it's 2 to nothing.", "transcript_id": 440, "action_detected": ["nothing but net"]},
{"text": "Great outlet pass to wonderful Feed and LeBron in stride target easily is the former Laker when telling 10 tell you what 60 of these 60 two shots had been.", "transcript_id": 441, "action_detected": ["outlet pass"]},
{"text": "Biggest challenge for him in the Smith Junior didn't get any rim that time on the little floater.", "transcript_id": 442, "action_detected": ["floater"]},
{"text": "Now Anderson has it pick and roll with Collins Swing and Swing to Prince.", "transcript_id": 443, "action_detected": ["pick and roll"]},
{"text": "As a matter of fact, impellers tipped in the League now is 60 dumps.", "transcript_id": 444, "action_detected": ["tip in"]},
{"text": "James Wiseman complete go drop coverage but then get all the way out there because of his length and really contest that well so wise men little post up early.", "transcript_id": 445, "action_detected": ["post up"]},
{"text": "Former Rookie of the Year there was that double team and then we end up having the out of bounds violation.", "transcript_id": 446, "action_detected": ["double team"]},
{"text": "So Cousins ends up going one for three at the foul line, little floater for Brogdon.", "transcript_id": 447, "action_detected": ["floater"]},
{"text": "So Cousins ends up going one for three at the foul line, little floater for Brogdon.", "transcript_id": 448, "action_detected": ["floater"]},
{"text": "No double team needed on that play.", "transcript_id": 449, "action_detected": ["double team"]},
{"text": "Oka 4 Double teamed immediately and he traveled.", "transcript_id": 450, "action_detected": ["double team"]},
{"text": "Art van guarded by Butler puts up a floater.", "transcript_id": 451, "action_detected": ["floater"]},
{"text": "At his best, that's when he can be super effective when you can pick and roll to the basket.", "transcript_id": 452, "action_detected": ["pick and roll"]},
{"text": "At his best, that's when he can be super effective when you can pick and roll to the basket.", "transcript_id": 453, "action_detected": ["pick and roll"]},
{"text": "At his best, that's when he can be super effective when you can pick and roll to the basket.", "transcript_id": 454, "action_detected": ["pick and roll"]},
{"text": "At his best, that's when he can be super effective when you can pick and roll to the basket.", "transcript_id": 455, "action_detected": ["pick and roll"]},
{"text": "Here's a 2 on one Millsap, the alley oop missed the layup and the other thing Mark just over a week ago.", "transcript_id": 456, "action_detected": ["alley oop"]},
{"text": "Ego doll and they double team.", "transcript_id": 457, "action_detected": ["double team"]},
{"text": "Outside for Derrick Rose little floater, from the Lane that won't fall.", "transcript_id": 458, "action_detected": ["floater"]},
{"text": "Now he's double teamed and now he's in trouble.", "transcript_id": 459, "action_detected": ["double team"]},
{"text": "Here's the floater again by Brockton.", "transcript_id": 460, "action_detected": ["floater"]},
{"text": "Lori left wing floater misses.", "transcript_id": 461, "action_detected": ["floater"]},
{"text": "Pick and roll.", "transcript_id": 462, "action_detected": ["pick and roll"]},
{"text": "Course recently took fighter to second straight Slam Dunk contest coming in second.", "transcript_id": 463, "action_detected": ["slam dunk"]},
{"text": "Transition defense would be very important and this is one of the reasons why, because they got five guys, all of them capable of taking it coast to coast.", "transcript_id": 464, "action_detected": ["coast to coast"]},
{"text": "Danny Green Jays swish", "transcript_id": 465, "action_detected": ["swish"]},
{"text": "Thank you, Rudy, but then promises them both and they get the easy tip in to pull within two 9189 Spurs answer.", "transcript_id": 466, "action_detected": ["tip in"]},
{"text": "Passes from Blake Griffin who draws a double team in the post.", "transcript_id": 467, "action_detected": ["double team"]},
{"text": "He can stretch you out, but also he is one of the best and being active around the glass and in the pick and roll.", "transcript_id": 468, "action_detected": ["pick and roll"]},
{"text": "And the tip in the backcourt to Derek White.", "transcript_id": 469, "action_detected": ["tip in"]},
{"text": "Sad OK for a late double team.", "transcript_id": 470, "action_detected": ["double team"]},
{"text": "Yeah, what you gotta do if your memory is gotta try to work him out a little bit and then the double team has to come.", "transcript_id": 471, "action_detected": ["double team"]},
{"text": "I did buy into the cool blow on a switch now Porzingis posting up turn around J fuel.", "transcript_id": 472, "action_detected": ["post up"]},
{"text": "He's a stretch for he can post up smaller players.", "transcript_id": 473, "action_detected": ["post up"]},
{"text": "The floater.", "transcript_id": 474, "action_detected": ["floater"]},
{"text": "Milwaukee's basketball first holiday, with a little floater from the Lane through holiday, fills it up.", "transcript_id": 475, "action_detected": ["floater"]},
{"text": "Out of a double team.", "transcript_id": 476, "action_detected": ["double team"]},
{"text": "You get an easy dunk poster.", "transcript_id": 477, "action_detected": ["poster"]},
{"text": "Flopping was back to his high school days in Louisiana.", "transcript_id": 478, "action_detected": ["flop"]},
{"text": "Simmons misses the finger roll.", "transcript_id": 479, "action_detected": ["finger roll"]},
{"text": "Patients are at the pump fake of the flyby.", "transcript_id": 480, "action_detected": ["pump fake"]},
{"text": "His fadeaway gun won't go.", "transcript_id": 481, "action_detected": ["fadeaway"]},
{"text": "Double team that time got it back for blood so this time.", "transcript_id": 482, "action_detected": ["double team"]},
{"text": "Will it will bounce Modert double team went between defenders had it ripped and they say he walks to the basketball.", "transcript_id": 483, "action_detected": ["double team"]},
{"text": "There's double team paper tries to turn the corner.", "transcript_id": 484, "action_detected": ["double team"]},
{"text": "Little weird awkward floater so Bowman 41% on threes.", "transcript_id": 485, "action_detected": ["floater"]},
{"text": "Or is that low pick and roll?", "transcript_id": 486, "action_detected": ["pick and roll"]},
{"text": "He's written a check for the League office for flopping a couple of times.", "transcript_id": 487, "action_detected": ["flop"]},
{"text": "Is Derek with a hot floater cut the rim?", "transcript_id": 488, "action_detected": ["floater"]},
{"text": "Miscommunication on the turnover and Rashawn Thomas, but tried to go behind the back in the reverse dunk.", "transcript_id": 489, "action_detected": ["behind the back", "reverse dunk"]},
{"text": "Now he's getting more post up basement.", "transcript_id": 490, "action_detected": ["post up"]},
{"text": "Now those floaters not falling.", "transcript_id": 491, "action_detected": ["floater"]},
{"text": "Again, to take it off to a quick start yesterday throwing up alley oop that doesn't workout is picked up.", "transcript_id": 492, "action_detected": ["alley oop"]},
{"text": "Again, to take it off to a quick start yesterday throwing up alley oop that doesn't workout is picked up.", "transcript_id": 493, "action_detected": ["alley oop"]},
{"text": "With Chester Turner pump faking Mr. Doe Bentley is love.", "transcript_id": 494, "action_detected": ["pump fake"]},
{"text": "Mark the size the length theory is trying to get the alley oop pass.", "transcript_id": 495, "action_detected": ["alley oop"]},
{"text": "Look at splash.", "transcript_id": 496, "action_detected": ["splash"]},
{"text": "Shock like down to five Levine looking to create Ombre tomb shuffles to his right, fires three and splashes intends to be hot.", "transcript_id": 497, "action_detected": ["splash"]},
{"text": "No double team help cover sets up the jump book.", "transcript_id": 498, "action_detected": ["double team"]},
{"text": "One turnovers out of Houston, but you gotta convert off of it as K CP, nothing but net.", "transcript_id": 499, "action_detected": ["nothing but net"]},
{"text": "Double team.", "transcript_id": 500, "action_detected": ["double team"]},
{"text": "There's a turnover block Coast to coast.", "transcript_id": 501, "action_detected": ["coast to coast"]},
{"text": "Dennis had a 2 for two at the line in the first app button over to their top for the alley Oop.", "transcript_id": 502, "action_detected": ["alley oop"]},
{"text": "There is that Birds Eye view of the pick and roll in the Sixers, defending it late rotation.", "transcript_id": 503, "action_detected": ["pick and roll"]},
{"text": "He'll take it to the rack for this delay up, but Ruby is there for the put back.", "transcript_id": 504, "action_detected": ["take it to the rack"]},
{"text": "Let's bear get that easy to CJ McCollum, the floater.", "transcript_id": 505, "action_detected": ["floater"]},
{"text": "That was a teardrop jumper, but", "transcript_id": 506, "action_detected": ["teardrop"]},
{"text": "No double team help.", "transcript_id": 507, "action_detected": ["double team"]},
{"text": "Horford looking to post up and they're gonna file.", "transcript_id": 508, "action_detected": ["post up"]},
{"text": "How do you stop hard not to pick and roll as soon as someone shows me weather travel to go under.", "transcript_id": 509, "action_detected": ["pick and roll"]},
{"text": "They double team play closer to four court nearly threw it away but didn't, and Luke Kennard is fouled.", "transcript_id": 510, "action_detected": ["double team"]},
{"text": "Notch double team five on the shot Clock to double team by Tiegs.", "transcript_id": 511, "action_detected": ["double team"]},
{"text": "Notch double team five on the shot Clock to double team by Tiegs.", "transcript_id": 512, "action_detected": ["double team"]},
{"text": "Double teamed and Lowry turns it over and the Wizards get the ball.", "transcript_id": 513, "action_detected": ["double team"]},
{"text": "He thought he was gonna pass it, but good recognition there and knocked down a tough little floater.", "transcript_id": 514, "action_detected": ["floater"]},
{"text": "Get that soft left hand floater.", "transcript_id": 515, "action_detected": ["floater"]},
{"text": "Will pick and roll with Lowry.", "transcript_id": 516, "action_detected": ["pick and roll"]},
{"text": "Harold, trying to post up Marvin Kimble comes over to help out breaks it away.", "transcript_id": 517, "action_detected": ["post up"]},
{"text": "By Looney there's a double team we talked about right away.", "transcript_id": 518, "action_detected": ["double team"]},
{"text": "Couple pump fakes and then the finish.", "transcript_id": 519, "action_detected": ["pump fake"]},
{"text": "Accepting the double teamed be coming more than leader.", "transcript_id": 520, "action_detected": ["double team"]},
{"text": "Toronto Raptors saying he's guarding the pick and roll and the ref says no.", "transcript_id": 521, "action_detected": ["pick and roll"]},
{"text": "We got double team.", "transcript_id": 522, "action_detected": ["double team"]},
{"text": "Mr Floater rebounded Brown.", "transcript_id": 523, "action_detected": ["floater"]},
{"text": "Bans the roll man with a nice finger roll inside in Boston.", "transcript_id": 524, "action_detected": ["finger roll"]},
{"text": "And that's his first, the Knicks are all heavy post up team.", "transcript_id": 525, "action_detected": ["post up"]},
{"text": "Little pump fake holiday corner three it's true at the expiration of the shot Clock.", "transcript_id": 526, "action_detected": ["pump fake"]},
{"text": "Then what he misses on the floater?", "transcript_id": 527, "action_detected": ["floater"]},
{"text": "For Maps whip it around Burke to try floater on the baseline.", "transcript_id": 528, "action_detected": ["floater"]},
{"text": "Kindly pick and roll valentinas attacks.", "transcript_id": 529, "action_detected": ["pick and roll"]},
{"text": "Jonas Valance Eunice with a slam dunk.", "transcript_id": 530, "action_detected": ["slam dunk"]},
{"text": "Would look at the angle on Atom 3 point swish Lopez.", "transcript_id": 531, "action_detected": ["swish"]},
{"text": "On that play, you have a double team on Lamarcus Aldridge, an.", "transcript_id": 532, "action_detected": ["double team"]},
{"text": "Early post up for Wiggins.", "transcript_id": 533, "action_detected": ["post up"]},
{"text": "Andre Drummond with the tip in.", "transcript_id": 534, "action_detected": ["tip in"]},
{"text": "Here's Neil Aquino posting up.", "transcript_id": 535, "action_detected": ["post up"]},
{"text": "Port is trying to post up.", "transcript_id": 536, "action_detected": ["post up"]},
{"text": "Daniel, get the start again tonight on the finger roll.", "transcript_id": 537, "action_detected": ["finger roll"]},
{"text": "Nichols draws a double team.", "transcript_id": 538, "action_detected": ["double team"]},
{"text": "This no good drum and Brown gobbles up the rebound long outlet pass.", "transcript_id": 539, "action_detected": ["outlet pass"]},
{"text": "Off the Adams screen, Alexander Walker, with a floater in it's for nothing.", "transcript_id": 540, "action_detected": ["floater"]},
{"text": "Going to see a lot of pick and roll tonight folks on virtually every play.", "transcript_id": 541, "action_detected": ["pick and roll"]},
{"text": "Hit your opponent in the face but land with the deep post up.", "transcript_id": 542, "action_detected": ["post up"]},
{"text": "Here's Courtney Lee, the veteran the floaters up.", "transcript_id": 543, "action_detected": ["floater"]},
{"text": "Goes Coast to coast for Toronto.", "transcript_id": 544, "action_detected": ["coast to coast"]},
{"text": "Pascal jab step and then between the legs on a pull up.", "transcript_id": 545, "action_detected": ["jab step", "between the legs"]},
{"text": "Pascal jab step and then between the legs on a pull up.", "transcript_id": 546, "action_detected": ["jab step", "between the legs"]},
{"text": "He can be a threat, especially when teams are looking to double team.", "transcript_id": 547, "action_detected": ["double team"]},
{"text": "Alakina gets into the paint, stops, double team back out.", "transcript_id": 548, "action_detected": ["double team"]},
{"text": "Inside that time the finger roll was nicely contested by Steven Adams.", "transcript_id": 549, "action_detected": ["finger roll"]},
{"text": "Here it goes with the floater.", "transcript_id": 550, "action_detected": ["floater"]},
{"text": "Here's favors jab step.", "transcript_id": 551, "action_detected": ["jab step"]},
{"text": "He has the ball on the poster going after this match.", "transcript_id": 552, "action_detected": ["poster"]},
{"text": "If you can't get stops and get an early outlet pass.", "transcript_id": 553, "action_detected": ["outlet pass"]},
{"text": "Dig down to help Tray on that post up.", "transcript_id": 554, "action_detected": ["post up"]},
{"text": "Can't shake Blake Griffin with a pump fake now?", "transcript_id": 555, "action_detected": ["pump fake"]},
{"text": "There's a floater baseline and the bucket is good.", "transcript_id": 556, "action_detected": ["floater"]},
{"text": "And here's the Dallas native Myles Turner, but one of the things Miles worked on was his post up game.", "transcript_id": 557, "action_detected": ["post up"]},
{"text": "An Rubio has been running the show with ease of Lake Hill, Dr Coast to coast and draws a foul.", "transcript_id": 558, "action_detected": ["coast to coast"]},
{"text": "Again, to take it off to a quick start yesterday throwing up alley oop that doesn't workout is picked up.", "transcript_id": 559, "action_detected": ["alley oop"]},
{"text": "Again, to take it off to a quick start yesterday throwing up alley oop that doesn't workout is picked up.", "transcript_id": 560, "action_detected": ["alley oop"]},
{"text": "Not easy steps, little floater and bring it home.", "transcript_id": 561, "action_detected": ["floater"]},
{"text": "Here comes a double team, this time on Ennis Kanter who could be a piece to deal with inside.", "transcript_id": 562, "action_detected": ["double team"]},
{"text": "Here's Trey with the floater and he crossed the whistle.", "transcript_id": 563, "action_detected": ["floater"]},
{"text": "Here is right form and working on the pick and roll game.", "transcript_id": 564, "action_detected": ["pick and roll"]},
{"text": "The class goes coast to coast.", "transcript_id": 565, "action_detected": ["coast to coast"]},
{"text": "Van Fleet forces up a runner and hits nothing but net.", "transcript_id": 566, "action_detected": ["nothing but net"]},
{"text": "Which three guys can score the ball consistently here in Boston but still too strong with the finger roll?", "transcript_id": 567, "action_detected": ["finger roll"]},
{"text": "Getting that is young off of Kevin Love trying to get him some post up touch.", "transcript_id": 568, "action_detected": ["post up"]},
{"text": "I like to double team over the Hill.", "transcript_id": 569, "action_detected": ["double team"]},
{"text": "Fraser had a screen from drama, gives it up to Drummond and back to Frazier Little floater off the window.", "transcript_id": 570, "action_detected": ["floater"]},
{"text": "Half court set here for Brooklyn Pick and Roll Action Allen over Go Berry doesn't get the role.", "transcript_id": 571, "action_detected": ["pick and roll"]},
{"text": "Campus pick and roll with seller and that is slap back by Dedmon was very poor shot there by Zella.", "transcript_id": 572, "action_detected": ["pick and roll"]},
{"text": "Games on the Drive shot Clock winding down bank shot Mrs Smart 3 pounds ahead to Tatum.", "transcript_id": 573, "action_detected": ["bank shot"]},
{"text": "If you leave truly on James Harden and Chris Paul out of the pick and roll, Harden catches fires.", "transcript_id": 574, "action_detected": ["pick and roll"]},
{"text": "Likes to post up, give it up against the double team.", "transcript_id": 575, "action_detected": ["post up", "double team"]},
{"text": "Likes to post up, give it up against the double team.", "transcript_id": 576, "action_detected": ["post up", "double team"]},
{"text": "You can already see that that defensive priority is going to be Bradley Beal, so he's gonna have opportunities floater in the Lane for RJ Barrett.", "transcript_id": 577, "action_detected": ["floater"]},
{"text": "Little floater would not fall.", "transcript_id": 578, "action_detected": ["floater"]},
{"text": "Rubio off balance along the baseline still splashes it.", "transcript_id": 579, "action_detected": ["splash"]},
{"text": "Simmons drawing a double team.", "transcript_id": 580, "action_detected": ["double team"]},
{"text": "But because of his absence, you'll see Ben post up a little bit more and he's exceptional town there.", "transcript_id": 581, "action_detected": ["post up"]},
{"text": "Simmons the floater doesn't go rebound embid scores and the foul.", "transcript_id": 582, "action_detected": ["floater"]},
{"text": "It stays with his shot tips in and it's starting to slow down that run by The Mavericks.", "transcript_id": 583, "action_detected": ["tip in"]},
{"text": "Look are trying to race past the double team needs.", "transcript_id": 584, "action_detected": ["double team"]},
{"text": "And 48 pick and roll.", "transcript_id": 585, "action_detected": ["pick and roll"]},
{"text": "Here's Troy Young Works the pick and roll here with death, and as Tobias Harrison's switch it, it goes to Duane.", "transcript_id": 586, "action_detected": ["pick and roll"]},
{"text": "Pick and roll up high.", "transcript_id": 587, "action_detected": ["pick and roll"]},
{"text": "And Ben Simmons has a little room service slam dunk Philadelphia on the board 1st.", "transcript_id": 588, "action_detected": ["slam dunk"]},
{"text": "Rubbed out again by George, who will attack splits the double team Eurosteps and hasn't blocked by lead.", "transcript_id": 589, "action_detected": ["double team", "euro step"]},
{"text": "That are anti comes back with a teardrop and puts it in.", "transcript_id": 590, "action_detected": ["teardrop"]},
{"text": "You gonna double team.", "transcript_id": 591, "action_detected": ["double team"]},
{"text": "Vallentuna's with the shot Clock ticking down gets a double team.", "transcript_id": 592, "action_detected": ["double team"]},
{"text": "Bryant has some skills he can handle it and shoot 3 between the legs.", "transcript_id": 593, "action_detected": ["between the legs"]},
{"text": "Stay on the pick and roll.", "transcript_id": 594, "action_detected": ["pick and roll"]},
{"text": "One footers, leaners, floaters, stepbacks, threes, be powers it down at the other end.", "transcript_id": 595, "action_detected": ["floater", "step back"]},
{"text": "There's that good double teaming defense by the Rockets hard right at the back of Gibson and Butler fouled Tucker.", "transcript_id": 596, "action_detected": ["double team"]},
{"text": "Gallon are gonna try to post Michael Porter Junior will pump fake Porter with the reach know off the mark.", "transcript_id": 597, "action_detected": ["post up", "pump fake"]},
{"text": "That'll be critical, and Vince Carter hits nothing but net.", "transcript_id": 598, "action_detected": ["nothing but net"]},
{"text": "John Collins gets the steal and Dunk Sexton with a floater.", "transcript_id": 599, "action_detected": ["floater"]},
{"text": "Yeah, shake Milton's gonna real good job scoring the basketball in bead with a pump fake and a drive down the Lane.", "transcript_id": 600, "action_detected": ["pump fake"]},
{"text": "Yep, so we gotta watch him getting posted up.", "transcript_id": 601, "action_detected": ["post up"]},
{"text": "He gets a knock away Aaron Gordon with the Bounce Pass and Isaac with the finger roll Jonathan Isaac in his third season out of Florida State.", "transcript_id": 602, "action_detected": ["finger roll"]},
{"text": "No slam dunk.", "transcript_id": 603, "action_detected": ["slam dunk"]},
{"text": "Nice post up.", "transcript_id": 604, "action_detected": ["post up"]},
{"text": "They believe that he's a good piece and something to really play and look forward to this year, Osman breaks it up on the alley Oop attempt to Jarrett Allen.", "transcript_id": 605, "action_detected": ["alley oop"]},
{"text": "Double team in the corner and throws a terrible pass bill I had to Peyton.", "transcript_id": 606, "action_detected": ["double team"]},
{"text": "Some floater good, each one more.", "transcript_id": 607, "action_detected": ["floater"]},
{"text": "No double team.", "transcript_id": 608, "action_detected": ["double team"]},
{"text": "Middleton floater would not go at Williams who was defending Giannis again, that time with the rebound, got numbers White supporter for free.", "transcript_id": 609, "action_detected": ["floater"]},
{"text": "Turtle Pass was deflected before he got it and Jodka little floater to get the Spurs on the board.", "transcript_id": 610, "action_detected": ["floater"]},
{"text": "As you're watching Simmons, just take it to the rack himself.", "transcript_id": 611, "action_detected": ["take it to the rack"]},
{"text": "Tough to double team the way the mixer spacing the floor with their shooters.", "transcript_id": 612, "action_detected": ["double team"]},
{"text": "Of course they do it at the end of this week with the home and home against Utah Bridges Coast to coast up to turn over.", "transcript_id": 613, "action_detected": ["coast to coast"]},
{"text": "Nothing but net for Lillard.", "transcript_id": 614, "action_detected": ["nothing but net"]},
{"text": "Poor Zynga's driving on Bonga takes it to the rack.", "transcript_id": 615, "action_detected": ["take it to the rack"]},
{"text": "Attack him at every opportunity Jackson pick and roll with Drummond.", "transcript_id": 616, "action_detected": ["pick and roll"]},
{"text": "You've got to get to more like size matches inside Whiteside on the nice pick and roll Mccollam found him.", "transcript_id": 617, "action_detected": ["pick and roll"]},
{"text": "It's with a floater little bit strong and rebounded by indeed.", "transcript_id": 618, "action_detected": ["floater"]},
{"text": "The breakdown the floater.", "transcript_id": 619, "action_detected": ["floater"]},
{"text": "See right off the bat involving joellen, a little pick and roll able.", "transcript_id": 620, "action_detected": ["pick and roll"]},
{"text": "Look at the pump fakes there by.", "transcript_id": 621, "action_detected": ["pump fake"]},
{"text": "He has that ability to block the South Hill off the bounce Dixit looking move off the pick and roll for George Hill.", "transcript_id": 622, "action_detected": ["pick and roll"]},
{"text": "He just wants him to take that with confidence in the rhythm off the pick and roll there's Durant.", "transcript_id": 623, "action_detected": ["pick and roll"]},
{"text": "If you guys with that trick this year stepping around DeAndre Jordan is Klay Thompson as good anticipation by Thompson and its Grand Slam dunk?", "transcript_id": 624, "action_detected": ["slam dunk"]},
{"text": "Barnes nice jab step frees himself, but can't put it in.", "transcript_id": 625, "action_detected": ["jab step"]},
{"text": "Bogdanovich puts it down on the floor with his left hand, goes to his right and can't finish with the finger roll.", "transcript_id": 626, "action_detected": ["finger roll"]},
{"text": "That's Lillard's pick and roll.", "transcript_id": 627, "action_detected": ["pick and roll"]},
{"text": "So philosophy of Steve Kerr is that Davis is not basically a post up player.", "transcript_id": 628, "action_detected": ["post up"]},
{"text": "Courier comes to double team.", "transcript_id": 629, "action_detected": ["double team"]},
{"text": "Well, that's 'cause they double team.", "transcript_id": 630, "action_detected": ["double team"]},
{"text": "Bank shot doesn't go out on the offensive board.", "transcript_id": 631, "action_detected": ["bank shot"]},
{"text": "Ricky posted up for free.", "transcript_id": 632, "action_detected": ["post up"]},
{"text": "They did a nice job of double teaming and recovering out of that.", "transcript_id": 633, "action_detected": ["double team"]},
{"text": "Burke plays a pick and roll game with Kyle Oquinn.", "transcript_id": 634, "action_detected": ["pick and roll"]},
{"text": "IT as opposed to wait here comes Paul George, Troy Brown, Junior getting back and Paul George with the finger roll.", "transcript_id": 635, "action_detected": ["finger roll"]},
{"text": "Playing with the post up in the baby Hook.", "transcript_id": 636, "action_detected": ["post up"]},
{"text": "Fournier, again looking for Gordon posting up against Oxymora.", "transcript_id": 637, "action_detected": ["post up"]},
{"text": "Look at splash.", "transcript_id": 638, "action_detected": ["splash"]},
{"text": "Stressed Millsap post up Dudley.", "transcript_id": 639, "action_detected": ["post up"]},
{"text": "Nice pump, fake resets for three, it's long.", "transcript_id": 640, "action_detected": ["pump fake"]},
{"text": "By Trevor Ariza Westbrook on the drive gets to the rim bank shot misses in a reason to rebound.", "transcript_id": 641, "action_detected": ["bank shot"]},
{"text": "Aren't the floater gets his own miss and scores on the second chance himself?", "transcript_id": 642, "action_detected": ["floater"]},
{"text": "So talk about tremendous skill Splash Mountain for you.", "transcript_id": 643, "action_detected": ["splash"]},
{"text": "Family little floater in the paint.", "transcript_id": 644, "action_detected": ["floater"]},
{"text": "Baseball pass from Jokic.", "transcript_id": 645, "action_detected": ["baseball pass"]},
{"text": "You don't have to send the double team to Lamarcus Aldridge.", "transcript_id": 646, "action_detected": ["double team"]},
{"text": "Brook Lopez Splash Mountain missing from downtown LA 7 footer.", "transcript_id": 647, "action_detected": ["splash"]},
{"text": "Wiggins trying to split that double team lost the handle.", "transcript_id": 648, "action_detected": ["double team"]},
{"text": "Try to split the double team.", "transcript_id": 649, "action_detected": ["double team"]},
{"text": "Zachry non javel in want that return post up match up will take the dunk instead.", "transcript_id": 650, "action_detected": ["post up", "jam"]},
{"text": "Throwing down 27 points a game, Anna double tip in his one and picked up by Portland.", "transcript_id": 651, "action_detected": ["tip in"]},
{"text": "Send me right to left and put up a floater for two so.", "transcript_id": 652, "action_detected": ["floater"]},
{"text": "Bites through a double team into the corner.", "transcript_id": 653, "action_detected": ["double team"]},
{"text": "With the Splash brothers erupting.", "transcript_id": 654, "action_detected": ["splash"]},
{"text": "Hit the fadeaway.", "transcript_id": 655, "action_detected": ["fadeaway"]},
{"text": "Here's that match up if we need to Bury beat spends works through the double team, can hit close range favors for the rebound.", "transcript_id": 656, "action_detected": ["double team"]},
{"text": "gets a screen, misses badly on the floater board with the rebound.", "transcript_id": 657, "action_detected": ["floater"]},
{"text": "Met his wife and clays mom Julie, right here in Portland, Draymond Green cleans it up inside with the tip in going to rebound out their Blazers.", "transcript_id": 658, "action_detected": ["tip in"]},
{"text": "Move to force a double team and that's great.", "transcript_id": 659, "action_detected": ["double team"]},
{"text": "Pick and roll with Katie.", "transcript_id": 660, "action_detected": ["pick and roll"]},
{"text": "baseball pass from Isaiah Thomas Detroit Brown.", "transcript_id": 661, "action_detected": ["baseball pass"]},
{"text": "They tried to give and go but the pass batted away.", "transcript_id": 662, "action_detected": ["give and go"]},
{"text": "Splash is up 3.", "transcript_id": 663, "action_detected": ["splash"]},
{"text": "Pick and roll with Dedmon.", "transcript_id": 664, "action_detected": ["pick and roll"]},
{"text": "Joe turns the corner pick and roll favors in the paint booth, shot up for two.", "transcript_id": 665, "action_detected": ["pick and roll"]},
{"text": "There's that floater.", "transcript_id": 666, "action_detected": ["floater"]},
{"text": "Double team him 48 for three.", "transcript_id": 667, "action_detected": ["double team"]},
{"text": "Little high pick and roll.", "transcript_id": 668, "action_detected": ["pick and roll"]},
{"text": "Donovan nothing but net all net shot.", "transcript_id": 669, "action_detected": ["nothing but net"]},
{"text": "Chris Paul steps back short.", "transcript_id": 670, "action_detected": ["step back"]},
{"text": "Fox steps back.", "transcript_id": 671, "action_detected": ["step back"]},
{"text": "Would with the ball fake on homes change their shots.", "transcript_id": 672, "action_detected": ["fake"]},
{"text": "Would with the ball fake on homes change their shots.", "transcript_id": 673, "action_detected": ["fake"]},
{"text": "good ball fake by Middleton.", "transcript_id": 674, "action_detected": ["fake"]},
{"text": "There's a backdoor lob layman using that athleticism to spring and jam with two hands.", "transcript_id": 675, "action_detected": ["backdoor", "lob", "jam"]},
{"text": "There's a backdoor lob layman using that athleticism to spring and jam with two hands.", "transcript_id": 676, "action_detected": ["backdoor", "lob", "jam"]},
{"text": "There's a backdoor lob layman using that athleticism to spring and jam with two hands.", "transcript_id": 677, "action_detected": ["backdoor", "lob", "jam"]},
{"text": "Zach, the step back short skips off the rim of the rebound Schreuder", "transcript_id": 678, "action_detected": ["step back"]},
{"text": "Backdoor cut by Villin Bruce Brown steps in front and steals.", "transcript_id": 679, "action_detected": ["backdoor"]},
{"text": "Palomar step back J2.", "transcript_id": 680, "action_detected": ["step back"]},
{"text": "Lillard, the lob to Whiteside, gets bumps and now that's where the offensive fellas.", "transcript_id": 681, "action_detected": ["lob"]},
{"text": "Nice little step back.", "transcript_id": 682, "action_detected": ["step back"]},
{"text": "Lillard steps back.", "transcript_id": 683, "action_detected": ["step back"]},
{"text": "Thomas goes baseline kickback hot Samora the pump shake step back jumper.", "transcript_id": 684, "action_detected": ["pump fake", "step back"]},
{"text": "James steps back.", "transcript_id": 685, "action_detected": ["step back"]},
{"text": "Agreed lob down low in a jam, just that quickly.", "transcript_id": 686, "action_detected": ["lob", "jam"]},
{"text": "Agreed lob down low in a jam, just that quickly.", "transcript_id": 687, "action_detected": ["lob", "jam"]},
{"text": "Trey takes the middle Lane, moves in, find space, steps back for three of them, would shorten over the rim.", "transcript_id": 688, "action_detected": ["step back"]},
{"text": "That died five man this year when he steps back, they do, but he's not playing long tip backed out to NERC.", "transcript_id": 689, "action_detected": ["step back"]},
{"text": "Schroder, who averaged a career high 19 last season for Atlanta, goes behind the back to set it up.", "transcript_id": 690, "action_detected": ["behind the back"]},
{"text": "Lakers want to push Randall on the run, two hand Jam and the Lakers needed for nothing.", "transcript_id": 691, "action_detected": ["jam"]},
{"text": "KD cutting backdoor you saw Dudley rotate.", "transcript_id": 692, "action_detected": ["backdoor"]},
{"text": "Step back jumper by Poyan is perfect.", "transcript_id": 693, "action_detected": ["step back"]},
{"text": "Now take a look and gather it in not get sick 1 three in a row from the Raptors two last season one this season Jokic step back and he's got it.", "transcript_id": 694, "action_detected": ["step back"]},
{"text": "And a step back bank, formerly Heil American, buried at.", "transcript_id": 695, "action_detected": ["step back", "bank shot"]},
{"text": "Olynyk with a pretty ball fake an easy layup for him.", "transcript_id": 696, "action_detected": ["fake"]},
{"text": "The jam didn't take him long.", "transcript_id": 697, "action_detected": ["jam"]},
{"text": "Step back three by Blake Griffin is good, and it's 7 nothing Pistons.", "transcript_id": 698, "action_detected": ["step back"]},
{"text": "His eyes, his vision could pass fake could find Sefolosha.", "transcript_id": 699, "action_detected": ["fake"]},
{"text": "Vucevic buys the head fake it.", "transcript_id": 700, "action_detected": ["fake"]},
{"text": "Paint gets in the paint steps back won't go.", "transcript_id": 701, "action_detected": ["step back"]},
{"text": "Paint gets in the paint steps back won't go.", "transcript_id": 702, "action_detected": ["step back"]},
{"text": "Paint gets in the paint steps back won't go.", "transcript_id": 703, "action_detected": ["step back"]},
{"text": "Fake you could see it there and then.", "transcript_id": 704, "action_detected": ["fake"]},
{"text": "And in Ellington with a ball fake one big escape dribble.", "transcript_id": 705, "action_detected": ["fake"]},
{"text": "Lowry ball fake.", "transcript_id": 706, "action_detected": ["fake"]},
{"text": "I had tried to jam it backhand and it's out of bounds.", "transcript_id": 707, "action_detected": ["jam"]},
{"text": "Step back three for James.", "transcript_id": 708, "action_detected": ["step back"]},
{"text": "I let it seven assists, dropping Dimes.", "transcript_id": 709, "action_detected": ["dime"]},
{"text": "Nice lob up to Brook Lopez.", "transcript_id": 710, "action_detected": ["lob"]},
{"text": "Alexander, with the fake on the handoff back out to Shavon Shields for three, he misses rebound Ellenson.", "transcript_id": 711, "action_detected": ["fake"]},
{"text": "Big Black Booty 8 ball fake driver shot over the rim.", "transcript_id": 712, "action_detected": ["fake"]},
{"text": "Culture Shock lock already down to four Jackson behind the back into the freight elevator and dumps.", "transcript_id": 713, "action_detected": ["behind the back", "jam"]},
{"text": "Kyle making a move cut off by Satiran Ski and the jumper is good as he steps back in Drillsite.", "transcript_id": 714, "action_detected": ["step back"]},
{"text": "Randall little step back.", "transcript_id": 715, "action_detected": ["step back"]},
{"text": "You see the quickness of quick step back there by Anthony did own underpants for Bam, was intercepted by Davis 2 early turnovers for the Heat is Bradley fires A3.", "transcript_id": 716, "action_detected": ["step back"]},
{"text": "Great length 7 one step back step back jumper.", "transcript_id": 717, "action_detected": ["step back"]},
{"text": "Euro lobbying for Chris Silva.", "transcript_id": 718, "action_detected": ["euro step", "lob"]},
{"text": "Liller to two steps back.", "transcript_id": 719, "action_detected": ["step back"]},
{"text": "Seductive Paul fake pulls the trigger after Mitchell does a brief flyby.", "transcript_id": 720, "action_detected": ["fake"]},
{"text": "Step back.", "transcript_id": 721, "action_detected": ["step back"]},
{"text": "Here comes the tribe in the Jam by pool.", "transcript_id": 722, "action_detected": ["jam"]},
{"text": "I don't think he's expecting that lob.", "transcript_id": 723, "action_detected": ["lob"]},
{"text": "Here's the lobby.", "transcript_id": 724, "action_detected": ["lob"]},
{"text": "Drew holiday backdoor.", "transcript_id": 725, "action_detected": ["backdoor"]},
{"text": "Former Oklahoma City Thunder got jammed up in the air and finish it.", "transcript_id": 726, "action_detected": ["jam"]},
{"text": "Lob it up to Chandler.", "transcript_id": 727, "action_detected": ["lob"]},
{"text": "Bill makes his move Satiran ski nice, little fake gets to the elbow.", "transcript_id": 728, "action_detected": ["fake"]},
{"text": "Great look for dirt to start this contest for Dallas, taking their first shot, another lob rockets to autobio.", "transcript_id": 729, "action_detected": ["lob"]},
{"text": "Been awhile, will step back.", "transcript_id": 730, "action_detected": ["step back"]},
{"text": "Backdoor Kevin Love as that one rejected Middleton got a hand on it.", "transcript_id": 731, "action_detected": ["backdoor"]},
{"text": "Hesitation head fake and he is at the Cup.", "transcript_id": 732, "action_detected": ["fake"]},
{"text": "First puppy shot fake.", "transcript_id": 733, "action_detected": ["fake"]},
{"text": "He is not used to banging with Zion and guys of that size is going to be a little step back for Crowder.", "transcript_id": 734, "action_detected": ["step back"]},
{"text": "Aminu, with the ball fake Anna Drive on holiday.", "transcript_id": 735, "action_detected": ["fake"]},
{"text": "step back.", "transcript_id": 736, "action_detected": ["step back"]},
{"text": "step back.", "transcript_id": 737, "action_detected": ["step back"]},
{"text": "A step back 3.", "transcript_id": 738, "action_detected": ["step back"]},
{"text": "There's Vince Carter once again this improved his shot by step into his right off the fake.", "transcript_id": 739, "action_detected": ["fake"]},
{"text": "Does he realize that the step back three is off?", "transcript_id": 740, "action_detected": ["step back"]},
{"text": "Nothing Butler with a step back midrange jumper off the mark.", "transcript_id": 741, "action_detected": ["step back"]},
{"text": "Run ice jam and start the game.", "transcript_id": 742, "action_detected": ["jam"]},
{"text": "Wendell Carter junior tried to make it tough on him at the basket, but a little side step.", "transcript_id": 743, "action_detected": ["side step"]},
{"text": "Long Bar is a little shake and bake, and it writes it on 100 draws the foul.", "transcript_id": 744, "action_detected": ["shake and bake"]},
{"text": "Is it to the big man who delivers to Jayson Tatum behind the back to smart for the long run it goes.", "transcript_id": 745, "action_detected": ["behind the back"]},
{"text": "Everyone contribute as well as J step back.", "transcript_id": 746, "action_detected": ["step back"]},
{"text": "The little subtle head fake.", "transcript_id": 747, "action_detected": ["fake"]},
{"text": "Now, Porter's owned by San Antonio Porter, guarded by Derrick White, is a good defender now Porter steps back three ball all the way.", "transcript_id": 748, "action_detected": ["step back"]},
{"text": "You know he's a guy who really is crafty with fakes, getting you up in the air and forcing precious into files.", "transcript_id": 749, "action_detected": ["fake"]},
{"text": "You know he's a guy who really is crafty with fakes, getting you up in the air and forcing precious into files.", "transcript_id": 750, "action_detected": ["fake"]},
{"text": "Regardless of who was on the floor, James Ball faked nearly lost it.", "transcript_id": 751, "action_detected": ["fake"]},
{"text": "So nice backdoor cut by Conley in a pitchout", "transcript_id": 752, "action_detected": ["backdoor"]},
{"text": "Step back for Lamarcus Aldridge.", "transcript_id": 753, "action_detected": ["step back"]},
{"text": "Get started, Paris tried the fake gun, he likes.", "transcript_id": 754, "action_detected": ["fake"]},
{"text": "steps back with that.", "transcript_id": 755, "action_detected": ["step back"]},
{"text": "Moody against ball nice fake Moody inside Black by Lonzo Ball.", "transcript_id": 756, "action_detected": ["fake"]},
{"text": "And then there's a great job with those head and shoulder fakes inside.", "transcript_id": 757, "action_detected": ["fake"]},
{"text": "Deleter steps back his three pointer is short long rebound out to Randall.", "transcript_id": 758, "action_detected": ["step back"]},
{"text": "The veteran Al Horford with a step back.", "transcript_id": 759, "action_detected": ["step back"]},
{"text": "Here's Harris little step back over Buddy Hield.", "transcript_id": 760, "action_detected": ["step back"]},
{"text": "Zach Lavine Bulls lobbying along the baseline.", "transcript_id": 761, "action_detected": []},
{"text": "You gotta look at use all averaging just under 18 again 18 and eight and a half rebounds a game at a lob.", "transcript_id": 762, "action_detected": ["lob"]},
{"text": "Damn nice move behind the back.", "transcript_id": 763, "action_detected": ["behind the back"]},
{"text": "Easy way down a step back.", "transcript_id": 764, "action_detected": ["step back"]},
{"text": "Middleton step back triple try the job.", "transcript_id": 765, "action_detected": ["step back"]},
{"text": "I've ever seen that for Pie showing little ball fake and then to touch on the floating shot.", "transcript_id": 766, "action_detected": ["fake", "floater"]},
{"text": "Shot Clock dwindling is kalo that tough step back shot.", "transcript_id": 767, "action_detected": ["step back"]},
{"text": "Little fake goes up and under and scores.", "transcript_id": 768, "action_detected": ["fake"]},
{"text": "No look pass favors.", "transcript_id": 769, "action_detected": ["no look pass"]},
{"text": "James steps back.", "transcript_id": 770, "action_detected": ["step back"]},
{"text": "Art behind the back to King.", "transcript_id": 771, "action_detected": ["behind the back"]},
{"text": "Barry Euro steps in couldn't finish get tonight at the rim still alive for the next year.", "transcript_id": 772, "action_detected": ["euro step"]},
{"text": "Here's Brooks Ball gets detached as they come to trap the ball and Brooks racers away to jam it down 6 nothing.", "transcript_id": 773, "action_detected": ["jam"]},
{"text": "Data science Hayward will step back and back again.", "transcript_id": 774, "action_detected": ["step back"]},
{"text": "Here's Klay Thompson with step back three at rims off but it would dolla.", "transcript_id": 775, "action_detected": ["step back"]},
{"text": "Very high score drains that step back deep.", "transcript_id": 776, "action_detected": ["step back"]},
{"text": "And that's Westbrooks now went to Murray, who picked up by Jeremy Grant steps back for three that's on the way, and it's off the heel.", "transcript_id": 777, "action_detected": ["step back"]},
{"text": "Nurkic sagging off Carter, as many teams have done White Hill step back that's off target.", "transcript_id": 778, "action_detected": ["step back"]},
{"text": "Nurkic sagging off Carter, as many teams have done White Hill step back that's off target.", "transcript_id": 779, "action_detected": ["step back"]},
{"text": "Kindly step back.", "transcript_id": 780, "action_detected": ["step back"]},
{"text": "It thinks about this step back three.", "transcript_id": 781, "action_detected": ["step back"]},
{"text": "To go behind the back, but through it beyond the reach of Qatar.", "transcript_id": 782, "action_detected": ["behind the back"]},
{"text": "Andrew Wiggins with the step back Wiseman already with a couple of rebound to the game who pray.", "transcript_id": 783, "action_detected": ["step back"]},
{"text": "Dreidel stepping back.", "transcript_id": 784, "action_detected": ["step back"]},
{"text": "Step back 3.", "transcript_id": 785, "action_detected": ["step back"]},
{"text": "Step back 3.", "transcript_id": 786, "action_detected": ["step back"]},
{"text": "Fournier step back to was good.", "transcript_id": 787, "action_detected": ["step back"]},
{"text": "The rebound Capella kicks it back out, Tucker fakes the three, gets it inside the Capello throws it down.", "transcript_id": 788, "action_detected": ["fake"]},
{"text": "He is not used to banging with Zion and guys of that size is going to be a little step back for Crowder.", "transcript_id": 789, "action_detected": ["step back"]},
{"text": "However, when he's healthy, pretty solid, average open up, trying to lob it to Dennis Smith, Junior couldn't control the fast.", "transcript_id": 790, "action_detected": ["lob"]},
{"text": "The step back?", "transcript_id": 791, "action_detected": ["step back"]},
{"text": "Like the step back 3 point shot.", "transcript_id": 792, "action_detected": ["step back"]},
{"text": "Darren Fox missing a triple Giannis stepping back out of the paint.", "transcript_id": 793, "action_detected": ["step back"]},
{"text": "But good to see Joe knock down one Jokic so good with a step back.", "transcript_id": 794, "action_detected": ["step back"]},
{"text": "There's Joanna step back 15 footer outcomes and Moody a right place.", "transcript_id": 795, "action_detected": ["step back"]},
{"text": "Now he's under baseline they missed him, ain't trying to jam the pass in there.", "transcript_id": 796, "action_detected": ["jam"]},
{"text": "Soccer fakes it.", "transcript_id": 797, "action_detected": ["fake"]},
{"text": "Gordon Tucker for the step back triple.", "transcript_id": 798, "action_detected": ["step back"]},
{"text": "Gordon Tucker for the step back triple.", "transcript_id": 799, "action_detected": ["step back"]},
{"text": "Backdoor cut by Villin Bruce Brown steps in front and steals.", "transcript_id": 800, "action_detected": ["backdoor"]},
{"text": "Pardon steps back controls another.", "transcript_id": 801, "action_detected": ["step back"]},
{"text": "Gonna step back at three.", "transcript_id": 802, "action_detected": ["step back"]},
{"text": "Clear great hands by Davis and then the behind the back.", "transcript_id": 803, "action_detected": ["behind the back"]},
{"text": "Rozier on the wing step back.", "transcript_id": 804, "action_detected": ["step back"]},
{"text": "Rozier on the wing step back.", "transcript_id": 805, "action_detected": ["step back"]},
{"text": "Rozier on the wing step back.", "transcript_id": 806, "action_detected": ["step back"]},
{"text": "Kyle Anderson with a nice fake and a finish.", "transcript_id": 807, "action_detected": ["fake"]},
{"text": "Now Nikolai Yokich don't work on Dedmon and a power it up behind the back, Lyles off the glass.", "transcript_id": 808, "action_detected": ["behind the back"]},
{"text": "Little euro step to hang the basket.", "transcript_id": 809, "action_detected": ["euro step"]},
{"text": "Paris lobbing it up.", "transcript_id": 810, "action_detected": ["lob"]},
{"text": "Chandler's kind of tough step back.", "transcript_id": 811, "action_detected": ["step back"]},
{"text": "Lowry ball fake.", "transcript_id": 812, "action_detected": ["fake"]},
{"text": "Grant closure of the baseline Russell, founded for the dime.", "transcript_id": 813, "action_detected": ["dime"]},
{"text": "Let teams play against your set half court defense, lob download.", "transcript_id": 814, "action_detected": ["lob"]},
{"text": "Hopsital step back threes and now is a great sign.", "transcript_id": 815, "action_detected": ["step back"]},
{"text": "Rockets 6th and drawn charges per game years of step Back Re buy hardwoods in.", "transcript_id": 816, "action_detected": ["step back"]},
{"text": "Jam it off the Terrell screen lost the handle out of bounds.", "transcript_id": 817, "action_detected": ["jam"]},
{"text": "Rebound comes all the way back out to Rozier, Rozier steps back.", "transcript_id": 818, "action_detected": ["step back"]},
{"text": "Step back.", "transcript_id": 819, "action_detected": ["step back"]},
{"text": "Davis behind the back drives and draws the foul.", "transcript_id": 820, "action_detected": ["behind the back"]},
{"text": "Fake fires that I've got fouled and will have free throws.", "transcript_id": 821, "action_detected": ["fake"]},
{"text": "We're doing the step back.", "transcript_id": 822, "action_detected": ["step back"]},
{"text": "I've watched team spaces the floor better than Milwaukee feel step back, jumper drops.", "transcript_id": 823, "action_detected": ["step back"]},
{"text": "Step back baseline off the rim.", "transcript_id": 824, "action_detected": ["step back"]},
{"text": "Step back baseline off the rim.", "transcript_id": 825, "action_detected": ["step back"]},
{"text": "Step back baseline off the rim.", "transcript_id": 826, "action_detected": ["step back"]},
{"text": "Damian Jones setting up late, didn't settle for the three the lob thrown and beautifully done.", "transcript_id": 827, "action_detected": ["lob"]},
{"text": "George stops on a dime, puts up a 20 footer.", "transcript_id": 828, "action_detected": ["dime"]},
{"text": "The step back for the long 2.", "transcript_id": 829, "action_detected": ["step back"]},
{"text": "Take the past the dirt and here's a step back from the corner.", "transcript_id": 830, "action_detected": ["step back"]},
{"text": "Beautiful backdoor cut that wasn't paid off.", "transcript_id": 831, "action_detected": ["backdoor"]},
{"text": "Scurry out of step back for five.", "transcript_id": 832, "action_detected": ["step back"]},
{"text": "Locked out of each step back.", "transcript_id": 833, "action_detected": ["step back"]},
{"text": "Now, Porter's owned by San Antonio Porter, guarded by Derrick White, is a good defender now Porter steps back three ball all the way.", "transcript_id": 834, "action_detected": ["step back"]},
{"text": "How about the drop off behind the back pass?", "transcript_id": 835, "action_detected": ["behind the back"]},
{"text": "Eric Gordon fakes the three.", "transcript_id": 836, "action_detected": ["fake"]},
{"text": "Folks thinking about the three Simmons the fake in the score, he used charts really there as bait and got too.", "transcript_id": 837, "action_detected": ["fake"]},
{"text": "Peel steps back and that's way off.", "transcript_id": 838, "action_detected": ["step back"]},
{"text": "now done circles into the Lane, lobs it for Thaddeus up and good.", "transcript_id": 839, "action_detected": ["lob"]},
{"text": "JJ Redick with the extra pass Damian Lillard off the high screen on the step back and the first four points belong to the Blazers.", "transcript_id": 840, "action_detected": ["step back"]},
{"text": "Step back 3 blocked by Ben Simmons.", "transcript_id": 841, "action_detected": ["step back"]},
{"text": "Euro steps in and dropped it off.", "transcript_id": 842, "action_detected": ["euro step"]},
{"text": "He's one of the players just went from a two way contract to a standard contract backdoor.", "transcript_id": 843, "action_detected": ["backdoor"]},
{"text": "Carmelo goes behind the back, but it hit him in the back.", "transcript_id": 844, "action_detected": ["behind the back"]},
{"text": "Brown put up fake.", "transcript_id": 845, "action_detected": ["fake"]},
{"text": "You want to make sure you don't go for any of the unusual fakes?", "transcript_id": 846, "action_detected": ["fake"]},
{"text": "Jackson works steps back.", "transcript_id": 847, "action_detected": ["step back"]},
{"text": "Thompson beautiful backdoor play and the foul on a gorgeous bounce fast from Draymond Green.", "transcript_id": 848, "action_detected": ["backdoor"]},
{"text": "Hayward guards Merante John step back three spoons out rebound.", "transcript_id": 849, "action_detected": ["step back"]},
{"text": "The fastest moving teams in the NBA in a lot of different ways is George steps back for that.", "transcript_id": 850, "action_detected": ["step back"]},
{"text": "Fake that driving Mr Duncan a good challenge.", "transcript_id": 851, "action_detected": ["fake"]},
{"text": "Bill got called for the offensive foul step back, jumpers off the mark in the rebound.", "transcript_id": 852, "action_detected": ["step back"]},
{"text": "Gilgeous Alexander shot Clock down to five little step back, three on the way.", "transcript_id": 853, "action_detected": ["step back"]},
{"text": "It's just a great backdoor cut.", "transcript_id": 854, "action_detected": ["backdoor"]},
{"text": "His ability to stretch the floor because of his shooting, but a couple dribbles he can step back as well.", "transcript_id": 855, "action_detected": ["step back"]},
{"text": "Drew on a step back.", "transcript_id": 856, "action_detected": ["step back"]},
{"text": "Not to get the step back, but Ray was right there.", "transcript_id": 857, "action_detected": ["step back"]},
{"text": "Avery Bradley steps back.", "transcript_id": 858, "action_detected": ["step back"]},
{"text": "Another key move there early in this game side Step 3 by Jaylen Brown, rebounded by Tobias Harris.", "transcript_id": 859, "action_detected": ["side step"]},
{"text": "First note on Dillon Brooks, no backdoors.", "transcript_id": 860, "action_detected": ["backdoor"]},
{"text": "Down the bottom part of the League and defending grants cutting backdoor Draymond found him nicely.", "transcript_id": 861, "action_detected": ["backdoor"]},
{"text": "Dangles steps back and bounced it to favors picked up.", "transcript_id": 862, "action_detected": ["step back"]},
{"text": "That time went for the fake.", "transcript_id": 863, "action_detected": ["fake"]},
{"text": "I had to Walker Walker's on the run behind the back pass.", "transcript_id": 864, "action_detected": ["behind the back"]},
{"text": "Fake on a fade away.", "transcript_id": 865, "action_detected": ["fake", "fadeaway"]},
{"text": "I am working on Fultz step back triple.", "transcript_id": 866, "action_detected": ["step back"]},
{"text": "Steph Curry fakes drives goes inside high off the glass is good.", "transcript_id": 867, "action_detected": ["fake"]},
{"text": "It all poker jam holding a green garden.", "transcript_id": 868, "action_detected": ["jam"]},
{"text": "DeAngelo Russell uses the shot fake to get the shot off ties the game.", "transcript_id": 869, "action_detected": ["fake"]},
{"text": "Zipsor a fake and the drive high off the window no, and the rebound to Evan Turner.", "transcript_id": 870, "action_detected": ["fake"]},
{"text": "Tobias Harris size mismatch it shows and indeed shot fake draws, the contact and.", "transcript_id": 871, "action_detected": ["fake"]},
{"text": "Free backdoor to Curry and Curry banks at home.", "transcript_id": 872, "action_detected": ["backdoor"]},
{"text": "He got the rebound in the steel played point forward, dropped the dime off to Barrett for the for the for the dunk.", "transcript_id": 873, "action_detected": ["dime"]},
{"text": "Step back Reaper.", "transcript_id": 874, "action_detected": ["step back"]},
{"text": "Single coverage for Kiss all the step back and he is so good with that particular shot.", "transcript_id": 875, "action_detected": ["step back"]},
{"text": "Picks up his first foul Serge Ibaka up, fake man, then looking for OG.", "transcript_id": 876, "action_detected": ["fake"]},
{"text": "Will be a step back for Wiggins and along two at that forward.", "transcript_id": 877, "action_detected": ["step back"]},
{"text": "Seven assist greatly if Lonzo first bucket of the game, the dime and a good drive right?", "transcript_id": 878, "action_detected": ["dime"]},
{"text": "See here 20 steps back.", "transcript_id": 879, "action_detected": ["step back"]},
{"text": "Olinick wanna nice fake little step back and knocks it down.", "transcript_id": 880, "action_detected": ["fake", "step back"]},
{"text": "Olinick wanna nice fake little step back and knocks it down.", "transcript_id": 881, "action_detected": ["fake", "step back"]},
{"text": "Step back three is through for Blake.", "transcript_id": 882, "action_detected": ["step back"]},
{"text": "The soul step back jumper.", "transcript_id": 883, "action_detected": ["step back"]},
{"text": "Little fake drive shot won't go.", "transcript_id": 884, "action_detected": ["fake"]},
{"text": "Steps back.", "transcript_id": 885, "action_detected": ["step back"]},
{"text": "Jeff Teague steps back.", "transcript_id": 886, "action_detected": ["step back"]},
{"text": "Yeah step back not there.", "transcript_id": 887, "action_detected": ["step back"]},
{"text": "Yeah step back not there.", "transcript_id": 888, "action_detected": ["step back"]},
{"text": "Step back J over diamond.", "transcript_id": 889, "action_detected": ["step back"]},
{"text": "This all steps out fakes on a 3 temple will take but not making the rebound kupas.", "transcript_id": 890, "action_detected": ["fake"]},
{"text": "A little turn, couple of fakes, and he will get the bank off the window.", "transcript_id": 891, "action_detected": ["fake", "bank shot"]},
{"text": "Blake Griffin step back long gun short.", "transcript_id": 892, "action_detected": ["step back"]},
{"text": "For the two hand Jam Blazers up for nothing.", "transcript_id": 893, "action_detected": ["jam"]},
{"text": "George George on the Dr Little fake Westbrook throws it up.", "transcript_id": 894, "action_detected": ["fake"]},
{"text": "Now look with a step back three over Baynes.", "transcript_id": 895, "action_detected": ["step back"]},
{"text": "Maybe should have stepped back a bit book quick pull up and Booker coming on the clearly his best overall game.", "transcript_id": 896, "action_detected": ["step back"]},
{"text": "Damian Lillard, little fake for shot.", "transcript_id": 897, "action_detected": ["fake"]},
{"text": "Now we can score and he's just gotta take a little bit of a step back behind McCollum and Lillard, but he's in.", "transcript_id": 898, "action_detected": ["step back"]},
{"text": "Reggie Jackson against Smith junior lobs.", "transcript_id": 899, "action_detected": ["lob"]},
{"text": "Find Rosier fakes the three.", "transcript_id": 900, "action_detected": ["fake"]},
{"text": "With a step back to shot.", "transcript_id": 901, "action_detected": ["step back"]},
{"text": "Well, first three points for the rookie and again big task filling in for joellen bead step Curry steps back a 2 pointer.", "transcript_id": 902, "action_detected": ["step back"]},
{"text": "No look pass.", "transcript_id": 903, "action_detected": ["no look pass"]},
{"text": "Asthma column on the step back.", "transcript_id": 904, "action_detected": ["step back"]},
{"text": "The last couple of games Randle's into his sort of step back jumper with the left hand.", "transcript_id": 905, "action_detected": ["step back"]},
{"text": "Running the old school we hear on the perimeter, Beverly will step back game Tide at 4.", "transcript_id": 906, "action_detected": ["step back"]},
{"text": "Fake Luke Canard wide open gun won't go tapped in by Blake.", "transcript_id": 907, "action_detected": ["fake"]},
{"text": "Here's Augustin step back for three.", "transcript_id": 908, "action_detected": ["step back"]},
{"text": "Great separation with step back moves averaged around 35% from downtown over in Europe is Andrew Wiggins gets the first point to this game.", "transcript_id": 909, "action_detected": ["step back"]},
{"text": "Backdoor cut by Mitch Creek.", "transcript_id": 910, "action_detected": ["backdoor"]},
{"text": "Trying to pick on Dellavedova was trying to pull his way to the hoop couple of good fakes.", "transcript_id": 911, "action_detected": ["fake"]},
{"text": "I had the college for the Jam and it's 5 nothing.", "transcript_id": 912, "action_detected": ["jam"]},
{"text": "By Willie Cauley, backdoor.", "transcript_id": 913, "action_detected": ["backdoor"]},
{"text": "Avery Bradley brought over from the Los Angeles Clippers starting his second game at the two guard opposite Delon Wright tonight steps back on Aldridge.", "transcript_id": 914, "action_detected": ["step back"]},
{"text": "Step back.", "transcript_id": 915, "action_detected": ["step back"]},
{"text": "Mitchell behind the back voice.", "transcript_id": 916, "action_detected": ["behind the back"]},
{"text": "Here is Carris LeVert stepping back.", "transcript_id": 917, "action_detected": ["step back"]},
{"text": "While trying to defend Horford on the low block step, back jumper is good for Horford in Philly.", "transcript_id": 918, "action_detected": ["step back"]},
{"text": "Step back, dogic that looked to be short out of the gate and it was.", "transcript_id": 919, "action_detected": ["step back"]},
{"text": "Step back, dogic that looked to be short out of the gate and it was.", "transcript_id": 920, "action_detected": ["step back"]},
{"text": "Now take a look and gather it in not get sick 1 three in a row from the Raptors two last season one this season Jokic step back and he's got it.", "transcript_id": 921, "action_detected": ["step back"]},
{"text": "He steps back for us three and hits.", "transcript_id": 922, "action_detected": ["step back"]},
{"text": "I once called like a fake Royal Rumble match and.", "transcript_id": 923, "action_detected": ["fake"]},
{"text": "McCallum Free throw line step back jumper good.", "transcript_id": 924, "action_detected": ["step back"]},
{"text": "That no look pass got deflected inside the atoms under clearly making it a point to get the ball inside the Adams.", "transcript_id": 925, "action_detected": ["no look pass"]},
{"text": "Purtle LeBron loves this match up little step back on Yakub no good remount taken by San Antonio's too.", "transcript_id": 926, "action_detected": ["step back"]},
{"text": "Little step back look good out of his hand.", "transcript_id": 927, "action_detected": ["step back"]},
{"text": "Step back jumper beautiful touch by Jaylen Brown.", "transcript_id": 928, "action_detected": ["step back"]},
{"text": "Behind the back and kicks to the corner.", "transcript_id": 929, "action_detected": ["behind the back"]},
{"text": "SJ steps back behind the line.", "transcript_id": 930, "action_detected": ["step back"]},
{"text": "Jack locks down to four step back, PJ.", "transcript_id": 931, "action_detected": ["step back"]},
{"text": "Jack locks down to four step back, PJ.", "transcript_id": 932, "action_detected": ["step back"]},
{"text": "Cody Zeller attack bowl, Euro step and he draws contact and he'll go to the free throw line.", "transcript_id": 933, "action_detected": ["euro step"]},
{"text": "Pat and step back.", "transcript_id": 934, "action_detected": ["step back"]},
{"text": "Fox steps back, left hands want short 3 pounds his own.", "transcript_id": 935, "action_detected": ["step back"]},
{"text": "Baynes roll into the bucket, put up fake.", "transcript_id": 936, "action_detected": ["fake"]},
{"text": "JJ Redick, nice shot fake.", "transcript_id": 937, "action_detected": ["fake"]},
{"text": "That's another step back.", "transcript_id": 938, "action_detected": ["step back"]},
{"text": "Step back three CJ McCollum.", "transcript_id": 939, "action_detected": ["step back"]},
{"text": "The lob for McGee back taps it.", "transcript_id": 940, "action_detected": ["lob"]},
{"text": "Great backdoor cut here by Gerald Green, His Manson, watching Chris Paul ball handle.", "transcript_id": 941, "action_detected": ["backdoor"]},
{"text": "Step back, triple try around it off Blake Griffin for Reggie and four court.", "transcript_id": 942, "action_detected": ["step back"]},
{"text": "Just a little step back.", "transcript_id": 943, "action_detected": ["step back"]},
{"text": "Guarding Doncic, they go to Matthews was Matthews is open, but this is Reggie Jackson, the forecourt, after the behind the back dribble.", "transcript_id": 944, "action_detected": ["behind the back"]},
{"text": "Looney sniffed out that lob right away.", "transcript_id": 945, "action_detected": ["lob"]},
{"text": "Watch the shot fake and then nobody steps up to help.", "transcript_id": 946, "action_detected": ["fake"]},
{"text": "Step back for Wendell Carter.", "transcript_id": 947, "action_detected": ["step back"]},
{"text": "Bogdanovich lobs it to Gobert an, misses the chip shot.", "transcript_id": 948, "action_detected": ["lob"]},
{"text": "Bogdanovich lobs it to Gobert an, misses the chip shot.", "transcript_id": 949, "action_detected": ["lob"]},
{"text": "Not bad when you can run a backdoor play for your 610 point guard.", "transcript_id": 950, "action_detected": ["backdoor"]},
{"text": "Step back.", "transcript_id": 951, "action_detected": ["step back"]},
{"text": "Kimbo with that lethal step back.", "transcript_id": 952, "action_detected": ["step back"]},
{"text": "Doris is there another step back in the League?", "transcript_id": 953, "action_detected": ["step back"]},
{"text": "Evans is on first time, second half step back jumper by Tyreek no good, and that's a loose ball foul keeping it there.", "transcript_id": 954, "action_detected": ["step back"]},
{"text": "Get past the elites of the fake.", "transcript_id": 955, "action_detected": ["fake"]},
{"text": "No look pass.", "transcript_id": 956, "action_detected": ["no look pass"]},
{"text": "Behind the back pass, Kemba misses the three.", "transcript_id": 957, "action_detected": ["behind the back"]},
{"text": "CJ Williams little ball fake.", "transcript_id": 958, "action_detected": ["fake"]},
{"text": "First half lob inside, knocked away by Mickey, but a foul, whoever tonight.", "transcript_id": 959, "action_detected": ["lob"]},
{"text": "By the Sun jam for DeAndre Ayton.", "transcript_id": 960, "action_detected": ["jam"]},
{"text": "Ricky step back 3.", "transcript_id": 961, "action_detected": ["step back"]},
{"text": "And that trade involving Brooklyn in all the pics as Lucas thought about the step back, went to Dennis Smith Junior.", "transcript_id": 962, "action_detected": ["step back"]},
{"text": "Unforced turnover Reggie Jackson behind the back.", "transcript_id": 963, "action_detected": ["behind the back"]},
{"text": "Big part of the problem has worn able to leak out and jams at home.", "transcript_id": 964, "action_detected": ["jam"]},
{"text": "Just turn a ball, faked him out of his squat, still able to get to the rim.", "transcript_id": 965, "action_detected": ["fake"]},
{"text": "Rubio nice fake tried to draw the foul on Jones but Rubio initiated the contact.", "transcript_id": 966, "action_detected": ["fake"]},
{"text": "Fines Turkey fakes two defenders in the air and he draws purple and white into the body.", "transcript_id": 967, "action_detected": ["fake"]},
{"text": "It used, it steps back.", "transcript_id": 968, "action_detected": ["step back"]},
{"text": "Jeremy Lamb step back behind the line.", "transcript_id": 969, "action_detected": ["step back"]},
{"text": "Clear great hands by Davis and then the behind the back.", "transcript_id": 970, "action_detected": ["behind the back"]},
{"text": "Here's Moran touching the paint behind the back to Anderson.", "transcript_id": 971, "action_detected": ["behind the back"]},
{"text": "Here's Moran touching the paint behind the back to Anderson.", "transcript_id": 972, "action_detected": ["behind the back"]},
{"text": "Here's Tatum coming off a 22 point effort in Game six, they lob it Horford for the Easy Jam.", "transcript_id": 973, "action_detected": ["lob", "jam"]},
{"text": "He's really played well in that game, Brandl step back Jumper missed it.", "transcript_id": 974, "action_detected": ["step back"]},
{"text": "He put a fake out and not as much why?", "transcript_id": 975, "action_detected": ["fake"]},
{"text": "But they were 24 and three prior to this St James isolated with Subocz shot Clock at 4A step back, three handed his face and Beverly the rebound.", "transcript_id": 976, "action_detected": ["step back"]},
{"text": "This is just that great euro step right there.", "transcript_id": 977, "action_detected": ["euro step"]},
{"text": "8785 victory to Welwyn Beetle step back and shoot a three and beat missing and bead played it.", "transcript_id": 978, "action_detected": ["step back"]},
{"text": "Backdoor lob intended for Turner, who was open at the rim, batted away by Lopez, Kentavious Caldwell.", "transcript_id": 979, "action_detected": ["backdoor", "lob"]},
{"text": "Backdoor lob intended for Turner, who was open at the rim, batted away by Lopez, Kentavious Caldwell.", "transcript_id": 980, "action_detected": ["backdoor", "lob"]},
{"text": "Here's Doncic lobbing it for Jordan.", "transcript_id": 981, "action_detected": ["lob"]},
{"text": "Nice backdoor.", "transcript_id": 982, "action_detected": ["backdoor"]},
{"text": "Rubio nice fake tried to draw the foul on Jones but Rubio initiated the contact.", "transcript_id": 983, "action_detected": ["fake"]},
{"text": "Ari fakes the three one dribble fires and hits.", "transcript_id": 984, "action_detected": ["fake"]},
{"text": "So step back over Aminu.", "transcript_id": 985, "action_detected": ["step back"]},
{"text": "But Anthony Davis going up top for the lobby.", "transcript_id": 986, "action_detected": ["lob"]},
{"text": "I drop a dime.", "transcript_id": 987, "action_detected": ["dime"]},
{"text": "Step back on a break the space right over the top.", "transcript_id": 988, "action_detected": ["step back"]},
{"text": "Green goes behind the back, gives it up for Murray.", "transcript_id": 989, "action_detected": ["behind the back"]},
{"text": "Allison, nice fake draws the foul and he's going to head to the free throw line.", "transcript_id": 990, "action_detected": ["fake"]},
{"text": "Going off another strong game last night against Washington early watch the Orpheus Harris with the step back, yes?", "transcript_id": 991, "action_detected": ["step back"]},
{"text": "Gallinari catches step back.", "transcript_id": 992, "action_detected": ["step back"]},
{"text": "McConnell on the step back had a good box off by Anthony Davis.", "transcript_id": 993, "action_detected": ["step back"]},
{"text": "The bronze behind the back bounce picked off.", "transcript_id": 994, "action_detected": ["behind the back"]},
{"text": "George pokes it away, George, just a Premier defender goes to that fake, and Marcus Morris knocks down his first shot.", "transcript_id": 995, "action_detected": ["fake"]},
{"text": "Nice backdoor pass by Jokic and Murray.", "transcript_id": 996, "action_detected": ["backdoor"]},
{"text": "Four on the 24th, James Steps back long.", "transcript_id": 997, "action_detected": ["step back"]},
{"text": "This little space on the three step back.", "transcript_id": 998, "action_detected": ["step back"]},
{"text": "Drag it with the side Step 3.", "transcript_id": 999, "action_detected": ["side step"]},
{"text": "Lips to Blake Griffin step back long gun short.", "transcript_id": 1000, "action_detected": ["step back"]},
{"text": "Fake, but still had enough time to get back.", "transcript_id": 1001, "action_detected": ["fake"]},
{"text": "Eric Gordon fakes the three.", "transcript_id": 1002, "action_detected": ["fake"]},
{"text": "Savannah's misses that double handed jam inside, with the reverse TJ Warren.", "transcript_id": 1003, "action_detected": ["jam", "reverse dunk"]},
{"text": "And here's a steal by ado behind the back.", "transcript_id": 1004, "action_detected": ["behind the back"]},
{"text": "Little fake.", "transcript_id": 1005, "action_detected": ["fake"]},
{"text": "Step back jumper by Giannis is little bit short, rosier with the rebound.", "transcript_id": 1006, "action_detected": ["step back"]},
{"text": "Gary Harris is inside great fake.", "transcript_id": 1007, "action_detected": ["fake"]},
{"text": "Is Giannis the long strides, the euro step and got the whistle came in late.", "transcript_id": 1008, "action_detected": ["euro step"]},
{"text": "Allen fakes Hunter drives kicks to the Lincoln Brook shot Clock expiring has Dillon Brooks put it down.", "transcript_id": 1009, "action_detected": ["fake"]},
{"text": "Joe goes behind the back, feeds it off to Rudy, gives a back Joe jingles inside to control Gobert.", "transcript_id": 1010, "action_detected": ["behind the back"]},
{"text": "If he can make this shot little confident builder confidence builder right there, stepping back and.", "transcript_id": 1011, "action_detected": ["step back"]},
{"text": "Curry, great backdoor caught.", "transcript_id": 1012, "action_detected": ["backdoor"]},
{"text": "Holiday steps back for the long two and another rebound another crack at it for the Pelicans.", "transcript_id": 1013, "action_detected": ["step back"]},
{"text": "That time will step back against Dockage defense, I thought.", "transcript_id": 1014, "action_detected": ["step back"]},
{"text": "Backdoor cut their black from behind by Oubre but then.", "transcript_id": 1015, "action_detected": ["backdoor"]},
{"text": "Tim Hardaway junior defended by none Toncic good shot fake missed his first opportunity tonight.", "transcript_id": 1016, "action_detected": ["fake"]},
{"text": "Both went to the ball, the lob was open.", "transcript_id": 1017, "action_detected": ["lob"]},
{"text": "Just for the roses down the Lane, the lob that app will not fall.", "transcript_id": 1018, "action_detected": ["lob"]},
{"text": "We've already seen one lob when Luca got into the page fire.", "transcript_id": 1019, "action_detected": ["lob"]},
{"text": "Free games on the step back doesn't get the roll.", "transcript_id": 1020, "action_detected": ["step back"]},
{"text": "Nice backdoor cut.", "transcript_id": 1021, "action_detected": ["backdoor"]},
{"text": "Mr Jam when we bound comes out to Tony Snell.", "transcript_id": 1022, "action_detected": ["jam"]},
{"text": "All opportunities ahead fake got two big defenders off balance.", "transcript_id": 1023, "action_detected": ["fake"]},
{"text": "Gary behind the back, and even he lays it up and then I might settle things down.", "transcript_id": 1024, "action_detected": ["behind the back"]},
{"text": "Nice little fake on Davis and there's the first bucket.", "transcript_id": 1025, "action_detected": ["fake"]},
{"text": "Backdoor cut by Villin Bruce Brown steps in front and steals.", "transcript_id": 1026, "action_detected": ["backdoor"]},
{"text": "Daniel how's step back three.", "transcript_id": 1027, "action_detected": ["step back"]},
{"text": "Backdoor run through holiday.", "transcript_id": 1028, "action_detected": ["backdoor"]},
{"text": "No look pass.", "transcript_id": 1029, "action_detected": ["no look pass"]},
{"text": "No look pass.", "transcript_id": 1030, "action_detected": ["no look pass"]},
{"text": "Looking for one of those lobs, but Dallas ready for Harper.", "transcript_id": 1031, "action_detected": ["lob"]},
{"text": "Their here's hard luck with that step back 3.", "transcript_id": 1032, "action_detected": ["step back"]},
{"text": "Nice little fake but he took an extra step.", "transcript_id": 1033, "action_detected": ["fake"]},
{"text": "Let's go a lot of lobs to him tray with.", "transcript_id": 1034, "action_detected": ["lob"]},
{"text": "Step back three by towns too strong.", "transcript_id": 1035, "action_detected": ["step back"]},
{"text": "Allen fakes Hunter drives kicks to the Lincoln Brook shot Clock expiring has Dillon Brooks put it down.", "transcript_id": 1036, "action_detected": ["fake"]},
{"text": "So name stops on a dime and says you want to come out a little bit further than that.", "transcript_id": 1037, "action_detected": []},
{"text": "Smart home fake now fires.", "transcript_id": 1038, "action_detected": ["fake"]},
{"text": "It's around the corner trying to fake out 18 in the hook shot is good.", "transcript_id": 1039, "action_detected": ["fake"]},
{"text": "Step back again.", "transcript_id": 1040, "action_detected": ["step back"]},
{"text": "Danny Green, little head fake.", "transcript_id": 1041, "action_detected": ["fake"]},
{"text": "Chase drew a foul as he went up to try and jam it.", "transcript_id": 1042, "action_detected": ["jam"]},
{"text": "McGee fakes out there comes back into nowhere.", "transcript_id": 1043, "action_detected": ["fake"]},
{"text": "Little step back action stays out as Josh Jackson clears it away.", "transcript_id": 1044, "action_detected": ["step back"]},
{"text": "Middleton lobs it up for antenna kupo", "transcript_id": 1045, "action_detected": ["lob"]},
{"text": "Lobs over the right shoulder.", "transcript_id": 1046, "action_detected": ["lob"]},
{"text": "Such backdoor corner feed Avery Bradley Long gun short rebound Rondo.", "transcript_id": 1047, "action_detected": ["backdoor"]},
{"text": "Barry Euro steps in couldn't finish get tonight at the rim still alive for the next year.", "transcript_id": 1048, "action_detected": ["euro step"]},
{"text": "Here's DeRozan behind the back to Kelvin.", "transcript_id": 1049, "action_detected": ["behind the back"]},
{"text": "Gordon Hayward dribbling through a bunch of defenders thanks low step back jumper won't go.", "transcript_id": 1050, "action_detected": ["step back"]},
{"text": "You can see fresher legs from the Nuggets, little euro step inside Tappin, try zubat Scott of them a third time on the George Smith.", "transcript_id": 1051, "action_detected": ["euro step"]},
{"text": "Andrew Wiggins with the step back Wiseman already with a couple of rebound to the game who pray.", "transcript_id": 1052, "action_detected": ["step back"]},
{"text": "Wow got away with that one now part is open he steps back for us.", "transcript_id": 1053, "action_detected": ["step back"]},
{"text": "RJ Barrett get ahead a little bit here to start this game off, Barrett on the lob and he's back.", "transcript_id": 1054, "action_detected": ["lob"]},
{"text": "Andre fakes the handoff to Bruce Brown gives it up Chellington.", "transcript_id": 1055, "action_detected": ["fake"]},
{"text": "Here comes a step back 310 down the Hatch.", "transcript_id": 1056, "action_detected": ["step back"]},
{"text": "Step back through holiday off the back of the rim and Nurkic clears it away.", "transcript_id": 1057, "action_detected": ["step back"]},
{"text": "Auto Porter on a step back and he answers.", "transcript_id": 1058, "action_detected": ["step back"]},
{"text": "There's the step back three over Brogdon.", "transcript_id": 1059, "action_detected": ["step back"]},
{"text": "Matched up with Cam reddish backdoor cutting.", "transcript_id": 1060, "action_detected": ["backdoor"]},
{"text": "Step back three over Oladipo, drills it.", "transcript_id": 1061, "action_detected": ["step back"]},
{"text": "Behind the back dribble, lost it, gets it back, roads it up, no good.", "transcript_id": 1062, "action_detected": ["behind the back"]},
{"text": "Thompson drives nice fake annaleise at home.", "transcript_id": 1063, "action_detected": ["fake"]},
{"text": "The lobster panga off the backboard.", "transcript_id": 1064, "action_detected": ["lob"]},
{"text": "Here's Harris little step back over Buddy Hield.", "transcript_id": 1065, "action_detected": ["step back"]},
{"text": "A terrific player now heard her steps back.", "transcript_id": 1066, "action_detected": ["step back"]},
{"text": "I say Cannonball fake stepped into three.", "transcript_id": 1067, "action_detected": ["fake"]},
{"text": "Take him back to Walker, fakes the three.", "transcript_id": 1068, "action_detected": ["fake"]},
{"text": "He's going to get more relaxed and more comfortable up here in the big step back 3.", "transcript_id": 1069, "action_detected": ["step back"]},
{"text": "Step back three for James.", "transcript_id": 1070, "action_detected": ["step back"]},
{"text": "Fox is going to throw lobs.", "transcript_id": 1071, "action_detected": ["lob"]},
{"text": "actor Paul George steps back Bob to the Hawks, knock it away and take it away.", "transcript_id": 1072, "action_detected": ["step back"]},
{"text": "Bird no look passing side and it's good by Gerson.", "transcript_id": 1073, "action_detected": ["no look pass"]},
{"text": "Jordan Bell goes inside, misses the jam, but he got hit.", "transcript_id": 1074, "action_detected": ["jam"]},
{"text": "Step back jumper is short long rebound craft taps it over to Harris.", "transcript_id": 1075, "action_detected": ["step back"]},
{"text": "What a fake.", "transcript_id": 1076, "action_detected": ["fake"]},
{"text": "Randle against Carmelo Anthony single coverage steps back his short jumper can't connect.", "transcript_id": 1077, "action_detected": ["step back"]},
{"text": "It hits the step back jump shot with that lethal middle range game of his Boston leading by pool.", "transcript_id": 1078, "action_detected": ["step back"]},
{"text": "And here comes OK. See they have a 3 on two tort with a little euro step inside to Adams and Robert Covington with a nice recovery.", "transcript_id": 1079, "action_detected": ["euro step"]},
{"text": "Tuluka makes those step back threes look easy.", "transcript_id": 1080, "action_detected": ["step back"]},
{"text": "Fake and the three is short.", "transcript_id": 1081, "action_detected": ["fake"]},
{"text": "Gotta step back down am it's resources, right?", "transcript_id": 1082, "action_detected": ["step back"]},
{"text": "Caldwell Pope cutting backdoor in behind ****** and it gets delayed.", "transcript_id": 1083, "action_detected": ["backdoor"]},
{"text": "He wants to shoot a step back.", "transcript_id": 1084, "action_detected": ["step back"]},
{"text": "Free young lobs it for Collins and a foul.", "transcript_id": 1085, "action_detected": ["lob"]},
{"text": "Right behind the back in front of Venice.", "transcript_id": 1086, "action_detected": ["behind the back"]},
{"text": "Here's lobby Seer backing down and he gets the role.", "transcript_id": 1087, "action_detected": []},
{"text": "Behind the back gets by angles, gets to the rack off the high glass.", "transcript_id": 1088, "action_detected": ["behind the back", "take it to the rack"]},
{"text": "Trey ***** in nice behind the back pass spots open.", "transcript_id": 1089, "action_detected": ["behind the back"]},
{"text": "Courage to step back in the tribe and they need more of that from them.", "transcript_id": 1090, "action_detected": ["step back"]},
{"text": "Here comes Donovan Mitchell into the Lane double teamed.", "transcript_id": 1091, "action_detected": ["double team"]},
{"text": "Lowry with an ice ball fake on Tucker.", "transcript_id": 1092, "action_detected": ["fake"]},
{"text": "Great little backdoor cut alcohol with a beautiful pass right on the money and Tony Parker just doing a lot of focus there.", "transcript_id": 1093, "action_detected": ["backdoor"]},
{"text": "On the right side, by terror the Delon Wright just makes a great play great finish against Tom Baker, inside backdoor Brogdon on the other side.", "transcript_id": 1094, "action_detected": ["backdoor"]},
{"text": "Here's green backdoor cut by step, partially blocked by Beverly.", "transcript_id": 1095, "action_detected": ["backdoor"]},
{"text": "Here's green backdoor cut by step, partially blocked by Beverly.", "transcript_id": 1096, "action_detected": ["backdoor"]},
{"text": "Lob right off.", "transcript_id": 1097, "action_detected": ["lob"]},
{"text": "But good to see Joe knock down one Jokic so good with a step back.", "transcript_id": 1098, "action_detected": ["step back"]},
{"text": "Been out on Doncic, let's step back for you something.", "transcript_id": 1099, "action_detected": ["step back"]},
{"text": "Not played well against Utah is Andrew Wiggins makes another backdoor cut.", "transcript_id": 1100, "action_detected": ["backdoor"]},
{"text": "Faked one's pants down the lamp.", "transcript_id": 1101, "action_detected": ["fake"]},
{"text": "Gary Harris inside great fake.", "transcript_id": 1102, "action_detected": ["fake"]},
{"text": "DeAngelo Russell would stop step back.", "transcript_id": 1103, "action_detected": ["step back"]},
{"text": "Try playing lobs it up for Jukebox.", "transcript_id": 1104, "action_detected": ["lob"]},
{"text": "Nice little fake.", "transcript_id": 1105, "action_detected": ["fake"]},
{"text": "He normally shoots a step back right there and miles I think is ready for it.", "transcript_id": 1106, "action_detected": ["step back"]},
{"text": "So he's going to use those shot fakes.", "transcript_id": 1107, "action_detected": ["fake"]},
{"text": "Damn rubbish with alot college with the jam and that's how he wore the Bulls out here last January.", "transcript_id": 1108, "action_detected": ["jam"]},
{"text": "Bunch of instaflex the attempted lob.", "transcript_id": 1109, "action_detected": ["lob"]},
{"text": "Step back.", "transcript_id": 1110, "action_detected": ["step back"]},
{"text": "Christiane steps back shoots the 3:00 AM berries.", "transcript_id": 1111, "action_detected": ["step back"]},
{"text": "The basket and a big jam by Yusuf Nurkic.", "transcript_id": 1112, "action_detected": ["jam"]},
{"text": "John Collins can't convert the lob inside.", "transcript_id": 1113, "action_detected": ["lob"]},
{"text": "Backdoor cut watch couldn't find him.", "transcript_id": 1114, "action_detected": ["backdoor"]},
{"text": "Fake getting up in the air and complete the play.", "transcript_id": 1115, "action_detected": ["fake"]},
{"text": "Here's Porter with a fake.", "transcript_id": 1116, "action_detected": ["fake"]},
{"text": "Little step back into Lane for two.", "transcript_id": 1117, "action_detected": ["step back"]}
]

if WRITE:
    df_pseudo_actions = pd.DataFrame(pseudo_actions_labels)
    df_pseudo_actions.to_csv("data/pseudo_actions_labels.csv", index=False)

    df_pseudo_actions = pd.read_csv("data/pseudo_actions_labels.csv")
    ids_mapping = pd.read_csv('data/ids_mapping.csv')
    joined_df = df_pseudo_actions.merge(ids_mapping[['sample_id', 'transcript_id', 'Text']].rename(columns={'Text': 'text'}), on=['transcript_id', 'text'], how='inner').drop('transcript_id', axis=1)
    joined_df.to_csv('data/pseudo_actions_labels_with_id.csv', index=False)


# Validate data

In [14]:
import json
with open("data/actions_processed_to_action.json") as f:
    actions_processed_to_action = json.load(f)

In [22]:
transcrpits_folds = pd.read_csv('data/transcripts_folds.csv')
transcrpits_text_actions = transcrpits_folds[['sample_id', 'Text', 'actions_in_text']]
chatgpt_labels = pd.read_csv('data/pseudo_actions_labels_with_id.csv')

chatgpt_labels = chatgpt_labels.rename(columns={"text": "llm_text", "action_detected": "llm_actions"})
chatgpt_labels['llm_actions_lema'] = chatgpt_labels['llm_actions'].apply(
    lambda actions: [actions_processed_to_action[a] for a in eval(actions)] if pd.notnull(actions) else actions
)


merged_df = pd.merge(transcrpits_text_actions, chatgpt_labels, on='sample_id', how='outer')
merged_df['actions_in_text'] = merged_df['actions_in_text'].apply(lambda x: sorted(eval(x)) if pd.notnull(x) else x)
# Sort each array in llm_actions_lema column
merged_df['llm_actions_lema'] = merged_df['llm_actions_lema'].apply(lambda x: sorted(x) if isinstance(x, list) else x)


print(merged_df.shape)
merged_df['actions_match'] = merged_df['actions_in_text'] == merged_df['llm_actions_lema']
merged_df['text_match'] = merged_df['Text'] == merged_df['llm_text']
print("Number of matching actions:", merged_df['actions_match'].sum())
print("Number of matching texts:", merged_df['text_match'].sum())

merged_df[~merged_df['actions_match']].iloc[:20]


(1105, 6)
Number of matching actions: 1068
Number of matching texts: 1105


,sample_id,Text,actions_in_text,llm_text,llm_actions,llm_actions_lema,actions_match,text_match
28,29,"Are underway John Michael Austin Carr an Angel Gray welcome aboard for our first broadcast of a 2021 Darius Garland will find college Sexton Sexton High posted Drummond rubbing works on Capella, finds a country in Larry ***** back out the X of all five Cavaliers touched the ball Garland midrange floater.",[floater],"Are underway John Michael Austin Carr an Angel Gray welcome aboard for our first broadcast of a 2021 Darius Garland will find college Sexton Sexton High posted Drummond rubbing works on Capella, finds a country in Larry ***** back out the X of all five Cavaliers touched the ball Garland midrange floater.","['floater', 'post up']","[floater, post_up]",False,True
29,30,"Are underway John Michael Austin Carr an Angel Gray welcome aboard for our first broadcast of a 2021 Darius Garland will find college Sexton Sexton High posted Drummond rubbing works on Capella, finds a country in Larry ***** back out the X of all five Cavaliers touched the ball Garland midrange floater.",[floater],"Are underway John Michael Austin Carr an Angel Gray welcome aboard for our first broadcast of a 2021 Darius Garland will find college Sexton Sexton High posted Drummond rubbing works on Capella, finds a country in Larry ***** back out the X of all five Cavaliers touched the ball Garland midrange floater.","['floater', 'post up']","[floater, post_up]",False,True
68,69,High rainbow shot.,[],High rainbow shot.,['rainbow shot'],[rainbow_shoot],False,True
134,135,Who would you like to Posterize in this League?,[],Who would you like to Posterize in this League?,['poster'],[poster],False,True
172,173,Knox comes to help out this week and doing a lot of dropping and double teaming.,[],Knox comes to help out this week and doing a lot of dropping and double teaming.,['double team'],[double_team],False,True
226,227,"George went through some of his numbers as Kevin Knox misses on the other end, but chases down the rebound and tipping in Robinson.",[],"George went through some of his numbers as Kevin Knox misses on the other end, but chases down the rebound and tipping in Robinson.",['tip in'],[tip_in],False,True
286,287,Nice up there and try to get the tip instead.,[],Nice up there and try to get the tip instead.,['tip in'],[tip_in],False,True
289,290,He was in flip flops.,[flop],He was in flip flops.,[],[],False,True
383,384,Melo taking it to the ram in the bank shot doesn't go Collins couldn't clean it up but does on the second chance.,[],Melo taking it to the ram in the bank shot doesn't go Collins couldn't clean it up but does on the second chance.,['bank shot'],[bank_shoot],False,True
393,394,The other night Nurkic gets inside bank shot won't go the second attempt.,[],The other night Nurkic gets inside bank shot won't go the second attempt.,['bank shot'],[bank_shoot],False,True


In [23]:
#todo bad 2 labels here, need to fix 
df_pseudo_actions[df_pseudo_actions['action_detected'].apply(lambda x: len(eval(x)) == 0)].merge(transcrpits_folds, on='sample_id')

,text,action_detected,sample_id,Text,EventNameCleaned,tokenized_event_name,tokenized_text,actions_in_text,events,Label,fold1,fold2,fold3,fold4,fold5
0,So I think the Knicks are gonna go to that more Nixa double teaming.,[],40,So I think the Knicks are gonna go to that more Nixa double teaming.,Missed Shot by Justin Jackson,"['miss', 'shot', 'justin', 'jackson']","['think', 'knicks', 'nixa', 'double', 'teaming']",[],['Missed Shot'],0,test,val,train,test,val
1,He was in flip flops.,[],290,He was in flip flops.,FT-Made by Thon Maker,"['ft_make', 'thon', 'maker']","['flip', 'flop']",['flop'],['FT-Made'],0,val,train,test,val,train
2,Zach Lavine Bulls lobbying along the baseline.,[],751,Zach Lavine Bulls lobbying along the baseline.,2-pointer by Zach LaVine,"['2_pointer', 'zach', 'lavine']","['zach', 'lavine', 'bulls', 'lobby', 'baseline']",[],['2-pointer'],1,test,val,test,val,train
3,So name stops on a dime and says you want to come out a little bit further than that.,[],1025,So name stops on a dime and says you want to come out a little bit further than that.,3-pointer by Damian Lillard,"['3_pointer', 'damian', 'lillard']","['stop', 'dime', 'want', 'come', 'little', 'bit', 'far']",['dime'],['3-pointer'],1,test,test,test,test,test
4,Here's lobby Seer backing down and he gets the role.,[],1075,Here's lobby Seer backing down and he gets the role.,2-pointer by Skal Labissiere,"['2_pointer', 'skal', 'labissiere']","['lobby', 'seer', 'role']",[],['2-pointer'],0,train,test,test,test,train


# Generate augmentations from data using LLM

transcrpits_folds = pd.read_csv('data/transcripts_train_folds.csv')

In [ ]:
transcrpits_folds = pd.read_csv('data/transcripts_folds_train.csv')[['transcript_id', 'Label']]
df_pseudo_actions = pd.read_csv("data/pseudo_actions_labels_with_id.csv")

# A1

In [139]:
generated_data_df = df_pseudo_actions.loc[df_pseudo_actions['action_detected'].apply(lambda x: len(eval(x)) == 1)] \
    .assign(action_detected=lambda df: df['action_detected'].apply(lambda x: eval(x)[0])) \
    .merge(transcrpits_folds, on='transcript_id')
generated_data_df

labels_mapping = {0: "action_not_executed", 1: "action_executed"}

row_dicts = generated_data_df.apply(
    lambda row: {
        "transcript_id": row["transcript_id"],
        "text": row["text"],
        "action_in_text": row["action_detected"],
        "label": labels_mapping[row["Label"]]
    }, axis=1
).tolist()
row_dicts[550:600]



[{'transcript_id': 570,
  'text': 'Fraser had a screen from drama, gives it up to Drummond and back to Frazier Little floater off the window.',
  'action_in_text': 'floater',
  'label': 'action_executed'},
 {'transcript_id': 571,
  'text': "Half court set here for Brooklyn Pick and Roll Action Allen over Go Berry doesn't get the role.",
  'action_in_text': 'pick and roll',
  'label': 'action_executed'},
 {'transcript_id': 572,
  'text': 'Campus pick and roll with seller and that is slap back by Dedmon was very poor shot there by Zella.',
  'action_in_text': 'pick and roll',
  'label': 'action_executed'},
 {'transcript_id': 573,
  'text': 'Games on the Drive shot Clock winding down bank shot Mrs Smart 3 pounds ahead to Tatum.',
  'action_in_text': 'bank shot',
  'label': 'action_executed'},
 {'transcript_id': 574,
  'text': 'If you leave truly on James Harden and Chris Paul out of the pick and roll, Harden catches fires.',
  'action_in_text': 'pick and roll',
  'label': 'action_not_exec

# Gen Start

In [26]:
augmented_data = [
  {
    "transcript_id": 0,
    "text": "If you go into that defensive circle and post up, you notice the defensive players behind you and policy and left that jump look a little bit short.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "0_1",
        "label": "action_executed",
        "text": "He spins into the paint, gets deep position, and firmly posts up with defenders shading behind."
      },
      {
        "new_transcript_id": "0_2",
        "label": "action_not_executed",
        "text": "Looked like he was about to post up inside, but the ball swung away before he established position."
      }
    ]
  },
  {
    "transcript_id": 1,
    "text": "He's double teamed.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "1_1",
        "label": "action_executed",
        "text": "As soon as he caught it near the elbow, two defenders converged and forced the double team pressure."
      },
      {
        "new_transcript_id": "1_2",
        "label": "action_not_executed",
        "text": "They showed a second defender near the baseline, but ultimately stayed home and no double team came."
      }
    ]
  },
  {
    "transcript_id": 2,
    "text": "Jordan trying to post up against Oxymora.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "2_1",
        "label": "action_executed",
        "text": "Jordan backs Oxymora down on the block, firmly establishing a post up before the pass arrives."
      },
      {
        "new_transcript_id": "2_2",
        "label": "action_not_executed",
        "text": "Jordan started to angle his body for a post up, but the entry never came and play reset outside."
      }
    ]
  },
  {
    "transcript_id": 3,
    "text": "They're going to come and help on all of his post ups.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "3_1",
        "label": "action_not_executed",
        "text": "Defense keeps shading whenever he sets up, but on this possession the post up hasn't happened yet."
      },
      {
        "new_transcript_id": "3_2",
        "label": "action_executed",
        "text": "He planted deep on the left block and the defense instantly collapsed to contest his post up move."
      }
    ]
  },
  {
    "transcript_id": 4,
    "text": "They're going to come and help on all of his post ups.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "4_1",
        "label": "action_not_executed",
        "text": "It’s a strategy note, saying they’ll collapse on any future post ups, but nothing occurred this trip."
      },
      {
        "new_transcript_id": "4_2",
        "label": "action_executed",
        "text": "He seals his man on the low block, executes a solid post up, and the help defender rotates quickly."
      }
    ]
  },
  {
    "transcript_id": 5,
    "text": "Maybe tonight not to double team as much.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "5_1",
        "label": "action_not_executed",
        "text": "Coaches suggest they might back off the double team tonight, indicating a lighter scheme approach."
      },
      {
        "new_transcript_id": "5_2",
        "label": "action_executed",
        "text": "Immediately when he drove into the lane, the second defender collapsed hard and the double team came."
      }
    ]
  },
  {
    "transcript_id": 6,
    "text": "Or for tried to tuck in, but Lowry always takes those post ups personally.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "6_1",
        "label": "action_not_executed",
        "text": "Lowry battled him physically, preventing the post up from ever truly establishing on this sequence."
      },
      {
        "new_transcript_id": "6_2",
        "label": "action_executed",
        "text": "Despite the contact, he sealed on the block and successfully executed a post up against Lowry."
      }
    ]
  },
  {
    "transcript_id": 7,
    "text": "Doesn't get the finger roll challenged at the rim.",
    "action": "finger roll",
    "augmentations": [
      {
        "new_transcript_id": "7_1",
        "label": "action_executed",
        "text": "He attacks downhill, tries the finger roll at the rim, but the defense contests and the shot falls short."
      },
      {
        "new_transcript_id": "7_2",
        "label": "action_not_executed",
        "text": "Looked ready to scoop it with a finger roll, but contact forced him to dish instead of finishing."
      }
    ]
  },
  {
    "transcript_id": 8,
    "text": "Now over tomorrow good pump, fake jokisch.",
    "action": "pump fake",
    "augmentations": [
      {
        "new_transcript_id": "8_1",
        "label": "action_executed",
        "text": "He gave a convincing pump fake on the wing, freezing the defender before putting the ball on the floor."
      },
      {
        "new_transcript_id": "8_2",
        "label": "action_not_executed",
        "text": "He looked as if to pump fake, but instead just held steady and swung the ball along the perimeter."
      }
    ]
  },
  {
    "transcript_id": 9,
    "text": "I was three shots and made 0 Russell penetration floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "9_1",
        "label": "action_executed",
        "text": "Russell drives hard into the lane, rises and lofts a floater softly toward the rim."
      },
      {
        "new_transcript_id": "9_2",
        "label": "action_not_executed",
        "text": "He started downhill like a floater was coming, but instead kicked it out to the corner shooter."
      }
    ]
  },
  {
    "transcript_id": 10,
    "text": "Well, I saw a different type of defense against the Raptors double teamed every time he touched the ball is Westbrook runs is for the midrange shot of fibrosing.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "10_1",
        "label": "action_not_executed",
        "text": "Tonight they only threatened, but the Raptors didn’t actually send the double team when he caught it."
      },
      {
        "new_transcript_id": "10_2",
        "label": "action_executed",
        "text": "As soon as he touched the ball, the Raptors swarmed with two defenders executing a strong double team."
      }
    ]
  },
  {
    "transcript_id": 11,
    "text": "Richardson came over from Philly last year and Cauley Stein went for the slam dunk.",
    "action": "slam dunk",
    "augmentations": [
      {
        "new_transcript_id": "11_1",
        "label": "action_executed",
        "text": "Cauley Stein elevated with force in transition and hammered home the slam dunk to ignite the crowd."
      },
      {
        "new_transcript_id": "11_2",
        "label": "action_not_executed",
        "text": "He rose like a slam dunk was coming, but the defender bumped him midair forcing a miss at the rim."
      }
    ]
  },
  {
    "transcript_id": 12,
    "text": "Richardson came over from Philly last year and Cauley Stein went for the slam dunk.",
    "action": "slam dunk",
    "augmentations": [
      {
        "new_transcript_id": "12_1",
        "label": "action_executed",
        "text": "Cauley Stein rolls hard to the basket, gets the pass, and finishes it emphatically with a slam dunk."
      },
      {
        "new_transcript_id": "12_2",
        "label": "action_not_executed",
        "text": "Looked like he wanted the slam dunk, but lost control and settled for a soft layup attempt instead."
      }
    ]
  },
  {
    "transcript_id": 13,
    "text": "In a pretty good pick and roll team as well.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "13_1",
        "label": "action_not_executed",
        "text": "They’re known for executing pick and roll often, but in this possession they never ran it at all."
      },
      {
        "new_transcript_id": "13_2",
        "label": "action_executed",
        "text": "Guard and big combined neatly, executed a textbook pick and roll, and opened space down the lane."
      }
    ]
  },
  {
    "transcript_id": 14,
    "text": "His floater is good.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "14_1",
        "label": "action_executed",
        "text": "He attacked the lane, released a high floater over the big man, and dropped it cleanly through."
      },
      {
        "new_transcript_id": "14_2",
        "label": "action_not_executed",
        "text": "Looked ready to launch the floater, but he pulled it back and reset at the top of the key."
      }
    ]
  },
  {
    "transcript_id": 15,
    "text": "Take that away and Brandon Goodwin takes it coast to coast against Prince.",
    "action": "coast to coast",
    "augmentations": [
      {
        "new_transcript_id": "15_1",
        "label": "action_executed",
        "text": "Goodwin snatched the rebound and went coast to coast, slicing through defenders to finish strong."
      },
      {
        "new_transcript_id": "15_2",
        "label": "action_not_executed",
        "text": "He began sprinting as if to go coast to coast, but slowed down and dished off in the half court."
      }
    ]
  },
  {
    "transcript_id": 16,
    "text": "And they just conceded a pick and roll.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "16_1",
        "label": "action_executed",
        "text": "The guard used the screen, rolled the ball inside, and the defense conceded the pick and roll bucket."
      },
      {
        "new_transcript_id": "16_2",
        "label": "action_not_executed",
        "text": "They lined up like a pick and roll was coming, but spacing broke down before the action unfolded."
      }
    ]
  },
  {
    "transcript_id": 17,
    "text": "Coast to coast kissed it off the glass short rebound white.",
    "action": "coast to coast",
    "augmentations": [
      {
        "new_transcript_id": "17_1",
        "label": "action_executed",
        "text": "He took the rebound himself, raced coast to coast, and attempted to bank it but left it short."
      },
      {
        "new_transcript_id": "17_2",
        "label": "action_not_executed",
        "text": "He pushed the pace like a coast to coast run, but passed off at half court instead of finishing."
      }
    ]
  },
  {
    "transcript_id": 18,
    "text": "Slam dunk champion.",
    "action": "slam dunk",
    "augmentations": [
      {
        "new_transcript_id": "18_1",
        "label": "action_not_executed",
        "text": "It’s just a mention of his status as slam dunk champion, not an actual dunk attempt in this play."
      },
      {
        "new_transcript_id": "18_2",
        "label": "action_executed",
        "text": "He soared into the lane and hammered down a thunderous slam dunk reminiscent of his contest wins."
      }
    ]
  },
  {
    "transcript_id": 19,
    "text": "Left handed floater for two, see this little hesitation move.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "19_1",
        "label": "action_executed",
        "text": "He hesitated briefly, then lofted a left handed floater over the defender that dropped cleanly."
      },
      {
        "new_transcript_id": "19_2",
        "label": "action_not_executed",
        "text": "He sold the hesitation like a floater was coming, but then passed out to the perimeter instead."
      }
    ]
  },
  {
    "transcript_id": 20,
    "text": "Little pick and roll game Kemba Walker to master at that and Daniel Theis unable to put it down.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "20_1",
        "label": "action_not_executed",
        "text": "They set up for the pick and roll, but Theis mishandled the pass and no finish came from the play."
      },
      {
        "new_transcript_id": "20_2",
        "label": "action_executed",
        "text": "Walker ran the pick and roll to perfection, slipping the ball inside, even if Theis missed the shot."
      }
    ]
  },
  {
    "transcript_id": 21,
    "text": "Downhill in the finger roll is good.",
    "action": "finger roll",
    "augmentations": [
      {
        "new_transcript_id": "21_1",
        "label": "action_executed",
        "text": "He drove hard, scooped with the right hand, and the finger roll softly kissed the glass for two."
      },
      {
        "new_transcript_id": "21_2",
        "label": "action_not_executed",
        "text": "He gathered speed downhill, looked to finger roll it, but dished to the corner before going up."
      }
    ]
  },
  {
    "transcript_id": 22,
    "text": "Just the second time he's had a losing record with Eric at the helm, there's the floater from Trae Young to start thinking.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "22_1",
        "label": "action_executed",
        "text": "Trae Young drove into the lane, lofted his signature floater high over the defender, and hit it."
      },
      {
        "new_transcript_id": "22_2",
        "label": "action_not_executed",
        "text": "He moved like a floater was coming, but pulled up and reset the offense without releasing it."
      }
    ]
  },
  {
    "transcript_id": 23,
    "text": "Here's the post up for Danilo.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "23_1",
        "label": "action_executed",
        "text": "Danilo established deep block position and executed a strong post up against the smaller defender."
      },
      {
        "new_transcript_id": "23_2",
        "label": "action_not_executed",
        "text": "They set him on the block like a post up, but the ball never entered and he drifted away."
      }
    ]
  },
  {
    "transcript_id": 24,
    "text": "There is an outlet pass.",
    "action": "outlet pass",
    "augmentations": [
      {
        "new_transcript_id": "24_1",
        "label": "action_executed",
        "text": "He grabbed the rebound and immediately fired the outlet pass to spark the fast break attack."
      },
      {
        "new_transcript_id": "24_2",
        "label": "action_not_executed",
        "text": "Looked to launch the outlet pass, but held the ball and waited for teammates to set the half court."
      }
    ]
  },
  {
    "transcript_id": 25,
    "text": "No way, no way here comes out around the corner up a ghost floater counted Mitchell 17.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "25_1",
        "label": "action_executed",
        "text": "Mitchell drives baseline, flips up a high ghost floater off the glass and it drops through."
      },
      {
        "new_transcript_id": "25_2",
        "label": "action_not_executed",
        "text": "Mitchell tried to line up the floater but got stripped before he could release the ball."
      }
    ]
  },
  {
    "transcript_id": 26,
    "text": "Ben Simmons got all the way to the bucket in the finger roll good.",
    "action": "finger roll",
    "augmentations": [
      {
        "new_transcript_id": "26_1",
        "label": "action_executed",
        "text": "Simmons glides into the lane, extends his arm, and kisses in a smooth finger roll."
      },
      {
        "new_transcript_id": "26_2",
        "label": "action_not_executed",
        "text": "Simmons looked like he was going for a finger roll but dished it out to Harris instead."
      }
    ]
  },
  {
    "transcript_id": 27,
    "text": "DeAngelo Russell leans in and gets the floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "27_1",
        "label": "action_executed",
        "text": "Russell steps through traffic and lofts a floater from eight feet that splashes in."
      },
      {
        "new_transcript_id": "27_2",
        "label": "action_not_executed",
        "text": "Russell looked for the floater in the lane but passed out when the defense collapsed."
      }
    ]
  },
  {
    "transcript_id": 30,
    "text": "The flooring goes to work seven to shoot, dancing in pump fakes, spins away from pressure late in.",
    "action": "pump fake",
    "augmentations": [
      {
        "new_transcript_id": "30_1",
        "label": "action_executed",
        "text": "With the shot clock winding down, he sells the defender on a pump fake and clears space."
      },
      {
        "new_transcript_id": "30_2",
        "label": "action_not_executed",
        "text": "He thought about the pump fake near the elbow but immediately fired the jumper instead."
      }
    ]
  },
  {
    "transcript_id": 31,
    "text": "He dropped in on angles and Rudy Little late coming over and at high floater for two.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "31_1",
        "label": "action_executed",
        "text": "Sliding past Gobert, he lofts a soft floater that bounces twice on the rim before falling."
      },
      {
        "new_transcript_id": "31_2",
        "label": "action_not_executed",
        "text": "He started the motion for a floater but Gobert blocked it clean before the shot left his hand."
      }
    ]
  },
  {
    "transcript_id": 32,
    "text": "You go under that pick and roll.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "32_1",
        "label": "action_executed",
        "text": "Guard curls off the screen, using the pick and roll to split defenders and attack the paint."
      },
      {
        "new_transcript_id": "32_2",
        "label": "action_not_executed",
        "text": "They set for a pick and roll, but the ballhandler pulled back without using the screen."
      }
    ]
  },
  {
    "transcript_id": 33,
    "text": "She's strong seasons with Orlando for the last couple of campaigns, a floater by Garland is no good.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "33_1",
        "label": "action_executed",
        "text": "Garland dribbles to his right hand, rises, and lofts a floater that rattles off the rim."
      },
      {
        "new_transcript_id": "33_2",
        "label": "action_not_executed",
        "text": "Garland shaped to take the floater, but drew a foul before the release on the drive."
      }
    ]
  },
  {
    "transcript_id": 34,
    "text": "Nice pick and roll with Dedmon turns cancer challenges.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "34_1",
        "label": "action_executed",
        "text": "Dedmon slips to the rim after the pick and roll and catches the pass for an easy finish."
      },
      {
        "new_transcript_id": "34_2",
        "label": "action_not_executed",
        "text": "They set up for a pick and roll, but the defender switched and cut off the passing lane."
      }
    ]
  },
  {
    "transcript_id": 35,
    "text": "Fournier, chased by Chalmers and the fadeaway, is in.",
    "action": "fadeaway",
    "augmentations": [
      {
        "new_transcript_id": "35_1",
        "label": "action_executed",
        "text": "Fournier pivots over the smaller defender, launches a contested fadeaway jumper, and buries it."
      },
      {
        "new_transcript_id": "35_2",
        "label": "action_not_executed",
        "text": "Fournier looked to create a fadeaway, but slipped and lost the handle before the shot."
      }
    ]
  }
,{
    "transcript_id": 36,
    "text": "And they're having some issues getting to the weak side and pick and roll so.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "36_1",
        "label": "action_not_executed",
        "text": "They tried to run a pick and roll on the wing, but poor spacing killed the possession."
      },
      {
        "new_transcript_id": "36_2",
        "label": "action_executed",
        "text": "Despite the pressure, they execute a clean pick and roll, slipping inside for a quick bucket."
      }
    ]
  },
  {
    "transcript_id": 37,
    "text": "So post up Robinson, teasing him with that stunt track never came really.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "37_1",
        "label": "action_not_executed",
        "text": "Robinson called for a post up, but the entry pass never came as the defense rotated."
      },
      {
        "new_transcript_id": "37_2",
        "label": "action_executed",
        "text": "Robinson seals his man deep on the block and powers up into a strong post up finish."
      }
    ]
  },
  {
    "transcript_id": 38,
    "text": "Is attempts have exploded with Milwaukee, took him a few extra years after Brooke, but he's there now too, as Robin wins the tip in Milwaukee controls.",
    "action": "tip in",
    "augmentations": [
      {
        "new_transcript_id": "38_1",
        "label": "action_not_executed",
        "text": "Robin crashed the glass but missed the tip in, and Milwaukee regrouped to reset the offense."
      },
      {
        "new_transcript_id": "38_2",
        "label": "action_executed",
        "text": "Robin rises above the crowd, secures the rebound, and softly lays in the tip in."
      }
    ]
  },
  {
    "transcript_id": 40,
    "text": "He gets it off of pick and rolls and things like that smart.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "40_1",
        "label": "action_executed",
        "text": "Smart feeds the big after running the pick and roll, leading to an easy dunk inside."
      },
      {
        "new_transcript_id": "40_2",
        "label": "action_not_executed",
        "text": "They called for a pick and roll, but Smart waved it off and pulled up for three instead."
      }
    ]
  },
  {
    "transcript_id": 41,
    "text": "Josh Tip in the crew chief.",
    "action": "tip in",
    "augmentations": [
      {
        "new_transcript_id": "41_1",
        "label": "action_not_executed",
        "text": "Josh went for a tip in attempt, but the ball rimmed out and play continued the other way."
      },
      {
        "new_transcript_id": "41_2",
        "label": "action_executed",
        "text": "Josh charges into the paint, elevates, and taps the ball back in with a tip in."
      }
    ]
  },
  {
    "transcript_id": 42,
    "text": "A little pick and roll action between yards and drew and drew his.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "42_1",
        "label": "action_executed",
        "text": "Holiday comes off the high screen, executes the pick and roll, and drops a dime to the roller."
      },
      {
        "new_transcript_id": "42_2",
        "label": "action_not_executed",
        "text": "They signaled for a pick and roll set, but miscommunication left the ballhandler stranded."
      }
    ]
  },
  {
    "transcript_id": 43,
    "text": "DeRozan to Belinelli the fadeaway off the window in Good Marco Belinelli.",
    "action": "fadeaway",
    "augmentations": [
      {
        "new_transcript_id": "43_1",
        "label": "action_executed",
        "text": "Belinelli leans back, releases a tough fadeaway off glass, and sinks the shot cleanly."
      },
      {
        "new_transcript_id": "43_2",
        "label": "action_not_executed",
        "text": "Belinelli started to rise for a fadeaway but got fouled before the shot could leave his hand."
      }
    ]
  },
  {
    "transcript_id": 44,
    "text": "Usually when your back is tight you're not able to get the floaters or go off the dribble like you like him.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "44_1",
        "label": "action_not_executed",
        "text": "He wanted to attempt a floater, but the sore back kept him from even lifting for the shot."
      },
      {
        "new_transcript_id": "44_2",
        "label": "action_executed",
        "text": "Despite his tight back, he manages to loft a short floater that rattles home for two."
      }
    ]
  },
  {
    "transcript_id": 45,
    "text": "We'll double team.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "45_1",
        "label": "action_not_executed",
        "text": "They considered a double team in the corner, but never actually brought the second defender."
      },
      {
        "new_transcript_id": "45_2",
        "label": "action_executed",
        "text": "Two defenders close in immediately, executing a tight double team to trap him near midcourt."
      }
    ]
  },
  {
    "transcript_id": 46,
    "text": "Just Jackson with Elena little floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "46_1",
        "label": "action_executed",
        "text": "Jackson dribbles into the lane, tosses up a soft floater, and watches it drop straight through."
      },
      {
        "new_transcript_id": "46_2",
        "label": "action_not_executed",
        "text": "Jackson drove hard but kicked it out before even attempting the floater in the lane."
      }
    ]
  },
  {
    "transcript_id": 47,
    "text": "Just Jackson with Elena little floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "47_1",
        "label": "action_executed",
        "text": "Jackson takes contact, hangs in the air, and nails a smooth floater from the left elbow."
      },
      {
        "new_transcript_id": "47_2",
        "label": "action_not_executed",
        "text": "Jackson faked the floater, drew defenders in, and lobbed it instead to the trailing big man."
      }
    ]
  },
  {
    "transcript_id": 48,
    "text": "Always starts the game posting up.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "48_1",
        "label": "action_executed",
        "text": "He opens the quarter with a strong post up, backing his man and scoring on a soft hook."
      },
      {
        "new_transcript_id": "48_2",
        "label": "action_not_executed",
        "text": "They planned to start with a post up, but the ball never found him on the block."
      }
    ]
  },
  {
    "transcript_id": 49,
    "text": "The good pump fake.",
    "action": "pump fake",
    "augmentations": [
      {
        "new_transcript_id": "49_1",
        "label": "action_executed",
        "text": "He pulls up, sells the defender with a pump fake, and then steps in for a clean jumper."
      },
      {
        "new_transcript_id": "49_2",
        "label": "action_not_executed",
        "text": "He looked like he might use a pump fake, but shot the ball quickly before the closeout."
      }
    ]
  },
  {
    "transcript_id": 50,
    "text": "Robert Williams into the game and immediately flushes in an alley Oop pass from Kemba.",
    "action": "alley oop",
    "augmentations": [
      {
        "new_transcript_id": "50_1",
        "label": "action_executed",
        "text": "Williams times the cut perfectly, catches Kemba’s lob, and hammers home the alley oop jam."
      },
      {
        "new_transcript_id": "50_2",
        "label": "action_not_executed",
        "text": "Kemba threw up the alley oop, but Williams mistimed his jump and the ball sailed out."
      }
    ]
  },
  {
    "transcript_id": 51,
    "text": "Gallinari posted up against Ingram.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "51_1",
        "label": "action_executed",
        "text": "Gallinari sets on the block, backs down Ingram, and scores cleanly on the post up hook."
      },
      {
        "new_transcript_id": "51_2",
        "label": "action_not_executed",
        "text": "Gallinari attempted to post up, but Ingram denied the entry pass, forcing a reset up top."
      }
    ]
  },
  {
    "transcript_id": 52,
    "text": "Here is MB sons will bring the double team and they forced the early turnover.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "52_1",
        "label": "action_executed",
        "text": "Two defenders rush to trap him, the double team forces a bad pass and a turnover."
      },
      {
        "new_transcript_id": "52_2",
        "label": "action_not_executed",
        "text": "They showed double team for a moment, but quickly backed off without applying real pressure."
      }
    ]
  },
  {
    "transcript_id": 53,
    "text": "Jackson drive the floater off the glass.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "53_1",
        "label": "action_executed",
        "text": "Jackson pushes hard into the lane and flips in a floater off the glass for two."
      },
      {
        "new_transcript_id": "53_2",
        "label": "action_not_executed",
        "text": "Jackson looked ready to float it high off the glass but pulled it back at the last second."
      }
    ]
  },
  {
    "transcript_id": 54,
    "text": "Initially it tried that finger roll in balance units in an effort to save it.",
    "action": "finger roll",
    "augmentations": [
      {
        "new_transcript_id": "54_1",
        "label": "action_executed",
        "text": "He attempts the finger roll while off balance, stretching toward the rim to keep the play alive."
      },
      {
        "new_transcript_id": "54_2",
        "label": "action_not_executed",
        "text": "Looked like he was going for a finger roll finish but lost his footing and passed it out instead."
      }
    ]
  },
  {
    "transcript_id": 56,
    "text": "Adams getting that dunk off a high pick and roll.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "56_1",
        "label": "action_not_executed",
        "text": "Adams set the screen but the guard never used the pick and roll to create a play."
      },
      {
        "new_transcript_id": "56_2",
        "label": "action_executed",
        "text": "Adams slips perfectly after the high pick and roll and slams it home with authority."
      }
    ]
  },
  {
    "transcript_id": 57,
    "text": "The young guys trying to find their rhythm play some pick and roll today.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "57_1",
        "label": "action_not_executed",
        "text": "The rookies are experimenting with pick and roll sets in this quarter but haven’t executed one yet."
      },
      {
        "new_transcript_id": "57_2",
        "label": "action_executed",
        "text": "They finally get into a solid pick and roll sequence, leading to an easy basket inside."
      }
    ]
  },
  {
    "transcript_id": 58,
    "text": "Yeah, pick and roll.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "58_1",
        "label": "action_not_executed",
        "text": "Commentator simply noting the pick and roll look, but no action follows on this possession."
      },
      {
        "new_transcript_id": "58_2",
        "label": "action_executed",
        "text": "They run a clean pick and roll here, freeing the guard to finish at the rim."
      }
    ]
  },
  {
    "transcript_id": 59,
    "text": "Nice pick and roll herder.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "59_1",
        "label": "action_executed",
        "text": "Huerter snakes the pick and roll, uses the screen, and drops a smooth dime for two."
      },
      {
        "new_transcript_id": "59_2",
        "label": "action_not_executed",
        "text": "Huerter motions for a pick and roll, but the defense switches early and they reset."
      }
    ]
  },
  {
    "transcript_id": 60,
    "text": "Is Derek with a hot floater cut the rim?",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "60_1",
        "label": "action_executed",
        "text": "Derrick darts into the paint and lofts a soft floater that kisses the rim and falls."
      },
      {
        "new_transcript_id": "60_2",
        "label": "action_not_executed",
        "text": "Derrick probes the lane, starts to float it up, but kicks out to the corner instead."
      }
    ]
  },
  {
    "transcript_id": 61,
    "text": "Alley Oop intercepted by Embiid Hornets.",
    "action": "alley oop",
    "augmentations": [
      {
        "new_transcript_id": "61_1",
        "label": "action_executed",
        "text": "They toss the alley oop toward the rim, but Embiid climbs the ladder and snatches it away."
      },
      {
        "new_transcript_id": "61_2",
        "label": "action_not_executed",
        "text": "They eye the alley oop, but Embiid jumps the passing lane early, forcing them to abort."
      }
    ]
  },
  {
    "transcript_id": 62,
    "text": "Alley Oop intercepted by Embiid Hornets.",
    "action": "alley oop",
    "augmentations": [
      {
        "new_transcript_id": "62_1",
        "label": "action_not_executed",
        "text": "Hornets load up for a lob, yet Embiid reads it and denies the alley oop before it leaves."
      },
      {
        "new_transcript_id": "62_2",
        "label": "action_executed",
        "text": "They throw the alley oop anyway, and Embiid intercepts midair to kill the play."
      }
    ]
  },
  {
    "transcript_id": 63,
    "text": "Alley Oop intercepted by Embiid Hornets.",
    "action": "alley oop",
    "augmentations": [
      {
        "new_transcript_id": "63_1",
        "label": "action_executed",
        "text": "Charlotte tries the alley oop, but Embiid elevates, picks it off, and starts the break."
      },
      {
        "new_transcript_id": "63_2",
        "label": "action_not_executed",
        "text": "Charlotte signals for the lob, yet Embiid jumps the route and they never release the alley oop."
      }
    ]
  },
  {
    "transcript_id": 64,
    "text": "Alley Oop intercepted by Embiid Hornets.",
    "action": "alley oop",
    "augmentations": [
      {
        "new_transcript_id": "64_1",
        "label": "action_not_executed",
        "text": "Seeing Embiid lurking, the Hornets abandon the alley oop setup and swing it back out top."
      },
      {
        "new_transcript_id": "64_2",
        "label": "action_executed",
        "text": "Despite Embiid shading the lane, they lob the alley oop and he plucks it clean."
      }
    ]
  },
  {
    "transcript_id": 65,
    "text": "Bring their vice Nights uniform, ever scouted that alley oop very well they'd like to go to Whiteside and Bam, Adebayo on those passes.",
    "action": "alley oop",
    "augmentations": [
      {
        "new_transcript_id": "65_1",
        "label": "action_not_executed",
        "text": "Heat were ready for the lob to Whiteside or Bam, so the defense shuts down the alley oop look."
      },
      {
        "new_transcript_id": "65_2",
        "label": "action_executed",
        "text": "They still try the alley oop toward Bam despite the scout, and the pass nearly connects above the rim."
      }
    ]
  },
  {
    "transcript_id": 66,
    "text": "Hard drives another alley oop.",
    "action": "alley oop",
    "augmentations": [
      {
        "new_transcript_id": "66_1",
        "label": "action_executed",
        "text": "Harden collapses the defense, tosses the alley oop, and his big flushes it home with two hands."
      },
      {
        "new_transcript_id": "66_2",
        "label": "action_not_executed",
        "text": "Harden penetrates looking lob, but the weakside help arrives and he resets rather than alley oop."
      }
    ]
  },
  {
    "transcript_id": 67,
    "text": "Santa ran ski with a floater and it's worn on the right.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "67_1",
        "label": "action_executed",
        "text": "Satoransky snakes middle, releases a right-hand floater from eight feet, and drops it softly."
      },
      {
        "new_transcript_id": "67_2",
        "label": "action_not_executed",
        "text": "Satoransky starts to loft a floater, but the big steps up and he dishes out instead."
      }
    ]
  },
  {
    "transcript_id": 68,
    "text": "High rainbow shot.",
    "action": "rainbow shot",
    "augmentations": [
      {
        "new_transcript_id": "68_1",
        "label": "action_executed",
        "text": "He arcs a high rainbow shot over the contest, scraping the rafters before splashing down."
      },
      {
        "new_transcript_id": "68_2",
        "label": "action_not_executed",
        "text": "He fakes the rainbow shot, sees the closeout coming, and swings it weakside to reset."
      }
    ]
  },
  {
    "transcript_id": 69,
    "text": "He's been relentless as a post up.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "69_1",
        "label": "action_not_executed",
        "text": "He calls for the block touch again, but the entry pass is denied and no post up occurs."
      },
      {
        "new_transcript_id": "69_2",
        "label": "action_executed",
        "text": "He seals deep on the block, receives the feed, and powers into a strong post up finish."
      }
    ]
  },
  {
    "transcript_id": 70,
    "text": "They dial up Julius Randle for post up right away.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "70_1",
        "label": "action_executed",
        "text": "Randle plants on the left block, backs down twice, spins middle, and scores from the post up."
      },
      {
        "new_transcript_id": "70_2",
        "label": "action_not_executed",
        "text": "They try to enter for a Randle post up, but help digs collapse and the ball never gets there."
      }
    ]
  },
  {
    "transcript_id": 71,
    "text": "The teardrop couldn't get it to go.",
    "action": "teardrop",
    "augmentations": [
      {
        "new_transcript_id": "71_1",
        "label": "action_executed",
        "text": "He lofts the teardrop over the shot blocker, but it rims out at the last moment."
      },
      {
        "new_transcript_id": "71_2",
        "label": "action_not_executed",
        "text": "He gears up for a teardrop in the lane, then sees help and bails to a corner shooter."
      }
    ]
  },
  {
    "transcript_id": 72,
    "text": "Thompson also had his first NBA 222 nights ago in that game against Indiana Garlands, exactly the foul line's teardrop is true.",
    "action": "teardrop",
    "augmentations": [
      {
        "new_transcript_id": "72_1",
        "label": "action_executed",
        "text": "Garland drifts to the nail and floats a pure teardrop that splashes straight through."
      },
      {
        "new_transcript_id": "72_2",
        "label": "action_not_executed",
        "text": "Garland approaches the foul line, fakes a teardrop, then skips it to the wing for three."
      }
    ]
  },
  {
    "transcript_id": 73,
    "text": "Gobert could not get the tip in on that end in Putun had it not able to convert.",
    "action": "tip in",
    "augmentations": [
      {
        "new_transcript_id": "73_1",
        "label": "action_not_executed",
        "text": "Gobert rises for the tip in but is boxed out, gets a hand on it, and cannot finish."
      },
      {
        "new_transcript_id": "73_2",
        "label": "action_executed",
        "text": "Gobert times the carom perfectly, tips it toward the cylinder, and the putback just rattles out."
      }
    ]
  },
  {
    "transcript_id": 74,
    "text": "The floater doesn't go for Simmons.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "74_1",
        "label": "action_executed",
        "text": "Simmons drops a soft floater from the dotted line, but it rolls off the front iron."
      },
      {
        "new_transcript_id": "74_2",
        "label": "action_not_executed",
        "text": "Simmons probes for a floater, sees the center step up, and kicks it out to the slot."
      }
    ]
  },
  {
    "transcript_id": 75,
    "text": "They're going to let these guards shoot these little floaters.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "75_1",
        "label": "action_not_executed",
        "text": "Defense sags to invite floaters, yet on this trip the guard declines and swings it around."
      },
      {
        "new_transcript_id": "75_2",
        "label": "action_executed",
        "text": "Given the drop coverage, the guard takes the bait and flips in a comfortable floater."
      }
    ]
  },
  {
    "transcript_id": 76,
    "text": "You may see some double teams from the wars that you gotta rotate out of it.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "76_1",
        "label": "action_not_executed",
        "text": "They anticipate a double team, but on this possession the defense stays home and no trap comes."
      },
      {
        "new_transcript_id": "76_2",
        "label": "action_executed",
        "text": "The Warriors spring the double team on the wing, forcing a hot potato pass out top."
      }
    ]
  },
  {
    "transcript_id": 77,
    "text": "Tatum double team.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "77_1",
        "label": "action_not_executed",
        "text": "They show help toward Tatum, bluffing a trap, but never commit to a full double team."
      },
      {
        "new_transcript_id": "77_2",
        "label": "action_executed",
        "text": "Two defenders swallow Tatum at the elbow, a quick double team forcing the ball out."
      }
    ]
  },
  {
    "transcript_id": 78,
    "text": "So how quick are these double teams coming for Detroit?",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "78_1",
        "label": "action_not_executed",
        "text": "They discuss Detroit’s timing, but on this trip there’s no double team, just a soft hedge."
      },
      {
        "new_transcript_id": "78_2",
        "label": "action_executed",
        "text": "Detroit springs a rapid double team on the catch, trapping the ballhandler near the nail."
      }
    ]
  },
  {
    "transcript_id": 79,
    "text": "Comics and Trey Floater from Trey's little law.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "79_1",
        "label": "action_executed",
        "text": "Trae snakes the screen and pops a delicate floater from ten feet straight down the chute."
      },
      {
        "new_transcript_id": "79_2",
        "label": "action_not_executed",
        "text": "Trae shapes for a floater, but the big steps up and he lobs out of it instead."
      }
    ]
  },
  {
    "transcript_id": 80,
    "text": "Millsap wants the post up.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "80_1",
        "label": "action_executed",
        "text": "Millsap seals on the right block, backs down patiently, and finishes through contact on the post up."
      },
      {
        "new_transcript_id": "80_2",
        "label": "action_not_executed",
        "text": "Millsap calls for a post up touch, but the entry angle disappears and they swing it."
      }
    ]
  },
  {
    "transcript_id": 81,
    "text": "15 foot high offer nothing but net and he wanted to back.",
    "action": "nothing but net",
    "augmentations": [
      {
        "new_transcript_id": "81_1",
        "label": "action_executed",
        "text": "From fifteen feet he rises in rhythm and drains it clean, nothing but net from midrange."
      },
      {
        "new_transcript_id": "81_2",
        "label": "action_not_executed",
        "text": "He starts to rise for the jumper, but the closeout deters him; no clean nothing-but-net look."
      }
    ]
  },
  {
    "transcript_id": 82,
    "text": "Gotta post up Cody Zeller.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "82_1",
        "label": "action_not_executed",
        "text": "They want Zeller on the block, but the feed is fronted and the post up never materializes."
      },
      {
        "new_transcript_id": "82_2",
        "label": "action_executed",
        "text": "They finally angle the entry, Zeller catches deep, and pounds in a decisive post up score."
      }
    ]
  },
  {
    "transcript_id": 83,
    "text": "Garrett Temple gets away from Josh Hart slam dunk.",
    "action": "slam dunk",
    "augmentations": [
      {
        "new_transcript_id": "83_1",
        "label": "action_executed",
        "text": "Temple shakes Hart on the wing, attacks the rim, and hammers a two-hand slam dunk."
      },
      {
        "new_transcript_id": "83_2",
        "label": "action_not_executed",
        "text": "Temple breaks free but is clipped on the drive, whistle blows before any slam dunk attempt."
      }
    ]
  },
  {
    "transcript_id": 84,
    "text": "Double teaming already thinking about Curry.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "84_1",
        "label": "action_not_executed",
        "text": "They’re planning to double team Curry, but on this possession he goes untrapped off the screen."
      },
      {
        "new_transcript_id": "84_2",
        "label": "action_executed",
        "text": "As Curry crosses half court, two defenders converge immediately in a hard double team."
      }
    ]
  },
  {
    "transcript_id": 85,
    "text": "Now it's Adams wanted to post up favors.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "85_1",
        "label": "action_executed",
        "text": "Adams buries Favors on his hip, receives it deep, and muscles home a strong post up."
      },
      {
        "new_transcript_id": "85_2",
        "label": "action_not_executed",
        "text": "Adams tries to set up the post, but fronting from Favors breaks the angle and they reset."
      }
    ]
  },
  {
    "transcript_id": 86,
    "text": "Gary Harris turns the corner now Millsap great pump fake.",
    "action": "pump fake",
    "augmentations": [
      {
        "new_transcript_id": "86_1",
        "label": "action_executed",
        "text": "Millsap shot-fakes the closeout into the air, takes one dribble, and finishes after the pump fake."
      },
      {
        "new_transcript_id": "86_2",
        "label": "action_not_executed",
        "text": "Millsap shows the ball but the defender stays grounded; no bite on the pump fake this time."
      }
    ]
  },
  {
    "transcript_id": 87,
    "text": "Taylor on the run to Bazemore in the corner and he splashes in the tree.",
    "action": "splash",
    "augmentations": [
      {
        "new_transcript_id": "87_1",
        "label": "action_executed",
        "text": "Bazemore catches in the corner in rhythm and lets it fly—pure splash from deep."
      },
      {
        "new_transcript_id": "87_2",
        "label": "action_not_executed",
        "text": "Bazemore loads up in the corner, but the defender recovers and he passes, no splash."
      }
    ]
  },
  {
    "transcript_id": 88,
    "text": "A little pick and roll action between yards and drew and drew his.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "88_1",
        "label": "action_executed",
        "text": "They initiate a snug pick and roll, ballhandler turns the corner and drops a pocket pass for two."
      },
      {
        "new_transcript_id": "88_2",
        "label": "action_not_executed",
        "text": "They signal pick and roll, but a switch stalls the action and they flow into horns."
      }
    ]
  },
  {
    "transcript_id": 89,
    "text": "Millsaps, trying to power his way to the basket, little finger, rolled over removed by Millsap, but good position.",
    "action": "finger roll",
    "augmentations": [
      {
        "new_transcript_id": "89_1",
        "label": "action_executed",
        "text": "Millsap slashes middle and kisses a delicate finger roll high off the window."
      },
      {
        "new_transcript_id": "89_2",
        "label": "action_not_executed",
        "text": "Millsap shows the finger roll, then the help closes and he pivots out to the shooter."
      }
    ]
  },
  {
    "transcript_id": 90,
    "text": "Randall pulls the mist down, brings it to four court NYT up 6 Randall in the Lane double team.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "90_1",
        "label": "action_executed",
        "text": "Randle drives into the lane and immediately meets a strong double team, ball pops out to Barrett."
      },
      {
        "new_transcript_id": "90_2",
        "label": "action_not_executed",
        "text": "Randle probes the paint, but the second defender never commits; no true double team forms."
      }
    ]
  },
  {
    "transcript_id": 91,
    "text": "Yeah, both teams try to score at the rim, but then there's Splash Mountain taking the team three let's their center.",
    "action": "splash",
    "augmentations": [
      {
        "new_transcript_id": "91_1",
        "label": "action_not_executed",
        "text": "Lopez pops beyond the arc, but they never find him; no splash from Splash Mountain here."
      },
      {
        "new_transcript_id": "91_2",
        "label": "action_executed",
        "text": "Splash Mountain trails the play, spots up, and buries a deep triple—pure splash."
      }
    ]
  },
  {
    "transcript_id": 92,
    "text": "post up the smaller Trae Young and Sweet Junior missing badly there.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "92_1",
        "label": "action_executed",
        "text": "They go right at the mismatch, post up Trae Young, and the hook rolls off the rim."
      },
      {
        "new_transcript_id": "92_2",
        "label": "action_not_executed",
        "text": "They look to post up Trae, but the double arrives and they kick it out instead."
      }
    ]
  },
  {
    "transcript_id": 93,
    "text": "Player for the Thunder last night to Billy Donovan relied on over 15 minutes is Mitchell gets inside and his floater drops in.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "93_1",
        "label": "action_executed",
        "text": "Mitchell knifes to the dotted line and releases a smooth floater that tumbles through."
      },
      {
        "new_transcript_id": "93_2",
        "label": "action_not_executed",
        "text": "Mitchell drives toward a floater, sees the big step up, and skips to the corner shooter."
      }
    ]
  },
  {
    "transcript_id": 94,
    "text": "Loves that little back down put post up there.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "94_1",
        "label": "action_not_executed",
        "text": "He wants the block touch, but the defender fronts perfectly; no post up entry thrown."
      },
      {
        "new_transcript_id": "94_2",
        "label": "action_executed",
        "text": "He establishes position on the left block, takes two power dribbles, and scores on the post up."
      }
    ]
  },
  {
    "transcript_id": 95,
    "text": "Nothing but net.",
    "action": "nothing but net",
    "augmentations": [
      {
        "new_transcript_id": "95_1",
        "label": "action_executed",
        "text": "He rises smoothly and cans the jumper, absolutely pure—nothing but net from the elbow."
      },
      {
        "new_transcript_id": "95_2",
        "label": "action_not_executed",
        "text": "He thinks about the jumper, pulls it down as the trap arrives; no nothing-but-net moment."
      }
    ]
  },
  {
    "transcript_id": 96,
    "text": "Here a little power throwing his body into Julie's random backs him up and then showing off his little finger roll.",
    "action": "finger roll",
    "augmentations": [
      {
        "new_transcript_id": "96_1",
        "label": "action_executed",
        "text": "He bumps the defender off his spot and finishes with a silky finger roll over the rim."
      },
      {
        "new_transcript_id": "96_2",
        "label": "action_not_executed",
        "text": "He initiates contact, shows the ball for a finger roll, then swings it out as help arrives."
      }
    ]
  },
  {
    "transcript_id": 97,
    "text": "Pascal look into post up strong take and it doesn't go.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "97_1",
        "label": "action_executed",
        "text": "Pascal backs down on the block, spins baseline, and the post up rolls off the rim."
      },
      {
        "new_transcript_id": "97_2",
        "label": "action_not_executed",
        "text": "Pascal flashes to the block, but the entry is denied and they never initiate the post up."
      }
    ]
  },
  {
    "transcript_id": 98,
    "text": "Catholic by three paint double team gets it back out to Taj Gibson.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "98_1",
        "label": "action_executed",
        "text": "He’s swarmed on the catch, double team in the paint, and he fires it out to Gibson."
      },
      {
        "new_transcript_id": "98_2",
        "label": "action_not_executed",
        "text": "Help shades toward the post, but no second defender commits; the double team never comes."
      }
    ]
  },
  {
    "transcript_id": 99,
    "text": "Depoorter the floater falls.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "99_1",
        "label": "action_executed",
        "text": "Deporter glides into the lane and drops a tidy floater that settles softly."
      },
      {
        "new_transcript_id": "99_2",
        "label": "action_not_executed",
        "text": "Deporter shapes for a floater, then threads a bounce pass when the center steps up."
      }
    ]
  },
  {
    "transcript_id": 100,
    "text": "But back at you with the alley Oop, older Harden gets the ball.",
    "action": "alley oop",
    "augmentations": [
      {
        "new_transcript_id": "100_1",
        "label": "action_executed",
        "text": "Harden answers immediately, drawing help and tossing the alley oop for a thunderous finish."
      },
      {
        "new_transcript_id": "100_2",
        "label": "action_not_executed",
        "text": "Harden looks for the alley oop, but the weakside tagger snuffs it and he resets."
      }
    ]
  },
  {
    "transcript_id": 101,
    "text": "Nearly had the tip in to get the same shot that Gordon got.",
    "action": "tip in",
    "augmentations": [
      {
        "new_transcript_id": "101_1",
        "label": "action_executed",
        "text": "He skies for the rebound and flicks a quick tip in that barely misses the mark."
      },
      {
        "new_transcript_id": "101_2",
        "label": "action_not_executed",
        "text": "He elevates for a tip in but is bumped underneath, whistle halts play before any attempt."
      }
    ]
  },
  {
    "transcript_id": 102,
    "text": "Boston between the legs short.",
    "action": "between the legs",
    "augmentations": [
      {
        "new_transcript_id": "102_1",
        "label": "action_not_executed",
        "text": "He starts a between-the-legs setup, then stumbles on the gather and never completes the move."
      },
      {
        "new_transcript_id": "102_2",
        "label": "action_executed",
        "text": "Boston crosses between the legs to shake his man, creating room before firing the jumper."
      }
    ]
  },
  {
    "transcript_id": 103,
    "text": "Good chemistry between those two, they play a lot of pick and roll together.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "103_1",
        "label": "action_executed",
        "text": "They flow into pick and roll seamlessly, guard turns the corner and hits the roller in stride."
      },
      {
        "new_transcript_id": "103_2",
        "label": "action_not_executed",
        "text": "They signal their usual pick and roll, but the defense switches and they abort the action."
      }
    ]
  },
   {
    "transcript_id": 104,
    "text": "Here's Rose Derrick Rose with that floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "104_1",
        "label": "action_executed",
        "text": "Derrick Rose drives inside and softly drops in the floater over the defense."
      },
      {
        "new_transcript_id": "104_2",
        "label": "action_not_executed",
        "text": "Rose looked ready to launch the floater but pulled back, kicking it to the corner instead."
      }
    ]
  },
  {
    "transcript_id": 105,
    "text": "70 shoot it gets into the paint on a tough fadeaway that's off the front of the iron.",
    "action": "fadeaway",
    "augmentations": [
      {
        "new_transcript_id": "105_1",
        "label": "action_executed",
        "text": "He spun into the lane and released a tough fadeaway jumper, clanking it off the rim."
      },
      {
        "new_transcript_id": "105_2",
        "label": "action_not_executed",
        "text": "He started to lean back as if for a fadeaway but passed the ball before lifting the shot."
      }
    ]
  },
  {
    "transcript_id": 106,
    "text": "Coast to coast now kicks it back out.",
    "action": "coast to coast",
    "augmentations": [
      {
        "new_transcript_id": "106_1",
        "label": "action_not_executed",
        "text": "He looked to go coast to coast but slowed down and swung it to the trailing guard."
      },
      {
        "new_transcript_id": "106_2",
        "label": "action_executed",
        "text": "He sprinted coast to coast past defenders, finishing strong at the rim before the defense reacted."
      }
    ]
  },
  {
    "transcript_id": 107,
    "text": "Coast to coast now kicks it back out.",
    "action": "coast to coast",
    "augmentations": [
      {
        "new_transcript_id": "107_1",
        "label": "action_not_executed",
        "text": "Pushed the tempo as if to go coast to coast but stopped short and kicked it to the wing."
      },
      {
        "new_transcript_id": "107_2",
        "label": "action_executed",
        "text": "He went coast to coast with speed, slicing through the lane for a finish at the basket."
      }
    ]
  },
  {
    "transcript_id": 108,
    "text": "Now over tomorrow good pump, fake jokisch.",
    "action": "pump fake",
    "augmentations": [
      {
        "new_transcript_id": "108_1",
        "label": "action_executed",
        "text": "Jokic sold the pump fake beautifully, drawing his defender off the floor before making his move."
      },
      {
        "new_transcript_id": "108_2",
        "label": "action_not_executed",
        "text": "Jokic thought about a pump fake but instead immediately swung the ball across the top."
      }
    ]
  },
  {
    "transcript_id": 109,
    "text": "Tristan Thompson guarding Geron Heroes to the bucket on a pick and roll.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "109_1",
        "label": "action_executed",
        "text": "They executed the pick and roll to perfection, rolling hard to the rim against Thompson."
      },
      {
        "new_transcript_id": "109_2",
        "label": "action_not_executed",
        "text": "They set up as if to run a pick and roll but the ball handler waved it off immediately."
      }
    ]
  },
  {
    "transcript_id": 110,
    "text": "Partner is going to be middle pick and roll defense.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "110_1",
        "label": "action_executed",
        "text": "The guard and big combined on a middle pick and roll, drawing help from the weak side."
      },
      {
        "new_transcript_id": "110_2",
        "label": "action_not_executed",
        "text": "They discussed middle pick and roll coverages but the actual possession never initiated it."
      }
    ]
  },
  {
    "transcript_id": 111,
    "text": "Quick move before the double team had a chance of getting there.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "111_1",
        "label": "action_not_executed",
        "text": "He attacked swiftly so the double team never actually arrived in time on this possession."
      },
      {
        "new_transcript_id": "111_2",
        "label": "action_executed",
        "text": "Two defenders closed down aggressively, forcing him into a tight double team on the block."
      }
    ]
  },
  {
    "transcript_id": 104,
    "text": "Here's Rose Derrick Rose with that floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "104_1",
        "label": "action_executed",
        "text": "Derrick Rose knifes into the lane and drops a soft floater over the big."
      },
      {
        "new_transcript_id": "104_2",
        "label": "action_not_executed",
        "text": "Rose shows the floater, then aborts and sprays a pass to the weak-side corner."
      }
    ]
  },
  {
    "transcript_id": 105,
    "text": "70 shoot it gets into the paint on a tough fadeaway that's off the front of the iron.",
    "action": "fadeaway",
    "augmentations": [
      {
        "new_transcript_id": "105_1",
        "label": "action_executed",
        "text": "He pivots baseline and launches a contested fadeaway that catches only front rim."
      },
      {
        "new_transcript_id": "105_2",
        "label": "action_not_executed",
        "text": "He starts to lean for a fadeaway, but kicks out to the slot before releasing."
      }
    ]
  },
  {
    "transcript_id": 106,
    "text": "Coast to coast now kicks it back out.",
    "action": "coast to coast",
    "augmentations": [
      {
        "new_transcript_id": "106_1",
        "label": "action_not_executed",
        "text": "He pushes in transition as if to go coast to coast, then circles out and resets."
      },
      {
        "new_transcript_id": "106_2",
        "label": "action_executed",
        "text": "He races coast to coast through traffic and finishes at the cup before help arrives."
      }
    ]
  },
  {
    "transcript_id": 107,
    "text": "Coast to coast now kicks it back out.",
    "action": "coast to coast",
    "augmentations": [
      {
        "new_transcript_id": "107_1",
        "label": "action_not_executed",
        "text": "He looks coast to coast but throttles down near the nail, swinging it to the wing."
      },
      {
        "new_transcript_id": "107_2",
        "label": "action_executed",
        "text": "He goes coast to coast in a burst, laying it in ahead of the recovering defense."
      }
    ]
  },
  {
    "transcript_id": 108,
    "text": "Now over tomorrow good pump, fake jokisch.",
    "action": "pump fake",
    "augmentations": [
      {
        "new_transcript_id": "108_1",
        "label": "action_executed",
        "text": "Jokic sells the pump fake, gets the defender airborne, then steps through cleanly."
      },
      {
        "new_transcript_id": "108_2",
        "label": "action_not_executed",
        "text": "He considers a pump fake at the elbow, but moves it along before the closeout."
      }
    ]
  },
  {
    "transcript_id": 109,
    "text": "Tristan Thompson guarding Geron Heroes to the bucket on a pick and roll.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "109_1",
        "label": "action_executed",
        "text": "They run pick and roll; the screener dives and catches, finishing through Thompson."
      },
      {
        "new_transcript_id": "109_2",
        "label": "action_not_executed",
        "text": "They signal pick and roll, but the handler rejects the screen and swings it out."
      }
    ]
  },
  {
    "transcript_id": 110,
    "text": "Partner is going to be middle pick and roll defense.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "110_1",
        "label": "action_executed",
        "text": "They initiate a middle pick and roll, drawing a tag from the weak-side corner."
      },
      {
        "new_transcript_id": "110_2",
        "label": "action_not_executed",
        "text": "They talk through middle pick and roll coverage, but never run the action this trip."
      }
    ]
  },
  {
    "transcript_id": 111,
    "text": "Quick move before the double team had a chance of getting there.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "111_1",
        "label": "action_not_executed",
        "text": "He attacks immediately, so the second defender never arrives to form a double."
      },
      {
        "new_transcript_id": "111_2",
        "label": "action_executed",
        "text": "Help comes fast; two defenders trap him on the block and force a pickup."
      }
    ]
  },
  {
    "transcript_id": 112,
    "text": "Here's Trey Burke, guarded by Bledsoe on the pick and roll.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "112_1",
        "label": "action_executed",
        "text": "Burke uses the screen against Bledsoe, snakes the lane, and finds the roll man."
      },
      {
        "new_transcript_id": "112_2",
        "label": "action_not_executed",
        "text": "Burke fakes toward the screen, declines it, and swings to the corner instead."
      }
    ]
  },
  {
    "transcript_id": 113,
    "text": "You could play the midrange game, he could post up other guards as Derrick favors.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "113_1",
        "label": "action_not_executed",
        "text": "He considers posting the smaller guard, but opts for a pull-up from the elbow."
      },
      {
        "new_transcript_id": "113_2",
        "label": "action_executed",
        "text": "He seals the mismatch on the block and begins a strong post up toward the middle."
      }
    ]
  },
  {
    "transcript_id": 116,
    "text": "He was double teamed over to McCollum Corner 3.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "116_1",
        "label": "action_executed",
        "text": "Two defenders converge, he escapes the double and fires it to McCollum for three."
      },
      {
        "new_transcript_id": "116_2",
        "label": "action_not_executed",
        "text": "Help shows briefly, then retreats; no true double forms before the kickout."
      }
    ]
  },
  {
    "transcript_id": 117,
    "text": "And also Westbrook guys that can defend post up players in this League.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "117_1",
        "label": "action_executed",
        "text": "Westbrook bodies his man on the block, initiating a firm post up toward the paint."
      },
      {
        "new_transcript_id": "117_2",
        "label": "action_not_executed",
        "text": "They talk about defending post ups, but the possession never features a back-down."
      }
    ]
  },
  {
    "transcript_id": 118,
    "text": "He beats her during transition and Tomahawks one arranging Tomahawk from Lehman to get things started here in Atlanta.",
    "action": "tomahawk",
    "augmentations": [
      {
        "new_transcript_id": "118_1",
        "label": "action_executed",
        "text": "In transition, he cocks it back and hammers a tomahawk to ignite the crowd."
      },
      {
        "new_transcript_id": "118_2",
        "label": "action_not_executed",
        "text": "He rises in transition but loses the handle, aborting the tomahawk for a reset."
      }
    ]
  },
  {
    "transcript_id": 119,
    "text": "He beats her during transition and Tomahawks one arranging Tomahawk from Lehman to get things started here in Atlanta.",
    "action": "tomahawk",
    "augmentations": [
      {
        "new_transcript_id": "119_1",
        "label": "action_executed",
        "text": "He sprints the break and thunders home a tomahawk slam, bench up instantly."
      },
      {
        "new_transcript_id": "119_2",
        "label": "action_not_executed",
        "text": "He loads for a tomahawk but is bumped midair, landing without attempting the dunk."
      }
    ]
  },
  {
    "transcript_id": 120,
    "text": "Listed at 51 and 28 third place in the West, parted with between the legs to set up Capella.",
    "action": "between the legs",
    "augmentations": [
      {
        "new_transcript_id": "120_1",
        "label": "action_executed",
        "text": "He dribbles between the legs to split the hedge and feeds Capela on the roll."
      },
      {
        "new_transcript_id": "120_2",
        "label": "action_not_executed",
        "text": "He fakes a between-the-legs move, keeps it high, and swings to the slot instead."
      }
    ]
  },
  {
    "transcript_id": 121,
    "text": "360 with a slam dunk.",
    "action": "slam dunk",
    "augmentations": [
      {
        "new_transcript_id": "121_1",
        "label": "action_executed",
        "text": "He spins 360 in the air and flushes a powerful slam to cap the break."
      },
      {
        "new_transcript_id": "121_2",
        "label": "action_not_executed",
        "text": "He gathers for a 360 attempt but lands awkwardly, resetting without a dunk."
      }
    ]
  },
  {
    "transcript_id": 122,
    "text": "Bacon pick and roll game with Vucevic off the dribble.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "122_1",
        "label": "action_executed",
        "text": "Bacon turns the corner off Vucevic’s screen and hits the big on the short roll."
      },
      {
        "new_transcript_id": "122_2",
        "label": "action_not_executed",
        "text": "They signal pick and roll, but the defense switches early and the action stalls."
      }
    ]
  },
  {
    "transcript_id": 123,
    "text": "Bacon pick and roll game with Vucevic off the dribble.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "123_1",
        "label": "action_executed",
        "text": "Bacon uses Vucevic’s screen, snakes middle, and finds the corner shooter."
      },
      {
        "new_transcript_id": "123_2",
        "label": "action_not_executed",
        "text": "He waves off the screen and resets the offense without initiating pick and roll."
      }
    ]
  },
  {
    "transcript_id": 124,
    "text": "Kowhai Leonard looking to post up Jamal Murray.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "124_1",
        "label": "action_executed",
        "text": "Leonard backs Murray down methodically, establishing a strong post on the left block."
      },
      {
        "new_transcript_id": "124_2",
        "label": "action_not_executed",
        "text": "He flashes to the block, then abandons the post to run a dribble handoff."
      }
    ]
  },
  {
    "transcript_id": 125,
    "text": "So can you start faster than pick and roll defense?",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "125_1",
        "label": "action_not_executed",
        "text": "They debate starting quicker against pick and roll, but never run the action."
      },
      {
        "new_transcript_id": "125_2",
        "label": "action_executed",
        "text": "Out of the timeout they initiate pick and roll immediately, catching the defense late."
      }
    ]
  },
  {
    "transcript_id": 126,
    "text": "Double team comms, Domasin Trouble and Harden makes the steal his heart came with double.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "126_1",
        "label": "action_executed",
        "text": "Harden springs the double with a teammate, trapping Domas and stripping the ball clean."
      },
      {
        "new_transcript_id": "126_2",
        "label": "action_not_executed",
        "text": "They threaten a trap, but the matchup peels away before a true double materializes."
      }
    ]
  },
  {
    "transcript_id": 127,
    "text": "On the wings of Bradley against Curry shot clocks down to seven pick and roll picked up off the floor nicely by Gortat, Johnson.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "127_1",
        "label": "action_executed",
        "text": "Late clock, they run pick and roll; Gortat scoops the bounce and finishes inside."
      },
      {
        "new_transcript_id": "127_2",
        "label": "action_not_executed",
        "text": "With seven on the clock, Curry jumps the screen and they abort the pick and roll."
      }
    ]
  },
  {
    "transcript_id": 128,
    "text": "He steps up on the pick and rolls and make sure that he's not hanging back in the Lane.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "128_1",
        "label": "action_not_executed",
        "text": "They talk hedging the pick and roll; no screen is actually set on this trip."
      },
      {
        "new_transcript_id": "128_2",
        "label": "action_executed",
        "text": "They call pick and roll; he steps up to hedge as the handler turns the corner."
      }
    ]
  },
  {
    "transcript_id": 129,
    "text": "It appears Lopez for three is nothing but net.",
    "action": "nothing but net",
    "augmentations": [
      {
        "new_transcript_id": "129_1",
        "label": "action_executed",
        "text": "Lopez rises and splashes it pure—nothing but net from straightaway."
      },
      {
        "new_transcript_id": "129_2",
        "label": "action_not_executed",
        "text": "Lopez fakes the jumper, drives a step, and kicks without releasing a shot."
      }
    ]
  },
  {
    "transcript_id": 130,
    "text": "Sacoche finger rolling at home.",
    "action": "finger roll",
    "augmentations": [
      {
        "new_transcript_id": "130_1",
        "label": "action_executed",
        "text": "He glides past the contest and kisses in a smooth finger roll."
      },
      {
        "new_transcript_id": "130_2",
        "label": "action_not_executed",
        "text": "He shows the finger roll, then pivots out and dishes to a corner shooter."
      }
    ]
  },
  {
    "transcript_id": 131,
    "text": "Bridges little floater in the Lane, no yellow and offensive rebound.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "131_1",
        "label": "action_executed",
        "text": "Bridges lofts a soft floater in the lane that rims out before a putback try."
      },
      {
        "new_transcript_id": "131_2",
        "label": "action_not_executed",
        "text": "Bridges probes for a floater look, then dumps it down instead of taking it."
      }
    ]
  },
  {
    "transcript_id": 132,
    "text": "Effectively touches fadeaway jumper.",
    "action": "fadeaway",
    "augmentations": [
      {
        "new_transcript_id": "132_1",
        "label": "action_executed",
        "text": "He fades from the midpost, releasing a high arcing jumper over the contest."
      },
      {
        "new_transcript_id": "132_2",
        "label": "action_not_executed",
        "text": "He squares for a fadeaway, then resets to the top without taking the shot."
      }
    ]
  },
  {
    "transcript_id": 133,
    "text": "Thomas Bank shot in it.",
    "action": "bank shot",
    "augmentations": [
      {
        "new_transcript_id": "133_1",
        "label": "action_executed",
        "text": "Thomas angles it off glass from midrange, a textbook bank shot."
      },
      {
        "new_transcript_id": "133_2",
        "label": "action_not_executed",
        "text": "Thomas feints the window shot, then pitches it out to the trailer instead."
      }
    ]
  },
  {
    "transcript_id": 134,
    "text": "Thomas Bank shot in it.",
    "action": "bank shot",
    "augmentations": [
      {
        "new_transcript_id": "134_1",
        "label": "action_executed",
        "text": "Off the elbow, Thomas kisses it off the glass for a clean bank."
      },
      {
        "new_transcript_id": "134_2",
        "label": "action_not_executed",
        "text": "Thomas gathers for a bank attempt, then kicks to a cutter before releasing."
      }
    ]
  },
  {
    "transcript_id": 135,
    "text": "Because of Josh Richardson's injury, he's already made a game winner this year for Christmas, and The Teardrop by Brogdon missed in.",
    "action": "teardrop",
    "augmentations": [
      {
        "new_transcript_id": "135_1",
        "label": "action_executed",
        "text": "Brogdon floats a teardrop in the lane; it hangs and rolls off the rim."
      },
      {
        "new_transcript_id": "135_2",
        "label": "action_not_executed",
        "text": "Brogdon threatens a teardrop but shovels a pass to the dunker spot instead."
      }
    ]
  },
  {
    "transcript_id": 136,
    "text": "Who would you like to Posterize in this League?",
    "action": "poster",
    "augmentations": [
      {
        "new_transcript_id": "136_1",
        "label": "action_not_executed",
        "text": "They discuss hypothetical poster dunks; no attempt happens on this possession."
      },
      {
        "new_transcript_id": "136_2",
        "label": "action_executed",
        "text": "He rises over the rotating center and throws down a vicious poster jam."
      }
    ]
  },
  {
    "transcript_id": 137,
    "text": "Murray, the floater, that Mrs Jokic tips it.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "137_1",
        "label": "action_executed",
        "text": "Murray lofts a floater that rims out before Jokic taps at the rebound."
      },
      {
        "new_transcript_id": "137_2",
        "label": "action_not_executed",
        "text": "Murray probes for a floater but dishes to Jokic on the short roll instead."
      }
    ]
  },
  {
    "transcript_id": 138,
    "text": "pick and roll with the big man.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "138_1",
        "label": "action_executed",
        "text": "They run pick and roll; the big dives hard and the pocket pass hits perfectly."
      },
      {
        "new_transcript_id": "138_2",
        "label": "action_not_executed",
        "text": "They motion for pick and roll, but the handler rejects and resets to the top."
      }
    ]
  },
  {
    "transcript_id": 139,
    "text": "He'll take it to the rack.",
    "action": "take it to the rack",
    "augmentations": [
      {
        "new_transcript_id": "139_1",
        "label": "action_executed",
        "text": "He explodes to the rack, absorbs contact, and flips it in off the window."
      },
      {
        "new_transcript_id": "139_2",
        "label": "action_not_executed",
        "text": "He starts downhill toward the rack, then pulls out and resets the offense."
      }
    ]
  },
  {
    "transcript_id": 140,
    "text": "Richardson, on his hips head, gets the role on the floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "140_1",
        "label": "action_executed",
        "text": "Richardson leans in and kisses a floater that dances in on the rim."
      },
      {
        "new_transcript_id": "140_2",
        "label": "action_not_executed",
        "text": "Richardson shapes for a floater but whips a skip pass to the opposite wing."
      }
    ]
  },
  {
    "transcript_id": 141,
    "text": "Double team their shot Clock down to two.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "141_1",
        "label": "action_executed",
        "text": "With the clock dwindling, two defenders trap the ball and force a rushed heave."
      },
      {
        "new_transcript_id": "141_2",
        "label": "action_not_executed",
        "text": "They bluff a trap late, then peel away; no true double forms before the buzzer."
      }
    ]
  },
  {
    "transcript_id": 142,
    "text": "So tough on the block through double team.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "142_1",
        "label": "action_executed",
        "text": "He powers through a firm double on the left block and still gets a look."
      },
      {
        "new_transcript_id": "142_2",
        "label": "action_not_executed",
        "text": "Help starts toward a double team, but retreats before it ever clamps down."
      }
    ]
  },
  {
    "transcript_id": 143,
    "text": "Then again, tips in his own mess.",
    "action": "tip in",
    "augmentations": [
      {
        "new_transcript_id": "143_1",
        "label": "action_executed",
        "text": "He follows the miss and flicks a quick tip-in over the crowd."
      },
      {
        "new_transcript_id": "143_2",
        "label": "action_not_executed",
        "text": "He jumps for the putback but is boxed out, never getting a tip on it."
      }
    ]
  },
  {
    "transcript_id": 144,
    "text": "Carlin has been more of the point guard, the penetration, the floater off the rim.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "144_1",
        "label": "action_executed",
        "text": "He drives middle and lofts a floater that lips out at the last moment."
      },
      {
        "new_transcript_id": "144_2",
        "label": "action_not_executed",
        "text": "He penetrates, shows the floater, then shovels to a cutter under the rim."
      }
    ]
  },
  {
    "transcript_id": 145,
    "text": "The teardrop no.",
    "action": "teardrop",
    "augmentations": [
      {
        "new_transcript_id": "145_1",
        "label": "action_executed",
        "text": "He floats a teardrop from eight feet, and it skips off the back iron."
      },
      {
        "new_transcript_id": "145_2",
        "label": "action_not_executed",
        "text": "He approaches for a teardrop, but dumps it off rather than releasing."
      }
    ]
  },
  {
    "transcript_id": 146,
    "text": "Zip surpassed the young finds a little people around the Horn Turner Rainbow shot the mark.",
    "action": "rainbow shot",
    "augmentations": [
      {
        "new_transcript_id": "146_1",
        "label": "action_executed",
        "text": "Turner arcs a high rainbow from the wing, a moonshot that draws the net."
      },
      {
        "new_transcript_id": "146_2",
        "label": "action_not_executed",
        "text": "He fakes the high-arcing jumper and swings it around the horn instead."
      }
    ]
  },
  {
    "transcript_id": 147,
    "text": "Nice past Cauley Stein attacking little finger roll put in the book and send them to the line.",
    "action": "finger roll",
    "augmentations": [
      {
        "new_transcript_id": "147_1",
        "label": "action_executed",
        "text": "Cauley-Stein glides in and finishes a delicate finger roll through contact."
      },
      {
        "new_transcript_id": "147_2",
        "label": "action_not_executed",
        "text": "He gathers for a finger roll but is fouled before the release, side-out."
      }
    ]
  },
  {
    "transcript_id": 148,
    "text": "Orlando goes to the pick and roll game with bolts driving it to the basket.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "148_1",
        "label": "action_executed",
        "text": "They run pick and roll; Fultz turns downhill and finishes strong at the rim."
      },
      {
        "new_transcript_id": "148_2",
        "label": "action_not_executed",
        "text": "They start pick and roll action, but a deflection kills the play before it develops."
      }
    ]
  },
  {
    "transcript_id": 149,
    "text": "Booker nearly had the tip in, and then he collides down low.",
    "action": "tip in",
    "augmentations": [
      {
        "new_transcript_id": "149_1",
        "label": "action_executed",
        "text": "Booker rises above the scrum and taps in the miss with quick hands."
      },
      {
        "new_transcript_id": "149_2",
        "label": "action_not_executed",
        "text": "Booker reaches for a tip-in but gets bumped, never making contact with the ball."
      }
    ]
  },
  {
    "transcript_id": 150,
    "text": "Very good at the pick and roll, and very bouncy.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "150_1",
        "label": "action_executed",
        "text": "They run pick and roll; the lob target springs free and finishes above the rim."
      },
      {
        "new_transcript_id": "150_2",
        "label": "action_not_executed",
        "text": "They praise his pick and roll game, but this trip never initiates the action."
      }
    ]
  },
  {
    "transcript_id": 151,
    "text": "Thomas Jefferson will go coast to coast.",
    "action": "coast to coast",
    "augmentations": [
      {
        "new_transcript_id": "151_1",
        "label": "action_executed",
        "text": "Jefferson grabs the board and bolts coast to coast for a layup in traffic."
      },
      {
        "new_transcript_id": "151_2",
        "label": "action_not_executed",
        "text": "He starts the break but pulls up near half court, abandoning coast to coast."
      }
    ]
  },
  {
    "transcript_id": 152,
    "text": "You wanna pick and roll?",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "152_1",
        "label": "action_executed",
        "text": "They agree, set the screen, and flow into a clean pick and roll possession."
      },
      {
        "new_transcript_id": "152_2",
        "label": "action_not_executed",
        "text": "They discuss running pick and roll, then call it off and swing the ball."
      }
    ]
  },
  {
    "transcript_id": 153,
    "text": "2 free throws for Josh Jackson and we are tide at four is frameworks that pick and roll this time on the side of steel by Jackson.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "153_1",
        "label": "action_executed",
        "text": "They run side pick and roll; Jackson jumps the lane and turns it into points."
      },
      {
        "new_transcript_id": "153_2",
        "label": "action_not_executed",
        "text": "They signal side pick and roll, but a miscue leads to a turnover before the screen."
      }
    ]
  },
  {
    "transcript_id": 154,
    "text": "Double teaming side by the Celtics.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "154_1",
        "label": "action_executed",
        "text": "Boston sends the second defender quickly, forming a strong sideline double team."
      },
      {
        "new_transcript_id": "154_2",
        "label": "action_not_executed",
        "text": "They shade with help toward the sideline but never commit to a full double."
      }
    ]
  },
  {
    "transcript_id": 155,
    "text": "Starting down the Lane, little floater tipped up and out.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "155_1",
        "label": "action_executed",
        "text": "He teardrops a floater in the lane; it’s nudged off the rim on a tip."
      },
      {
        "new_transcript_id": "155_2",
        "label": "action_not_executed",
        "text": "He starts to float it, then shovels a pass to the dunker spot instead."
      }
    ]
  },
  {
    "transcript_id": 156,
    "text": "Leslie Matthews inside gets the floater to fall.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "156_1",
        "label": "action_executed",
        "text": "Matthews drives down the lane, lofts the floater softly, and it drops straight through the net."
      },
      {
        "new_transcript_id": "156_2",
        "label": "action_not_executed",
        "text": "Matthews fakes a floater in the paint, but kicks it out instead without attempting the shot."
      }
    ]
  },
  {
    "transcript_id": 157,
    "text": "He had a double team.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "157_1",
        "label": "action_executed",
        "text": "Two defenders collapse quickly, forming a tough double team and forcing him to pass out of trouble."
      },
      {
        "new_transcript_id": "157_2",
        "label": "action_not_executed",
        "text": "They considered sending a double team, but stayed home on shooters and never pressured him."
      }
    ]
  },
  {
    "transcript_id": 158,
    "text": "His forms giving chase gotta contest those jumpers as he'll knocks down the fadeaway jumper.",
    "action": "fadeaway",
    "augmentations": [
      {
        "new_transcript_id": "158_1",
        "label": "action_executed",
        "text": "Defender crowding, he spins baseline and drains the fadeaway jumper over the outstretched arm."
      },
      {
        "new_transcript_id": "158_2",
        "label": "action_not_executed",
        "text": "He looked ready to take a fadeaway, but pulled it back down and swung the ball around instead."
      }
    ]
  },
  {
    "transcript_id": 159,
    "text": "MB pump fakes and drives in on land.",
    "action": "pump fake",
    "augmentations": [
      {
        "new_transcript_id": "159_1",
        "label": "action_executed",
        "text": "MB sells the pump fake at the arc, gets his defender in the air, then attacks the rim aggressively."
      },
      {
        "new_transcript_id": "159_2",
        "label": "action_not_executed",
        "text": "MB thought about a pump fake but kept the ball low, choosing to reset the offense instead."
      }
    ]
  },
  {
    "transcript_id": 160,
    "text": "Little leaner floater that is straight in future.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "160_1",
        "label": "action_executed",
        "text": "He leans into the paint, lofts a floater high off the glass, and it falls cleanly through."
      },
      {
        "new_transcript_id": "160_2",
        "label": "action_not_executed",
        "text": "He starts the motion for a floater but pulls back, looking for a teammate on the perimeter instead."
      }
    ]
  },
  {
    "transcript_id": 166,
    "text": "You'll see him post up.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "166_1",
        "label": "action_not_executed",
        "text": "They set up on the wing but he never actually tries to post up on this trip."
      },
      {
        "new_transcript_id": "166_2",
        "label": "action_executed",
        "text": "He seals his man deep in the paint and goes into a strong post up under the rim."
      }
    ]
  },
  {
    "transcript_id": 167,
    "text": "They want to put him in the pick and roll.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "167_1",
        "label": "action_executed",
        "text": "They bring the screener high above the arc and run the pick and roll to perfection."
      },
      {
        "new_transcript_id": "167_2",
        "label": "action_not_executed",
        "text": "They signal for a pick and roll set but quickly swing the ball away without running it."
      }
    ]
  },
  {
    "transcript_id": 168,
    "text": "And here's the Dallas native Myles Turner for one of the things Miles worked on was his post up game.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "168_1",
        "label": "action_executed",
        "text": "Turner gets the ball on the block and executes a solid post up move toward the middle."
      },
      {
        "new_transcript_id": "168_2",
        "label": "action_not_executed",
        "text": "They mention Turner practicing his post up skills, but no actual post up occurs on this play."
      }
    ]
  },
  {
    "transcript_id": 169,
    "text": "Middleton, great job swish.",
    "action": "swish",
    "augmentations": [
      {
        "new_transcript_id": "169_1",
        "label": "action_executed",
        "text": "Middleton pulls up from the elbow and drains it cleanly, nothing but net, pure swish."
      },
      {
        "new_transcript_id": "169_2",
        "label": "action_not_executed",
        "text": "Middleton lines it up but the shot rattles off the rim, no swish this time around."
      }
    ]
  },
  {
    "transcript_id": 170,
    "text": "Conley's teardrop.",
    "action": "teardrop",
    "augmentations": [
      {
        "new_transcript_id": "170_1",
        "label": "action_executed",
        "text": "Conley dribbles into the lane and floats up a teardrop that drops softly through the net."
      },
      {
        "new_transcript_id": "170_2",
        "label": "action_not_executed",
        "text": "Conley looks for the teardrop but pulls it back and swings it out to the perimeter instead."
      }
    ]
  },
  {
    "transcript_id": 171,
    "text": "He couldn't find the hell down along the alley Oop.",
    "action": "alley oop",
    "augmentations": [
      {
        "new_transcript_id": "171_1",
        "label": "action_executed",
        "text": "He rises up in transition and finishes the alley oop with a thunderous slam to ignite the crowd."
      },
      {
        "new_transcript_id": "171_2",
        "label": "action_not_executed",
        "text": "They attempt the alley oop lob, but the pass sails too high and no connection is made."
      }
    ]
  },
  {
    "transcript_id": 172,
    "text": "He couldn't find the hell down along the alley Oop.",
    "action": "alley oop",
    "augmentations": [
      {
        "new_transcript_id": "172_1",
        "label": "action_executed",
        "text": "The lob goes up and he times it perfectly, finishing the alley oop with authority above the rim."
      },
      {
        "new_transcript_id": "172_2",
        "label": "action_not_executed",
        "text": "They signal for the alley oop but the defense clogs the lane and the pass never goes up."
      }
    ]
  },
  {
    "transcript_id": 173,
    "text": "He couldn't find the hell down along the alley Oop.",
    "action": "alley oop",
    "augmentations": [
      {
        "new_transcript_id": "173_1",
        "label": "action_executed",
        "text": "He streaks baseline, catches the lob midair, and hammers home the alley oop dunk emphatically."
      },
      {
        "new_transcript_id": "173_2",
        "label": "action_not_executed",
        "text": "They tried to set up an alley oop, but the timing was off and the ball ricocheted away."
      }
    ]
  },
  {
    "transcript_id": 174,
    "text": "Knox comes to help out this week and doing a lot of dropping and double teaming.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "174_1",
        "label": "action_executed",
        "text": "Knox rushes over from the weak side, joining his teammate in a tight double team trap near the sideline."
      },
      {
        "new_transcript_id": "174_2",
        "label": "action_not_executed",
        "text": "Knox looks to double team but stays home on his assignment, never committing to the trap."
      }
    ]
  },
  {
    "transcript_id": 177,
    "text": "Bogdanovich puts it down on the floor with his left hand, goes to his right and can't finish with the finger roll.",
    "action": "finger roll",
    "augmentations": [
      {
        "new_transcript_id": "177_1",
        "label": "action_executed",
        "text": "Bogdanovich drives into traffic and lofts up the finger roll, the ball kissing the glass before falling off."
      },
      {
        "new_transcript_id": "177_2",
        "label": "action_not_executed",
        "text": "Bogdanovich begins to drive but kicks out to the corner instead, never attempting the finger roll."
      }
    ]
  },
  {
    "transcript_id": 178,
    "text": "Driving with the finger roll the rim coming All Star looking moving.",
    "action": "finger roll",
    "augmentations": [
      {
        "new_transcript_id": "178_1",
        "label": "action_executed",
        "text": "He weaves through defenders and lays in the finger roll softly off the front of the rim."
      },
      {
        "new_transcript_id": "178_2",
        "label": "action_not_executed",
        "text": "He looked set to finish with a finger roll, but pulled back and reset the offense instead."
      }
    ]
  },
  {
    "transcript_id": 179,
    "text": "Wait for the double team.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "179_1",
        "label": "action_not_executed",
        "text": "He pauses in the corner signaling to wait for the double team, but the second defender never comes."
      },
      {
        "new_transcript_id": "179_2",
        "label": "action_executed",
        "text": "Two defenders close in aggressively, forming a strong double team and forcing the ball out of his hands."
      }
    ]
  },
  {
    "transcript_id": 182,
    "text": "Fat shoot nothing but net for Casey gotta find more shots for Casey's meeting.",
    "action": "nothing but net",
    "augmentations": [
      {
        "new_transcript_id": "182_1",
        "label": "action_executed",
        "text": "Casey rises from deep and nails the jumper clean, straight through with nothing but net."
      },
      {
        "new_transcript_id": "182_2",
        "label": "action_not_executed",
        "text": "Casey spots up but the jumper clanks off the back iron, no nothing but net this possession."
      }
    ]
  },
  {
    "transcript_id": 183,
    "text": "He had never had a major injury in his life and he was out for the entire season, as garlands floater doesn't go.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "183_1",
        "label": "action_executed",
        "text": "Garland drifts into the paint, flips up a floater high over the defender, but it rolls off the rim."
      },
      {
        "new_transcript_id": "183_2",
        "label": "action_not_executed",
        "text": "They mention Garland’s floater in conversation, but on this play he never attempts it at the rim."
      }
    ]
  },
  {
    "transcript_id": 184,
    "text": "He had never had a major injury in his life and he was out for the entire season, as garlands floater doesn't go.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "184_1",
        "label": "action_executed",
        "text": "Garland puts up a high arcing floater in the lane that bounces twice before rattling out."
      },
      {
        "new_transcript_id": "184_2",
        "label": "action_not_executed",
        "text": "The floater is discussed during commentary, but he never goes into that move during the possession."
      }
    ]
  },
  {
    "transcript_id": 185,
    "text": "Two of the best alley oop growers in the League.",
    "action": "alley oop",
    "augmentations": [
      {
        "new_transcript_id": "185_1",
        "label": "action_not_executed",
        "text": "The broadcast highlights their alley oop chemistry, but no lob attempt takes place on this trip."
      },
      {
        "new_transcript_id": "185_2",
        "label": "action_executed",
        "text": "He cuts baseline, receives the lob and slams the alley oop emphatically over the rotating defender."
      }
    ]
  },
  {
    "transcript_id": 186,
    "text": "Try the switching pick and roll.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "186_1",
        "label": "action_executed",
        "text": "They run the pick and roll at the top of the key and punish the switch with a quick jumper."
      },
      {
        "new_transcript_id": "186_2",
        "label": "action_not_executed",
        "text": "They signal for a pick and roll, but the ballhandler waves it off and resets the play outside."
      }
    ]
  },
  {
    "transcript_id": 187,
    "text": "Pick and roll played very nicely.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "187_1",
        "label": "action_not_executed",
        "text": "Commentary praises the pick and roll set in theory, but they never actually execute it on this possession."
      },
      {
        "new_transcript_id": "187_2",
        "label": "action_executed",
        "text": "They run the high pick and roll cleanly, the ballhandler using the screen to create space for a jumper."
      }
    ]
  },
  {
    "transcript_id": 188,
    "text": "Running floater is good for Serge Ibaka.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "188_1",
        "label": "action_executed",
        "text": "Ibaka glides into the lane and lofts up a smooth running floater that drops perfectly through."
      },
      {
        "new_transcript_id": "188_2",
        "label": "action_not_executed",
        "text": "Ibaka looked ready for a floater but kicks it out instead, so no floater attempt is made."
      }
    ]
  },
  {
    "transcript_id": 189,
    "text": "Tastic pump fakes slash and move without it he put on a show indeed he did and we are underway.",
    "action": "pump fake",
    "augmentations": [
      {
        "new_transcript_id": "189_1",
        "label": "action_not_executed",
        "text": "He gestures a pump fake but never actually sells it, instead swinging the ball away quickly."
      },
      {
        "new_transcript_id": "189_2",
        "label": "action_executed",
        "text": "He draws the defender off balance with a convincing pump fake, then drives strong to the rim."
      }
    ]
  },
  {
    "transcript_id": 190,
    "text": "Nice floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "190_1",
        "label": "action_not_executed",
        "text": "The announcer calls out a nice floater, but the ball never leaves his hands this time."
      },
      {
        "new_transcript_id": "190_2",
        "label": "action_executed",
        "text": "He tosses up a soft floater in the paint, dropping cleanly over the outstretched arms of the defender."
      }
    ]
  },
  {
    "transcript_id": 191,
    "text": "Nice floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "191_1",
        "label": "action_not_executed",
        "text": "The crowd anticipates a floater, but instead he dishes to the corner without shooting."
      },
      {
        "new_transcript_id": "191_2",
        "label": "action_executed",
        "text": "He glides to the paint and releases a beautiful floater that splashes softly through the net."
      }
    ]
  },
 {
    "transcript_id": 192,
    "text": "Miles with a pump fake back to Van Fleet.",
    "action": "pump fake",
    "augmentations": [
      {
        "new_transcript_id": "192_1",
        "label": "action_executed",
        "text": "Miles fakes the shot with a sharp pump fake, freezing the defender before kicking it back to VanVleet."
      },
      {
        "new_transcript_id": "192_2",
        "label": "action_not_executed",
        "text": "Miles hesitates but never commits to a pump fake, simply swinging it to VanVleet on the wing."
      }
    ]
  },
  {
    "transcript_id": 193,
    "text": "The 4th is Julia wanted to post up ******.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "193_1",
        "label": "action_not_executed",
        "text": "Julia was calling for a post up opportunity, but the ball never entered the paint on that possession."
      },
      {
        "new_transcript_id": "193_2",
        "label": "action_executed",
        "text": "She establishes deep position and seals her defender, executing a strong post up under the rim."
      }
    ]
  },
  {
    "transcript_id": 194,
    "text": "Kevin at RIM run thereby Davis forced the mismatching great recognition Rim run to a post up and he asked you shot jump out of DVD.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "194_1",
        "label": "action_not_executed",
        "text": "They tried to find Davis in a post up mismatch, but the pass never connected and the play reset."
      },
      {
        "new_transcript_id": "194_2",
        "label": "action_executed",
        "text": "Davis catches deep in the block, turning quickly for a smooth post up score inside."
      }
    ]
  },
  {
    "transcript_id": 195,
    "text": "Here's the floater again by Brockton.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "195_1",
        "label": "action_executed",
        "text": "Brogdon curls into the lane and lifts up a soft floater that bounces in off the front rim."
      },
      {
        "new_transcript_id": "195_2",
        "label": "action_not_executed",
        "text": "They mention Brogdon’s floater, but on this trip he never attempts it and swings it outside."
      }
    ]
  },
  {
    "transcript_id": 196,
    "text": "Current is a double team.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "196_1",
        "label": "action_executed",
        "text": "The defense collapses with two players, executing a strong double team at the top of the key."
      },
      {
        "new_transcript_id": "196_2",
        "label": "action_not_executed",
        "text": "They talk about a double team forming, but the second defender stays home on his matchup."
      }
    ]
  },
  {
    "transcript_id": 197,
    "text": "Here is a Baka using the jab steps just going on.",
    "action": "jab step",
    "augmentations": [
      {
        "new_transcript_id": "197_1",
        "label": "action_executed",
        "text": "Ibaka plants his foot and repeatedly uses jab steps to keep the defender guessing before rising for a jumper."
      },
      {
        "new_transcript_id": "197_2",
        "label": "action_not_executed",
        "text": "Ibaka looks poised for a jab step but quickly swings the ball to the perimeter without using it."
      }
    ]
  },
  {
    "transcript_id": 198,
    "text": "Outlet pass broken up and then Capella.",
    "action": "outlet pass",
    "augmentations": [
      {
        "new_transcript_id": "198_1",
        "label": "action_executed",
        "text": "He secures the rebound and fires a long outlet pass ahead to Capela streaking down the floor."
      },
      {
        "new_transcript_id": "198_2",
        "label": "action_not_executed",
        "text": "He looked ready for the outlet pass but lost control and never delivered the long throw ahead."
      }
    ]
  },
  {
    "transcript_id": 199,
    "text": "Split a double team gets his own miss.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "199_1",
        "label": "action_executed",
        "text": "He weaves between two defenders, splitting the double team before grabbing his rebound inside."
      },
      {
        "new_transcript_id": "199_2",
        "label": "action_not_executed",
        "text": "The defenders close as if to double team, but he never actually faces both at once before missing."
      }
    ]
  },
  {
    "transcript_id": 200,
    "text": "After he gets there you can go to the floater Luka Doncic's can do the same thing.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "200_1",
        "label": "action_not_executed",
        "text": "The announcer notes Doncic could use a floater, but he doesn’t attempt one on this sequence."
      },
      {
        "new_transcript_id": "200_2",
        "label": "action_executed",
        "text": "Doncic drives into the paint and lofts up a floater high over the contest that drops through."
      }
    ]
  },
  {
    "transcript_id": 201,
    "text": "Allison goes coast to coast with the left hand layup off the glass.",
    "action": "coast to coast",
    "augmentations": [
      {
        "new_transcript_id": "201_1",
        "label": "action_executed",
        "text": "Allison takes the rebound and races coast to coast, finishing with a smooth lefty layup off glass."
      },
      {
        "new_transcript_id": "201_2",
        "label": "action_not_executed",
        "text": "Allison pushes upcourt but stops short of coast to coast, slowing it down into a half-court set."
      }
    ]
  },
  {
    "transcript_id": 202,
    "text": "What makes the coast to coast?",
    "action": "coast to coast",
    "augmentations": [
      {
        "new_transcript_id": "202_1",
        "label": "action_executed",
        "text": "He grabs the board and sprints untouched, finishing the coast to coast with a right-hand layup."
      },
      {
        "new_transcript_id": "202_2",
        "label": "action_not_executed",
        "text": "The commentary mentions coast to coast runs, but no one actually goes end to end on this play."
      }
    ]
  },
  {
    "transcript_id": 203,
    "text": "Chocolate down to two hard double team got found.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "203_1",
        "label": "action_executed",
        "text": "With the clock winding, two defenders swarm aggressively, forcing a tough pass out of the double team."
      },
      {
        "new_transcript_id": "203_2",
        "label": "action_not_executed",
        "text": "They looked set to double team near the sideline, but instead stayed home on their individual matchups."
      }
    ]
  },
  {
    "transcript_id": 204,
    "text": "Pick and roll with Capella.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "204_1",
        "label": "action_executed",
        "text": "Capela sets a strong screen at the top, the guard uses it perfectly to run the pick and roll."
      },
      {
        "new_transcript_id": "204_2",
        "label": "action_not_executed",
        "text": "Capela starts to move into screen position, but they reset before the pick and roll develops."
      }
    ]
  },
  {
    "transcript_id": 205,
    "text": "Harden backs off Simmons Simmons with a floater no good.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "205_1",
        "label": "action_executed",
        "text": "Simmons attacks the lane and tosses up a floater that bounces off the rim and misses."
      },
      {
        "new_transcript_id": "205_2",
        "label": "action_not_executed",
        "text": "Simmons fakes like he’ll try a floater, but instead kicks it to the corner shooter without attempting it."
      }
    ]
  },
  {
    "transcript_id": 206,
    "text": "Missing the free throw line floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "206_1",
        "label": "action_executed",
        "text": "He launches a floater from the free throw line that rims out and misses badly."
      },
      {
        "new_transcript_id": "206_2",
        "label": "action_not_executed",
        "text": "The play was drawn up for a floater, but he never attempts it, passing to the wing instead."
      }
    ]
  },
  {
    "transcript_id": 207,
    "text": "Missing the free throw line floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "207_1",
        "label": "action_executed",
        "text": "He steps into the lane, flicks up a floater from the line, and it bounces out no good."
      },
      {
        "new_transcript_id": "207_2",
        "label": "action_not_executed",
        "text": "He appears to set for a floater, but resets and dishes outside without putting it up."
      }
    ]
  },
  {
    "transcript_id": 208,
    "text": "Point shot is going to be looking to get into the paint post up.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "208_1",
        "label": "action_not_executed",
        "text": "The commentary mentions a post up option, but the ball never gets inside for him to establish position."
      },
      {
        "new_transcript_id": "208_2",
        "label": "action_executed",
        "text": "He muscles his way into the paint, sealing his man for a solid post up play on the block."
      }
    ]
  },
  {
    "transcript_id": 209,
    "text": "He can stretch you out, but also he is one of the best and being active around the glass and in the pick and roll.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "209_1",
        "label": "action_not_executed",
        "text": "They describe his pick and roll ability, but he never actually runs it during this possession."
      },
      {
        "new_transcript_id": "209_2",
        "label": "action_executed",
        "text": "He sets the screen cleanly and rolls hard, completing the pick and roll action inside."
      }
    ]
  },
  {
    "transcript_id": 210,
    "text": "Davis gets into it quickly on a fadeaway rings out regarded by Aldridge.",
    "action": "fadeaway",
    "augmentations": [
      {
        "new_transcript_id": "210_1",
        "label": "action_executed",
        "text": "Anthony Davis rises at the elbow, hits a tough fadeaway over Aldridge’s outstretched arm."
      },
      {
        "new_transcript_id": "210_2",
        "label": "action_not_executed",
        "text": "Davis looked ready for the fadeaway, but he passed inside before releasing the shot."
      }
    ]
  },
  {
    "transcript_id": 211,
    "text": "The floater box Ray rebounded by zoo Patch.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "211_1",
        "label": "action_executed",
        "text": "Ray slips into the lane and lofts a floater, soft touch off the glass before Zubac boards it."
      },
      {
        "new_transcript_id": "211_2",
        "label": "action_not_executed",
        "text": "Ray faked the floater midair and shoveled a quick dish to Zubac instead."
      }
    ]
  },
  {
    "transcript_id": 212,
    "text": "Denver on the push and Millsap trying to go coast to coast miss with the left.",
    "action": "coast to coast",
    "augmentations": [
      {
        "new_transcript_id": "212_1",
        "label": "action_executed",
        "text": "Millsap takes the rebound end to end, coast to coast drive finishing with the left hand miss."
      },
      {
        "new_transcript_id": "212_2",
        "label": "action_not_executed",
        "text": "Millsap looked like he would go coast to coast but pulled up and reset at the arc."
      }
    ]
  },
  {
    "transcript_id": 213,
    "text": "Coast to coast Simmons lazatin.",
    "action": "coast to coast",
    "augmentations": [
      {
        "new_transcript_id": "213_1",
        "label": "action_executed",
        "text": "Ben Simmons grabbed the board and went coast to coast, laying it in before the defense collapsed."
      },
      {
        "new_transcript_id": "213_2",
        "label": "action_not_executed",
        "text": "Simmons started a coast to coast push but lost control at midcourt, forced to pass it away."
      }
    ]
  },
  {
    "transcript_id": 214,
    "text": "On the NBA campus, tried to give and go.",
    "action": "give and go",
    "augmentations": [
      {
        "new_transcript_id": "214_1",
        "label": "action_executed",
        "text": "They executed a textbook give and go, cutting backdoor for the finish inside the restricted area."
      },
      {
        "new_transcript_id": "214_2",
        "label": "action_not_executed",
        "text": "He looked to start the give and go but the defender cut off the lane, breaking the sequence."
      }
    ]
  },
  {
    "transcript_id": 215,
    "text": "Wait posting up Hayward.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "215_1",
        "label": "action_executed",
        "text": "Waiters backs down Hayward on the left block, working into a strong post up move."
      },
      {
        "new_transcript_id": "215_2",
        "label": "action_not_executed",
        "text": "He set up like he would post up Hayward, but immediately kicked the ball back out top."
      }
    ]
  },
  {
    "transcript_id": 216,
    "text": "Thought he was fouled and one day missing on the floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "216_1",
        "label": "action_executed",
        "text": "He drove the lane, tossed up a floater through contact, but it rimmed off no whistle."
      },
      {
        "new_transcript_id": "216_2",
        "label": "action_not_executed",
        "text": "He looked ready to float it up but lost balance, never releasing the floater at the rim."
      }
    ]
  },
  {
    "transcript_id": 217,
    "text": "Inside, Alan spins into a double team.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "217_1",
        "label": "action_executed",
        "text": "Allen spins baseline and is immediately smothered by a collapsing double team under the rim."
      },
      {
        "new_transcript_id": "217_2",
        "label": "action_not_executed",
        "text": "Allen looked like he’d spin into a double, but only one defender stayed with him in the paint."
      }
    ]
  },
  {
    "transcript_id": 218,
    "text": "DeMarcus Cousins now Jerrod Jackson table layout to start the game and there's the pick and roll I'm talking about.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "218_1",
        "label": "action_executed",
        "text": "Cousins set the screen, Jackson came off strong in the pick and roll to open the game."
      },
      {
        "new_transcript_id": "218_2",
        "label": "action_not_executed",
        "text": "They signaled a pick and roll but Cousins slipped out early and the action never developed."
      }
    ]
  },
  {
    "transcript_id": 219,
    "text": "Dillard's passes deflected ends up in Aminu's hands anyway, out of the double team Feeding Harkless Harkless with five to Shoot Boy active hands by the Grizzlies.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "219_1",
        "label": "action_executed",
        "text": "Aminu escaped the collapsing double team, dishing to Harkless before the shot clock expired."
      },
      {
        "new_transcript_id": "219_2",
        "label": "action_not_executed",
        "text": "The Grizzlies threatened a double team but stayed home, forcing Aminu to pass under pressure."
      }
    ]
  },
  {
    "transcript_id": 220,
    "text": "Very little floater and who found him.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "220_1",
        "label": "action_executed",
        "text": "He tossed a soft floater from the lane, nearly dropping before the defense swarmed the rebound."
      },
      {
        "new_transcript_id": "220_2",
        "label": "action_not_executed",
        "text": "He faked the floater, then slipped a pass to the cutter instead of finishing himself."
      }
    ]
  },
  {
    "transcript_id": 221,
    "text": "a lot of that is Cameron Payne with a beautiful floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "221_1",
        "label": "action_executed",
        "text": "Cameron Payne drove down the lane, floated it up beautifully for two points."
      },
      {
        "new_transcript_id": "221_2",
        "label": "action_not_executed",
        "text": "Payne appeared to float it but dished last second, never taking the floater shot."
      }
    ]
  },
  {
    "transcript_id": 222,
    "text": "Now you gotta wear this out if they're not going to double team, they're just milking Kevin Love.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "222_1",
        "label": "action_not_executed",
        "text": "The defense stayed home, no double team came, leaving Kevin Love to work single coverage."
      },
      {
        "new_transcript_id": "222_2",
        "label": "action_executed",
        "text": "Love caught it deep and immediately drew a quick double team, forcing him to kick it out."
      }
    ]
  },
  {
    "transcript_id": 223,
    "text": "But the basically as we see Tobias with the little post up.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "223_1",
        "label": "action_executed",
        "text": "Tobias Harris backed his man down with a controlled post up on the left block."
      },
      {
        "new_transcript_id": "223_2",
        "label": "action_not_executed",
        "text": "Harris looked to post up briefly but reversed it out to the wing instead of backing in."
      }
    ]
  },
  {
    "transcript_id": 224,
    "text": "Murray keeps it, can't hit the floater, batted out.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "224_1",
        "label": "action_executed",
        "text": "Jamal Murray attacked the paint, lifted the floater, but it clanked off the rim and was tipped out."
      },
      {
        "new_transcript_id": "224_2",
        "label": "action_not_executed",
        "text": "Murray hesitated, started to float it, then pivoted and reset without attempting the shot."
      }
    ]
  },
  {
    "transcript_id": 226,
    "text": "Double team comes from none for to shoot.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "226_1",
        "label": "action_executed",
        "text": "With the clock winding down, the defense sent a quick double team trapping in the corner."
      },
      {
        "new_transcript_id": "226_2",
        "label": "action_not_executed",
        "text": "They looked ready to double, but only one defender pressured as the clock ticked down."
      }
    ]
  },
  {
    "transcript_id": 227,
    "text": "We play on simple and easy stealing love outlet passes for the basket.",
    "action": "outlet pass",
    "augmentations": [
      {
        "new_transcript_id": "227_1",
        "label": "action_executed",
        "text": "Love grabbed the rebound and fired a textbook outlet pass leading to a breakaway bucket."
      },
      {
        "new_transcript_id": "227_2",
        "label": "action_not_executed",
        "text": "Love considered the outlet pass but held on and waited for guards to circle back."
      }
    ]
  },
  {
    "transcript_id": 228,
    "text": "I'll give it another try to Porter the floater falls.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "228_1",
        "label": "action_executed",
        "text": "Michael Porter Jr. tossed up a floater in the paint, soft touch dropping through the net."
      },
      {
        "new_transcript_id": "228_2",
        "label": "action_not_executed",
        "text": "Porter shaped for a floater but slipped it off to a cutting teammate before taking the shot."
      }
    ]
  },
  {
    "transcript_id": 229,
    "text": "Double team aren't Vanvleet goes the other way down to four on the shot Clock on an OB to the bucket.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "229_1",
        "label": "action_executed",
        "text": "Toronto beat the double team pressure, VanVleet swung it over and Anunoby attacked the rim."
      },
      {
        "new_transcript_id": "229_2",
        "label": "action_not_executed",
        "text": "The Raptors expected a double team, but it never came, leaving VanVleet to reset the play."
      }
    ]
  },
  {
    "transcript_id": 230,
    "text": "George went through some of his numbers as Kevin Knox misses on the other end, but chases down the rebound and tipping in Robinson.",
    "action": "tip in",
    "augmentations": [
      {
        "new_transcript_id": "230_1",
        "label": "action_executed",
        "text": "Kevin Knox missed inside but hustled, tipping the rebound in for Robinson’s easy two."
      },
      {
        "new_transcript_id": "230_2",
        "label": "action_not_executed",
        "text": "Knox missed at the rim but Robinson couldn’t get position, so no tip in occurred."
      }
    ]
  },
  {
    "transcript_id": 231,
    "text": "Patrick Beverley still coming back from that calf injury that floater halfway down and out.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "231_1",
        "label": "action_executed",
        "text": "Beverley dribbled inside, floated it over the big man, ball rattled halfway down and popped out."
      },
      {
        "new_transcript_id": "231_2",
        "label": "action_not_executed",
        "text": "Beverley set up like he’d attempt a floater but kicked it out before rising for the shot."
      }
    ]
  },
  {
    "transcript_id": 232,
    "text": "Post up for dropping right away, banging with Alex Len.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "232_1",
        "label": "action_executed",
        "text": "He planted on the low block, posted up Alex Len, muscling in for position near the rim."
      },
      {
        "new_transcript_id": "232_2",
        "label": "action_not_executed",
        "text": "He looked to post up against Len but quickly reset beyond the arc instead."
      }
    ]
  },
  {
    "transcript_id": 233,
    "text": "There just shouldn't happen, and you need not double teaming Jokic on the low block if you're gonna stay home with their shooting.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "233_1",
        "label": "action_not_executed",
        "text": "Denver spaced the floor, and the defense refused to double team Jokic, sticking to shooters."
      },
      {
        "new_transcript_id": "233_2",
        "label": "action_executed",
        "text": "The defenders collapsed hard, sending a double team at Jokic on the low block quickly."
      }
    ]
  },
  {
    "transcript_id": 234,
    "text": "Aldridge gets rid before the double team, but trust the shot.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "234_1",
        "label": "action_executed",
        "text": "Aldridge felt the double team closing and passed away just in time before it trapped him."
      },
      {
        "new_transcript_id": "234_2",
        "label": "action_not_executed",
        "text": "The help defender looked ready to double, but never committed, leaving Aldridge single covered."
      }
    ]
  },
  {
    "transcript_id": 235,
    "text": "Having some trouble in the post up and the loose ball is picked up by the white power pole.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "235_1",
        "label": "action_executed",
        "text": "He battled on the post up, lost the ball, but still worked to regain possession quickly."
      },
      {
        "new_transcript_id": "235_2",
        "label": "action_not_executed",
        "text": "He tried to establish a post up but fumbled before fully backing down into the move."
      }
    ]
  },
  {
    "transcript_id": 236,
    "text": "Double teams going away by Lonzo.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "236_1",
        "label": "action_executed",
        "text": "Lonzo Ball quickly rotated and sent the double team to force the ball out of the post."
      },
      {
        "new_transcript_id": "236_2",
        "label": "action_not_executed",
        "text": "It looked like Lonzo might double team, but he backed away, leaving only single coverage."
      }
    ]
  },
  {
    "transcript_id": 237,
    "text": "Slam dunk champ.",
    "action": "slam dunk",
    "augmentations": [
      {
        "new_transcript_id": "237_1",
        "label": "action_not_executed",
        "text": "Commentary referenced the slam dunk champ, but no dunk was actually performed on this possession."
      },
      {
        "new_transcript_id": "237_2",
        "label": "action_executed",
        "text": "The slam dunk champ rose high in transition and finished with a thunderous dunk at the rim."
      }
    ]
  },
  {
    "transcript_id": 238,
    "text": "Locked out of five temple looking for the pick, pick and roll without fighting fires.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "238_1",
        "label": "action_executed",
        "text": "Temple initiated the pick and roll, coming off the screen and attacking the lane effectively."
      },
      {
        "new_transcript_id": "238_2",
        "label": "action_not_executed",
        "text": "Temple tried to set up a pick and roll but the spacing collapsed, and the action broke down."
      }
    ]
  },
  {
    "transcript_id": 239,
    "text": "The fadeaway for two.",
    "action": "fadeaway",
    "augmentations": [
      {
        "new_transcript_id": "239_1",
        "label": "action_executed",
        "text": "He knocked down a smooth fadeaway jumper from the midrange for two points."
      },
      {
        "new_transcript_id": "239_2",
        "label": "action_not_executed",
        "text": "He squared up as if to attempt a fadeaway but instead passed to the corner shooter."
      }
    ]
  },
  {
    "transcript_id": 240,
    "text": "But get to Adams quickly on that pick and roll.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "240_1",
        "label": "action_executed",
        "text": "Steven Adams rolled to the basket perfectly after the pick and roll, catching the feed inside."
      },
      {
        "new_transcript_id": "240_2",
        "label": "action_not_executed",
        "text": "They attempted to initiate a pick and roll but Adams slipped away early, breaking the sequence."
      }
    ]
  },
  {
    "transcript_id": 241,
    "text": "Oh, the left hand floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "241_1",
        "label": "action_executed",
        "text": "He glided into the lane and finished with a soft left-handed floater over the defender."
      },
      {
        "new_transcript_id": "241_2",
        "label": "action_not_executed",
        "text": "He looked like he would float it left-handed but instead dished it to the trailing big man."
      }
    ]
  },
  {
    "transcript_id": 242,
    "text": "Now those floaters not falling.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "242_1",
        "label": "action_executed",
        "text": "He kept trying the floater in the lane, but once again it rattled off the rim without dropping."
      },
      {
        "new_transcript_id": "242_2",
        "label": "action_not_executed",
        "text": "They expected him to shoot the floater, but he never put it up on this trip down."
      }
    ]
  },
  {
    "transcript_id": 243,
    "text": "Rodney Hood gets inside, Floater, misses ball, knock loose, Hood gets it back.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "243_1",
        "label": "action_executed",
        "text": "Rodney Hood drove inside, released a floater that missed, then scrambled for the rebound himself."
      },
      {
        "new_transcript_id": "243_2",
        "label": "action_not_executed",
        "text": "Hood faked the floater motion, lost the handle midair, and never released the shot cleanly."
      }
    ]
  },
  {
    "transcript_id": 244,
    "text": "Middleton tries to take advantage double team comp.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "244_1",
        "label": "action_executed",
        "text": "Khris Middleton faced a collapsing double team, splitting it and kicking out to the perimeter."
      },
      {
        "new_transcript_id": "244_2",
        "label": "action_not_executed",
        "text": "The defense threatened a double on Middleton but ultimately stayed home, leaving him isolated."
      }
    ]
  },
  {
    "transcript_id": 245,
    "text": "Full head of Steam Marvin Teardrop in and out.",
    "action": "teardrop",
    "augmentations": [
      {
        "new_transcript_id": "245_1",
        "label": "action_executed",
        "text": "Marvin attacked hard, tossed up a teardrop that spun in and out of the rim."
      },
      {
        "new_transcript_id": "245_2",
        "label": "action_not_executed",
        "text": "Marvin looked ready to attempt a teardrop but instead skipped it to the corner shooter."
      }
    ]
  },
  {
    "transcript_id": 246,
    "text": "Tatum with a nice floater over the top Kemba.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "246_1",
        "label": "action_not_executed",
        "text": "The commentary mentioned a floater from Tatum, but he never actually released it over Kemba."
      },
      {
        "new_transcript_id": "246_2",
        "label": "action_executed",
        "text": "Jayson Tatum slipped past the defense and lofted a soft floater right over Kemba Walker."
      }
    ]
  },
  {
    "transcript_id": 247,
    "text": "can't get the floater to fall foul.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "247_1",
        "label": "action_executed",
        "text": "He drove the lane, released a floater that rimmed out, drawing a foul on the attempt."
      },
      {
        "new_transcript_id": "247_2",
        "label": "action_not_executed",
        "text": "He started the floater motion but was fouled before he could actually get the shot off."
      }
    ]
  },
  {
    "transcript_id": 248,
    "text": "Of course in scoring tray with a floater or try again.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "248_1",
        "label": "action_executed",
        "text": "Trae Young darted inside and connected on a quick floater to keep the scoring rolling."
      },
      {
        "new_transcript_id": "248_2",
        "label": "action_not_executed",
        "text": "Trae faked toward a floater but reset the dribble and passed back out instead of shooting."
      }
    ]
  },
  {
    "transcript_id": 249,
    "text": "McKee on the pick and roll and that's how you.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "249_1",
        "label": "action_executed",
        "text": "JaVale McGee executed the pick and roll perfectly, diving to the rim for the easy finish."
      },
      {
        "new_transcript_id": "249_2",
        "label": "action_not_executed",
        "text": "McGee started to set the pick and roll but slipped away, leaving the play unfinished."
      }
    ]
  },
  {
    "transcript_id": 250,
    "text": "The floaters won't go rebound deflected and taken by Peyton.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "250_1",
        "label": "action_executed",
        "text": "The floater attempt bounced off iron and Elfrid Payton chased down the deflected rebound."
      },
      {
        "new_transcript_id": "250_2",
        "label": "action_not_executed",
        "text": "Peyton looked like he’d try the floater but lost the handle and never got it off."
      }
    ]
  },
  {
    "transcript_id": 251,
    "text": "Damien outside double team bounces over to Dirk dumps at Low Brain.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "251_1",
        "label": "action_executed",
        "text": "Damian Lillard was quickly swarmed by a double team near the arc before swinging it to Dirk."
      },
      {
        "new_transcript_id": "251_2",
        "label": "action_not_executed",
        "text": "The defense looked to double Damian outside but stayed spaced, never sending the second defender."
      }
    ]
  },
  {
    "transcript_id": 252,
    "text": "Lillard posting up against Brunson finds a cutter for the weak side and Dirks defense against Yusif.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "252_1",
        "label": "action_executed",
        "text": "Damian Lillard backed down Brunson on the post up before finding the cutter on the weak side."
      },
      {
        "new_transcript_id": "252_2",
        "label": "action_not_executed",
        "text": "Lillard looked to post up but quickly faced up and dumped it out instead of backing in."
      }
    ]
  },
  {
    "transcript_id": 253,
    "text": "We've got flop Cam standing by as well as Kyle Lowry berries at three.",
    "action": "flop",
    "augmentations": [
      {
        "new_transcript_id": "253_1",
        "label": "action_executed",
        "text": "Lowry sold the contact with a flop, drawing a whistle before draining a three pointer."
      },
      {
        "new_transcript_id": "253_2",
        "label": "action_not_executed",
        "text": "They mentioned the flop cam but no actual flop occurred as play continued normally."
      }
    ]
  },
  {
    "transcript_id": 254,
    "text": "Statistically worst defense in the NBA is Isaiah Thomas who misses on the floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "254_1",
        "label": "action_executed",
        "text": "Isaiah Thomas drove the lane, tossed up a floater that rimmed out against the defense."
      },
      {
        "new_transcript_id": "254_2",
        "label": "action_not_executed",
        "text": "Thomas faked toward a floater but dropped it off to a teammate instead of shooting."
      }
    ]
  },
  {
    "transcript_id": 255,
    "text": "The Left handed player at Bettys shut the majority of those floaters with his right hand.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "255_1",
        "label": "action_not_executed",
        "text": "He normally shoots floaters, but this possession he kept it in his right without taking one."
      },
      {
        "new_transcript_id": "255_2",
        "label": "action_executed",
        "text": "Despite being left-handed, he lofted a floater with his right hand over the defense."
      }
    ]
  },
  {
    "transcript_id": 256,
    "text": "He loves that left side of the floor and a fadeaway jumper over the Magic 7 foot center.",
    "action": "fadeaway",
    "augmentations": [
      {
        "new_transcript_id": "256_1",
        "label": "action_executed",
        "text": "From his sweet spot on the left side, he drilled a fadeaway over the seven footer."
      },
      {
        "new_transcript_id": "256_2",
        "label": "action_not_executed",
        "text": "He looked ready to fade away but quickly passed when the center closed out."
      }
    ]
  },
  {
    "transcript_id": 257,
    "text": "Pick and roll every single time as you see KD coming off of Cousins.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "257_1",
        "label": "action_executed",
        "text": "Kevin Durant used Cousins’ screen in the pick and roll, curling into a jumper."
      },
      {
        "new_transcript_id": "257_2",
        "label": "action_not_executed",
        "text": "They motioned for a pick and roll, but Cousins never fully set the screen for Durant."
      }
    ]
  },
  {
    "transcript_id": 258,
    "text": "Fox, the little floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "258_1",
        "label": "action_executed",
        "text": "De’Aaron Fox knifed into the lane and floated it up softly for two."
      },
      {
        "new_transcript_id": "258_2",
        "label": "action_not_executed",
        "text": "Fox drove inside looking for the floater but pulled it back and reset at the perimeter."
      }
    ]
  },
  {
    "transcript_id": 259,
    "text": "What makes the coast to coast?",
    "action": "coast to coast",
    "augmentations": [
      {
        "new_transcript_id": "259_1",
        "label": "action_executed",
        "text": "He went coast to coast, grabbing the rebound and sprinting the length of the floor for a finish."
      },
      {
        "new_transcript_id": "259_2",
        "label": "action_not_executed",
        "text": "He looked to push coast to coast but slowed up and waited for teammates to join."
      }
    ]
  },
  {
    "transcript_id": 260,
    "text": "The double team finds Trent Junior Swanigan swings it around.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "260_1",
        "label": "action_executed",
        "text": "The defense trapped with a quick double team, forcing Trent Jr. to swing it to Swanigan."
      },
      {
        "new_transcript_id": "260_2",
        "label": "action_not_executed",
        "text": "They looked like they might double team Trent Jr., but instead stayed with their own men."
      }
    ]
  },
   {
    "transcript_id": 261,
    "text": "A good sign that Conley makes his first floater Simmons, with a little baby hook shot in the paint.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "261_1",
        "label": "action_not_executed",
        "text": "Conley looked ready but pulled back on the floater, letting Simmons try a hook instead."
      },
      {
        "new_transcript_id": "261_2",
        "label": "action_executed",
        "text": "Conley crosses mid-lane, calmly rises and drops in his first floater over the defense."
      }
    ]
  },
  {
    "transcript_id": 262,
    "text": "Have to take a fadeaway contested shot TJ Warren.",
    "action": "fadeaway",
    "augmentations": [
      {
        "new_transcript_id": "262_1",
        "label": "action_executed",
        "text": "Warren leans back, fading over the contest, and knocks down the tough jumper off the wing."
      },
      {
        "new_transcript_id": "262_2",
        "label": "action_not_executed",
        "text": "Warren fakes a fadeaway but puts the ball on the floor, choosing to reset the possession."
      }
    ]
  },
  {
    "transcript_id": 263,
    "text": "Rescreening is Gibson Teague with the floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "263_1",
        "label": "action_executed",
        "text": "After Gibson sets the rescreen, Teague floats one up from the lane and buries it."
      },
      {
        "new_transcript_id": "263_2",
        "label": "action_not_executed",
        "text": "Teague looked to launch the floater after Gibson’s rescreen, but kicked it out instead."
      }
    ]
  },
  {
    "transcript_id": 264,
    "text": "Brown with a floater banks it in Bruce Brown the rookie out of Miami of Florida.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "264_1",
        "label": "action_executed",
        "text": "Bruce Brown glides into the paint, lofts a floater, and watches it bank softly off glass."
      },
      {
        "new_transcript_id": "264_2",
        "label": "action_not_executed",
        "text": "Brown started to lift for a floater but lost balance, passing it out before taking the shot."
      }
    ]
  },
  {
    "transcript_id": 265,
    "text": "Little finger roll goals in a chance for three point play and then it's out to a quick corner bead.",
    "action": "finger roll",
    "augmentations": [
      {
        "new_transcript_id": "265_1",
        "label": "action_executed",
        "text": "He slashes through traffic, flicks in the finger roll and draws contact for the and-one."
      },
      {
        "new_transcript_id": "265_2",
        "label": "action_not_executed",
        "text": "Looked like he wanted the finger roll, but he dished out to the wing before lifting it."
      }
    ]
  },
  {
    "transcript_id": 266,
    "text": "Great touch with the floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "266_1",
        "label": "action_executed",
        "text": "With soft touch, he drops in a floater that arcs gently over the rim and through."
      },
      {
        "new_transcript_id": "266_2",
        "label": "action_not_executed",
        "text": "He faked the floater, drew the defense, and instead kicked it out for a perimeter look."
      }
    ]
  },
  {
    "transcript_id": 267,
    "text": "Yeah really can hurt you off the pick and roll game is approved as a 3 point shooter hurt Miami beyond the arc from the dotted line.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "267_1",
        "label": "action_not_executed",
        "text": "He surveyed after the screen but never executed the pick and roll, spacing to the perimeter instead."
      },
      {
        "new_transcript_id": "267_2",
        "label": "action_executed",
        "text": "Running the high pick and roll, he attacked downhill, reading Miami’s defense for a clean look."
      }
    ]
  },
  {
    "transcript_id": 268,
    "text": "Great young little floater rolls in.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "268_1",
        "label": "action_executed",
        "text": "Young pushes into the lane, softly lofts a floater, and watches it drop straight through."
      },
      {
        "new_transcript_id": "268_2",
        "label": "action_not_executed",
        "text": "Young looked to float it up but got stripped, never getting the floater attempt away."
      }
    ]
  },
  {
    "transcript_id": 269,
    "text": "Here comes holiday office screen Double teamed over Rondo into the Lane drops in the more ball cap.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "269_1",
        "label": "action_executed",
        "text": "Holiday comes off the high screen and immediately draws a double team, forcing the ball away from Rondo."
      },
      {
        "new_transcript_id": "269_2",
        "label": "action_not_executed",
        "text": "Holiday looked to draw the double but defenders stayed home, so no trap came on that play."
      }
    ]
  },
  {
    "transcript_id": 270,
    "text": "Johnson also had his first NBA 222 nights ago in that game against Indiana Garlands, exactly the foul line's teardrop is true.",
    "action": "teardrop",
    "augmentations": [
      {
        "new_transcript_id": "270_1",
        "label": "action_not_executed",
        "text": "Garland lined up a teardrop near the foul line but instead deferred, pulling it back to reset."
      },
      {
        "new_transcript_id": "270_2",
        "label": "action_executed",
        "text": "From the foul line Garland arcs a high teardrop, floating it in cleanly over the Indiana defense."
      }
    ]
  },
  {
    "transcript_id": 271,
    "text": "But when he comes cross selling off their brush green, he's a good post up player.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "271_1",
        "label": "action_executed",
        "text": "He establishes deep position off the screen, turning into a strong post up play near the block."
      },
      {
        "new_transcript_id": "271_2",
        "label": "action_not_executed",
        "text": "He looked to back his man down for a post up but quickly swung it out before posting."
      }
    ]
  },
  {
    "transcript_id": 272,
    "text": "Here's Quarry, watched by Hood, picked off by Zubats, Hawaii with the floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "272_1",
        "label": "action_executed",
        "text": "Kawhi Leonard slips past Hood, pulls up with touch, and drops in the floater over Zubats’ man."
      },
      {
        "new_transcript_id": "272_2",
        "label": "action_not_executed",
        "text": "Kawhi sized up for a floater but had his dribble poked loose, unable to release the shot."
      }
    ]
  },
  {
    "transcript_id": 273,
    "text": "Chief works the inline drops it to Nurkic, draws a double team of a couple of seven footers.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "273_1",
        "label": "action_executed",
        "text": "Nurkic receives it inside, immediately attracting a double team as two bigs collapse on him."
      },
      {
        "new_transcript_id": "273_2",
        "label": "action_not_executed",
        "text": "He caught it deep but the second defender never came, so no real double materialized."
      }
    ]
  },
  {
    "transcript_id": 274,
    "text": "Don't try and judge us until we are whole and healthy and they double team balance Unison.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "274_1",
        "label": "action_executed",
        "text": "Balanced possession leads to a trap, defenders converging quickly to double team the post entry."
      },
      {
        "new_transcript_id": "274_2",
        "label": "action_not_executed",
        "text": "They prepared to double, but both defenders stayed spaced, leaving single coverage intact."
      }
    ]
  },
  {
    "transcript_id": 275,
    "text": "Thanks for the floater, and that's the first story again.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "275_1",
        "label": "action_executed",
        "text": "He knifes into the lane and gently lofts a floater, earning the team’s opening basket."
      },
      {
        "new_transcript_id": "275_2",
        "label": "action_not_executed",
        "text": "He looked to launch a floater but got cut off, passing out to the wing instead."
      }
    ]
  },
  {
    "transcript_id": 276,
    "text": "He was six for seven on floaters last night.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "276_1",
        "label": "action_not_executed",
        "text": "Commentators recall he made six of seven floaters last game, though none attempted here yet."
      },
      {
        "new_transcript_id": "276_2",
        "label": "action_executed",
        "text": "He continues the rhythm, steps inside and executes another floater like those from last night."
      }
    ]
  },
  {
    "transcript_id": 277,
    "text": "Veteran head coach in the NBA, Williams off to turn over Anna Randall with a slam dunk.",
    "action": "slam dunk",
    "augmentations": [
      {
        "new_transcript_id": "277_1",
        "label": "action_executed",
        "text": "Randall takes the turnover coast to coast, exploding for a two-handed slam dunk finish."
      },
      {
        "new_transcript_id": "277_2",
        "label": "action_not_executed",
        "text": "Randall ran the floor but was fouled before he could rise up for a slam dunk attempt."
      }
    ]
  },
  {
    "transcript_id": 278,
    "text": "Westbrook drives and kicks Ferguson, who had a couple of trees early in that game, finds Grant into the link to the glass finger, rolled up no Panther defense.",
    "action": "finger roll",
    "augmentations": [
      {
        "new_transcript_id": "278_1",
        "label": "action_executed",
        "text": "Grant cuts through the lane and gently flicks a finger roll off glass over late defense."
      },
      {
        "new_transcript_id": "278_2",
        "label": "action_not_executed",
        "text": "Grant slashed in but never lifted the finger roll, dishing to the corner instead."
      }
    ]
  },
  {
    "transcript_id": 279,
    "text": "Welsh posting up.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "279_1",
        "label": "action_executed",
        "text": "Welsh seals his man on the block and establishes a strong post up position inside."
      },
      {
        "new_transcript_id": "279_2",
        "label": "action_not_executed",
        "text": "Welsh flashed toward the post but never settled, retreating outside without a post up."
      }
    ]
  },
  {
    "transcript_id": 280,
    "text": "Of course immediately double team Jokic who passes cross court to Murray.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "280_1",
        "label": "action_executed",
        "text": "Jokic touches the ball and is instantly double teamed, forcing him to swing it to Murray."
      },
      {
        "new_transcript_id": "280_2",
        "label": "action_not_executed",
        "text": "He got the ball but the second defender hesitated, leaving him single covered this time."
      }
    ]
  },
  {
    "transcript_id": 281,
    "text": "A7 footer had never done before, never done and then they created the post up at the free throw line.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "281_1",
        "label": "action_not_executed",
        "text": "They talked about creating a post up at the foul line but it never materialized on the possession."
      },
      {
        "new_transcript_id": "281_2",
        "label": "action_executed",
        "text": "He set up at the free throw line, turning his back and executing the post up move."
      }
    ]
  },
  {
    "transcript_id": 282,
    "text": "It's interesting they trying to switch some pick and rolls.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "282_1",
        "label": "action_executed",
        "text": "They ran another pick and roll, testing the switch, which led to penetration toward the rim."
      },
      {
        "new_transcript_id": "282_2",
        "label": "action_not_executed",
        "text": "They talked about switching pick and rolls but never ran one during this sequence."
      }
    ]
  },
  {
    "transcript_id": 283,
    "text": "Chased by Kuczma pick and roll, it Gobert loss to handle.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "283_1",
        "label": "action_executed",
        "text": "Kuzma chased hard, and the pick and roll freed Gobert inside, though he bobbled the catch."
      },
      {
        "new_transcript_id": "283_2",
        "label": "action_not_executed",
        "text": "The pick and roll action was started, but Gobert lost the ball before completing it."
      }
    ]
  },
  {
    "transcript_id": 284,
    "text": "Two nights ago in San Antonio, Osmonds floater doesn't go Thompson with the offensive rebound.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "284_1",
        "label": "action_executed",
        "text": "Osmond drove and lofted a floater from the baseline, though it rimmed out to Thompson."
      },
      {
        "new_transcript_id": "284_2",
        "label": "action_not_executed",
        "text": "Looked like he’d take the floater but pulled it back, missing the chance before rebounding."
      }
    ]
  },
  {
    "transcript_id": 285,
    "text": "Cunningham swings it forms chased off the line floater for Brent.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "285_1",
        "label": "action_executed",
        "text": "Forbes gets it on the swing, dribbles into the paint and lofts a floater right over the big."
      },
      {
        "new_transcript_id": "285_2",
        "label": "action_not_executed",
        "text": "Forbes moved inside as if for a floater but kicked to the wing before releasing it."
      }
    ]
  },
  {
    "transcript_id": 286,
    "text": "Well, this is a pick and roll.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "286_1",
        "label": "action_executed",
        "text": "They set up the pick and roll, guard uses the screen and dives into the lane decisively."
      },
      {
        "new_transcript_id": "286_2",
        "label": "action_not_executed",
        "text": "They lined up for a pick and roll but never executed the screen or dive."
      }
    ]
  },
  {
    "transcript_id": 287,
    "text": "Well, if you're not going to send the double team and he gives it that deep, there really isn't much you can do about it so long and creative, easy finish thanks.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "287_1",
        "label": "action_not_executed",
        "text": "They discussed doubling him down low but never actually sent the double team on the play."
      },
      {
        "new_transcript_id": "287_2",
        "label": "action_executed",
        "text": "He caught it deep and the defense sent a quick double team, trying to crowd his space."
      }
    ]
  },
  {
    "transcript_id": 288,
    "text": "Pick and roll involving Brunson, Kleber, Brunson in the paint looking for the first basket of the game and there it is looks like a vet man.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "288_1",
        "label": "action_executed",
        "text": "Brunson uses Kleber’s screen to perfection, executing the pick and roll and finishing inside."
      },
      {
        "new_transcript_id": "288_2",
        "label": "action_not_executed",
        "text": "They motioned into pick and roll but Brunson pulled back, resetting without taking the shot."
      }
    ]
  },
  {
    "transcript_id": 289,
    "text": "Look and book settle for that fadeaway jump shot.",
    "action": "fadeaway",
    "augmentations": [
      {
        "new_transcript_id": "289_1",
        "label": "action_not_executed",
        "text": "He looked like he might go fadeaway but pivoted out, choosing a pass instead of shooting."
      },
      {
        "new_transcript_id": "289_2",
        "label": "action_executed",
        "text": "He squares up, leans back, and drains the contested fadeaway jumper from mid-range."
      }
    ]
  },
  {
    "transcript_id": 290,
    "text": "Nice up there and try to get the tip instead.",
    "action": "tip in",
    "augmentations": [
      {
        "new_transcript_id": "290_1",
        "label": "action_not_executed",
        "text": "He leaped in traffic, tried to attempt the tip in, but never managed to get the ball on rim."
      },
      {
        "new_transcript_id": "290_2",
        "label": "action_executed",
        "text": "Off the miss, he skies and flicks the tip in cleanly off the backboard for two."
      }
    ]
  },
  {
    "transcript_id": 291,
    "text": "Carl attack that front foot in Nice give and go right there.",
    "action": "give and go",
    "augmentations": [
      {
        "new_transcript_id": "291_1",
        "label": "action_executed",
        "text": "They run a textbook give and go, cutting hard to the basket for a smooth lay-in finish."
      },
      {
        "new_transcript_id": "291_2",
        "label": "action_not_executed",
        "text": "The cut was set up for a give and go, but he fumbled it and pulled back outside."
      }
    ]
  },
  {
    "transcript_id": 292,
    "text": "Here's Connolly into the pain and missing the floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "292_1",
        "label": "action_executed",
        "text": "Conley snakes into the lane, gets it up on a floater that rims out at the cup."
      },
      {
        "new_transcript_id": "292_2",
        "label": "action_not_executed",
        "text": "Conley drove but lost it before releasing the floater, unable to get the attempt away."
      }
    ]
  },
  {
    "transcript_id": 296,
    "text": "It's tough little floater from the baseline over the top of that.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "296_1",
        "label": "action_executed",
        "text": "He works baseline and lofts a high floater, dropping it in cleanly over the defender."
      },
      {
        "new_transcript_id": "296_2",
        "label": "action_not_executed",
        "text": "He looked baseline for a floater but was stripped before ever releasing the shot."
      }
    ]
  },
  {
    "transcript_id": 297,
    "text": "Donovan, like the left hand, work behind the screen of Rudy Gobert, the high floater banked it off the glass, and the ball comes loose.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "297_1",
        "label": "action_executed",
        "text": "Mitchell curls behind Gobert’s screen and flips a lefty floater off glass that bounces out."
      },
      {
        "new_transcript_id": "297_2",
        "label": "action_not_executed",
        "text": "Mitchell set for a floater but got hit midair, ball knocked away before release."
      }
    ]
  },
  {
    "transcript_id": 298,
    "text": "Drew holiday in the little floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "298_1",
        "label": "action_executed",
        "text": "Holiday snakes into the paint and calmly drops in a smooth floater with the right hand."
      },
      {
        "new_transcript_id": "298_2",
        "label": "action_not_executed",
        "text": "Holiday started to rise for a floater but got bumped and kicked it out before releasing."
      }
    ]
  },
  {
    "transcript_id": 299,
    "text": "They double teaming quickly in the post.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "299_1",
        "label": "action_executed",
        "text": "As soon as he touches the block, defenders swarm him with a quick double team trap."
      },
      {
        "new_transcript_id": "299_2",
        "label": "action_not_executed",
        "text": "Looked like the help was ready, but no double team actually came in the post that trip."
      }
    ]
  },
  {
    "transcript_id": 300,
    "text": "Post up for Jokic, dump it into cragan he felt.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "300_1",
        "label": "action_executed",
        "text": "Denver feeds Jokic on the block, he backs down and executes a patient post up move."
      },
      {
        "new_transcript_id": "300_2",
        "label": "action_not_executed",
        "text": "They looked for Jokic to post up, but the entry pass never got through the defense."
      }
    ]
  },
  {
    "transcript_id": 301,
    "text": "The first point to the game with a nice floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "301_1",
        "label": "action_executed",
        "text": "He scores the opener by gliding inside and finishing softly with a floater over traffic."
      },
      {
        "new_transcript_id": "301_2",
        "label": "action_not_executed",
        "text": "He drove toward the paint as if for a floater, but turned it over before releasing."
      }
    ]
  },
  {
    "transcript_id": 302,
    "text": "Well you seem a double teams right now 'cause the Blazers have number.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "302_1",
        "label": "action_executed",
        "text": "Portland sends two at the ball, a clear double team trying to slow the possession."
      },
      {
        "new_transcript_id": "302_2",
        "label": "action_not_executed",
        "text": "It looked like a double team was coming, but the defense stayed home on shooters."
      }
    ]
  },
  {
    "transcript_id": 303,
    "text": "Doubled Rivers Reborn Rivers defender came over to double team James Horton.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "303_1",
        "label": "action_executed",
        "text": "Two defenders converge hard on Harden, forcing the ball out with an aggressive double team."
      },
      {
        "new_transcript_id": "303_2",
        "label": "action_not_executed",
        "text": "Harden was lined up for a double team, but the second defender never committed fully."
      }
    ]
  },
  {
    "transcript_id": 304,
    "text": "JV update drives 18 stays with the bank shot.",
    "action": "bank shot",
    "augmentations": [
      {
        "new_transcript_id": "304_1",
        "label": "action_executed",
        "text": "He muscles his way inside and calmly banks in the mid-range shot off the window."
      },
      {
        "new_transcript_id": "304_2",
        "label": "action_not_executed",
        "text": "He lined up as if for a bank shot but got stripped before putting it off the glass."
      }
    ]
  },
  {
    "transcript_id": 305,
    "text": "What at this, and I think that went between the legs of Aten.",
    "action": "between the legs",
    "augmentations": [
      {
        "new_transcript_id": "305_1",
        "label": "action_not_executed",
        "text": "He faked a flashy between-the-legs dribble but pulled back, never executing the move."
      },
      {
        "new_transcript_id": "305_2",
        "label": "action_executed",
        "text": "He split the defender cleanly with a smooth between-the-legs dribble, crossing midcourt."
      }
    ]
  },
  {
    "transcript_id": 306,
    "text": "And here comes Troy Brown Junior Goes Coast to Coast pizza but swatted away by MTSU.",
    "action": "coast to coast",
    "augmentations": [
      {
        "new_transcript_id": "306_1",
        "label": "action_executed",
        "text": "Troy Brown Jr. pushes the rebound himself coast to coast, taking it all the way before contact."
      },
      {
        "new_transcript_id": "306_2",
        "label": "action_not_executed",
        "text": "He started on a coast to coast drive but got stripped near midcourt, never reaching the rim."
      }
    ]
  },
  {
    "transcript_id": 307,
    "text": "Former slam dunk letter J are put on there with nice men blue.",
    "action": "slam dunk",
    "augmentations": [
      {
        "new_transcript_id": "307_1",
        "label": "action_not_executed",
        "text": "He rose like a slam dunk attempt was coming, but got fouled before he could finish it."
      },
      {
        "new_transcript_id": "307_2",
        "label": "action_executed",
        "text": "He explodes toward the rim, rising high and hammering home a slam dunk over the defense."
      }
    ]
  },
  {
    "transcript_id": 308,
    "text": "At home we played two of the first three at Home Alley Oop attempt Aaron Drummond, right, there, not a smart pass.",
    "action": "alley oop",
    "augmentations": [
      {
        "new_transcript_id": "308_1",
        "label": "action_executed",
        "text": "They toss it up for Drummond, who finishes the alley oop powerfully above the rim."
      },
      {
        "new_transcript_id": "308_2",
        "label": "action_not_executed",
        "text": "The lob for Drummond was too high, and the alley oop never connected cleanly."
      }
    ]
  },
  {
    "transcript_id": 309,
    "text": "Looking for the pretty alley Oop Jazz will take it the other way.",
    "action": "alley oop",
    "augmentations": [
      {
        "new_transcript_id": "309_1",
        "label": "action_not_executed",
        "text": "They tried to set up the alley oop but the pass was stolen, so no finish came."
      },
      {
        "new_transcript_id": "309_2",
        "label": "action_executed",
        "text": "He rises above the rim and throws down the alley oop in highlight fashion."
      }
    ]
  },
  {
    "transcript_id": 310,
    "text": "But running the pick and rolls to perfection that last time to Curry Johnson might have gotten away with the travel, but he scores off glass.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "310_1",
        "label": "action_not_executed",
        "text": "They mentioned running the pick and roll earlier, but on this play it wasn’t executed."
      },
      {
        "new_transcript_id": "310_2",
        "label": "action_executed",
        "text": "Curry executes the pick and roll seamlessly, feeding Johnson who finishes through contact."
      }
    ]
  },
  {
    "transcript_id": 311,
    "text": "Will take the three little splash it money.",
    "action": "splash",
    "augmentations": [
      {
        "new_transcript_id": "311_1",
        "label": "action_executed",
        "text": "He pulls up from deep, pure splash as the three-pointer rips through the net."
      },
      {
        "new_transcript_id": "311_2",
        "label": "action_not_executed",
        "text": "He faked the splash three but drove inside, never actually releasing the long shot."
      }
    ]
  },
  {
    "transcript_id": 312,
    "text": "The floater guides it home an one.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "312_1",
        "label": "action_executed",
        "text": "He slips into the lane, kisses the floater in and draws the foul for a three-point play."
      },
      {
        "new_transcript_id": "312_2",
        "label": "action_not_executed",
        "text": "He attempted to rise for a floater but was fouled before the shot could be released."
      }
    ]
  },
  {
    "transcript_id": 313,
    "text": "This is the floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "313_1",
        "label": "action_executed",
        "text": "He pulls up in the lane and softly sends the floater straight through the rim."
      },
      {
        "new_transcript_id": "313_2",
        "label": "action_not_executed",
        "text": "He looked poised for a floater but instead pivoted, never letting the ball go."
      }
    ]
  },
  {
    "transcript_id": 314,
    "text": "The alley oop.",
    "action": "alley oop",
    "augmentations": [
      {
        "new_transcript_id": "314_1",
        "label": "action_executed",
        "text": "Beautiful alley oop connection, lobbed high and finished with power at the rim."
      },
      {
        "new_transcript_id": "314_2",
        "label": "action_not_executed",
        "text": "They set up for an alley oop, but the defender tipped the pass away before completion."
      }
    ]
  },
  {
    "transcript_id": 315,
    "text": "Necessary to throw that high type of floater, but he's just showing off his skills that's involved with Supreme confidence.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "315_1",
        "label": "action_not_executed",
        "text": "He gestures like he’ll release a floater, but instead resets and kicks the ball out."
      },
      {
        "new_transcript_id": "315_2",
        "label": "action_executed",
        "text": "Confidently drives the lane, lofts the floater high off the glass, and it drops through."
      }
    ]
  },
  {
    "transcript_id": 316,
    "text": "Are double team on Luka?",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "316_1",
        "label": "action_executed",
        "text": "Two defenders collapse hard on Luka near the wing, forcing him to pass out of the double team."
      },
      {
        "new_transcript_id": "316_2",
        "label": "action_not_executed",
        "text": "They start to show help on Luka, but quickly retreat, never fully committing to a double team."
      }
    ]
  },
  {
    "transcript_id": 318,
    "text": "Splendid Alley oop that time to pump.",
    "action": "alley oop",
    "augmentations": [
      {
        "new_transcript_id": "318_1",
        "label": "action_not_executed",
        "text": "They attempt a lob for the alley oop, but the pass sails too long and goes out of bounds."
      },
      {
        "new_transcript_id": "318_2",
        "label": "action_executed",
        "text": "He times his jump perfectly, catching the lob for a thunderous alley oop slam."
      }
    ]
  },
  {
    "transcript_id": 319,
    "text": "Will post up against Kobe White.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "319_1",
        "label": "action_executed",
        "text": "He seals Kobe White deep in the paint, backs him down, and establishes a strong post up position."
      },
      {
        "new_transcript_id": "319_2",
        "label": "action_not_executed",
        "text": "Looked ready to post up Kobe White, but instead drifts back to the perimeter to reset."
      }
    ]
  },
  {
    "transcript_id": 321,
    "text": "Start with Cody Zeller, PJ Washington trying to post up Prince.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "321_1",
        "label": "action_executed",
        "text": "Washington squares up Prince, plants himself on the block, and powers into a solid post up."
      },
      {
        "new_transcript_id": "321_2",
        "label": "action_not_executed",
        "text": "PJ Washington begins to back Prince down, but quickly passes out before completing the post up."
      }
    ]
  },
  {
    "transcript_id": 322,
    "text": "post up the smaller Trae Young and Sweet Junior missing badly there.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "322_1",
        "label": "action_executed",
        "text": "He posts up the smaller Trae Young, powers into the lane, and rises for a short jumper."
      },
      {
        "new_transcript_id": "322_2",
        "label": "action_not_executed",
        "text": "He looks to post Trae Young, but the ball slips loose before he can get into position."
      }
    ]
  },
  {
    "transcript_id": 323,
    "text": "Zatarain's key with the floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "323_1",
        "label": "action_executed",
        "text": "Zatarain floats it softly over the big man, the floater bouncing gently through the net."
      },
      {
        "new_transcript_id": "323_2",
        "label": "action_not_executed",
        "text": "He starts to gather for the floater, but dishes off to the corner before leaving his feet."
      }
    ]
  },
  {
    "transcript_id": 324,
    "text": "Glad you're bored here on Fox Sports Ohio as they go right away for a post up to Morse, triple team swatted away by Alec Burks.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "324_1",
        "label": "action_executed",
        "text": "They feed Morse in the block for a post up, but the triple team collapses and blocks his attempt."
      },
      {
        "new_transcript_id": "324_2",
        "label": "action_not_executed",
        "text": "Looked to go for a post up to Morse, but Burks denies the entry pass before it develops."
      }
    ]
  },
  {
    "transcript_id": 325,
    "text": "That one is tipped in by McGee.",
    "action": "tip in",
    "augmentations": [
      {
        "new_transcript_id": "325_1",
        "label": "action_executed",
        "text": "McGee skies for the rebound, extending his arm to secure the tip in over two defenders."
      },
      {
        "new_transcript_id": "325_2",
        "label": "action_not_executed",
        "text": "McGee goes up for the tip in, but the ball rims out and the rebound falls to the defense."
      }
    ]
  },
  {
    "transcript_id": 326,
    "text": "Oregon State's Gary Payton, defending lunar here's Whiteside runs into the Lane, takes it strong in the Cup to lay it in the pick and roll action right there.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "326_1",
        "label": "action_not_executed",
        "text": "They try setting up the pick and roll, but Whiteside breaks it off and just drives straight to the rim."
      },
      {
        "new_transcript_id": "326_2",
        "label": "action_executed",
        "text": "Off the screen, Whiteside rolls hard into the paint, completing a textbook pick and roll layup."
      }
    ]
  },
  {
    "transcript_id": 327,
    "text": "Nobody wants to get on that poster.",
    "action": "poster",
    "augmentations": [
      {
        "new_transcript_id": "327_1",
        "label": "action_not_executed",
        "text": "Crowd reacts to the near attempt, but no actual dunk poster happens on that possession."
      },
      {
        "new_transcript_id": "327_2",
        "label": "action_executed",
        "text": "He rises with authority, slamming home the poster dunk over the helpless defender."
      }
    ]
  },
  {
    "transcript_id": 328,
    "text": "Could help by favors on the pick and roll Hardaway from deep.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "328_1",
        "label": "action_executed",
        "text": "Favors shows help on the pick and roll, Hardaway uses it and scores before the defense recovers."
      },
      {
        "new_transcript_id": "328_2",
        "label": "action_not_executed",
        "text": "They signal for the pick and roll, but Hardaway skips it and just pulls up from deep instead."
      }
    ]
  },
  {
    "transcript_id": 329,
    "text": "He's tide up on a double team Good help defense and a travel.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "329_1",
        "label": "action_executed",
        "text": "Two defenders swarm him in the double team, forcing a travel violation in the paint."
      },
      {
        "new_transcript_id": "329_2",
        "label": "action_not_executed",
        "text": "They shade toward a double team, but only one defender stays on him as the whistle blows."
      }
    ]
  },
  {
    "transcript_id": 330,
    "text": "Will wait for the double team and then I'll give it up.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "330_1",
        "label": "action_executed",
        "text": "He holds the ball patiently until the double team arrives, then swings it to the corner."
      },
      {
        "new_transcript_id": "330_2",
        "label": "action_not_executed",
        "text": "He anticipates a double team but none comes, so he just resets at the top of the arc."
      }
    ]
  },
  {
    "transcript_id": 331,
    "text": "The floater, the bounce, the bucket.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "331_1",
        "label": "action_executed",
        "text": "He lofts a soft floater, the ball bounces twice on the rim and finally falls through."
      },
      {
        "new_transcript_id": "331_2",
        "label": "action_not_executed",
        "text": "Looks like he’ll shoot a floater, but instead fakes and kicks it out to the perimeter."
      }
    ]
  },
  {
    "transcript_id": 332,
    "text": "On the post up, Jeffrey Pretty Easy start coming.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "332_1",
        "label": "action_executed",
        "text": "Jeffrey backs his man down on the block, establishing a clean post up for the easy look."
      },
      {
        "new_transcript_id": "332_2",
        "label": "action_not_executed",
        "text": "Jeffrey tried to get position for the post up, but the entry pass never arrived."
      }
    ]
  },
  {
    "transcript_id": 333,
    "text": "He puts up a little floater that's no good.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "333_1",
        "label": "action_executed",
        "text": "He drives hard into the paint, launches the floater over the big man, but misses off back iron."
      },
      {
        "new_transcript_id": "333_2",
        "label": "action_not_executed",
        "text": "Starts to go for the floater but changes his mind, passing it outside before the shot goes up."
      }
    ]
  },
  {
    "transcript_id": 334,
    "text": "He's going against a lot of smaller guards so that post up games becoming important Hemi.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "334_1",
        "label": "action_not_executed",
        "text": "Commentary notes his post up could be useful, but he doesn’t actually post up on that trip."
      },
      {
        "new_transcript_id": "334_2",
        "label": "action_executed",
        "text": "He exploits the mismatch, sealing the smaller guard deep and finishing the post up with ease."
      }
    ]
  },
  {
    "transcript_id": 335,
    "text": "Nice dish underneath that ball swatted away baseball pass downfield.",
    "action": "baseball pass",
    "augmentations": [
      {
        "new_transcript_id": "335_1",
        "label": "action_not_executed",
        "text": "He thought about the baseball pass downcourt, but defense closed and he reset the offense."
      },
      {
        "new_transcript_id": "335_2",
        "label": "action_executed",
        "text": "After the block, he gathers the loose ball and fires a baseball pass the length of the court."
      }
    ]
  },
  {
    "transcript_id": 336,
    "text": "And finger roll short, therefore Capella.",
    "action": "finger roll",
    "augmentations": [
      {
        "new_transcript_id": "336_1",
        "label": "action_executed",
        "text": "Capella glides to the rim, attempts the finger roll, but it comes up short off the front rim."
      },
      {
        "new_transcript_id": "336_2",
        "label": "action_not_executed",
        "text": "Capella fakes like he’ll try a finger roll, but instead kicks it to the wing shooter."
      }
    ]
  },
  {
    "transcript_id": 337,
    "text": "So already 2 forced turnovers from the Clipper defense and their setting pick and rolls with Porzingis.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "337_1",
        "label": "action_not_executed",
        "text": "They line up for the pick and roll with Porzingis, but the defense forces a turnover before execution."
      },
      {
        "new_transcript_id": "337_2",
        "label": "action_executed",
        "text": "Porzingis comes high, sets the screen, and rolls perfectly as the guard feeds him inside."
      }
    ]
  },
  {
    "transcript_id": 338,
    "text": "Green comes up short, James on the tip in 27 points for LeBron James.",
    "action": "tip in",
    "augmentations": [
      {
        "new_transcript_id": "338_1",
        "label": "action_executed",
        "text": "LeBron crashes the glass after the miss, rising above everyone for the emphatic tip in."
      },
      {
        "new_transcript_id": "338_2",
        "label": "action_not_executed",
        "text": "LeBron goes for the tip in but it rims out, and the rebound falls to the opponent."
      }
    ]
  },
  {
    "transcript_id": 339,
    "text": "We've got flop Cam standing by as well as Kyle Lowry berries at three.",
    "action": "flop",
    "augmentations": [
      {
        "new_transcript_id": "339_1",
        "label": "action_executed",
        "text": "Lowry draws contact and goes down selling the flop, bringing chuckles from the broadcast crew."
      },
      {
        "new_transcript_id": "339_2",
        "label": "action_not_executed",
        "text": "Mention of a flop cam, but in reality no flop occurred on that particular play."
      }
    ]
  },
  {
    "transcript_id": 340,
    "text": "Ana Steele and Ferguson comes here and finger rolls it up again.",
    "action": "finger roll",
    "augmentations": [
      {
        "new_transcript_id": "340_1",
        "label": "action_executed",
        "text": "Ferguson glides into the paint and softly flips in a smooth finger roll over the defense."
      },
      {
        "new_transcript_id": "340_2",
        "label": "action_not_executed",
        "text": "Ferguson looked like he’d go for the finger roll, but pulled back and reset to the top instead."
      }
    ]
  },
  {
    "transcript_id": 341,
    "text": "I still got the sweet shot swish.",
    "action": "swish",
    "augmentations": [
      {
        "new_transcript_id": "341_1",
        "label": "action_executed",
        "text": "He releases a clean jumper, perfect rotation, nothing but net—pure swish."
      },
      {
        "new_transcript_id": "341_2",
        "label": "action_not_executed",
        "text": "Talks about having that swish touch, but no actual shot was taken during the sequence."
      }
    ]
  },
  {
    "transcript_id": 342,
    "text": "Yeah, they want a little side pick and roll for who roots.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "342_1",
        "label": "action_executed",
        "text": "They run the side pick and roll, screen set cleanly, and the ballhandler finds the rolling big."
      },
      {
        "new_transcript_id": "342_2",
        "label": "action_not_executed",
        "text": "They signal for a side pick and roll, but never actually execute as the clock winds down."
      }
    ]
  },
   {
    "transcript_id": 343,
    "text": "Now you gotta see the double team areas and opportunities for those in there.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "343_1",
        "label": "action_not_executed",
        "text": "Commentator points out potential double team zones, but the defense never actually commits on that trip."
      },
      {
        "new_transcript_id": "343_2",
        "label": "action_executed",
        "text": "Two defenders converge quickly, creating the double team trap along the baseline corner."
      }
    ]
  },
  {
    "transcript_id": 344,
    "text": "Is Jackson looking along the post up here to Griffith?",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "344_1",
        "label": "action_executed",
        "text": "Jackson delivers the ball inside, Griffith seals his man and establishes strong post up position."
      },
      {
        "new_transcript_id": "344_2",
        "label": "action_not_executed",
        "text": "They eyed the post up for Griffith, but the pass never came and the play reset outside."
      }
    ]
  },
  {
    "transcript_id": 345,
    "text": "Quickest turn around, this fires hurry the outlet pass through Durant.",
    "action": "outlet pass",
    "augmentations": [
      {
        "new_transcript_id": "345_1",
        "label": "action_executed",
        "text": "Off the miss, Curry whips an outlet pass downcourt to Durant streaking ahead for the bucket."
      },
      {
        "new_transcript_id": "345_2",
        "label": "action_not_executed",
        "text": "Looked to throw an outlet pass toward Durant, but held onto the ball under backcourt pressure."
      }
    ]
  },
  {
    "transcript_id": 346,
    "text": "He can go up high, he's got the hops so he's one of those guys on the pick and roll.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "346_1",
        "label": "action_not_executed",
        "text": "They mention his hops on pick and roll plays, but none actually unfolds on this possession."
      },
      {
        "new_transcript_id": "346_2",
        "label": "action_executed",
        "text": "He sets the screen, rolls to the rim, and elevates high for the lob finish on the pick and roll."
      }
    ]
  },
  {
    "transcript_id": 347,
    "text": "Play it, but it is tipped in by John Collins.",
    "action": "tip in",
    "augmentations": [
      {
        "new_transcript_id": "347_1",
        "label": "action_executed",
        "text": "Collins crashes the glass, gets his fingertips on it, and converts the second-chance tip in."
      },
      {
        "new_transcript_id": "347_2",
        "label": "action_not_executed",
        "text": "Collins leaps for the tip in but the ball bounces off the rim and out of play."
      }
    ]
  },
  {
    "transcript_id": 348,
    "text": "Post up opportunity for it to white Howard Anna kills in and you seen a lot of that issue.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "348_1",
        "label": "action_executed",
        "text": "Howard backs down his man, muscles inside the paint, and scores through contact on the post up."
      },
      {
        "new_transcript_id": "348_2",
        "label": "action_not_executed",
        "text": "They try to set Howard for a post up, but the defense fronted him and denied the entry pass."
      }
    ]
  },
  {
    "transcript_id": 349,
    "text": "Splash is the 3 pointer.",
    "action": "splash",
    "augmentations": [
      {
        "new_transcript_id": "349_1",
        "label": "action_executed",
        "text": "He lets it fly from deep, perfect rotation and arc, straight splash through the net."
      },
      {
        "new_transcript_id": "349_2",
        "label": "action_not_executed",
        "text": "Fans shout splash, but the three-point attempt rattled off the rim and did not fall."
      }
    ]
  },
  {
    "transcript_id": 350,
    "text": "As tears started to show, some promise as a roller to the hoop off pick and rolls, Shotblocker averaged just under 10 points per game.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "350_1",
        "label": "action_not_executed",
        "text": "They discuss his role as a roller on pick and roll, but no such play actually occurs here."
      },
      {
        "new_transcript_id": "350_2",
        "label": "action_executed",
        "text": "He dives perfectly after setting the screen, the guard finds him and he finishes the pick and roll."
      }
    ]
  },
  {
    "transcript_id": 351,
    "text": "Unable to connect on the floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "351_1",
        "label": "action_executed",
        "text": "He lofts up the floater in the lane but it bounces off the rim and misses badly."
      },
      {
        "new_transcript_id": "351_2",
        "label": "action_not_executed",
        "text": "He considered the floater but dumped it off inside before ever releasing the shot."
      }
    ]
  },
  {
    "transcript_id": 352,
    "text": "Shooting in that game against New York, here's a blocker with a floater inside and he knocks it down.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "352_1",
        "label": "action_executed",
        "text": "He drives to the paint, arcs up a smooth floater, and it drops softly through the net."
      },
      {
        "new_transcript_id": "352_2",
        "label": "action_not_executed",
        "text": "He looks like he’ll take the floater but instead kicks it out to the shooter on the wing."
      }
    ]
  },
  {
    "transcript_id": 353,
    "text": "I wasn't a double team so brands like I'm used to that.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "353_1",
        "label": "action_not_executed",
        "text": "He mentions the double team never came, the defense stayed single coverage throughout the play."
      },
      {
        "new_transcript_id": "353_2",
        "label": "action_executed",
        "text": "He gets trapped by the defense, the double team forces him into an awkward turnover."
      }
    ]
  },
  {
    "transcript_id": 354,
    "text": "Little pick and roll game.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "354_1",
        "label": "action_executed",
        "text": "They run a crisp pick and roll, screen set high, and the roll man gets the ball cleanly."
      },
      {
        "new_transcript_id": "354_2",
        "label": "action_not_executed",
        "text": "They gesture toward a pick and roll, but the guard rejects the screen and goes isolation."
      }
    ]
  },
  {
    "transcript_id": 355,
    "text": "You get an easy dunk poster.",
    "action": "poster",
    "augmentations": [
      {
        "new_transcript_id": "355_1",
        "label": "action_not_executed",
        "text": "Broadcaster calls it a poster dunk setup, but in reality the defender fouled before it happened."
      },
      {
        "new_transcript_id": "355_2",
        "label": "action_executed",
        "text": "He elevates, hammers it down with authority, delivering the highlight poster over his opponent."
      }
    ]
  },
  {
    "transcript_id": 356,
    "text": "Posting up in the paint.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "356_1",
        "label": "action_not_executed",
        "text": "He looks to post up in the paint, but the ball swings away before he receives the entry pass."
      },
      {
        "new_transcript_id": "356_2",
        "label": "action_executed",
        "text": "He establishes position deep in the paint, backing down into a powerful post up possession."
      }
    ]
  },
  {
    "transcript_id": 357,
    "text": "It could be a floater 30 foot 3 pointer, didn't score.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "357_1",
        "label": "action_not_executed",
        "text": "Speculation about a floater chance, but the attempt never actually materialized on that trip."
      },
      {
        "new_transcript_id": "357_2",
        "label": "action_executed",
        "text": "He drives inside, lifts a floater over the big man, but it clangs off the iron without scoring."
      }
    ]
  },
  {
    "transcript_id": 358,
    "text": "Lowry coast to coast rejected inside, but a good follow by Ibaka.",
    "action": "coast to coast",
    "augmentations": [
      {
        "new_transcript_id": "358_1",
        "label": "action_executed",
        "text": "Lowry races coast to coast with speed, takes it to the rim, and draws the shot block attempt."
      },
      {
        "new_transcript_id": "358_2",
        "label": "action_not_executed",
        "text": "Lowry looked to go coast to coast but pulled up near half court, never attacking the rim."
      }
    ]
  },
  {
    "transcript_id": 359,
    "text": "Again, Turner will be open, moving and grooving nicely at that time to get the finger roll.",
    "action": "finger roll",
    "augmentations": [
      {
        "new_transcript_id": "359_1",
        "label": "action_executed",
        "text": "Turner cuts sharply, takes the feed, and finishes gracefully with a clean finger roll at the cup."
      },
      {
        "new_transcript_id": "359_2",
        "label": "action_not_executed",
        "text": "Turner made the cut, but defense forced him to pass instead of taking the finger roll attempt."
      }
    ]
  },
  {
    "transcript_id": 360,
    "text": "Again, Turner will be open, moving and grooving nicely at that time to get the finger roll.",
    "action": "finger roll",
    "augmentations": [
      {
        "new_transcript_id": "360_1",
        "label": "action_executed",
        "text": "Turner times the cut, gets the pass and finishes off the window with a soft finger roll."
      },
      {
        "new_transcript_id": "360_2",
        "label": "action_not_executed",
        "text": "Turner moved inside, but defense denied the angle and he kicked it back out without a finger roll."
      }
    ]
  },
  {
    "transcript_id": 361,
    "text": "I see there's a double team.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "361_1",
        "label": "action_not_executed",
        "text": "He notes a potential double team, but the defenders quickly retreat without actually trapping."
      },
      {
        "new_transcript_id": "361_2",
        "label": "action_executed",
        "text": "Two defenders close in fast, successfully executing the double team near the top of the arc."
      }
    ]
  },
  {
    "transcript_id": 362,
    "text": "No double team.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "362_1",
        "label": "action_not_executed",
        "text": "Commentary emphasizes the defense stayed single coverage, no double team came that time."
      },
      {
        "new_transcript_id": "362_2",
        "label": "action_executed",
        "text": "Defense sends the second defender late, completing the double team at the elbow."
      }
    ]
  },
  {
    "transcript_id": 363,
    "text": "Steven Adams usually likes to get out and guard the pick and roll in attack and Blitz.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "363_1",
        "label": "action_not_executed",
        "text": "They describe Adams’s usual pick and roll defense, but no such play is run in this possession."
      },
      {
        "new_transcript_id": "363_2",
        "label": "action_executed",
        "text": "Adams jumps out on the screen, blitzing the ballhandler hard on the pick and roll action."
      }
    ]
  },
  {
    "transcript_id": 364,
    "text": "No help, no double team.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "364_1",
        "label": "action_not_executed",
        "text": "Defense sticks to man coverage; no help defender comes and no double team develops."
      },
      {
        "new_transcript_id": "364_2",
        "label": "action_executed",
        "text": "A second defender jumps over late, surprising him with the double team trap on the wing."
      }
    ]
  },
  {
    "transcript_id": 365,
    "text": "Posting up the smaller Lowry no get Lowry for the fouls.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "365_1",
        "label": "action_executed",
        "text": "He posts up Lowry deep in the lane, powering inside to draw the foul from the smaller guard."
      },
      {
        "new_transcript_id": "365_2",
        "label": "action_not_executed",
        "text": "He tried to post Lowry, but the ball never entered the paint as the play reset outside."
      }
    ]
  },
   {
    "transcript_id": 366,
    "text": "Their percentage, low Giannis late getting into the action so Henson has to work through a double team and he draws a foul goddess got popped.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "366_1",
        "label": "action_executed",
        "text": "Giannis delays the set, forcing Henson to grind through a strong double team before contact is whistled."
      },
      {
        "new_transcript_id": "366_2",
        "label": "action_not_executed",
        "text": "Henson expected a double team, but the help never really arrived and play continued without pressure."
      }
    ]
  },
  {
    "transcript_id": 367,
    "text": "That's will certainly run their share of pick and rolls.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "367_1",
        "label": "action_not_executed",
        "text": "The broadcast notes this team typically runs plenty of pick and roll actions during games."
      },
      {
        "new_transcript_id": "367_2",
        "label": "action_executed",
        "text": "Right here they set the screen, execute the pick and roll, and the guard dives into the lane."
      }
    ]
  },
  {
    "transcript_id": 368,
    "text": "nothing but net of the bad variety PJ, every goal breaks the seal.",
    "action": "nothing but net",
    "augmentations": [
      {
        "new_transcript_id": "368_1",
        "label": "action_not_executed",
        "text": "Commentator remarks about nothing but net moments, but in this sequence the rim is unkind and misses pile up."
      },
      {
        "new_transcript_id": "368_2",
        "label": "action_executed",
        "text": "PJ steps into rhythm, his jumper pure, nothing but net as the crowd rises to its feet."
      }
    ]
  },
  {
    "transcript_id": 369,
    "text": "Did a good job against the Nets to knockdown floaters in the paint Dillon Brooks pump.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "369_1",
        "label": "action_not_executed",
        "text": "They recall how Brooks had success with floaters against Brooklyn, but here he pulled up without attempting one."
      },
      {
        "new_transcript_id": "369_2",
        "label": "action_executed",
        "text": "Brooks drives into the lane, rises above the defense and softly drops in a floater off the glass."
      }
    ]
  },
  {
    "transcript_id": 370,
    "text": "Double team on James.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "370_1",
        "label": "action_not_executed",
        "text": "The defense shows help toward James but backs off quickly, never committing to a full double team."
      },
      {
        "new_transcript_id": "370_2",
        "label": "action_executed",
        "text": "As James catches the ball near the wing, two defenders swarm him with an immediate double team trap."
      }
    ]
  },
  {
    "transcript_id": 371,
    "text": "Anthony Pick and roll with Vucevic.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "371_1",
        "label": "action_executed",
        "text": "Anthony dribbles off Vucevic’s screen, executes the pick and roll cleanly, and finds him rolling toward the rim."
      },
      {
        "new_transcript_id": "371_2",
        "label": "action_not_executed",
        "text": "They call for a pick and roll with Vucevic, but instead Anthony pulls back and resets without using it."
      }
    ]
  },
  {
    "transcript_id": 372,
    "text": "Covington the floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "372_1",
        "label": "action_executed",
        "text": "Covington glides into the paint, lofts a floater over the big man, and the shot drops through softly."
      },
      {
        "new_transcript_id": "372_2",
        "label": "action_not_executed",
        "text": "Covington looked set to attempt a floater, but pulled the ball out and kicked to the corner instead."
      }
    ]
  },
  {
    "transcript_id": 373,
    "text": "Now the beautiful floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "373_1",
        "label": "action_not_executed",
        "text": "They highlight a beautiful floater earlier, but in this possession the shot never goes up as the defense closes."
      },
      {
        "new_transcript_id": "373_2",
        "label": "action_executed",
        "text": "Driving left, he lofts a beautiful floater that sails over the shot blocker and splashes home."
      }
    ]
  },
  {
    "transcript_id": 374,
    "text": "Rozier gets into the paint and he gets the rainbow teardrop.",
    "action": "teardrop",
    "augmentations": [
      {
        "new_transcript_id": "374_1",
        "label": "action_executed",
        "text": "Rozier knifes into the lane, arcs a rainbow teardrop high above the rim, and it falls perfectly."
      },
      {
        "new_transcript_id": "374_2",
        "label": "action_not_executed",
        "text": "Rozier penetrates the paint but elects to pass it back out, never getting off the rainbow teardrop attempt."
      }
    ]
  },
  {
    "transcript_id": 375,
    "text": "There's a floater Jonathan Isaac on the attack.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "375_1",
        "label": "action_not_executed",
        "text": "Isaac drives hard looking to set up a floater, but the ball is poked away before he can release it."
      },
      {
        "new_transcript_id": "375_2",
        "label": "action_executed",
        "text": "Jonathan Isaac curls into the lane, floats it up softly and converts despite the contest inside."
      }
    ]
  },
   {
    "transcript_id": 376,
    "text": "Here's ball floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "376_1",
        "label": "action_executed",
        "text": "Ball drives into the lane, releases a floater over the defense and it splashes cleanly through."
      },
      {
        "new_transcript_id": "376_2",
        "label": "action_not_executed",
        "text": "Ball dribbled toward the paint but chose not to attempt a floater, instead passing out to the perimeter."
      }
    ]
  },
  {
    "transcript_id": 377,
    "text": "Here's the cruder again, an undrafted player out of Kansas State and Richard Super Floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "377_1",
        "label": "action_executed",
        "text": "The Kansas State product soars into the lane and finishes with a super floater that drops straight in."
      },
      {
        "new_transcript_id": "377_2",
        "label": "action_not_executed",
        "text": "He looked to line up his trademark super floater, but instead dishes the ball out to the corner."
      }
    ]
  },
  {
    "transcript_id": 378,
    "text": "Utah is a big pick and roll team.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "378_1",
        "label": "action_not_executed",
        "text": "The commentator reminds everyone that Utah often runs heavy pick and roll sets throughout the season."
      },
      {
        "new_transcript_id": "378_2",
        "label": "action_executed",
        "text": "Mitchell and Gobert execute a crisp pick and roll, freeing Gobert for an easy finish inside."
      }
    ]
  },
  {
    "transcript_id": 379,
    "text": "No pick and roll action.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "379_1",
        "label": "action_not_executed",
        "text": "The defense sets up, and once again there’s no pick and roll action initiated on this possession."
      },
      {
        "new_transcript_id": "379_2",
        "label": "action_executed",
        "text": "They set a high screen and roll effectively, pulling off the pick and roll to perfection."
      }
    ]
  },
  {
    "transcript_id": 380,
    "text": "Tough fadeaway that ones off well defended by Adams with good jobs is not letting into the paint.",
    "action": "fadeaway",
    "augmentations": [
      {
        "new_transcript_id": "380_1",
        "label": "action_executed",
        "text": "With Adams contesting, the shooter spins and nails a tough fadeaway jumper over the outstretched arm."
      },
      {
        "new_transcript_id": "380_2",
        "label": "action_not_executed",
        "text": "He considered taking the fadeaway but Adams’s defense forced him to pass instead of shooting."
      }
    ]
  },
  {
    "transcript_id": 381,
    "text": "The floater being one of 'em I think young had 38 in the season opener.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "381_1",
        "label": "action_not_executed",
        "text": "The announcer recalls how floaters are part of Young’s arsenal, referencing his earlier season opener."
      },
      {
        "new_transcript_id": "381_2",
        "label": "action_executed",
        "text": "Young pushes into the lane and softly drops a floater over the defender’s contest for two."
      }
    ]
  },
  {
    "transcript_id": 382,
    "text": "He'll take it to the rack.",
    "action": "take it to the rack",
    "augmentations": [
      {
        "new_transcript_id": "382_1",
        "label": "action_executed",
        "text": "He explodes to the rim, taking it to the rack and finishing with authority at the cup."
      },
      {
        "new_transcript_id": "382_2",
        "label": "action_not_executed",
        "text": "He looked like he would take it to the rack but pulled up and reset the play."
      }
    ]
  },
  {
    "transcript_id": 383,
    "text": "Horford is is better really difficult match up in post up situations for Karl Anthony Towns.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "383_1",
        "label": "action_not_executed",
        "text": "Analysts explain how Horford is tough in post up matchups, making it a challenge for Towns."
      },
      {
        "new_transcript_id": "383_2",
        "label": "action_executed",
        "text": "Horford plants deep in the paint, works the block and executes a solid post up move on Towns."
      }
    ]
  },
  {
    "transcript_id": 384,
    "text": "Balance Unix under the pump fake.",
    "action": "pump fake",
    "augmentations": [
      {
        "new_transcript_id": "384_1",
        "label": "action_executed",
        "text": "Valanciunas sells the pump fake, gets his defender in the air, and attacks the open lane."
      },
      {
        "new_transcript_id": "384_2",
        "label": "action_not_executed",
        "text": "Valanciunas appeared ready to use a pump fake, but released the shot immediately without faking."
      }
    ]
  },
  {
    "transcript_id": 385,
    "text": "He takes it a post up.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "385_1",
        "label": "action_executed",
        "text": "He establishes position, takes the ball on the block, and works into a classic post up move."
      },
      {
        "new_transcript_id": "385_2",
        "label": "action_not_executed",
        "text": "He moved toward post up positioning but quickly swung the ball back out to the perimeter."
      }
    ]
  },
  {
    "transcript_id": 386,
    "text": "Probably very good with those little floaters.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "386_1",
        "label": "action_executed",
        "text": "He slips inside, softly lofts a floater over the defender’s arms and it falls right through."
      },
      {
        "new_transcript_id": "386_2",
        "label": "action_not_executed",
        "text": "Commentators mention how effective he is with floaters, but on this trip he dishes to the wing instead."
      }
    ]
  },
  {
    "transcript_id": 387,
    "text": "fadeaway no short shot.",
    "action": "fadeaway",
    "augmentations": [
      {
        "new_transcript_id": "387_1",
        "label": "action_executed",
        "text": "He pivots, fades away over the smaller defender, and drains the tough midrange jumper."
      },
      {
        "new_transcript_id": "387_2",
        "label": "action_not_executed",
        "text": "Looked to set up a fadeaway but lost balance, never actually releasing the attempt."
      }
    ]
  },
  {
    "transcript_id": 388,
    "text": "Melo taking it to the ram in the bank shot doesn't go Collins couldn't clean it up but does on the second chance.",
    "action": "bank shot",
    "augmentations": [
      {
        "new_transcript_id": "388_1",
        "label": "action_executed",
        "text": "Melo drives strong, puts up a bank shot off the glass that misses, but Collins scores the putback."
      },
      {
        "new_transcript_id": "388_2",
        "label": "action_not_executed",
        "text": "Melo started toward the bank shot but lost control, and the possession ended without him getting it off."
      }
    ]
  },
  {
    "transcript_id": 389,
    "text": "It's better to switch post up dance against pink.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "389_1",
        "label": "action_not_executed",
        "text": "Analysts agree it’s smarter to switch instead of forcing post up plays in this matchup."
      },
      {
        "new_transcript_id": "389_2",
        "label": "action_executed",
        "text": "They isolate down low, spin into the paint and work a traditional post up possession."
      }
    ]
  },
  {
    "transcript_id": 390,
    "text": "Jack turns the corner looking for can or stolen away by Snell Outlet Pass.",
    "action": "outlet pass",
    "augmentations": [
      {
        "new_transcript_id": "390_1",
        "label": "action_not_executed",
        "text": "Jack’s drive is stripped by Snell, preventing any outlet pass from being made on the break."
      },
      {
        "new_transcript_id": "390_2",
        "label": "action_executed",
        "text": "Snell grabs the ball and fires an outlet pass downcourt to ignite the fast break."
      }
    ]
  },
  {
    "transcript_id": 391,
    "text": "Booker was very close to an up and down there as pain splashes into three.",
    "action": "splash",
    "augmentations": [
      {
        "new_transcript_id": "391_1",
        "label": "action_executed",
        "text": "Pain rises from deep, splashes home the three-pointer cleanly as Booker avoided the violation."
      },
      {
        "new_transcript_id": "391_2",
        "label": "action_not_executed",
        "text": "Booker shuffled his feet nearly traveling, and Pain’s shot from outside clanked off, no splash here."
      }
    ]
  },
  {
    "transcript_id": 392,
    "text": "Friend Forbes delivery that was an ice pick and roll, thereby Lamarcus Aldridge to free up forms.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "392_1",
        "label": "action_executed",
        "text": "Aldridge sets the screen, Forbes uses it and executes the pick and roll for an open jumper."
      },
      {
        "new_transcript_id": "392_2",
        "label": "action_not_executed",
        "text": "They set for a pick and roll but the defense cut it off before any action developed."
      }
    ]
  },
  {
    "transcript_id": 393,
    "text": "Coast to coast and finishes with the left hand.",
    "action": "coast to coast",
    "augmentations": [
      {
        "new_transcript_id": "393_1",
        "label": "action_executed",
        "text": "He grabs the rebound and goes coast to coast, finishing strong with his left hand at the rim."
      },
      {
        "new_transcript_id": "393_2",
        "label": "action_not_executed",
        "text": "He began to push coast to coast, but lost control and kicked it back before reaching the paint."
      }
    ]
  },
  {
    "transcript_id": 394,
    "text": "Pick and roll Whiteside right down the middle of the Lane and nobody here and play it up in a great start from here yet.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "394_1",
        "label": "action_executed",
        "text": "Whiteside rolls hard off the pick, catches the pass and scores down the middle of the lane."
      },
      {
        "new_transcript_id": "394_2",
        "label": "action_not_executed",
        "text": "They motion into a pick and roll look, but the play stalls before Whiteside can roll to the rim."
      }
    ]
  },
  {
    "transcript_id": 395,
    "text": "The Thunder win the tip in here is Westbrook.",
    "action": "tip in",
    "augmentations": [
      {
        "new_transcript_id": "395_1",
        "label": "action_not_executed",
        "text": "Thunder secure possession after the jump ball, but there was no tip in attempt on this play."
      },
      {
        "new_transcript_id": "395_2",
        "label": "action_executed",
        "text": "Off the miss, Westbrook crashes the glass and scores the putback tip in for two."
      }
    ]
  },
  {
    "transcript_id": 396,
    "text": "In the second meeting, and Wesley Matthews splashes A3.",
    "action": "splash",
    "augmentations": [
      {
        "new_transcript_id": "396_1",
        "label": "action_executed",
        "text": "Matthews rises on the wing and splashes in a three-pointer during this rematch contest."
      },
      {
        "new_transcript_id": "396_2",
        "label": "action_not_executed",
        "text": "Matthews lined up for a three but rimmed it out, no splash on this attempt."
      }
    ]
  },
  {
    "transcript_id": 397,
    "text": "Here's Carmelo Anthony inside on the post up.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "397_1",
        "label": "action_executed",
        "text": "Anthony sets his position on the block, backs down his man and executes a classic post up."
      },
      {
        "new_transcript_id": "397_2",
        "label": "action_not_executed",
        "text": "Anthony drifted inside but never established a true post up, swinging the ball back outside."
      }
    ]
  },
  {
    "transcript_id": 398,
    "text": "The other night Nurkic gets inside bank shot won't go the second attempt.",
    "action": "bank shot",
    "augmentations": [
      {
        "new_transcript_id": "398_1",
        "label": "action_executed",
        "text": "Nurkic gets inside, lays up a bank shot that rattles out before putting it back on the next try."
      },
      {
        "new_transcript_id": "398_2",
        "label": "action_not_executed",
        "text": "Nurkic was in position but mishandled the ball, never releasing a true bank shot on this play."
      }
    ]
  },
  {
    "transcript_id": 399,
    "text": "Kemba Walker, again dealing with some knee discomfort that he's had throughout much of the season, misses his first jumper, but the ball tipped in Matthews in.",
    "action": "tip in",
    "augmentations": [
      {
        "new_transcript_id": "399_1",
        "label": "action_executed",
        "text": "Walker missed the jumper, but Matthews crashed in and scored with a clean tip in finish."
      },
      {
        "new_transcript_id": "399_2",
        "label": "action_not_executed",
        "text": "Walker’s jumper missed, and Matthews reached but failed to complete the tip in attempt."
      }
    ]
  },
  {
    "transcript_id": 400,
    "text": "Tyler Johnson with the floater is off the mark.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "400_1",
        "label": "action_executed",
        "text": "Johnson drives into the lane, floats it up softly, but the floater bounces off the rim no good."
      },
      {
        "new_transcript_id": "400_2",
        "label": "action_not_executed",
        "text": "Johnson looked ready to shoot a floater but instead pivoted and passed it to a teammate outside."
      }
    ]
  },
  {
    "transcript_id": 401,
    "text": "Barton little pick and roll with Jokic.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "401_1",
        "label": "action_executed",
        "text": "Barton runs a pick and roll with Jokic, hitting him perfectly on the roll to the basket."
      },
      {
        "new_transcript_id": "401_2",
        "label": "action_not_executed",
        "text": "They looked to set a pick and roll, but the play broke down before any execution."
      }
    ]
  },
  {
    "transcript_id": 402,
    "text": "Switch pick and rolls between KP and smart.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "402_1",
        "label": "action_not_executed",
        "text": "They discuss switching defensive assignments on pick and rolls between Porzingis and Smart."
      },
      {
        "new_transcript_id": "402_2",
        "label": "action_executed",
        "text": "Porzingis sets the screen for Smart, executing the pick and roll with a bounce pass to the roller."
      }
    ]
  },
  {
    "transcript_id": 403,
    "text": "A chance for Simmons post up the smaller Brooks.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "403_1",
        "label": "action_executed",
        "text": "Simmons establishes low post position, backs down Brooks, and scores from the block."
      },
      {
        "new_transcript_id": "403_2",
        "label": "action_not_executed",
        "text": "Simmons had the chance for a post up, but passed out of the mismatch instead."
      }
    ]
  },
  {
    "transcript_id": 404,
    "text": "CJ toying with Wiseman nothing but net.",
    "action": "nothing but net",
    "augmentations": [
      {
        "new_transcript_id": "404_1",
        "label": "action_executed",
        "text": "CJ shakes Wiseman, rises and drains the jumper clean, nothing but net for two points."
      },
      {
        "new_transcript_id": "404_2",
        "label": "action_not_executed",
        "text": "CJ dribbled around Wiseman but missed badly, no nothing but net this time."
      }
    ]
  },
  {
    "transcript_id": 405,
    "text": "Rudy Gay double team swings too wide.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "405_1",
        "label": "action_executed",
        "text": "Two defenders converge on Rudy Gay, executing the double team and forcing the ball out."
      },
      {
        "new_transcript_id": "405_2",
        "label": "action_not_executed",
        "text": "The defense shaded toward Gay but never committed fully to a double team."
      }
    ]
  },
  {
    "transcript_id": 406,
    "text": "They have to quickly double team Perkins.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "406_1",
        "label": "action_executed",
        "text": "Perkins draws an immediate double team, with defenders collapsing fast as soon as he touches the ball."
      },
      {
        "new_transcript_id": "406_2",
        "label": "action_not_executed",
        "text": "They considered doubling Perkins but rotated away before any true double team was applied."
      }
    ]
  },
  {
    "transcript_id": 407,
    "text": "There they go to eat an early post upon homes.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "407_1",
        "label": "action_executed",
        "text": "They feed Holmes early on the block, letting him work into a post up move inside."
      },
      {
        "new_transcript_id": "407_2",
        "label": "action_not_executed",
        "text": "They looked to establish Holmes in a post up, but never delivered the entry pass."
      }
    ]
  },
  {
    "transcript_id": 408,
    "text": "There they go to eat an early post upon homes.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "408_1",
        "label": "action_executed",
        "text": "Holmes catches low, muscles inside and executes a quick post up finish at the rim."
      },
      {
        "new_transcript_id": "408_2",
        "label": "action_not_executed",
        "text": "Holmes moved toward the block but the ball never entered, so no post up developed."
      }
    ]
  },
  {
    "transcript_id": 409,
    "text": "There they go to eat an early post upon homes.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "409_1",
        "label": "action_executed",
        "text": "Early in the possession, they find Holmes deep and he executes a post up play."
      },
      {
        "new_transcript_id": "409_2",
        "label": "action_not_executed",
        "text": "Holmes set for a post up but the ball swung away, leaving him unused in the sequence."
      }
    ]
  },
  {
    "transcript_id": 410,
    "text": "No double team help from the Kings and which makes them pay router which always looking across that inside shoulder to see where the defense is coming from.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "410_1",
        "label": "action_not_executed",
        "text": "The Kings give no double team help on the post, leaving the defender stranded one-on-one."
      },
      {
        "new_transcript_id": "410_2",
        "label": "action_executed",
        "text": "Kings quickly collapse on the post, applying a double team that forces a difficult kick-out."
      }
    ]
  },
  {
    "transcript_id": 411,
    "text": "Double team.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "411_1",
        "label": "action_executed",
        "text": "Two defenders swarm immediately, executing a tight double team near the baseline."
      },
      {
        "new_transcript_id": "411_2",
        "label": "action_not_executed",
        "text": "The help defender stunted but retreated, leaving no real double team pressure."
      }
    ]
  },
  {
    "transcript_id": 412,
    "text": "Just buy one tough fadeaway",
    "action": "fadeaway",
    "augmentations": [
      {
        "new_transcript_id": "412_1",
        "label": "action_executed",
        "text": "He squares up and nails a tough fadeaway jumper with the shot clock winding down."
      },
      {
        "new_transcript_id": "412_2",
        "label": "action_not_executed",
        "text": "He looked to rise for a fadeaway but instead passed to a teammate on the perimeter."
      }
    ]
  },
  {
    "transcript_id": 413,
    "text": "Here's the fadeaways.",
    "action": "fadeaway",
    "augmentations": [
      {
        "new_transcript_id": "413_1",
        "label": "action_executed",
        "text": "He pivots baseline, rises over the defender and sinks the fadeaway jumper with smooth touch."
      },
      {
        "new_transcript_id": "413_2",
        "label": "action_not_executed",
        "text": "He fakes as if going for a fadeaway, but instead kicks it out to the corner for a reset."
      }
    ]
  },
  {
    "transcript_id": 414,
    "text": "Andre screens for Reggie Jackson floater in the Lane, fills it up.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "414_1",
        "label": "action_executed",
        "text": "Reggie Jackson uses Andre’s screen, penetrates the lane and floats it in cleanly for two."
      },
      {
        "new_transcript_id": "414_2",
        "label": "action_not_executed",
        "text": "Jackson came off Andre’s screen but never attempted a floater, choosing to pass to the wing."
      }
    ]
  },
  {
    "transcript_id": 415,
    "text": "Middleton floater would not go at Williams who was defending Giannis again, that time with the rebound, got numbers White supporter for free.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "415_1",
        "label": "action_executed",
        "text": "Middleton drives in, releases the floater over Williams but it rims out, leading to a rebound chance."
      },
      {
        "new_transcript_id": "415_2",
        "label": "action_not_executed",
        "text": "Middleton drove into the paint but hesitated, never releasing the floater, instead swinging the ball out."
      }
    ]
  },
  {
    "transcript_id": 416,
    "text": "Point shot is going to be looking to get into the paint post up so you gotta be able to be ready for all situations.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "416_1",
        "label": "action_not_executed",
        "text": "He looked ready to post up in the paint, but never actually went into the move that trip."
      },
      {
        "new_transcript_id": "416_2",
        "label": "action_executed",
        "text": "He sealed his man deep in the lane and went right into a strong post up finish."
      }
    ]
  },
  {
    "transcript_id": 417,
    "text": "See if the Warriors were able to kind of rectify that tonight with some double teaming or creative defensive matchups.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "417_1",
        "label": "action_not_executed",
        "text": "They discussed using a double team defensively, but in that sequence it never materialized."
      },
      {
        "new_transcript_id": "417_2",
        "label": "action_executed",
        "text": "The Warriors swarmed the ball handler immediately, sending a hard double team near the wing."
      }
    ]
  },
  {
    "transcript_id": 418,
    "text": "fadeaway no short shot.",
    "action": "fadeaway",
    "augmentations": [
      {
        "new_transcript_id": "418_1",
        "label": "action_executed",
        "text": "He turned over his shoulder and hit a tough fadeaway jumper right before the buzzer."
      },
      {
        "new_transcript_id": "418_2",
        "label": "action_not_executed",
        "text": "Looked like he was setting for a fadeaway, but passed it off instead of taking the shot."
      }
    ]
  },
  {
    "transcript_id": 419,
    "text": "Post up opportunity.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "419_1",
        "label": "action_executed",
        "text": "He backed his defender down with a steady post up on the left block for two points."
      },
      {
        "new_transcript_id": "419_2",
        "label": "action_not_executed",
        "text": "He had position to post up, but quickly kicked it back out before making a move."
      }
    ]
  },
  {
    "transcript_id": 420,
    "text": "Hannah's double teams.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "420_1",
        "label": "action_executed",
        "text": "Hannah trapped the guard aggressively, springing a well-timed double team near half court."
      },
      {
        "new_transcript_id": "420_2",
        "label": "action_not_executed",
        "text": "They considered sending help, but no double team actually came on that play."
      }
    ]
  },
  {
    "transcript_id": 421,
    "text": "Here is Jokic little give and go inside to Millsap.",
    "action": "give and go",
    "augmentations": [
      {
        "new_transcript_id": "421_1",
        "label": "action_executed",
        "text": "Jokic handed it off, cut hard, and finished the give and go perfectly with Millsap."
      },
      {
        "new_transcript_id": "421_2",
        "label": "action_not_executed",
        "text": "They tried a give and go action, but the pass was deflected and the play broke down."
      }
    ]
  },
  {
    "transcript_id": 422,
    "text": "Joe puts a shoulder down pick and roll.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "422_1",
        "label": "action_executed",
        "text": "Joe initiated the pick and roll, used the screen, and attacked the lane decisively."
      },
      {
        "new_transcript_id": "422_2",
        "label": "action_not_executed",
        "text": "He called for a pick and roll but ended up resetting the offense without using it."
      }
    ]
  },
  {
    "transcript_id": 423,
    "text": "So here's that Lillard Nurkic kind of high pick and roll.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "423_1",
        "label": "action_executed",
        "text": "Lillard and Nurkic ran their signature high pick and roll, leading to a clean midrange look."
      },
      {
        "new_transcript_id": "423_2",
        "label": "action_not_executed",
        "text": "They lined up for the high pick and roll, but Nurkic slipped out early and no action developed."
      }
    ]
  },
  {
    "transcript_id": 424,
    "text": "Buccament with the floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "424_1",
        "label": "action_executed",
        "text": "Buccament floated it softly over the defender and dropped it through the net."
      },
      {
        "new_transcript_id": "424_2",
        "label": "action_not_executed",
        "text": "He looked to shoot a floater, but instead pulled it back out and reset the play."
      }
    ]
  },
  {
    "transcript_id": 425,
    "text": "floater is no good gets it back.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "425_1",
        "label": "action_executed",
        "text": "He tossed up a floater in the lane, missed, but hustled to grab the rebound."
      },
      {
        "new_transcript_id": "425_2",
        "label": "action_not_executed",
        "text": "He thought about a floater but dropped it off to a teammate cutting baseline."
      }
    ]
  },
  {
    "transcript_id": 426,
    "text": "Cook with the alley Oop and Jones with the throw down.",
    "action": "alley oop",
    "augmentations": [
      {
        "new_transcript_id": "426_1",
        "label": "action_executed",
        "text": "Cook lobbed it perfectly, and Jones soared for the alley oop dunk."
      },
      {
        "new_transcript_id": "426_2",
        "label": "action_not_executed",
        "text": "Cook tried to set up the alley oop, but the pass sailed out of bounds."
      }
    ]
  },
  {
    "transcript_id": 427,
    "text": "Again trying to muscle his way up despite the double team.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "427_1",
        "label": "action_executed",
        "text": "He backed down inside while facing a strong double team, forcing up a contested shot."
      },
      {
        "new_transcript_id": "427_2",
        "label": "action_not_executed",
        "text": "The defense showed help, but no real double team came and he scored easily."
      }
    ]
  },
  {
    "transcript_id": 428,
    "text": "Where he went coast to coast I kept.",
    "action": "coast to coast",
    "augmentations": [
      {
        "new_transcript_id": "428_1",
        "label": "action_not_executed",
        "text": "They mentioned going coast to coast, but he was stopped near midcourt and passed out."
      },
      {
        "new_transcript_id": "428_2",
        "label": "action_executed",
        "text": "He grabbed the rebound, sprinted coast to coast, and finished with a strong layup."
      }
    ]
  },
  {
    "transcript_id": 429,
    "text": "Conley floater Valentinas taps the rebound but cannot keep it in place.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "429_1",
        "label": "action_executed",
        "text": "Conley lofted a floater over the big man, missed, and Valentinas lost the rebound out."
      },
      {
        "new_transcript_id": "429_2",
        "label": "action_not_executed",
        "text": "Conley faked the floater, but pulled it back and dished it off to the wing instead."
      }
    ]
  },
  {
    "transcript_id": 430,
    "text": "Selden with a tough looking floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "430_1",
        "label": "action_executed",
        "text": "Selden elevated in traffic and launched a difficult floater over two defenders."
      },
      {
        "new_transcript_id": "430_2",
        "label": "action_not_executed",
        "text": "Selden thought about the floater, but swung it to the corner before taking the shot."
      }
    ]
  },
  {
    "transcript_id": 432,
    "text": "Done a nice little floater in and out.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "432_1",
        "label": "action_executed",
        "text": "He released a quick floater in the paint that rimmed out before being rebounded."
      },
      {
        "new_transcript_id": "432_2",
        "label": "action_not_executed",
        "text": "He started to gather for a floater but dropped the ball off to the trailing big."
      }
    ]
  },
  {
    "transcript_id": 433,
    "text": "Orange splits a double team works his way to the rim, and somehow finagles delay up out of that.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "433_1",
        "label": "action_executed",
        "text": "Orange sliced through the double team and converted a crafty layup at the rim."
      },
      {
        "new_transcript_id": "433_2",
        "label": "action_not_executed",
        "text": "He looked to split the double team but got tied up before getting a shot off."
      }
    ]
  },
  {
    "transcript_id": 434,
    "text": "Pascal jab step.",
    "action": "jab step",
    "augmentations": [
      {
        "new_transcript_id": "434_1",
        "label": "action_executed",
        "text": "Pascal faked with a jab step, creating space before attacking the basket."
      },
      {
        "new_transcript_id": "434_2",
        "label": "action_not_executed",
        "text": "He shaped up for a jab step but never actually made the move."
      }
    ]
  },
  {
    "transcript_id": 435,
    "text": "Nothing but net.",
    "action": "nothing but net",
    "augmentations": [
      {
        "new_transcript_id": "435_1",
        "label": "action_executed",
        "text": "He let it fly from deep, nothing but net on a clean swish."
      },
      {
        "new_transcript_id": "435_2",
        "label": "action_not_executed",
        "text": "The crowd expected nothing but net, but the shot clanked off iron instead."
      }
    ]
  },
  {
    "transcript_id": 436,
    "text": "Taking the floater in the Lane hits the backroom goes and gets his own miss.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "436_1",
        "label": "action_executed",
        "text": "He lofted a floater that rattled out, then chased it down for the rebound."
      },
      {
        "new_transcript_id": "436_2",
        "label": "action_not_executed",
        "text": "He started to float it up, but reset the offense without taking the shot."
      }
    ]
  },
  {
    "transcript_id": 437,
    "text": "Go coast to coast and flush it in.",
    "action": "coast to coast",
    "augmentations": [
      {
        "new_transcript_id": "437_1",
        "label": "action_executed",
        "text": "He went coast to coast off the rebound and hammered home the dunk."
      },
      {
        "new_transcript_id": "437_2",
        "label": "action_not_executed",
        "text": "He pushed it up but didn’t complete the coast to coast, pulling back near the arc."
      }
    ]
  },
  {
    "transcript_id": 438,
    "text": "Post up by Lukas right into the Lane, and hello, nothing like starting your game with an easy basket.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "438_1",
        "label": "action_executed",
        "text": "Lukas sealed inside, executed a strong post up, and scored with ease early."
      },
      {
        "new_transcript_id": "438_2",
        "label": "action_not_executed",
        "text": "He tried to establish post up position, but the defense forced him to kick it out."
      }
    ]
  },
  {
    "transcript_id": 439,
    "text": "Rage on college work the side pick and roll Tray gets into the paint at.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "439_1",
        "label": "action_executed",
        "text": "They ran the side pick and roll, freeing Trae to slice into the paint."
      },
      {
        "new_transcript_id": "439_2",
        "label": "action_not_executed",
        "text": "They looked to set the side pick and roll but aborted before the action developed."
      }
    ]
  },
  {
    "transcript_id": 440,
    "text": "And it's nothing but net and it's 2 to nothing.",
    "action": "nothing but net",
    "augmentations": [
      {
        "new_transcript_id": "440_1",
        "label": "action_executed",
        "text": "The jumper dropped clean, nothing but net, giving them the first points of the night."
      },
      {
        "new_transcript_id": "440_2",
        "label": "action_not_executed",
        "text": "He pulled up expecting nothing but net, but the ball rimmed out instead."
      }
    ]
  },
  {
    "transcript_id": 441,
    "text": "Great outlet pass to wonderful Feed and LeBron in stride target easily is the former Laker when telling 10 tell you what 60 of these 60 two shots had been.",
    "action": "outlet pass",
    "augmentations": [
      {
        "new_transcript_id": "441_1",
        "label": "action_not_executed",
        "text": "They mentioned an outlet pass opportunity, but the ball was never advanced quickly."
      },
      {
        "new_transcript_id": "441_2",
        "label": "action_executed",
        "text": "He fired a perfect outlet pass to LeBron, who caught it in stride for a finish."
      }
    ]
  },
  {
    "transcript_id": 442,
    "text": "Biggest challenge for him in the Smith Junior didn't get any rim that time on the little floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "442_1",
        "label": "action_executed",
        "text": "Smith Jr. lofted up a floater that missed everything, sailing over the rim."
      },
      {
        "new_transcript_id": "442_2",
        "label": "action_not_executed",
        "text": "Smith Jr. shaped up for a floater but passed the ball off instead."
      }
    ]
  },
  {
    "transcript_id": 443,
    "text": "Now Anderson has it pick and roll with Collins Swing and Swing to Prince.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "443_1",
        "label": "action_executed",
        "text": "Anderson ran the pick and roll with Collins and swung it to Prince in rhythm."
      },
      {
        "new_transcript_id": "443_2",
        "label": "action_not_executed",
        "text": "They lined up for the pick and roll, but Anderson skipped it across court without using it."
      }
    ]
  },
  {
    "transcript_id": 444,
    "text": "As a matter of fact, impellers tipped in the League now is 60 dumps.",
    "action": "tip in",
    "augmentations": [
      {
        "new_transcript_id": "444_1",
        "label": "action_not_executed",
        "text": "They referenced a tip in, but no second chance bucket actually happened on this play."
      },
      {
        "new_transcript_id": "444_2",
        "label": "action_executed",
        "text": "He grabbed the offensive board and finished it with a quick tip in over the defender."
      }
    ]
  },
  {
    "transcript_id": 445,
    "text": "James Wiseman complete go drop coverage but then get all the way out there because of his length and really contest that well so wise men little post up early.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "445_1",
        "label": "action_executed",
        "text": "Wiseman took early position inside and powered into a post up move on the block."
      },
      {
        "new_transcript_id": "445_2",
        "label": "action_not_executed",
        "text": "He set for a post up but quickly passed out when the double team arrived."
      }
    ]
  },
  {
    "transcript_id": 446,
    "text": "Former Rookie of the Year there was that double team and then we end up having the out of bounds violation.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "446_1",
        "label": "action_executed",
        "text": "They trapped him with a double team that led directly to a turnover on the sideline."
      },
      {
        "new_transcript_id": "446_2",
        "label": "action_not_executed",
        "text": "They prepared to send the double team but stayed home, forcing only a tough pass."
      }
    ]
  },
  {
    "transcript_id": 447,
    "text": "So Cousins ends up going one for three at the foul line, little floater for Brogdon.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "447_1",
        "label": "action_executed",
        "text": "Brogdon pulled up for a floater in the lane after Cousins’ free throws."
      },
      {
        "new_transcript_id": "447_2",
        "label": "action_not_executed",
        "text": "He lined up a floater but opted to swing it out to the perimeter instead."
      }
    ]
  },
  {
    "transcript_id": 448,
    "text": "So Cousins ends up going one for three at the foul line, little floater for Brogdon.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "448_1",
        "label": "action_executed",
        "text": "After Cousins’ trip to the line, Brogdon pushed into the paint and hit a floater."
      },
      {
        "new_transcript_id": "448_2",
        "label": "action_not_executed",
        "text": "They thought about Brogdon taking a floater, but he never released the shot."
      }
    ]
  },
  {
    "transcript_id": 449,
    "text": "No double team needed on that play.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "449_1",
        "label": "action_not_executed",
        "text": "The defense stayed home, no double team was necessary with the weak-side rotation ready."
      },
      {
        "new_transcript_id": "449_2",
        "label": "action_executed",
        "text": "They clamped down hard, sending a double team to force the ball out of his hands."
      }
    ]
  },
  {
    "transcript_id": 450,
    "text": "Oka 4 Double teamed immediately and he traveled.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "450_1",
        "label": "action_executed",
        "text": "Okafor was swarmed with a double team right away, leading to a travel violation."
      },
      {
        "new_transcript_id": "450_2",
        "label": "action_not_executed",
        "text": "He braced for a double team, but the defenders stayed home and no trap came."
      }
    ]
  },
  {
    "transcript_id": 451,
    "text": "Art van guarded by Butler puts up a floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "451_1",
        "label": "action_executed",
        "text": "He lifted a soft floater over Butler’s contest, right down the middle of the lane."
      },
      {
        "new_transcript_id": "451_2",
        "label": "action_not_executed",
        "text": "He looked to float it up, but swung the ball back out instead."
      }
    ]
  },
  {
    "transcript_id": 452,
    "text": "At his best, that's when he can be super effective when you can pick and roll to the basket.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "452_1",
        "label": "action_executed",
        "text": "He ran the pick and roll perfectly, turning the corner and finishing strong at the rim."
      },
      {
        "new_transcript_id": "452_2",
        "label": "action_not_executed",
        "text": "They talked about running the pick and roll, but the action didn’t happen there."
      }
    ]
  },
  {
    "transcript_id": 453,
    "text": "At his best, that's when he can be super effective when you can pick and roll to the basket.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "453_1",
        "label": "action_not_executed",
        "text": "They highlighted his ability in the pick and roll, but he didn’t use it that trip."
      },
      {
        "new_transcript_id": "453_2",
        "label": "action_executed",
        "text": "He and the screener executed a sharp pick and roll, slicing straight to the hoop."
      }
    ]
  },
  {
    "transcript_id": 454,
    "text": "At his best, that's when he can be super effective when you can pick and roll to the basket.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "454_1",
        "label": "action_executed",
        "text": "They ran the pick and roll to perfection, giving him an open driving lane."
      },
      {
        "new_transcript_id": "454_2",
        "label": "action_not_executed",
        "text": "The setup looked like a pick and roll, but he peeled off before using the screen."
      }
    ]
  },
  {
    "transcript_id": 455,
    "text": "At his best, that's when he can be super effective when you can pick and roll to the basket.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "455_1",
        "label": "action_not_executed",
        "text": "They described his skill in the pick and roll, but it wasn’t actually run this possession."
      },
      {
        "new_transcript_id": "455_2",
        "label": "action_executed",
        "text": "He called for the screen, ran the pick and roll, and attacked straight to the hoop."
      }
    ]
  },
  {
    "transcript_id": 456,
    "text": "Here's a 2 on one Millsap, the alley oop missed the layup and the other thing Mark just over a week ago.",
    "action": "alley oop",
    "augmentations": [
      {
        "new_transcript_id": "456_1",
        "label": "action_executed",
        "text": "Millsap went up for the alley oop on the break but couldn’t convert the layup."
      },
      {
        "new_transcript_id": "456_2",
        "label": "action_not_executed",
        "text": "They tried to connect on an alley oop, but the pass never reached him."
      }
    ]
  },
  {
    "transcript_id": 457,
    "text": "Ego doll and they double team.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "457_1",
        "label": "action_executed",
        "text": "They immediately swarmed Iguodala with a suffocating double team near the sideline."
      },
      {
        "new_transcript_id": "457_2",
        "label": "action_not_executed",
        "text": "They looked like they might double team him, but the help never came."
      }
    ]
  },
  {
    "transcript_id": 458,
    "text": "Outside for Derrick Rose little floater, from the Lane that won't fall.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "458_1",
        "label": "action_executed",
        "text": "Rose lofted a floater in the lane, but it rolled off the rim and out."
      },
      {
        "new_transcript_id": "458_2",
        "label": "action_not_executed",
        "text": "Rose set for a floater but kicked it out to the wing before releasing."
      }
    ]
  },
  {
    "transcript_id": 459,
    "text": "Now he's double teamed and now he's in trouble.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "459_1",
        "label": "action_executed",
        "text": "He was immediately trapped by the double team, leaving him with no passing lanes."
      },
      {
        "new_transcript_id": "459_2",
        "label": "action_not_executed",
        "text": "The defense threatened a double team, but stayed home, keeping him one-on-one."
      }
    ]
  },
  {
    "transcript_id": 460,
    "text": "Here's the floater again by Brockton.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "460_1",
        "label": "action_executed",
        "text": "Brogdon rose in the lane and softly floated it toward the rim again."
      },
      {
        "new_transcript_id": "460_2",
        "label": "action_not_executed",
        "text": "He shaped to shoot a floater but chose to dump it off to the corner instead."
      }
    ]
  },
  {
    "transcript_id": 461,
    "text": "Lori left wing floater misses.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "461_1",
        "label": "action_executed",
        "text": "From the left wing, Lauri launched a floater that rolled off the rim no good."
      },
      {
        "new_transcript_id": "461_2",
        "label": "action_not_executed",
        "text": "He faked a floater on the left wing but pulled it out to reset the play."
      }
    ]
  },
  {
    "transcript_id": 462,
    "text": "Pick and roll.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "462_1",
        "label": "action_executed",
        "text": "They executed a simple pick and roll, freeing the guard to attack the paint."
      },
      {
        "new_transcript_id": "462_2",
        "label": "action_not_executed",
        "text": "They called for the pick and roll but never actually used the screen that possession."
      }
    ]
  },
  {
    "transcript_id": 463,
    "text": "Course recently took fighter to second straight Slam Dunk contest coming in second.",
    "action": "slam dunk",
    "augmentations": [
      {
        "new_transcript_id": "463_1",
        "label": "action_not_executed",
        "text": "They mentioned his slam dunk contest history, but no dunk happened during this play."
      },
      {
        "new_transcript_id": "463_2",
        "label": "action_executed",
        "text": "He soared through the lane and threw down a thunderous slam dunk in traffic."
      }
    ]
  },
  {
    "transcript_id": 464,
    "text": "Transition defense would be very important and this is one of the reasons why, because they got five guys, all of them capable of taking it coast to coast.",
    "action": "coast to coast",
    "augmentations": [
      {
        "new_transcript_id": "464_1",
        "label": "action_executed",
        "text": "Off the board, he sprinted coast to coast and finished strong at the rim."
      },
      {
        "new_transcript_id": "464_2",
        "label": "action_not_executed",
        "text": "They highlighted that he could go coast to coast, but on this trip he slowed it down."
      }
    ]
  },
  {
    "transcript_id": 465,
    "text": "Danny Green Jays swish",
    "action": "swish",
    "augmentations": [
      {
        "new_transcript_id": "465_1",
        "label": "action_executed",
        "text": "Danny Green caught it in rhythm and drained a jumper, pure swish."
      },
      {
        "new_transcript_id": "465_2",
        "label": "action_not_executed",
        "text": "The shot from Green looked good but rattled out, not the swish the crowd wanted."
      }
    ]
  },
  {
    "transcript_id": 466,
    "text": "Thank you, Rudy, but then promises them both and they get the easy tip in to pull within two 9189 Spurs answer.",
    "action": "tip in",
    "augmentations": [
      {
        "new_transcript_id": "466_1",
        "label": "action_not_executed",
        "text": "They referenced a tip in chance, but the Spurs couldn’t finish the second opportunity."
      },
      {
        "new_transcript_id": "466_2",
        "label": "action_executed",
        "text": "Gobert grabbed the offensive rebound and tipped it back in to cut the deficit."
      }
    ]
  },
  {
    "transcript_id": 467,
    "text": "Passes from Blake Griffin who draws a double team in the post.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "467_1",
        "label": "action_not_executed",
        "text": "Griffin pivots in the paint, defenders lean toward him but no full double team is committed."
      },
      {
        "new_transcript_id": "467_2",
        "label": "action_executed",
        "text": "Blake Griffin draws an aggressive double team on the block, forcing him to pass out to the corner."
      }
    ]
  },
  {
    "transcript_id": 468,
    "text": "He can stretch you out, but also he is one of the best and being active around the glass and in the pick and roll.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "468_1",
        "label": "action_not_executed",
        "text": "Commentary praises his activity in pick and roll sets, though he isn’t executing one at this moment."
      },
      {
        "new_transcript_id": "468_2",
        "label": "action_executed",
        "text": "He sets the high screen, rolls decisively, and completes the pick and roll with a layup at the rim."
      }
    ]
  },
  {
    "transcript_id": 469,
    "text": "And the tip in the backcourt to Derek White.",
    "action": "tip in",
    "augmentations": [
      {
        "new_transcript_id": "469_1",
        "label": "action_executed",
        "text": "White follows the miss and secures two points with a clean tip in under pressure."
      },
      {
        "new_transcript_id": "469_2",
        "label": "action_not_executed",
        "text": "White stretches for the tip in but the ball caroms away, no finish this time."
      }
    ]
  },
  {
    "transcript_id": 470,
    "text": "Sad OK for a late double team.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "470_1",
        "label": "action_not_executed",
        "text": "Defenders consider a late double team but hesitate, leaving him single covered throughout the possession."
      },
      {
        "new_transcript_id": "470_2",
        "label": "action_executed",
        "text": "A second defender arrives belatedly, forming a late double team in the post."
      }
    ]
  },
  {
    "transcript_id": 471,
    "text": "Yeah, what you gotta do if your memory is gotta try to work him out a little bit and then the double team has to come.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "471_1",
        "label": "action_not_executed",
        "text": "Commentary notes the double team should come eventually, but no second defender actually shows up on this trip."
      },
      {
        "new_transcript_id": "471_2",
        "label": "action_executed",
        "text": "As he backs down, a second defender rushes in to enforce the double team and force the pass out."
      }
    ]
  },
  {
    "transcript_id": 472,
    "text": "I did buy into the cool blow on a switch now Porzingis posting up turn around J fuel.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "472_1",
        "label": "action_executed",
        "text": "Porzingis gets deep position, posts up strong, and drains a turnaround jumper over his man."
      },
      {
        "new_transcript_id": "472_2",
        "label": "action_not_executed",
        "text": "Porzingis starts to post up, but the ball swings out before he makes a move."
      }
    ]
  },
  {
    "transcript_id": 473,
    "text": "He's a stretch for he can post up smaller players.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "473_1",
        "label": "action_not_executed",
        "text": "Analysts highlight he can post up mismatches, but he doesn’t go to the block this time."
      },
      {
        "new_transcript_id": "473_2",
        "label": "action_executed",
        "text": "Matched on a guard, he quickly posts up and powers in for two points inside."
      }
    ]
  },
  {
    "transcript_id": 474,
    "text": "The floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "474_1",
        "label": "action_executed",
        "text": "He drives into the lane and tosses up a soft floater that kisses off the glass."
      },
      {
        "new_transcript_id": "474_2",
        "label": "action_not_executed",
        "text": "He fakes like he’ll shoot a floater, but instead resets and kicks to the wing."
      }
    ]
  },
  {
    "transcript_id": 475,
    "text": "Milwaukee's basketball first holiday, with a little floater from the Lane through holiday, fills it up.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "475_1",
        "label": "action_executed",
        "text": "Holiday drifts into the paint and releases a controlled floater that falls straight through."
      },
      {
        "new_transcript_id": "475_2",
        "label": "action_not_executed",
        "text": "Holiday looked to rise for the floater but dishes outside before getting the shot away."
      }
    ]
  },
  {
    "transcript_id": 476,
    "text": "Out of a double team.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "476_1",
        "label": "action_executed",
        "text": "Trapped on the baseline, he whips a pass out of the strong double team closing in."
      },
      {
        "new_transcript_id": "476_2",
        "label": "action_not_executed",
        "text": "The defense feints toward a trap, but no true double team materializes this trip."
      }
    ]
  },
  {
    "transcript_id": 477,
    "text": "You get an easy dunk poster.",
    "action": "poster",
    "augmentations": [
      {
        "new_transcript_id": "477_1",
        "label": "action_not_executed",
        "text": "He rises looking for a poster dunk, but the defender contests and the attempt never finishes."
      },
      {
        "new_transcript_id": "477_2",
        "label": "action_executed",
        "text": "He elevates strong over the defender and slams home a poster dunk that energizes the crowd."
      }
    ]
  },
  {
    "transcript_id": 478,
    "text": "Flopping was back to his high school days in Louisiana.",
    "action": "flop",
    "augmentations": [
      {
        "new_transcript_id": "478_1",
        "label": "action_not_executed",
        "text": "They mention his old flopping habits, but no actual flop occurs during this possession."
      },
      {
        "new_transcript_id": "478_2",
        "label": "action_executed",
        "text": "He dramatically falls to the hardwood, selling contact for the flop call."
      }
    ]
  },
  {
    "transcript_id": 479,
    "text": "Simmons misses the finger roll.",
    "action": "finger roll",
    "augmentations": [
      {
        "new_transcript_id": "479_1",
        "label": "action_executed",
        "text": "Simmons glides into the lane, tries the finger roll, but the shot spins out."
      },
      {
        "new_transcript_id": "479_2",
        "label": "action_not_executed",
        "text": "Simmons looked like he might attempt a finger roll but adjusted midair and pulled it back."
      }
    ]
  },
  {
    "transcript_id": 480,
    "text": "Patients are at the pump fake of the flyby.",
    "action": "pump fake",
    "augmentations": [
      {
        "new_transcript_id": "480_1",
        "label": "action_executed",
        "text": "He sells the pump fake, sending the defender flying past before resetting his feet."
      },
      {
        "new_transcript_id": "480_2",
        "label": "action_not_executed",
        "text": "He thinks about using the pump fake, but opts for a quick release instead."
      }
    ]
  },
  {
    "transcript_id": 481,
    "text": "His fadeaway gun won't go.",
    "action": "fadeaway",
    "augmentations": [
      {
        "new_transcript_id": "481_1",
        "label": "action_executed",
        "text": "He turns and takes the fadeaway jumper, but the shot rims out to the side."
      },
      {
        "new_transcript_id": "481_2",
        "label": "action_not_executed",
        "text": "He sets for a fadeaway but is fouled before getting the attempt off."
      }
    ]
  },
  {
    "transcript_id": 482,
    "text": "Double team that time got it back for blood so this time.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "482_1",
        "label": "action_executed",
        "text": "He gets trapped in a double team but fights through to recover the ball and reset the play."
      },
      {
        "new_transcript_id": "482_2",
        "label": "action_not_executed",
        "text": "The defense shows like they’ll double team, but peel away before making the trap real."
      }
    ]
  },
  {
    "transcript_id": 483,
    "text": "Will it will bounce Modert double team went between defenders had it ripped and they say he walks to the basketball.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "483_1",
        "label": "action_executed",
        "text": "Caught in the double team, he dribbles through traffic, loses it, and is whistled for traveling."
      },
      {
        "new_transcript_id": "483_2",
        "label": "action_not_executed",
        "text": "He thought a double team was coming, but no second defender commits before the turnover."
      }
    ]
  },
  {
    "transcript_id": 484,
    "text": "There's double team paper tries to turn the corner.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "484_1",
        "label": "action_executed",
        "text": "He’s caught in a double team on the wing, forcing him to retreat and swing the ball away."
      },
      {
        "new_transcript_id": "484_2",
        "label": "action_not_executed",
        "text": "He anticipates the double team, but the help defender holds off and never fully traps."
      }
    ]
  },
  {
    "transcript_id": 485,
    "text": "Little weird awkward floater so Bowman 41% on threes.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "485_1",
        "label": "action_not_executed",
        "text": "He looked ready for a floater but never releases it, pulling back for a reset instead."
      },
      {
        "new_transcript_id": "485_2",
        "label": "action_executed",
        "text": "He awkwardly tosses up the floater in the lane and it bounces off the rim."
      }
    ]
  },
  {
    "transcript_id": 486,
    "text": "Or is that low pick and roll?",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "486_1",
        "label": "action_executed",
        "text": "He sets the screen, rolls to the low block, and the guard feeds him on the pick and roll."
      },
      {
        "new_transcript_id": "486_2",
        "label": "action_not_executed",
        "text": "They discussed running a low pick and roll, but the offense resets without running it."
      }
    ]
  },
  {
    "transcript_id": 487,
    "text": "He's written a check for the League office for flopping a couple of times.",
    "action": "flop",
    "augmentations": [
      {
        "new_transcript_id": "487_1",
        "label": "action_not_executed",
        "text": "Mention of past flopping fines, but on this play he stands his ground with no flop attempt."
      },
      {
        "new_transcript_id": "487_2",
        "label": "action_executed",
        "text": "He exaggerates the contact and falls to the court, selling the flop to the officials."
      }
    ]
  },
  {
    "transcript_id": 488,
    "text": "Is Derek with a hot floater cut the rim?",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "488_1",
        "label": "action_executed",
        "text": "Derrick drives the lane and floats it high, the ball kissing the rim and dropping through."
      },
      {
        "new_transcript_id": "488_2",
        "label": "action_not_executed",
        "text": "Derrick approaches like he’ll shoot the floater, but pulls back and kicks to a teammate instead."
      }
    ]
  },
  {
    "transcript_id": 490,
    "text": "Now he's getting more post up basement.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "490_1",
        "label": "action_not_executed",
        "text": "They emphasize his increased post up opportunities, though none occur on this sequence."
      },
      {
        "new_transcript_id": "490_2",
        "label": "action_executed",
        "text": "He seals his man and executes a post up on the left block, powering to the rim."
      }
    ]
  },
  {
    "transcript_id": 491,
    "text": "Now those floaters not falling.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "491_1",
        "label": "action_executed",
        "text": "He tosses up another floater, but once again it clangs off the iron."
      },
      {
        "new_transcript_id": "491_2",
        "label": "action_not_executed",
        "text": "The talk centers on floaters not going in, but on this possession no floater was taken."
      }
    ]
  },
  {
    "transcript_id": 492,
    "text": "Again, to take it off to a quick start yesterday throwing up alley oop that doesn't workout is picked up.",
    "action": "alley oop",
    "augmentations": [
      {
        "new_transcript_id": "492_1",
        "label": "action_not_executed",
        "text": "They attempt an alley oop lob early, but it’s off target and picked off by the defense."
      },
      {
        "new_transcript_id": "492_2",
        "label": "action_executed",
        "text": "He connects on the lob pass and finishes the alley oop emphatically with two hands."
      }
    ]
  },
  {
    "transcript_id": 493,
    "text": "Again, to take it off to a quick start yesterday throwing up alley oop that doesn't workout is picked up.",
    "action": "alley oop",
    "augmentations": [
      {
        "new_transcript_id": "493_1",
        "label": "action_not_executed",
        "text": "Another failed lob attempt, the alley oop doesn’t connect and the ball goes to the other team."
      },
      {
        "new_transcript_id": "493_2",
        "label": "action_executed",
        "text": "The lob is timed perfectly this time, and the alley oop is flushed down with authority."
      }
    ]
  },
  {
    "transcript_id": 494,
    "text": "With Chester Turner pump faking Mr. Doe Bentley is love.",
    "action": "pump fake",
    "augmentations": [
      {
        "new_transcript_id": "494_1",
        "label": "action_executed",
        "text": "Turner fakes the shot with a convincing pump fake, sending his defender leaping past."
      },
      {
        "new_transcript_id": "494_2",
        "label": "action_not_executed",
        "text": "Turner sizes up like he’ll pump fake, but lets the shot fly immediately instead."
      }
    ]
  },
   {
    "transcript_id": 495,
    "text": "Mark the size the length theory is trying to get the alley oop pass.",
    "action": "alley oop",
    "augmentations": [
      {
        "new_transcript_id": "495_1",
        "label": "action_executed",
        "text": "He elevates to meet the lob at the rim and throws down the alley oop finish in traffic."
      },
      {
        "new_transcript_id": "495_2",
        "label": "action_not_executed",
        "text": "He rises expecting the alley oop, but the pass sails long and no connection is made."
      }
    ]
  },
  {
    "transcript_id": 496,
    "text": "Look at splash.",
    "action": "splash",
    "augmentations": [
      {
        "new_transcript_id": "496_1",
        "label": "action_executed",
        "text": "He steps behind the arc and buries the jumper clean, nothing but splash from deep."
      },
      {
        "new_transcript_id": "496_2",
        "label": "action_not_executed",
        "text": "He lines up a long three, but the shot clangs hard off the back iron instead of a splash."
      }
    ]
  },
  {
    "transcript_id": 497,
    "text": "Shock like down to five Levine looking to create Ombre tomb shuffles to his right, fires three and splashes intends to be hot.",
    "action": "splash",
    "augmentations": [
      {
        "new_transcript_id": "497_1",
        "label": "action_executed",
        "text": "With the clock winding down, Levine rises from deep and drains the three for a splash."
      },
      {
        "new_transcript_id": "497_2",
        "label": "action_not_executed",
        "text": "Levine attempts the three at the buzzer, but the shot bounces out, no splash this time."
      }
    ]
  },
  {
    "transcript_id": 498,
    "text": "No double team help cover sets up the jump book.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "498_1",
        "label": "action_not_executed",
        "text": "He faces single coverage the whole possession, as no double team help ever comes."
      },
      {
        "new_transcript_id": "498_2",
        "label": "action_executed",
        "text": "Suddenly the weak side defender joins, creating a firm double team on the ballhandler."
      }
    ]
  },
  {
    "transcript_id": 499,
    "text": "One turnovers out of Houston, but you gotta convert off of it as K CP, nothing but net.",
    "action": "nothing but net",
    "augmentations": [
      {
        "new_transcript_id": "499_1",
        "label": "action_executed",
        "text": "KCP pulls up in transition and nails the jumper, pure nothing but net."
      },
      {
        "new_transcript_id": "499_2",
        "label": "action_not_executed",
        "text": "KCP lines it up but the shot rattles out, denying the clean nothing but net."
      }
    ]
  },
  {
    "transcript_id": 500,
    "text": "Double team.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "500_1",
        "label": "action_executed",
        "text": "The ballhandler is immediately swarmed by a strong double team at the top of the key."
      },
      {
        "new_transcript_id": "500_2",
        "label": "action_not_executed",
        "text": "The defense signals for a double team, but the second defender never actually commits."
      }
    ]
  },
  {
    "transcript_id": 501,
    "text": "There's a turnover block Coast to coast.",
    "action": "coast to coast",
    "augmentations": [
      {
        "new_transcript_id": "501_1",
        "label": "action_executed",
        "text": "After the steal, he races coast to coast, finishing at the rim before defenders can react."
      },
      {
        "new_transcript_id": "501_2",
        "label": "action_not_executed",
        "text": "He tries to go coast to coast after the turnover, but the defense stops him short of the rim."
      }
    ]
  },
  {
    "transcript_id": 502,
    "text": "Dennis had a 2 for two at the line in the first app button over to their top for the alley Oop.",
    "action": "alley oop",
    "augmentations": [
      {
        "new_transcript_id": "502_1",
        "label": "action_executed",
        "text": "The lob is placed perfectly and the alley oop is hammered down above the rim."
      },
      {
        "new_transcript_id": "502_2",
        "label": "action_not_executed",
        "text": "The alley oop pass is off target, clanking out of bounds before the dunk attempt."
      }
    ]
  },
  {
    "transcript_id": 503,
    "text": "There is that Birds Eye view of the pick and roll in the Sixers, defending it late rotation.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "503_1",
        "label": "action_executed",
        "text": "They run the pick and roll high, forcing the Sixers into a late defensive rotation."
      },
      {
        "new_transcript_id": "503_2",
        "label": "action_not_executed",
        "text": "The discussion highlights their pick and roll schemes, but they don’t actually execute one this trip."
      }
    ]
  },
  {
    "transcript_id": 504,
    "text": "He'll take it to the rack for this delay up, but Ruby is there for the put back.",
    "action": "take it to the rack",
    "augmentations": [
      {
        "new_transcript_id": "504_1",
        "label": "action_executed",
        "text": "He drives strong and takes it to the rack, absorbing contact before Ruby cleans up the miss."
      },
      {
        "new_transcript_id": "504_2",
        "label": "action_not_executed",
        "text": "He looks like he’ll take it to the rack, but pulls up short and passes instead."
      }
    ]
  },
  {
    "transcript_id": 505,
    "text": "Let's bear get that easy to CJ McCollum, the floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "505_1",
        "label": "action_executed",
        "text": "McCollum floats it up softly over the defender, the floater bouncing gently through."
      },
      {
        "new_transcript_id": "505_2",
        "label": "action_not_executed",
        "text": "McCollum threatens to shoot the floater but swings it to the corner instead."
      }
    ]
  },
  {
    "transcript_id": 506,
    "text": "That was a teardrop jumper, but",
    "action": "teardrop",
    "augmentations": [
      {
        "new_transcript_id": "506_1",
        "label": "action_not_executed",
        "text": "It looked like he was setting for a teardrop, but he never got the shot off cleanly."
      },
      {
        "new_transcript_id": "506_2",
        "label": "action_executed",
        "text": "He pushes into the lane and arcs the teardrop over the big man for two."
      }
    ]
  },
  {
    "transcript_id": 507,
    "text": "No double team help.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "507_1",
        "label": "action_not_executed",
        "text": "He faces the defender straight up, as no double team help arrives from the weak side."
      },
      {
        "new_transcript_id": "507_2",
        "label": "action_executed",
        "text": "Eventually the weak side defender collapses, creating a surprise double team late in the clock."
      }
    ]
  },
  {
    "transcript_id": 508,
    "text": "Horford looking to post up and they're gonna file.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "508_1",
        "label": "action_executed",
        "text": "Horford establishes position and begins a post up move before drawing the foul inside."
      },
      {
        "new_transcript_id": "508_2",
        "label": "action_not_executed",
        "text": "Horford prepares to post up, but the ball swings away before he makes a move."
      }
    ]
  },
  {
    "transcript_id": 509,
    "text": "How do you stop hard not to pick and roll as soon as someone shows me weather travel to go under.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "509_1",
        "label": "action_not_executed",
        "text": "They discuss strategies for defending the pick and roll, but no set is run at this moment."
      },
      {
        "new_transcript_id": "509_2",
        "label": "action_executed",
        "text": "He sets the screen, dives hard, and the guard delivers to complete the pick and roll."
      }
    ]
  },
  {
    "transcript_id": 510,
    "text": "They double team play closer to four court nearly threw it away but didn't, and Luke Kennard is fouled.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "510_1",
        "label": "action_executed",
        "text": "The defense pressures with a double team near half court, nearly forcing the turnover."
      },
      {
        "new_transcript_id": "510_2",
        "label": "action_not_executed",
        "text": "They look ready to double team, but back off before actually committing to the trap."
      }
    ]
  },
  {
    "transcript_id": 511,
    "text": "Notch double team five on the shot Clock to double team by Tiegs.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "511_1",
        "label": "action_executed",
        "text": "With the shot clock winding, Tiegs applies a hard double team that forces the ball out."
      },
      {
        "new_transcript_id": "511_2",
        "label": "action_not_executed",
        "text": "Tiegs looks like he will trap, but retreats, leaving no real double team formed."
      }
    ]
  },
  {
    "transcript_id": 512,
    "text": "Notch double team five on the shot Clock to double team by Tiegs.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "512_1",
        "label": "action_executed",
        "text": "The shot clock at five, Tiegs joins in a smothering double team near the sideline."
      },
      {
        "new_transcript_id": "512_2",
        "label": "action_not_executed",
        "text": "The defense bluffs a double team, but Tiegs pulls back and stays with his man."
      }
    ]
  },
  {
    "transcript_id": 513,
    "text": "Double teamed and Lowry turns it over and the Wizards get the ball.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "513_1",
        "label": "action_executed",
        "text": "Lowry is swarmed in a double team, coughs it up, and Washington capitalizes on the turnover."
      },
      {
        "new_transcript_id": "513_2",
        "label": "action_not_executed",
        "text": "Lowry anticipates a double team but no trap actually comes, and he misplays the ball himself."
      }
    ]
  },
  {
    "transcript_id": 514,
    "text": "He thought he was gonna pass it, but good recognition there and knocked down a tough little floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "514_1",
        "label": "action_not_executed",
        "text": "He fakes a floater attempt, dishes instead, and never actually gets the floater off."
      },
      {
        "new_transcript_id": "514_2",
        "label": "action_executed",
        "text": "He fools the defense and then knocks down a tough floater over the help defender."
      }
    ]
  },
  {
    "transcript_id": 515,
    "text": "Get that soft left hand floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "515_1",
        "label": "action_not_executed",
        "text": "He looked ready to loft the left hand floater but chooses a pass out to the corner."
      },
      {
        "new_transcript_id": "515_2",
        "label": "action_executed",
        "text": "He rises in the paint and drops in a soft left hand floater over the rim."
      }
    ]
  },
  {
    "transcript_id": 516,
    "text": "Will pick and roll with Lowry.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "516_1",
        "label": "action_executed",
        "text": "They run the pick and roll with Lowry handling, the screener diving for an easy look."
      },
      {
        "new_transcript_id": "516_2",
        "label": "action_not_executed",
        "text": "They motion toward a pick and roll with Lowry, but reset without actually running it."
      }
    ]
  },
  {
    "transcript_id": 517,
    "text": "Harold, trying to post up Marvin Kimble comes over to help out breaks it away.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "517_1",
        "label": "action_executed",
        "text": "Harrell establishes position on the block, starts a post up, but loses it to the help defender."
      },
      {
        "new_transcript_id": "517_2",
        "label": "action_not_executed",
        "text": "He looked for the post up entry, but the ball was poked away before he could initiate."
      }
    ]
  },
  {
    "transcript_id": 518,
    "text": "By Looney there's a double team we talked about right away.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "518_1",
        "label": "action_executed",
        "text": "Looney instantly draws the double team near the nail, just as we mentioned at the tip."
      },
      {
        "new_transcript_id": "518_2",
        "label": "action_not_executed",
        "text": "Looney looked ready for a double team, but defenders stayed home and never sent the extra pressure."
      }
    ]
  },
  {
    "transcript_id": 519,
    "text": "Couple pump fakes and then the finish.",
    "action": "pump fake",
    "augmentations": [
      {
        "new_transcript_id": "519_1",
        "label": "action_not_executed",
        "text": "He rises immediately for the jumper, skipping any pump fake before knocking it down clean."
      },
      {
        "new_transcript_id": "519_2",
        "label": "action_executed",
        "text": "With defenders flying by, he delivers two crafty pump fakes before laying it softly off the glass."
      }
    ]
  },
  {
    "transcript_id": 520,
    "text": "Accepting the double teamed be coming more than leader.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "520_1",
        "label": "action_not_executed",
        "text": "He anticipates the double team but no second defender comes, leaving him isolated on the wing."
      },
      {
        "new_transcript_id": "520_2",
        "label": "action_executed",
        "text": "As he drives baseline, the defense collapses quickly, swarming him with a hard double team."
      }
    ]
  },
  {
    "transcript_id": 521,
    "text": "Toronto Raptors saying he's guarding the pick and roll and the ref says no.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "521_1",
        "label": "action_not_executed",
        "text": "They planned to run a pick and roll, but the whistle stopped play before the screen was set."
      },
      {
        "new_transcript_id": "521_2",
        "label": "action_executed",
        "text": "Lowry sets up high, calls for the screen, and they execute a smooth pick and roll action."
      }
    ]
  },
  {
    "transcript_id": 522,
    "text": "We got double team.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "522_1",
        "label": "action_executed",
        "text": "The ball handler crosses midcourt and immediately draws the double team near the sideline."
      },
      {
        "new_transcript_id": "522_2",
        "label": "action_not_executed",
        "text": "He thought the trap was coming, but both defenders stayed single coverage with no double team."
      }
    ]
  },
  {
    "transcript_id": 523,
    "text": "Mr Floater rebounded Brown.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "523_1",
        "label": "action_executed",
        "text": "Brown snatches the rebound after the soft floater rims out at the free-throw line."
      },
      {
        "new_transcript_id": "523_2",
        "label": "action_not_executed",
        "text": "He drove inside but opted for a straight layup, never pulling up for a floater this trip."
      }
    ]
  },
  {
    "transcript_id": 524,
    "text": "Bans the roll man with a nice finger roll inside in Boston.",
    "action": "finger roll",
    "augmentations": [
      {
        "new_transcript_id": "524_1",
        "label": "action_executed",
        "text": "He glides past the defender and finishes a smooth finger roll right over the rim."
      },
      {
        "new_transcript_id": "524_2",
        "label": "action_not_executed",
        "text": "Cutting into the lane, he considered a finger roll but settled for a power layup instead."
      }
    ]
  },
  {
    "transcript_id": 525,
    "text": "And that's his first, the Knicks are all heavy post up team.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "525_1",
        "label": "action_not_executed",
        "text": "The Knicks emphasize post ups in their system, but none occurred on this possession."
      },
      {
        "new_transcript_id": "525_2",
        "label": "action_executed",
        "text": "Randall backs his man down on the block, establishing deep position for a strong post up."
      }
    ]
  },
  {
    "transcript_id": 526,
    "text": "Little pump fake holiday corner three it's true at the expiration of the shot Clock.",
    "action": "pump fake",
    "augmentations": [
      {
        "new_transcript_id": "526_1",
        "label": "action_executed",
        "text": "Holiday sells the corner pump fake, sidesteps the defender, and buries the three as the clock expires."
      },
      {
        "new_transcript_id": "526_2",
        "label": "action_not_executed",
        "text": "Holiday rises immediately for the corner triple without using a pump fake at all."
      }
    ]
  },
  {
    "transcript_id": 527,
    "text": "Then what he misses on the floater?",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "527_1",
        "label": "action_executed",
        "text": "Dinwiddie drives inside and puts up the floater, but it bounces off the rim no good."
      },
      {
        "new_transcript_id": "527_2",
        "label": "action_not_executed",
        "text": "He thought about a floater in the lane, but instead pulled back for a mid-range jumper."
      }
    ]
  },
  {
    "transcript_id": 528,
    "text": "For Maps whip it around Burke to try floater on the baseline.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "528_1",
        "label": "action_executed",
        "text": "Burke gets the swing pass, attacks baseline, and lofts up a floater that drops softly in."
      },
      {
        "new_transcript_id": "528_2",
        "label": "action_not_executed",
        "text": "Burke fakes a baseline floater attempt but dishes out to the corner for a three instead."
      }
    ]
  },
  {
    "transcript_id": 529,
    "text": "Kindly pick and roll valentinas attacks.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "529_1",
        "label": "action_executed",
        "text": "They execute the pick and roll cleanly, and Valanciunas rolls hard to the rim for a finish."
      },
      {
        "new_transcript_id": "529_2",
        "label": "action_not_executed",
        "text": "They set up as if to run the pick and roll, but the ball handler pulls it back out."
      }
    ]
  },
  {
    "transcript_id": 530,
    "text": "Jonas Valance Eunice with a slam dunk.",
    "action": "slam dunk",
    "augmentations": [
      {
        "new_transcript_id": "530_1",
        "label": "action_executed",
        "text": "Valanciunas powers down the lane and punctuates it with a ferocious slam dunk."
      },
      {
        "new_transcript_id": "530_2",
        "label": "action_not_executed",
        "text": "He rose up for what looked like a slam dunk, but lost control and laid it in instead."
      }
    ]
  },
  {
    "transcript_id": 531,
    "text": "Would look at the angle on Atom 3 point swish Lopez.",
    "action": "swish",
    "augmentations": [
      {
        "new_transcript_id": "531_1",
        "label": "action_executed",
        "text": "Lopez drains the triple clean, nothing but net on that smooth swish."
      },
      {
        "new_transcript_id": "531_2",
        "label": "action_not_executed",
        "text": "The shot looked perfect but rattled out, no swish on that release."
      }
    ]
  },
  {
    "transcript_id": 532,
    "text": "On that play, you have a double team on Lamarcus Aldridge, an.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "532_1",
        "label": "action_executed",
        "text": "Two defenders immediately close on Aldridge, forcing him to pass out of the double team."
      },
      {
        "new_transcript_id": "532_2",
        "label": "action_not_executed",
        "text": "The defense shaded toward Aldridge, but never actually committed a double team."
      }
    ]
  },
  {
    "transcript_id": 533,
    "text": "Early post up for Wiggins.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "533_1",
        "label": "action_executed",
        "text": "Wiggins establishes position deep in the paint and begins the post up attack."
      },
      {
        "new_transcript_id": "533_2",
        "label": "action_not_executed",
        "text": "They looked to set Wiggins on the block, but he never executed the post up."
      }
    ]
  },
  {
    "transcript_id": 534,
    "text": "Andre Drummond with the tip in.",
    "action": "tip in",
    "augmentations": [
      {
        "new_transcript_id": "534_1",
        "label": "action_executed",
        "text": "Drummond skies for the offensive rebound and tips it back in instantly."
      },
      {
        "new_transcript_id": "534_2",
        "label": "action_not_executed",
        "text": "Drummond reached for the rebound but missed the chance for a tip in."
      }
    ]
  },
  {
    "transcript_id": 535,
    "text": "Here's Neil Aquino posting up.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "535_1",
        "label": "action_executed",
        "text": "Ntilikina plants himself on the block and starts a steady post up sequence."
      },
      {
        "new_transcript_id": "535_2",
        "label": "action_not_executed",
        "text": "Ntilikina drifted toward the post but never truly executed a post up move."
      }
    ]
  },
  {
    "transcript_id": 536,
    "text": "Port is trying to post up.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "536_1",
        "label": "action_executed",
        "text": "Portis gets into the paint, plants strong, and executes a deliberate post up."
      },
      {
        "new_transcript_id": "536_2",
        "label": "action_not_executed",
        "text": "Portis tried to establish inside position but the entry pass never came, no post up."
      }
    ]
  },
  {
    "transcript_id": 537,
    "text": "Daniel, get the start again tonight on the finger roll.",
    "action": "finger roll",
    "augmentations": [
      {
        "new_transcript_id": "537_1",
        "label": "action_executed",
        "text": "Daniel glides past the defender and lays it in with a soft finger roll."
      },
      {
        "new_transcript_id": "537_2",
        "label": "action_not_executed",
        "text": "Daniel looked to attempt a finger roll but was stripped before releasing the shot."
      }
    ]
  },
  {
    "transcript_id": 538,
    "text": "Nichols draws a double team.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "538_1",
        "label": "action_executed",
        "text": "Nichols attracts two defenders as they converge hard, forming the double team on the wing."
      },
      {
        "new_transcript_id": "538_2",
        "label": "action_not_executed",
        "text": "Nichols looked ready to be doubled, but the second defender never left his man."
      }
    ]
  },
  {
    "transcript_id": 539,
    "text": "This no good drum and Brown gobbles up the rebound long outlet pass.",
    "action": "outlet pass",
    "augmentations": [
      {
        "new_transcript_id": "539_1",
        "label": "action_executed",
        "text": "Brown secures the board and instantly fires a perfect outlet pass up the floor."
      },
      {
        "new_transcript_id": "539_2",
        "label": "action_not_executed",
        "text": "Brown corrals the rebound but holds the ball instead of making an outlet pass."
      }
    ]
  },
  {
    "transcript_id": 540,
    "text": "Off the Adams screen, Alexander Walker, with a floater in it's for nothing.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "540_1",
        "label": "action_executed",
        "text": "Walker curls off the Adams pick and drops in a smooth floater to extend the lead."
      },
      {
        "new_transcript_id": "540_2",
        "label": "action_not_executed",
        "text": "He drove hard off the Adams screen but kicked it out instead of taking a floater."
      }
    ]
  },
  {
    "transcript_id": 541,
    "text": "Going to see a lot of pick and roll tonight folks on virtually every play.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "541_1",
        "label": "action_not_executed",
        "text": "Analysts predict heavy pick and roll usage, but on this trip they didn’t run one."
      },
      {
        "new_transcript_id": "541_2",
        "label": "action_executed",
        "text": "Right here they go into another pick and roll, guard uses the screen and attacks."
      }
    ]
  },
  {
    "transcript_id": 542,
    "text": "Hit your opponent in the face but land with the deep post up.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "542_1",
        "label": "action_not_executed",
        "text": "He looked to establish a post up but never actually sealed his man inside."
      },
      {
        "new_transcript_id": "542_2",
        "label": "action_executed",
        "text": "He finds deep position on the block, backing his man down with a strong post up."
      }
    ]
  },
  {
    "transcript_id": 543,
    "text": "Here's Courtney Lee, the veteran the floaters up.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "543_1",
        "label": "action_executed",
        "text": "Lee lofts a soft floater over the defender, and it kisses the front rim before dropping."
      },
      {
        "new_transcript_id": "543_2",
        "label": "action_not_executed",
        "text": "Lee drove into the lane but pulled up for a jumper instead of a floater."
      }
    ]
  },
  {
    "transcript_id": 544,
    "text": "Goes Coast to coast for Toronto.",
    "action": "coast to coast",
    "augmentations": [
      {
        "new_transcript_id": "544_1",
        "label": "action_executed",
        "text": "He grabs the rebound and races coast to coast, finishing with a tough layup."
      },
      {
        "new_transcript_id": "544_2",
        "label": "action_not_executed",
        "text": "He looked like he’d go coast to coast, but pulled it out and waited for teammates."
      }
    ]
  },
  {
    "transcript_id": 547,
    "text": "He can be a threat, especially when teams are looking to double team.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "547_1",
        "label": "action_not_executed",
        "text": "Scouting reports warn about sending a double team, though none actually happened on that trip."
      },
      {
        "new_transcript_id": "547_2",
        "label": "action_executed",
        "text": "The defense quickly collapses, sending two bodies at him in an aggressive double team."
      }
    ]
  },
  {
    "transcript_id": 548,
    "text": "Alakina gets into the paint, stops, double team back out.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "548_1",
        "label": "action_executed",
        "text": "Ntilikina draws the double team in the lane, then kicks it back to the perimeter."
      },
      {
        "new_transcript_id": "548_2",
        "label": "action_not_executed",
        "text": "Ntilikina drove inside but defenders never fully committed a double team, leaving him one-on-one."
      }
    ]
  },
  {
    "transcript_id": 549,
    "text": "Inside that time the finger roll was nicely contested by Steven Adams.",
    "action": "finger roll",
    "augmentations": [
      {
        "new_transcript_id": "549_1",
        "label": "action_executed",
        "text": "He attacked the rim and floated up a finger roll, but Adams got a hand in the way."
      },
      {
        "new_transcript_id": "549_2",
        "label": "action_not_executed",
        "text": "He drove inside but didn’t attempt a finger roll, instead trying a tough bank shot."
      }
    ]
  },
  {
    "transcript_id": 550,
    "text": "Here it goes with the floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "550_1",
        "label": "action_executed",
        "text": "He rises off one foot and arcs a floater high, watching it drop through the net."
      },
      {
        "new_transcript_id": "550_2",
        "label": "action_not_executed",
        "text": "He gathered near the paint but passed out instead of putting up the floater."
      }
    ]
  },
  {
    "transcript_id": 551,
    "text": "Here's favors jab step.",
    "action": "jab step",
    "augmentations": [
      {
        "new_transcript_id": "551_1",
        "label": "action_executed",
        "text": "Favors uses a quick jab step to freeze his defender before driving baseline."
      },
      {
        "new_transcript_id": "551_2",
        "label": "action_not_executed",
        "text": "Favors faked a jab step but didn’t actually execute the move, opting to swing the ball."
      }
    ]
  },
  {
    "transcript_id": 552,
    "text": "He has the ball on the poster going after this match.",
    "action": "poster",
    "augmentations": [
      {
        "new_transcript_id": "552_1",
        "label": "action_not_executed",
        "text": "He was isolated but never rose for a poster dunk against his matchup."
      },
      {
        "new_transcript_id": "552_2",
        "label": "action_executed",
        "text": "He attacks strong and slams over the defender, completing a highlight-reel poster."
      }
    ]
  },
  {
    "transcript_id": 553,
    "text": "If you can't get stops and get an early outlet pass.",
    "action": "outlet pass",
    "augmentations": [
      {
        "new_transcript_id": "553_1",
        "label": "action_not_executed",
        "text": "The break never developed, so there wasn’t an outlet pass to spark transition."
      },
      {
        "new_transcript_id": "553_2",
        "label": "action_executed",
        "text": "Off the rebound, he immediately triggers transition with a crisp outlet pass upcourt."
      }
    ]
  },
  {
    "transcript_id": 554,
    "text": "Dig down to help Tray on that post up.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "554_1",
        "label": "action_not_executed",
        "text": "They sent extra help anticipating a post up, but no actual post move came."
      },
      {
        "new_transcript_id": "554_2",
        "label": "action_executed",
        "text": "Young backs his defender down, executing a determined post up as help rotates late."
      }
    ]
  },
  {
    "transcript_id": 555,
    "text": "Can't shake Blake Griffin with a pump fake now?",
    "action": "pump fake",
    "augmentations": [
      {
        "new_transcript_id": "555_1",
        "label": "action_executed",
        "text": "He tries a pump fake on Griffin, but the veteran doesn’t bite, staying grounded."
      },
      {
        "new_transcript_id": "555_2",
        "label": "action_not_executed",
        "text": "He faced Griffin straight up, taking the shot without using any pump fake."
      }
    ]
  },
  {
    "transcript_id": 556,
    "text": "There's a floater baseline and the bucket is good.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "556_1",
        "label": "action_executed",
        "text": "He drives baseline, floats it up softly, and the floater rattles in for two."
      },
      {
        "new_transcript_id": "556_2",
        "label": "action_not_executed",
        "text": "He attacked baseline but went for a reverse layup, not a floater."
      }
    ]
  },
  {
    "transcript_id": 557,
    "text": "And here's the Dallas native Myles Turner, but one of the things Miles worked on was his post up game.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "557_1",
        "label": "action_executed",
        "text": "Turner establishes himself deep and works into a strong post up, using his size effectively."
      },
      {
        "new_transcript_id": "557_2",
        "label": "action_not_executed",
        "text": "They highlighted his post up training, but in this possession he didn’t attempt one."
      }
    ]
  },
  {
    "transcript_id": 558,
    "text": "An Rubio has been running the show with ease of Lake Hill, Dr Coast to coast and draws a foul.",
    "action": "coast to coast",
    "augmentations": [
      {
        "new_transcript_id": "558_1",
        "label": "action_executed",
        "text": "Rubio grabs the rebound and goes coast to coast, drawing contact at the rim for free throws."
      },
      {
        "new_transcript_id": "558_2",
        "label": "action_not_executed",
        "text": "Rubio slowed near half court and reset the offense instead of pushing coast to coast."
      }
    ]
  },
  {
    "transcript_id": 559,
    "text": "Again, to take it off to a quick start yesterday throwing up alley oop that doesn't workout is picked up.",
    "action": "alley oop",
    "augmentations": [
      {
        "new_transcript_id": "559_1",
        "label": "action_not_executed",
        "text": "They tried setting up the alley oop, but the lob was too high and the play broke down."
      },
      {
        "new_transcript_id": "559_2",
        "label": "action_executed",
        "text": "He catches the lob perfectly and hammers home the alley oop in transition."
      }
    ]
  },
  {
    "transcript_id": 560,
    "text": "Again, to take it off to a quick start yesterday throwing up alley oop that doesn't workout is picked up.",
    "action": "alley oop",
    "augmentations": [
      {
        "new_transcript_id": "560_1",
        "label": "action_not_executed",
        "text": "The lob looked like an alley oop setup, but the timing was off and it failed."
      },
      {
        "new_transcript_id": "560_2",
        "label": "action_executed",
        "text": "The lob is on target, and the finish completes the alley oop with authority."
      }
    ]
  },
  {
    "transcript_id": 561,
    "text": "Not easy steps, little floater and bring it home.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "561_1",
        "label": "action_executed",
        "text": "He takes a couple of tough steps, floats it up and nails the floater through traffic."
      },
      {
        "new_transcript_id": "561_2",
        "label": "action_not_executed",
        "text": "He maneuvered into the lane but opted for a jump shot instead of a floater."
      }
    ]
  },
  {
    "transcript_id": 562,
    "text": "Here comes a double team, this time on Ennis Kanter who could be a piece to deal with inside.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "562_1",
        "label": "action_executed",
        "text": "Kanter feels the pressure as the second defender arrives, forming a solid double team."
      },
      {
        "new_transcript_id": "562_2",
        "label": "action_not_executed",
        "text": "It looked like the defense would double Kanter, but they backed off before trapping him."
      }
    ]
  },
  {
    "transcript_id": 563,
    "text": "Here's Trey with the floater and he crossed the whistle.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "563_1",
        "label": "action_executed",
        "text": "Young drives into the paint and floats it up, drawing a whistle as the floater drops."
      },
      {
        "new_transcript_id": "563_2",
        "label": "action_not_executed",
        "text": "Young drew the foul before attempting the floater, so the shot never left his hands."
      }
    ]
  },
  {
    "transcript_id": 564,
    "text": "Here is right form and working on the pick and roll game.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "564_1",
        "label": "action_executed",
        "text": "They set a solid screen, and the guard uses it to run a crisp pick and roll action."
      },
      {
        "new_transcript_id": "564_2",
        "label": "action_not_executed",
        "text": "They discussed practicing the pick and roll, but it wasn’t executed on this play."
      }
    ]
  },
  {
    "transcript_id": 565,
    "text": "The class goes coast to coast.",
    "action": "coast to coast",
    "augmentations": [
      {
        "new_transcript_id": "565_1",
        "label": "action_executed",
        "text": "He pushes the pace, drives the full length, and scores going coast to coast."
      },
      {
        "new_transcript_id": "565_2",
        "label": "action_not_executed",
        "text": "Looked like he would go coast to coast, but he slowed down and reset the half-court offense."
      }
    ]
  },
  {
    "transcript_id": 566,
    "text": "Van Fleet forces up a runner and hits nothing but net.",
    "action": "nothing but net",
    "augmentations": [
      {
        "new_transcript_id": "566_1",
        "label": "action_executed",
        "text": "VanVleet launches the runner, and it drops straight through, nothing but net."
      },
      {
        "new_transcript_id": "566_2",
        "label": "action_not_executed",
        "text": "VanVleet’s shot looked pure, but it rimmed out—no clean nothing but net this time."
      }
    ]
  },
  {
    "transcript_id": 567,
    "text": "Which three guys can score the ball consistently here in Boston but still too strong with the finger roll?",
    "action": "finger roll",
    "augmentations": [
      {
        "new_transcript_id": "567_1",
        "label": "action_executed",
        "text": "He attacks the rim with a finger roll but it’s too strong, bouncing off the glass."
      },
      {
        "new_transcript_id": "567_2",
        "label": "action_not_executed",
        "text": "He approached the paint but didn’t attempt the finger roll, opting for a pass instead."
      }
    ]
  },
  {
    "transcript_id": 568,
    "text": "Getting that is young off of Kevin Love trying to get him some post up touch.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "568_1",
        "label": "action_executed",
        "text": "They feed Young on the block, and he works into a deliberate post up move."
      },
      {
        "new_transcript_id": "568_2",
        "label": "action_not_executed",
        "text": "They tried to establish Young inside, but he never received the ball for a post up."
      }
    ]
  },
  {
    "transcript_id": 569,
    "text": "I like to double team over the Hill.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "569_1",
        "label": "action_not_executed",
        "text": "They talked about sending the double team, but stayed home and never trapped over the hill."
      },
      {
        "new_transcript_id": "569_2",
        "label": "action_executed",
        "text": "They commit two defenders, springing the double team as Hill tries to cross midcourt."
      }
    ]
  },
  {
    "transcript_id": 570,
    "text": "Fraser had a screen from drama, gives it up to Drummond and back to Frazier Little floater off the window.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "570_1",
        "label": "action_executed",
        "text": "Frazier comes off the high screen, dishes it back, then finishes with a floater banking softly off glass."
      },
      {
        "new_transcript_id": "570_2",
        "label": "action_not_executed",
        "text": "Frazier used the screen and looked for the floater, but the attempt rolled off without finding the glass."
      }
    ]
  },
  {
    "transcript_id": 571,
    "text": "Half court set here for Brooklyn Pick and Roll Action Allen over Go Berry doesn't get the role.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "571_1",
        "label": "action_executed",
        "text": "Brooklyn slowed into the half court, running a pick and roll with Allen diving hard against Gobert."
      },
      {
        "new_transcript_id": "571_2",
        "label": "action_not_executed",
        "text": "Brooklyn tried setting up the pick and roll, but the timing faltered and Allen never completed the action."
      }
    ]
  },
  {
    "transcript_id": 572,
    "text": "Campus pick and roll with seller and that is slap back by Dedmon was very poor shot there by Zella.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "572_1",
        "label": "action_executed",
        "text": "Zeller initiated the pick and roll up top, slipped inside but Dedmon swatted the attempt back emphatically."
      },
      {
        "new_transcript_id": "572_2",
        "label": "action_not_executed",
        "text": "They tried to run the pick and roll with Zeller, but Dedmon read it early and shut the play down."
      }
    ]
  },
  {
    "transcript_id": 573,
    "text": "Games on the Drive shot Clock winding down bank shot Mrs Smart 3 pounds ahead to Tatum.",
    "action": "bank shot",
    "augmentations": [
      {
        "new_transcript_id": "573_1",
        "label": "action_executed",
        "text": "With the clock at two, Smart drove inside and kissed the bank shot home before releasing to Tatum."
      },
      {
        "new_transcript_id": "573_2",
        "label": "action_not_executed",
        "text": "Smart pulled up late for a bank shot, but it clanged away as the shot clock buzzer sounded."
      }
    ]
  },
  {
    "transcript_id": 574,
    "text": "If you leave truly on James Harden and Chris Paul out of the pick and roll, Harden catches fires.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "574_1",
        "label": "action_not_executed",
        "text": "Leaving Harden and Paul together might suggest pick and roll, but this trip Harden simply rose up instead."
      },
      {
        "new_transcript_id": "574_2",
        "label": "action_executed",
        "text": "Harden and Paul executed the pick and roll on the wing, Harden popping free for a clean look."
      }
    ]
  },
  {
    "transcript_id": 577,
    "text": "You can already see that that defensive priority is going to be Bradley Beal, so he's gonna have opportunities floater in the Lane for RJ Barrett.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "577_1",
        "label": "action_executed",
        "text": "Barrett sliced into the paint, ignored the double on Beal, and calmly floated one over the rim for two."
      },
      {
        "new_transcript_id": "577_2",
        "label": "action_not_executed",
        "text": "Barrett looked poised for the floater in the lane, but he kicked it out instead without attempting it."
      }
    ]
  },
  {
    "transcript_id": 578,
    "text": "Little floater would not fall.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "578_1",
        "label": "action_executed",
        "text": "He rose in traffic and released the soft floater, but it rattled out off the back iron."
      },
      {
        "new_transcript_id": "578_2",
        "label": "action_not_executed",
        "text": "He thought about a little floater inside, but pulled it back and reset the offense on the wing."
      }
    ]
  },
  {
    "transcript_id": 579,
    "text": "Rubio off balance along the baseline still splashes it.",
    "action": "splash",
    "augmentations": [
      {
        "new_transcript_id": "579_1",
        "label": "action_executed",
        "text": "Off balance on the baseline fade, Rubio still splashed it straight through the net with confidence."
      },
      {
        "new_transcript_id": "579_2",
        "label": "action_not_executed",
        "text": "Rubio nearly splashed that baseline jumper, but it rimmed out and was cleared by the defense."
      }
    ]
  },
  {
    "transcript_id": 580,
    "text": "Simmons drawing a double team.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "580_1",
        "label": "action_executed",
        "text": "Simmons posted near the nail, quickly drawing a swarming double team from the weak side defenders."
      },
      {
        "new_transcript_id": "580_2",
        "label": "action_not_executed",
        "text": "It looked like Simmons might face a double team, but the help never actually committed on that touch."
      }
    ]
  },
  {
    "transcript_id": 581,
    "text": "But because of his absence, you'll see Ben post up a little bit more and he's exceptional town there.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "581_1",
        "label": "action_not_executed",
        "text": "Commentators note Simmons will likely post up more in this stretch, though on this trip he didn’t."
      },
      {
        "new_transcript_id": "581_2",
        "label": "action_executed",
        "text": "Simmons sealed his defender deep on the block and powered into a strong post up finish."
      }
    ]
  },
  {
    "transcript_id": 582,
    "text": "Simmons the floater doesn't go rebound embid scores and the foul.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "582_1",
        "label": "action_executed",
        "text": "Simmons drove middle, tossed up a floater that bounced out, Embiid cleaning the glass for an and-one."
      },
      {
        "new_transcript_id": "582_2",
        "label": "action_not_executed",
        "text": "Simmons looked like he wanted the floater, but instead dished it off before Embiid drew the foul."
      }
    ]
  },
  {
    "transcript_id": 583,
    "text": "It stays with his shot tips in and it's starting to slow down that run by The Mavericks.",
    "action": "tip in",
    "augmentations": [
      {
        "new_transcript_id": "583_1",
        "label": "action_executed",
        "text": "He followed his own miss perfectly, soaring back up for a clean tip in to halt the Dallas surge."
      },
      {
        "new_transcript_id": "583_2",
        "label": "action_not_executed",
        "text": "He tried for the quick tip in after the miss, but the ball slipped away into Maverick hands."
      }
    ]
  },
  {
    "transcript_id": 584,
    "text": "Look are trying to race past the double team needs.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "584_1",
        "label": "action_executed",
        "text": "Doncic pushed in transition, trying to split the quick double team closing near half court."
      },
      {
        "new_transcript_id": "584_2",
        "label": "action_not_executed",
        "text": "It looked like a double team was coming at Doncic, but the second defender peeled back late."
      }
    ]
  },
  {
    "transcript_id": 585,
    "text": "And 48 pick and roll.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "585_1",
        "label": "action_executed",
        "text": "Fournier orchestrated the pick and roll, using the screen to curl into the paint for a look."
      },
      {
        "new_transcript_id": "585_2",
        "label": "action_not_executed",
        "text": "Fournier started to call the pick and roll, but spacing broke down before it materialized."
      }
    ]
  },
  {
    "transcript_id": 586,
    "text": "Here's Troy Young Works the pick and roll here with death, and as Tobias Harrison's switch it, it goes to Duane.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "586_1",
        "label": "action_executed",
        "text": "Trae Young ran the pick and roll smoothly, drawing the switch and hitting the roll man inside."
      },
      {
        "new_transcript_id": "586_2",
        "label": "action_not_executed",
        "text": "Trae Young called for the pick and roll, but the defense disrupted the screen and they reset."
      }
    ]
  },
  {
    "transcript_id": 587,
    "text": "Pick and roll up high.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "587_1",
        "label": "action_executed",
        "text": "They set a high pick and roll above the arc, guard turning the corner into the lane."
      },
      {
        "new_transcript_id": "587_2",
        "label": "action_not_executed",
        "text": "The high pick and roll was called, but no shot or drive came from it on that set."
      }
    ]
  },
  {
    "transcript_id": 588,
    "text": "And Ben Simmons has a little room service slam dunk Philadelphia on the board 1st.",
    "action": "slam dunk",
    "augmentations": [
      {
        "new_transcript_id": "588_1",
        "label": "action_executed",
        "text": "Simmons leaked out ahead and hammered down an easy slam dunk to get Philly started early."
      },
      {
        "new_transcript_id": "588_2",
        "label": "action_not_executed",
        "text": "Simmons appeared ready for a slam dunk, but the defender fouled hard before he could finish it."
      }
    ]
  },
  {
    "transcript_id": 590,
    "text": "That are anti comes back with a teardrop and puts it in.",
    "action": "teardrop",
    "augmentations": [
      {
        "new_transcript_id": "590_1",
        "label": "action_executed",
        "text": "Giannis responded immediately with a soft teardrop in the paint that fell straight through."
      },
      {
        "new_transcript_id": "590_2",
        "label": "action_not_executed",
        "text": "Giannis looked to float a teardrop, but changed course midair and dished it away instead."
      }
    ]
  },
  {
    "transcript_id": 591,
    "text": "You gonna double team.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "591_1",
        "label": "action_not_executed",
        "text": "They were considering the double team on that possession, but no second defender actually came."
      },
      {
        "new_transcript_id": "591_2",
        "label": "action_executed",
        "text": "The defense quickly sent an extra man, forcing the ball out under the sudden double team."
      }
    ]
  },
  {
    "transcript_id": 592,
    "text": "Vallentuna's with the shot Clock ticking down gets a double team.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "592_1",
        "label": "action_executed",
        "text": "With the clock expiring, Valanciunas drew a late double team, forcing him to kick it away."
      },
      {
        "new_transcript_id": "592_2",
        "label": "action_not_executed",
        "text": "As the clock ran down, Valanciunas expected a double team, but defenders stayed home on shooters."
      }
    ]
  },
  {
    "transcript_id": 593,
    "text": "Bryant has some skills he can handle it and shoot 3 between the legs.",
    "action": "between the legs",
    "augmentations": [
      {
        "new_transcript_id": "593_1",
        "label": "action_executed",
        "text": "Bryant sized up his man, dribbled between the legs, then calmly rose for the outside jumper."
      },
      {
        "new_transcript_id": "593_2",
        "label": "action_not_executed",
        "text": "Bryant faked a dribble between the legs but kept it simple, never pulling off the move that time."
      }
    ]
  },
  {
    "transcript_id": 594,
    "text": "Stay on the pick and roll.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "594_1",
        "label": "action_executed",
        "text": "They went right back to the pick and roll, screen freeing space for the guard to attack inside."
      },
      {
        "new_transcript_id": "594_2",
        "label": "action_not_executed",
        "text": "Coach yelled to stay on the pick and roll, but the play dissolved before the screen was even set."
      }
    ]
  },
  {
    "transcript_id": 596,
    "text": "There's that good double teaming defense by the Rockets hard right at the back of Gibson and Butler fouled Tucker.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "596_1",
        "label": "action_not_executed",
        "text": "The Rockets prepared the double team look, but before they executed it, a foul halted the play."
      },
      {
        "new_transcript_id": "596_2",
        "label": "action_executed",
        "text": "The Rockets collapsed with a textbook double team, forcing Gibson into trouble before the foul."
      }
    ]
  },
  {
    "transcript_id": 598,
    "text": "That'll be critical, and Vince Carter hits nothing but net.",
    "action": "nothing but net",
    "augmentations": [
      {
        "new_transcript_id": "598_1",
        "label": "action_executed",
        "text": "Carter pulled up from deep and drilled it clean, nothing but net to energize the bench."
      },
      {
        "new_transcript_id": "598_2",
        "label": "action_not_executed",
        "text": "Carter launched the shot that looked like nothing but net, but it rimmed off the back iron."
      }
    ]
  },
  {
    "transcript_id": 599,
    "text": "John Collins gets the steal and Dunk Sexton with a floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "599_1",
        "label": "action_executed",
        "text": "After Collins’ steal and finish, Sexton answered with a floater in the lane that dropped in."
      },
      {
        "new_transcript_id": "599_2",
        "label": "action_not_executed",
        "text": "Sexton probed into the lane looking for a floater, but defenders cut him off and he dished out."
      }
    ]
  },
  {
    "transcript_id": 600,
    "text": "Yeah, shake Milton's gonna real good job scoring the basketball in bead with a pump fake and a drive down the Lane.",
    "action": "pump fake",
    "augmentations": [
      {
        "new_transcript_id": "600_1",
        "label": "action_executed",
        "text": "Embiid used a convincing pump fake at the elbow, froze the defense, and attacked straight down the lane."
      },
      {
        "new_transcript_id": "600_2",
        "label": "action_not_executed",
        "text": "Embiid thought about the pump fake, but never sold it, instead quickly swinging the ball away."
      }
    ]
  },
  {
    "transcript_id": 601,
    "text": "Yep, so we gotta watch him getting posted up.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "601_1",
        "label": "action_not_executed",
        "text": "Commentary warns about him getting posted up, though on this trip he never actually did."
      },
      {
        "new_transcript_id": "601_2",
        "label": "action_executed",
        "text": "He sealed his defender in the paint and started the post up action, looking for position inside."
      }
    ]
  },
  {
    "transcript_id": 602,
    "text": "He gets a knock away Aaron Gordon with the Bounce Pass and Isaac with the finger roll Jonathan Isaac in his third season out of Florida State.",
    "action": "finger roll",
    "augmentations": [
      {
        "new_transcript_id": "602_1",
        "label": "action_executed",
        "text": "Isaac took the feed in stride, finishing smooth at the rim with a quick finger roll over the defense."
      },
      {
        "new_transcript_id": "602_2",
        "label": "action_not_executed",
        "text": "Isaac cut toward the rim appearing ready for a finger roll, but he kicked the ball out to the wing instead."
      }
    ]
  },
  {
    "transcript_id": 603,
    "text": "No slam dunk.",
    "action": "slam dunk",
    "augmentations": [
      {
        "new_transcript_id": "603_1",
        "label": "action_executed",
        "text": "He rose up powerfully and hammered home a slam dunk to punctuate the possession."
      },
      {
        "new_transcript_id": "603_2",
        "label": "action_not_executed",
        "text": "He looked to elevate for a slam dunk but lost the ball midair, so the chance slipped away."
      }
    ]
  },
  {
    "transcript_id": 604,
    "text": "Nice post up.",
    "action": "post up",
    "augmentations": [
      {
        "new_transcript_id": "604_1",
        "label": "action_executed",
        "text": "He backed his man down deliberately, sealing deep in the paint for a clean post up bucket."
      },
      {
        "new_transcript_id": "604_2",
        "label": "action_not_executed",
        "text": "He tried to establish a post up position, but the entry pass was denied and they reset."
      }
    ]
  },
  {
    "transcript_id": 605,
    "text": "They believe that he's a good piece and something to really play and look forward to this year, Osman breaks it up on the alley Oop attempt to Jarrett Allen.",
    "action": "alley oop",
    "augmentations": [
      {
        "new_transcript_id": "605_1",
        "label": "action_executed",
        "text": "They lobbed the alley oop toward Allen, but Osman broke it up before the connection could finish."
      },
      {
        "new_transcript_id": "605_2",
        "label": "action_not_executed",
        "text": "They started to look for the alley oop to Allen, but never released the pass under pressure."
      }
    ]
  },
  {
    "transcript_id": 606,
    "text": "Double team in the corner and throws a terrible pass bill I had to Peyton.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "606_1",
        "label": "action_executed",
        "text": "Trapped in the corner by a double team, he floated a bad pass that was nearly stolen."
      },
      {
        "new_transcript_id": "606_2",
        "label": "action_not_executed",
        "text": "He expected a double team in the corner, but the defense stayed single coverage that time."
      }
    ]
  },
  {
    "transcript_id": 607,
    "text": "Some floater good, each one more.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "607_1",
        "label": "action_executed",
        "text": "E’Twaun Moore lofted up the soft floater in the lane, dropping it through with a touch finish."
      },
      {
        "new_transcript_id": "607_2",
        "label": "action_not_executed",
        "text": "Moore drove toward the lane, looked for a floater, but pulled it back without releasing the shot."
      }
    ]
  },
  {
    "transcript_id": 608,
    "text": "No double team.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "608_1",
        "label": "action_not_executed",
        "text": "The defense held steady on that possession, never sending the extra defender for a double team."
      },
      {
        "new_transcript_id": "608_2",
        "label": "action_executed",
        "text": "They quickly collapsed with a double team on the wing, forcing the pass out of his hands."
      }
    ]
  },
  {
    "transcript_id": 609,
    "text": "Middleton floater would not go at Williams who was defending Giannis again, that time with the rebound, got numbers White supporter for free.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "609_1",
        "label": "action_executed",
        "text": "Middleton drove hard and lifted a floater over Williams, but the attempt rolled off the rim."
      },
      {
        "new_transcript_id": "609_2",
        "label": "action_not_executed",
        "text": "Middleton faked the floater, drew the defender, then dished it to the wing for the open three."
      }
    ]
  },
  {
    "transcript_id": 610,
    "text": "Turtle Pass was deflected before he got it and Jodka little floater to get the Spurs on the board.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "610_1",
        "label": "action_executed",
        "text": "After the deflection, Jodka recovered and floated one softly in the lane to score the first points."
      },
      {
        "new_transcript_id": "610_2",
        "label": "action_not_executed",
        "text": "Jodka looked ready for the floater, but the ball was tipped away before he could release it."
      }
    ]
  },
  {
    "transcript_id": 611,
    "text": "As you're watching Simmons, just take it to the rack himself.",
    "action": "take it to the rack",
    "augmentations": [
      {
        "new_transcript_id": "611_1",
        "label": "action_executed",
        "text": "Simmons saw a seam and took it straight to the rack himself for the easy finish at the rim."
      },
      {
        "new_transcript_id": "611_2",
        "label": "action_not_executed",
        "text": "He looked like he wanted to take it to the rack, but settled for a reset instead."
      }
    ]
  },
  {
    "transcript_id": 612,
    "text": "Tough to double team the way the mixer spacing the floor with their shooters.",
    "action": "double team",
    "augmentations": [
      {
        "new_transcript_id": "612_1",
        "label": "action_not_executed",
        "text": "Spacing with shooters made a double team tough, so no second defender actually came across."
      },
      {
        "new_transcript_id": "612_2",
        "label": "action_executed",
        "text": "Despite the spacing, the Knicks collapsed aggressively to double team on that post touch."
      }
    ]
  },
  {
    "transcript_id": 613,
    "text": "Of course they do it at the end of this week with the home and home against Utah Bridges Coast to coast up to turn over.",
    "action": "coast to coast",
    "augmentations": [
      {
        "new_transcript_id": "613_1",
        "label": "action_executed",
        "text": "Bridges went coast to coast off the turnover, sprinting the full length for an uncontested layup."
      },
      {
        "new_transcript_id": "613_2",
        "label": "action_not_executed",
        "text": "Bridges tried to push coast to coast after the turnover, but was cut off before finishing."
      }
    ]
  },
  {
    "transcript_id": 614,
    "text": "Nothing but net for Lillard.",
    "action": "nothing but net",
    "augmentations": [
      {
        "new_transcript_id": "614_1",
        "label": "action_executed",
        "text": "Damian Lillard rose up confidently, drilling the jumper clean through with nothing but net."
      },
      {
        "new_transcript_id": "614_2",
        "label": "action_not_executed",
        "text": "Lillard pulled up looking for nothing but net, but the shot rimmed out late in the clock."
      }
    ]
  },
  {
    "transcript_id": 615,
    "text": "Poor Zynga's driving on Bonga takes it to the rack.",
    "action": "take it to the rack",
    "augmentations": [
      {
        "new_transcript_id": "615_1",
        "label": "action_executed",
        "text": "Porzingis put it on the deck and muscled past Bonga, taking it all the way to the rack."
      },
      {
        "new_transcript_id": "615_2",
        "label": "action_not_executed",
        "text": "Porzingis started toward the rack but dished the ball away before completing the drive."
      }
    ]
  },
  {
    "transcript_id": 616,
    "text": "Attack him at every opportunity Jackson pick and roll with Drummond.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "616_1",
        "label": "action_executed",
        "text": "Jackson initiated the pick and roll with Drummond, freeing him for a strong dive to the basket."
      },
      {
        "new_transcript_id": "616_2",
        "label": "action_not_executed",
        "text": "Jackson signaled for the pick and roll, but the defense switched early and disrupted the play."
      }
    ]
  },
  {
    "transcript_id": 617,
    "text": "You've got to get to more like size matches inside Whiteside on the nice pick and roll Mccollam found him.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "617_1",
        "label": "action_executed",
        "text": "McCollum orchestrated the pick and roll perfectly, dropping it to Whiteside for an easy bucket."
      },
      {
        "new_transcript_id": "617_2",
        "label": "action_not_executed",
        "text": "McCollum gestured for the pick and roll, but Whiteside never freed himself as the defense adjusted."
      }
    ]
  },
  {
    "transcript_id": 618,
    "text": "It's with a floater little bit strong and rebounded by indeed.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "618_1",
        "label": "action_executed",
        "text": "He floated it up from mid-lane, came up a touch strong, and Embiid cleaned the rebound."
      },
      {
        "new_transcript_id": "618_2",
        "label": "action_not_executed",
        "text": "He looked for the floater in the paint, but passed away before releasing the shot."
      }
    ]
  },
  {
    "transcript_id": 619,
    "text": "The breakdown the floater.",
    "action": "floater",
    "augmentations": [
      {
        "new_transcript_id": "619_1",
        "label": "action_executed",
        "text": "Breaking down his man, he rose in the lane and dropped in the floater over the help defense."
      },
      {
        "new_transcript_id": "619_2",
        "label": "action_not_executed",
        "text": "He started to break down the defender, looked ready for a floater, but dished outside instead."
      }
    ]
  },
  {
    "transcript_id": 620,
    "text": "See right off the bat involving joellen, a little pick and roll able.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "620_1",
        "label": "action_not_executed",
        "text": "They mentioned involving Joel in a pick and roll, but that possession never got executed."
      },
      {
        "new_transcript_id": "620_2",
        "label": "action_executed",
        "text": "They opened the game with a pick and roll through Embiid, creating an early scoring chance."
      }
    ]
  },
  {
    "transcript_id": 621,
    "text": "Look at the pump fakes there by.",
    "action": "pump fake",
    "augmentations": [
      {
        "new_transcript_id": "621_1",
        "label": "action_executed",
        "text": "He froze the defender with a slick pump fake, then attacked after creating separation."
      },
      {
        "new_transcript_id": "621_2",
        "label": "action_not_executed",
        "text": "He showed the ball slightly, hinting at a pump fake, but never sold the move completely."
      }
    ]
  },
  {
    "transcript_id": 622,
    "text": "He has that ability to block the South Hill off the bounce Dixit looking move off the pick and roll for George Hill.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "622_1",
        "label": "action_executed",
        "text": "George Hill flowed off the bounce, executing the pick and roll and slipping into space."
      },
      {
        "new_transcript_id": "622_2",
        "label": "action_not_executed",
        "text": "They hinted at setting the pick and roll for Hill, but the ball never went his direction."
      }
    ]
  },
  {
    "transcript_id": 623,
    "text": "He just wants him to take that with confidence in the rhythm off the pick and roll there's Durant.",
    "action": "pick and roll",
    "augmentations": [
      {
        "new_transcript_id": "623_1",
        "label": "action_executed",
        "text": "Durant popped free off the pick and roll, catching in rhythm and knocking down the jumper."
      },
      {
        "new_transcript_id": "623_2",
        "label": "action_not_executed",
        "text": "Durant prepared to use the pick and roll, but instead cleared out to isolate against his man."
      }
    ]
  },
  {
    "transcript_id": 624,
    "text": "If you guys with that trick this year stepping around DeAndre Jordan is Klay Thompson as good anticipation by Thompson and its Grand Slam dunk?",
    "action": "slam dunk",
    "augmentations": [
      {
        "new_transcript_id": "624_1",
        "label": "action_executed",
        "text": "Thompson anticipated perfectly, stepped around Jordan and hammered a grand slam dunk in transition."
      },
      {
        "new_transcript_id": "624_2",
        "label": "action_not_executed",
        "text": "Thompson seemed ready for the slam dunk, but Jordan fouled him before he could rise to finish."
      }
    ]
  }]


  
  










# Gen end

In [ ]:
# Flatten the augmented_data into a list of dicts for the DataFrame
rows = []
for item in augmented_data:
    for aug in item['augmentations']:
        rows.append({
            'transcript_id': item['transcript_id'],
            'origin_text': item['text'],
            'new_transcript_id': aug['new_transcript_id'],
            'action': item['action'],
            'label': 1 if aug['label'] == 'action_executed' else 0,
            'augmented_text': aug['text']
        })

df = pd.DataFrame(rows, columns=['transcript_id', 'origin_text', 'new_transcript_id', 'action', 'label', 'augmented_text'])
ids_mapping = pd.read_csv('data/ids_mapping.csv')
df_merged = df.merge(ids_mapping[['transcript_id', 'sample_id']], on='transcript_id', how='inner').drop('transcript_id', axis=1)
df_merged.to_csv('data/augmented_texts.csv', index=False)